In [1]:
from EMDA import EMDA, __version__
print("EMDA version is:", __version__)

from numpy import average

import matplotlib.pyplot as plt

/Users/mikicanyelles/miniconda3/envs/emda-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EMDA version is: 0.2.0a3


# Load parameters and trajectory <a class="anchor" id="load"></a>

In [2]:
parameters = 'parameters.prmtop' # AMBER parameters file
trajectory = 'trajectory.nc'     # AMBER NetCDF coordinates (10 frames)

In [3]:
emda = EMDA(parameters=parameters, trajectory=trajectory)

Trajectory has been loaded!


# Select the atoms to be used for the analysis <a class="anchor" id="select"></a>

In [4]:
emda.select('C10', 'C10', sel_type='at_name')
emda.select('C11', 'C11', sel_type='at_name')
emda.select('C12', 'C12', sel_type='at_name')
emda.select('C13', 'C13', sel_type='at_name')
emda.select('C14', 'C14', sel_type='at_name')

emda.select('H12', ['H12A', 'H12B'], sel_type='at_name')
emda.select('cof', 10597, sel_type='at_num')

emda.select('COO', [10599, 10600, 10601], sel_type='at_num')

subs = emda.universe.select_atoms('resid 666')

In [5]:
print(emda.selections['C12'])

<AtomGroup [<Atom 10626: C12 of type c3 of resname DHA, resid 666 and segid SYSTEM>]>


# Analysis of the trajectory <a class="anchor" id="trajanalysis"></a>

In [6]:
emda.add_distance('dist_H12', 'cof', 'H12')
emda.add_angle('angle_C11C12C13', 'C11', 'C12', 'C13')
emda.add_dihedral('dihe_C10C11C13C14', 'C10', 'C11', 'C13', 'C14', domain=180)
emda.add_contacts('contacts_COO', 'COO', sel_env=5, interactions='all', include_WAT=True, measure_distances=True)
emda.add_contacts('contacts_prot', 'protein', sel_env=5)
#emda.add_pKa('pka', excluded_ions=['Na+'], keep_pdb=True, keep_pka=True)
emda.add_RMSD('RMSD_subs', subs)

## Run the measurements

In [7]:
print(emda.measures)

{'dist_H12': Measure dataclass with:
	Name:   dist_H12
	Type:   distance
	Sel:    [<AtomGroup with 1 atom>, <AtomGroup with 2 atoms>]
	Status: Not calculated
, 'angle_C11C12C13': Measure dataclass with:
	Name:   angle_C11C12C13
	Type:   angle
	Sel:    [<AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>]
	Status: Not calculated
, 'dihe_C10C11C13C14': Measure dataclass with:
	Name:   dihe_C10C11C13C14
	Type:   dihedral
	Sel:    [<AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>, <AtomGroup with 1 atom>]
	Status: Not calculated
, 'contacts_COO': Measure dataclass with:
	Name:   contacts_COO
	Type:   contacts
	Sel:    [<AtomGroup with 3 atoms>, <AtomGroup with 77 atoms, with selection 'around 5 group select' on the entire Universe.>]
	Status: Not calculated
, 'contacts_prot': Measure dataclass with:
	Name:   contacts_prot
	Type:   contacts
	Sel:    [<AtomGroup with 10510 atoms>, 5]
	Status: Not calculated
, 'RMSD_subs': Measure dataclass with:

In [8]:
emda.run()

Measuring:   0%|          | 0/10 [00:00<?, ?Frame/s]

Measuring: 100%|██████████| 10/10 [00:12<00:00,  1.23s/Frame]


## Analyse distances, angles and dihedrals

In [9]:
emda.analyse_value('dist_H12_bool', 'dist_H12', 5)

In [10]:
print('angle_C11C12C13 average is:', average(emda.measures['angle_C11C12C13'].result))
print('angle_C11C12C13 min is:', min(emda.measures['angle_C11C12C13'].result))
print('angle_C11C12C13 max is:', max(emda.measures['angle_C11C12C13'].result))

angle_C11C12C13 average is: 112.20870900390655
angle_C11C12C13 min is: 107.89308253365704
angle_C11C12C13 max is: 118.05900472297759


In [11]:
emda.analyse_value('angle_C11C12C13_bool', 'angle_C11C12C13', 110, 5, mode='tol')

In [12]:
print(emda.analyses)

{'dist_H12_bool': Analysis dataclass with:
	Name:   dist_H12_bool
	Type:   value
	Related mesure:    dist_H12
	Status: Calculated
, 'angle_C11C12C13_bool': Analysis dataclass with:
	Name:   angle_C11C12C13_bool
	Type:   value
	Related mesure:    angle_C11C12C13
	Status: Calculated
}


In [13]:
emda.analyse_NACs('dist_angle_NACs', ['dist_H12_bool', 'angle_C11C12C13_bool'], inverse=False)


In [14]:
print('dist_H12_bool:\t\t\t', emda.analyses['dist_H12_bool'].result)
print('angle_C11C12C13_bool:\t\t', emda.analyses['angle_C11C12C13_bool'].result)
print('dist_angle_NACs:\t\t', emda.analyses['dist_angle_NACs'].result)

dist_H12_bool:			 [True, True, True, True, True, True, True, True, True, True]
angle_C11C12C13_bool:		 [True, True, True, True, True, False, True, True, False, True]
dist_angle_NACs:		 [True, True, True, True, True, False, True, True, False, True]


## Analyse contacts

In [15]:
emda.analyse_contacts_frequency('contacts_COO_freq', 'contacts_COO')
emda.analyse_contacts_frequency('contacts_COO_freq_perc', 'contacts_COO', percentage=True)

In [16]:
print(emda.analyses['contacts_COO_freq'].result)
print(emda.analyses['contacts_COO_freq_perc'].result)

{'PHE174': 10, 'LEU178': 10, 'ARG402': 10, 'GLN406': 10, 'LEU407': 1, 'WAT1926': 10, 'WAT2448': 3, 'WAT2882': 2, 'WAT4132': 10, 'WAT6949': 3, 'WAT9053': 3, 'WAT10696': 8, 'WAT13753': 10, 'WAT13796': 7, 'WAT20116': 5, 'WAT20580': 10, 'WAT21098': 1, 'WAT21107': 10, 'WAT22202': 8, 'WAT10954': 2, 'WAT18329': 7, 'WAT20907': 3, 'WAT7078': 2, 'WAT21978': 6, 'WAT14657': 6, 'WAT1514': 1, 'WAT3156': 4, 'WAT5929': 2, 'WAT12490': 2}
{'PHE174': 100.0, 'LEU178': 100.0, 'ARG402': 100.0, 'GLN406': 100.0, 'LEU407': 10.0, 'WAT1926': 100.0, 'WAT2448': 30.0, 'WAT2882': 20.0, 'WAT4132': 100.0, 'WAT6949': 30.0, 'WAT9053': 30.0, 'WAT10696': 80.0, 'WAT13753': 100.0, 'WAT13796': 70.0, 'WAT20116': 50.0, 'WAT20580': 100.0, 'WAT21098': 10.0, 'WAT21107': 100.0, 'WAT22202': 80.0, 'WAT10954': 20.0, 'WAT18329': 70.0, 'WAT20907': 30.0, 'WAT7078': 20.0, 'WAT21978': 60.0, 'WAT14657': 60.0, 'WAT1514': 10.0, 'WAT3156': 40.0, 'WAT5929': 20.0, 'WAT12490': 20.0}


In [19]:
emda.analyse_contacts_amount('contacts_COO_amount', 'contacts_COO')

In [20]:
print(emda.analyses['contacts_COO_amount'].result)

[19, 17, 19, 15, 15, 17, 19, 14, 15, 16]


In [24]:
emda.analyse_contacts_frequency('contacts_prot_freq', 'contacts_prot')
emda.analyse_contacts_amount('contacts_prot_amount', 'contacts_prot')

In [25]:
print(emda.analyses['contacts_prot_freq'].result)
print(emda.analyses['contacts_prot_amount'].result)

{'MET1': {'PRO86': 10, 'VAL53': 10, 'ALA54': 10, 'ASP56': 10, 'TYR4': 9, 'LEU57': 7, 'GLY2': 10, 'ARG3': 10, 'GLU55': 10}, 'GLY2': {'PRO86': 10, 'VAL53': 10, 'ASP52': 10, 'ALA54': 10, 'GLY85': 2, 'TYR4': 10, 'ARG3': 10, 'HID51': 8, 'GLU55': 2}, 'ARG3': {'MET1': 10, 'PRO86': 10, 'GLN84': 10, 'ASP52': 10, 'VAL53': 10, 'GLY85': 10, 'TYR4': 10, 'GLY87': 5, 'ARG5': 10, 'HID51': 10, 'ASP50': 10}, 'TYR4': {'PRO86': 10, 'GLN84': 10, 'ASP52': 10, 'VAL53': 10, 'ASP56': 8, 'ARG5': 10, 'LEU60': 10, 'MET1': 9, 'CYS89': 1, 'GLY85': 10, 'LEU57': 10, 'LEU26': 10, 'ILE6': 10, 'PHE49': 10, 'ASP50': 10, 'HID51': 10, 'GLY58': 5, 'VAL92': 8, 'VAL83': 10, 'GLU55': 9, 'GLY2': 10, 'LEU35': 10}, 'ARG5': {'ARG7': 10, 'THR82': 10, 'GLN84': 10, 'GLU48': 7, 'GLY85': 10, 'ARG3': 10, 'PHE49': 10, 'ILE6': 10, 'ASP50': 10, 'HID51': 10, 'VAL83': 10}, 'ILE6': {'LEU37': 10, 'ARG7': 10, 'GLU47': 10, 'THR82': 10, 'GLN84': 10, 'VAL83': 10, 'TYR4': 10, 'VAL8': 10, 'VAL63': 9, 'ILE81': 10, 'LEU24': 10, 'LEU26': 10, 'PHE49': 1

In [1]:
a = {'MET1': {'PRO86': 10, 'VAL53': 10, 'ALA54': 10, 'ASP56': 10, 'TYR4': 9, 'LEU57': 7, 'GLY2': 10, 'ARG3': 10, 'GLU55': 10}, 'GLY2': {'PRO86': 10, 'VAL53': 10, 'ASP52': 10, 'ALA54': 10, 'GLY85': 2, 'TYR4': 10, 'ARG3': 10, 'HID51': 8, 'GLU55': 2}, 'ARG3': {'MET1': 10, 'PRO86': 10, 'GLN84': 10, 'ASP52': 10, 'VAL53': 10, 'GLY85': 10, 'TYR4': 10, 'GLY87': 5, 'ARG5': 10, 'HID51': 10, 'ASP50': 10}, 'TYR4': {'PRO86': 10, 'GLN84': 10, 'ASP52': 10, 'VAL53': 10, 'ASP56': 8, 'ARG5': 10, 'LEU60': 10, 'MET1': 9, 'CYS89': 1, 'GLY85': 10, 'LEU57': 10, 'LEU26': 10, 'ILE6': 10, 'PHE49': 10, 'ASP50': 10, 'HID51': 10, 'GLY58': 5, 'VAL92': 8, 'VAL83': 10, 'GLU55': 9, 'GLY2': 10, 'LEU35': 10}, 'ARG5': {'ARG7': 10, 'THR82': 10, 'GLN84': 10, 'GLU48': 7, 'GLY85': 10, 'ARG3': 10, 'PHE49': 10, 'ILE6': 10, 'ASP50': 10, 'HID51': 10, 'VAL83': 10}, 'ILE6': {'LEU37': 10, 'ARG7': 10, 'GLU47': 10, 'THR82': 10, 'GLN84': 10, 'VAL83': 10, 'TYR4': 10, 'VAL8': 10, 'VAL63': 9, 'ILE81': 10, 'LEU24': 10, 'LEU26': 10, 'PHE49': 10, 'GLU48': 10, 'LEU35': 10, 'ASP50': 10, 'HID51': 10, 'LEU39': 10}, 'ARG7': {'LEU39': 7, 'GLU47': 10, 'THR82': 10, 'ALA9': 10, 'VAL8': 10, 'ILE81': 10, 'ARG5': 10, 'PHE49': 10, 'GLU48': 10, 'GLU46': 10, 'VAL83': 10, 'ARG80': 10}, 'VAL8': {'LEU24': 10, 'THR10': 10, 'THR82': 10, 'ASP79': 8, 'ILE6': 10, 'GLU48': 10, 'PHE49': 3, 'TRP76': 10, 'ILE81': 10, 'LEU39': 10, 'ARG80': 10, 'VAL22': 10, 'GLU47': 10, 'CYS78': 8, 'LEU65': 1, 'ALA9': 10, 'GLU46': 10, 'GLU45': 10, 'PRO41': 10}, 'ALA9': {'ARG7': 10, 'ARG80': 10, 'GLU47': 10, 'CYS78': 10, 'THR82': 7, 'TRP76': 10, 'GLY11': 10, 'ILE81': 10, 'ASP79': 10, 'GLU46': 10, 'GLU45': 10, 'PHE77': 6, 'THR10': 10, 'PRO41': 9}, 'THR10': {'VAL8': 10, 'ASP79': 10, 'GLU45': 10, 'SER16': 10, 'ARG43': 10, 'TRP76': 10, 'SER18': 10, 'GLY11': 10, 'GLY44': 10, 'TRP13': 1, 'GLY17': 10, 'ALA42': 10, 'ARG80': 4, 'GLU47': 9, 'CYS78': 9, 'ALA12': 10, 'GLU46': 10, 'PHE77': 10, 'PRO41': 10}, 'GLY11': {'SER16': 10, 'ARG43': 10, 'TRP76': 2, 'ALA9': 10, 'GLY44': 10, 'ALA12': 10, 'ASP79': 10, 'TRP13': 10, 'GLY17': 1, 'GLU45': 10, 'PHE77': 10, 'ALA42': 4}, 'ALA12': {'SER16': 10, 'ARG43': 10, 'GLY44': 10, 'TRP13': 10, 'LEU14': 10, 'GLU45': 3, 'THR10': 10}, 'TRP13': {'SER16': 10, 'ARG43': 10, 'LYS611': 8, 'GLY11': 10, 'GLY44': 8, 'GLU169': 5, 'ASP79': 2, 'GLU612': 10, 'THR10': 1, 'GLU168': 10, 'LEU14': 10, 'GLY17': 9, 'PHE15': 10, 'PHE77': 10, 'HIE167': 10, 'VAL605': 2, 'ASP603': 10}, 'LEU14': {'SER16': 10, 'ARG43': 10, 'GLU169': 10, 'ALA12': 10, 'GLU168': 10, 'PHE15': 10, 'LEU172': 10, 'HIE167': 10}, 'PHE15': {'SER16': 10, 'ARG43': 10, 'ALA75': 2, 'TYR19': 6, 'TRP13': 10, 'GLU168': 10, 'GLY17': 10, 'LEU172': 7, 'PHE77': 9, 'ARG171': 2}, 'SER16': {'ARG43': 10, 'ALA75': 9, 'TRP76': 10, 'SER18': 10, 'GLY11': 10, 'GLY44': 5, 'ALA12': 10, 'TYR19': 2, 'TRP13': 10, 'LEU14': 10, 'GLY17': 10, 'PHE77': 10, 'THR10': 10}, 'GLY17': {'ARG43': 10, 'ALA75': 10, 'TRP76': 10, 'SER18': 10, 'ASN20': 1, 'LYS67': 8, 'TYR19': 10, 'GLY44': 2, 'GLY11': 1, 'TRP13': 9, 'PHE15': 10, 'PHE77': 7, 'ALA42': 10, 'THR10': 10, 'PRO41': 10}, 'SER18': {'ARG21': 10, 'SER16': 10, 'ARG43': 10, 'ALA75': 10, 'HIE68': 1, 'TRP76': 10, 'ASN20': 10, 'LYS67': 10, 'TYR19': 10, 'ARG40': 10, 'ASP74': 3, 'VAL22': 10, 'LEU39': 6, 'PHE77': 7, 'ALA42': 10, 'THR10': 10, 'PRO41': 10}, 'TYR19': {'ARG21': 10, 'SER16': 2, 'ARG43': 1, 'ASN20': 10, 'LYS67': 10, 'ARG40': 10, 'ASP74': 9, 'GLY17': 10, 'PHE15': 6, 'PRO41': 5}, 'ASN20': {'ARG21': 10, 'TRP70': 10, 'HIE68': 10, 'ASP73': 2, 'SER18': 10, 'LYS67': 10, 'HID69': 10, 'ARG40': 10, 'ASP74': 10, 'GLY17': 1, 'VAL22': 10, 'LEU39': 10, 'PRO41': 10}, 'ARG21': {'LEU37': 10, 'GLN38': 10, 'GLN23': 10, 'HIE68': 10, 'SER18': 10, 'ARG66': 10, 'GLU36': 10, 'LYS67': 10, 'TYR19': 10, 'ARG40': 10, 'LEU24': 5, 'GLU34': 5, 'LEU35': 10, 'VAL22': 10, 'LEU39': 10, 'PRO41': 10}, 'VAL22': {'LEU37': 6, 'GLN38': 3, 'CYS78': 1, 'GLN23': 10, 'LEU65': 10, 'SER18': 10, 'ARG66': 10, 'ASN20': 10, 'VAL8': 10, 'LYS67': 10, 'HIE68': 10, 'TRP76': 10, 'ARG40': 10, 'LEU24': 10, 'ILE81': 10, 'VAL100': 7, 'LEU39': 10, 'PRO41': 10}, 'GLN23': {'ARG21': 10, 'HIE68': 10, 'LEU65': 10, 'ARG64': 10, 'ARG66': 10, 'LYS67': 10, 'HID69': 3, 'LEU24': 10, 'GLU34': 4, 'ILE105': 8, 'LEU35': 3, 'TRP25': 10, 'LEU39': 3}, 'LEU24': {'TRP25': 10, 'LEU37': 10, 'VAL8': 10, 'LEU26': 10, 'ILE6': 10, 'ARG21': 5, 'ALA33': 10, 'TRP76': 8, 'ARG64': 10, 'ILE81': 10, 'LEU108': 1, 'VAL83': 1, 'LEU39': 10, 'LEU65': 10, 'ARG66': 10, 'VAL63': 10, 'GLU34': 10, 'LEU35': 10, 'VAL22': 10}, 'TRP25': {'ALA33': 10, 'PHE62': 10, 'GLN23': 10, 'LEU65': 10, 'ARG64': 10, 'VAL27': 10, 'ARG66': 10, 'VAL63': 10, 'GLU34': 10, 'ILE105': 10, 'LEU26': 10, 'LEU106': 1, 'LEU35': 10, 'GLU32': 10}, 'LEU26': {'VAL53': 10, 'LEU24': 10, 'LEU60': 10, 'PHE62': 10, 'PHE94': 10, 'LEU57': 10, 'GLY28': 10, 'ILE6': 10, 'HID51': 2, 'ALA33': 10, 'VAL92': 2, 'ARG64': 10, 'ILE81': 9, 'GLY31': 10, 'VAL83': 10, 'GLU32': 10, 'TYR4': 10, 'VAL27': 10, 'VAL63': 10, 'GLU34': 10, 'LEU35': 10}, 'VAL27': {'MET123': 10, 'ALA33': 10, 'PHE94': 4, 'ARG64': 10, 'LEU57': 10, 'GLY28': 10, 'VAL63': 10, 'HID127': 7, 'THR29': 10, 'LEU60': 10, 'GLN61': 10, 'LEU59': 7, 'SER107': 2, 'PHE62': 10, 'TRP25': 10, 'GLY31': 10, 'GLU32': 10}, 'GLY28': {'MET123': 10, 'ARG30': 10, 'LEU57': 10, 'THR29': 10, 'LEU60': 10, 'GLN61': 10, 'LEU26': 10, 'LEU59': 10, 'PHE62': 10, 'GLY31': 10, 'GLY58': 10, 'GLU32': 10}, 'THR29': {'MET123': 10, 'ARG30': 10, 'VAL27': 10, 'LEU57': 10, 'LEU121': 3, 'LEU60': 10, 'GLN61': 10, 'LEU59': 10, 'GLY31': 10, 'GLY58': 10}, 'ARG30': {'ASP56': 10, 'GLY58': 10, 'LEU57': 10, 'GLY28': 10, 'LEU59': 10, 'GLY31': 10, 'GLU32': 10, 'GLU55': 10}, 'GLY31': {'MET123': 2, 'ALA33': 10, 'ASP56': 10, 'VAL27': 10, 'LEU57': 10, 'GLY28': 10, 'THR29': 10, 'LEU26': 10, 'GLU55': 10, 'GLY58': 7, 'GLU32': 10}, 'GLU32': {'ARG30': 10, 'ALA33': 10, 'ARG64': 10, 'VAL27': 10, 'LEU57': 10, 'GLY28': 10, 'GLU34': 10, 'LEU26': 10, 'TRP25': 10, 'GLU55': 10}, 'ALA33': {'VAL53': 10, 'ALA54': 2, 'VAL27': 10, 'LEU57': 10, 'LEU24': 10, 'GLU34': 10, 'LEU26': 10, 'HID51': 2, 'LEU35': 10, 'TRP25': 10, 'GLY31': 10, 'GLU55': 10}, 'GLU34': {'ARG21': 5, 'VAL53': 7, 'GLN23': 4, 'ASP52': 1, 'GLU36': 10, 'LEU24': 10, 'LEU26': 10, 'HID51': 10, 'LEU35': 10, 'TRP25': 10, 'GLU32': 10}, 'LEU35': {'ARG21': 10, 'LEU37': 10, 'ALA33': 10, 'ASP52': 8, 'VAL53': 10, 'VAL83': 9, 'GLN23': 3, 'TYR4': 10, 'LEU57': 1, 'GLU36': 10, 'VAL63': 8, 'LEU24': 10, 'LEU26': 10, 'ILE6': 10, 'HID51': 10, 'TRP25': 10, 'LEU39': 8}, 'GLU36': {'ARG21': 10, 'LEU37': 10, 'GLN38': 10, 'GLU34': 10, 'HID51': 10}, 'LEU37': {'ARG21': 10, 'GLN38': 10, 'LEU24': 10, 'ILE6': 10, 'PHE49': 10, 'LEU35': 10, 'VAL22': 6, 'ASP50': 10, 'LEU39': 10, 'HID51': 10}, 'GLN38': {'ARG21': 10, 'GLU36': 10, 'ARG40': 10, 'PHE49': 10, 'VAL22': 3, 'LEU39': 10}, 'LEU39': {'ARG21': 10, 'LEU37': 10, 'ARG7': 7, 'GLU47': 10, 'GLN23': 3, 'TRP76': 3, 'SER18': 6, 'ASN20': 10, 'VAL8': 10, 'ILE81': 7, 'ARG40': 10, 'LEU24': 10, 'ILE6': 10, 'GLU48': 8, 'LEU35': 8, 'VAL22': 10, 'PHE49': 10, 'PRO41': 10}, 'ARG40': {'ARG21': 10, 'GLN38': 10, 'GLU47': 10, 'SER18': 10, 'ASN20': 10, 'TYR19': 10, 'PHE49': 9, 'VAL22': 10, 'ALA42': 10, 'PRO41': 10}, 'PRO41': {'ARG21': 10, 'GLU45': 7, 'ARG43': 10, 'GLU47': 10, 'TRP76': 10, 'ALA9': 9, 'SER18': 10, 'ASN20': 10, 'VAL8': 10, 'TYR19': 5, 'GLU46': 3, 'PHE49': 1, 'GLY17': 10, 'VAL22': 10, 'LEU39': 10, 'ALA42': 10, 'THR10': 10}, 'ALA42': {'ARG43': 10, 'GLU47': 10, 'TRP76': 4, 'SER18': 10, 'GLY44': 10, 'ARG40': 10, 'GLY11': 4, 'GLY17': 10, 'GLU45': 10, 'THR10': 10}, 'ARG43': {'SER16': 10, 'SER18': 10, 'GLY11': 10, 'GLY44': 10, 'ALA12': 10, 'GLU169': 10, 'TYR19': 1, 'TRP13': 10, 'LEU14': 10, 'GLY17': 10, 'PHE15': 10, 'GLU45': 10, 'THR10': 10, 'PRO41': 10}, 'GLY44': {'SER16': 5, 'GLY11': 10, 'ALA12': 10, 'TRP13': 8, 'GLU46': 10, 'GLY17': 2, 'GLU45': 10, 'ALA42': 10, 'THR10': 10}, 'GLU45': {'ARG43': 10, 'GLU47': 10, 'ALA9': 10, 'VAL8': 10, 'GLY11': 10, 'ALA12': 3, 'GLU46': 10, 'ALA42': 10, 'THR10': 10, 'PRO41': 7}, 'GLU46': {'ARG7': 10, 'ARG80': 10, 'GLU47': 10, 'THR82': 10, 'ALA9': 10, 'VAL8': 10, 'GLY44': 10, 'ILE81': 2, 'GLU48': 10, 'THR10': 10, 'PRO41': 3}, 'GLU47': {'ARG7': 10, 'ALA9': 10, 'VAL8': 10, 'ARG40': 10, 'ILE6': 10, 'GLU48': 10, 'PHE49': 10, 'GLU45': 10, 'LEU39': 10, 'ALA42': 10, 'THR10': 9, 'PRO41': 10}, 'GLU48': {'ARG7': 10, 'VAL8': 10, 'ARG5': 7, 'ILE6': 10, 'PHE49': 10, 'ASP50': 10, 'GLU46': 10, 'LEU39': 8}, 'PHE49': {'LEU37': 10, 'ARG7': 10, 'GLN38': 10, 'GLU47': 10, 'TYR4': 10, 'VAL8': 3, 'ARG5': 10, 'ARG40': 9, 'ILE6': 10, 'ASP50': 10, 'HID51': 10, 'LEU39': 10, 'PRO41': 1}, 'ASP50': {'LEU37': 10, 'ASP52': 10, 'TYR4': 10, 'ARG3': 10, 'ARG5': 10, 'ILE6': 10, 'GLU48': 10, 'HID51': 10}, 'HID51': {'LEU37': 10, 'ALA33': 2, 'ASP52': 10, 'VAL53': 10, 'TYR4': 10, 'GLU36': 10, 'GLY2': 8, 'ARG3': 10, 'ARG5': 10, 'GLU34': 10, 'LEU26': 2, 'ILE6': 10, 'PHE49': 10, 'LEU35': 10}, 'ASP52': {'VAL53': 10, 'ALA54': 10, 'TYR4': 10, 'GLY2': 10, 'ARG3': 10, 'GLU34': 1, 'ASP50': 10, 'LEU35': 8}, 'VAL53': {'MET1': 10, 'PRO86': 3, 'ALA33': 10, 'ALA54': 10, 'ASP56': 10, 'TYR4': 10, 'LEU57': 10, 'GLY2': 10, 'ARG3': 10, 'GLU34': 7, 'LEU26': 10, 'HID51': 10, 'LEU35': 10, 'GLU55': 10}, 'ALA54': {'MET1': 10, 'ALA33': 2, 'ASP52': 10, 'ASP56': 10, 'GLY2': 10, 'GLU55': 10}, 'GLU55': {'MET1': 10, 'ARG30': 10, 'ALA33': 10, 'VAL53': 10, 'ASP56': 10, 'TYR4': 9, 'LEU57': 10, 'GLY2': 2, 'GLY31': 10, 'GLU32': 10}, 'ASP56': {'MET1': 10, 'ARG30': 10, 'VAL53': 10, 'ALA54': 10, 'TYR4': 8, 'LEU57': 10, 'GLY31': 10, 'GLY58': 10}, 'LEU57': {'MET1': 7, 'ARG30': 10, 'ALA33': 10, 'VAL53': 10, 'TYR4': 10, 'VAL27': 10, 'GLY28': 10, 'THR29': 10, 'LEU60': 10, 'GLN61': 2, 'LEU26': 10, 'LEU59': 10, 'LEU35': 1, 'GLY31': 10, 'GLU55': 10, 'GLY58': 10, 'GLU32': 10}, 'GLY58': {'ARG30': 10, 'ASP56': 10, 'TYR4': 5, 'GLY28': 10, 'THR29': 10, 'LEU60': 10, 'GLN61': 3, 'LEU59': 10, 'GLY31': 7}, 'LEU59': {'MET123': 10, 'ARG30': 10, 'VAL92': 4, 'VAL27': 7, 'LEU57': 10, 'GLY28': 10, 'THR29': 10, 'LEU60': 10, 'GLN61': 10, 'ARG114': 4, 'PHE124': 2, 'PHE62': 1}, 'LEU60': {'MET123': 5, 'GLN84': 1, 'PHE94': 10, 'VAL92': 10, 'GLU110': 10, 'GLY85': 2, 'TYR4': 10, 'VAL27': 10, 'GLY28': 10, 'LEU57': 10, 'VAL63': 10, 'THR29': 10, 'GLN61': 10, 'LEU26': 10, 'PHE62': 10, 'VAL83': 10, 'GLY58': 10}, 'GLN61': {'MET123': 10, 'LEU108': 10, 'PHE94': 10, 'VAL92': 10, 'GLU110': 10, 'VAL27': 10, 'LEU57': 2, 'GLY28': 10, 'VAL63': 10, 'THR29': 10, 'PRO109': 10, 'ARG114': 10, 'PHE124': 10, 'LEU59': 10, 'ASP122': 2, 'PHE62': 10, 'GLY58': 3}, 'PHE62': {'GLU110': 10, 'ARG128': 5, 'LEU60': 10, 'LEU59': 1, 'TRP25': 10, 'MET123': 10, 'PHE94': 10, 'GLY28': 10, 'LEU26': 10, 'LEU382': 10, 'VAL92': 3, 'ARG64': 10, 'SER107': 10, 'LEU108': 10, 'VAL27': 10, 'VAL63': 10, 'HID127': 10, 'PRO109': 10, 'PHE124': 10}, 'VAL63': {'LEU108': 10, 'PHE94': 10, 'GLU110': 1, 'LEU65': 10, 'ARG64': 10, 'VAL27': 10, 'ILE81': 10, 'PRO109': 10, 'LEU24': 10, 'LEU60': 10, 'GLN61': 10, 'LEU26': 10, 'SER107': 10, 'ILE6': 9, 'LEU106': 10, 'LEU35': 8, 'TRP25': 10, 'VAL83': 8}, 'ARG64': {'LEU108': 10, 'GLN23': 10, 'LEU65': 10, 'VAL27': 10, 'ARG66': 10, 'HID127': 10, 'LEU24': 10, 'ILE105': 10, 'LYS130': 1, 'LEU26': 10, 'SER107': 10, 'LEU106': 10, 'PHE62': 10, 'TRP25': 10, 'GLU32': 10}, 'LEU65': {'GLN23': 10, 'LEU24': 10, 'TRP25': 10, 'ASP104': 10, 'VAL100': 10, 'VAL8': 1, 'CYS96': 7, 'TRP76': 10, 'ILE81': 10, 'ARG98': 8, 'ILE105': 10, 'SER107': 10, 'LEU106': 10, 'LEU108': 10, 'CYS78': 10, 'ARG66': 10, 'VAL63': 10, 'LYS67': 10, 'VAL22': 10}, 'ARG66': {'ARG21': 10, 'GLN23': 10, 'HIE68': 10, 'ARG64': 10, 'GLN101': 1, 'ASP104': 10, 'VAL100': 10, 'LYS67': 10, 'HID69': 2, 'TRP76': 1, 'LEU24': 10, 'ILE105': 10, 'GLU103': 10, 'LEU106': 10, 'VAL22': 10, 'TRP25': 10, 'GLY102': 10}, 'LYS67': {'ALA75': 10, 'GLN23': 10, 'HIE68': 10, 'HID69': 10, 'ASP104': 1, 'VAL100': 10, 'TYR19': 10, 'ASP74': 10, 'ARG21': 10, 'TRP70': 1, 'TRP76': 10, 'SER18': 10, 'ASN20': 10, 'GLY17': 8, 'LEU106': 9, 'GLY102': 10, 'ASP73': 6, 'LEU65': 10, 'GLN101': 10, 'VAL22': 10}, 'HIE68': {'ARG21': 10, 'TRP70': 10, 'GLN23': 10, 'SER18': 1, 'ARG66': 10, 'ASN20': 10, 'HID69': 10, 'ASP74': 9, 'VAL22': 10}, 'HID69': {'ALA75': 7, 'VAL72': 10, 'TRP70': 10, 'ASP73': 10, 'GLN23': 3, 'GLN101': 10, 'ARG66': 2, 'ASN20': 10, 'VAL100': 1, 'LYS67': 10, 'ASP74': 10, 'GLU103': 10, 'LEU71': 10, 'GLY102': 10}, 'TRP70': {'VAL72': 10, 'ASP73': 10, 'HIE68': 10, 'ASN20': 10, 'LYS67': 1, 'ASP74': 4, 'LEU71': 10}, 'LEU71': {'VAL72': 10, 'HID69': 10, 'ASP73': 10}, 'VAL72': {'ASP74': 10, 'HID69': 10, 'ASP73': 10, 'TRP70': 10}, 'ASP73': {'ALA75': 10, 'TRP70': 10, 'GLN101': 10, 'ASN20': 2, 'HID69': 10, 'LYS67': 6, 'ASP74': 10, 'LEU71': 10, 'GLY102': 5}, 'ASP74': {'ALA75': 10, 'VAL72': 10, 'TRP70': 4, 'HIE68': 9, 'TRP76': 10, 'GLN101': 10, 'SER18': 3, 'ASN20': 10, 'VAL100': 8, 'HID69': 10, 'LYS67': 10, 'TYR19': 9, 'GLY102': 1}, 'ALA75': {'SER16': 9, 'ASP73': 10, 'TRP76': 10, 'SER18': 10, 'GLN101': 10, 'VAL100': 10, 'HID69': 7, 'LYS67': 10, 'TRP99': 10, 'GLY17': 10, 'PHE15': 2, 'PHE77': 10}, 'TRP76': {'LEU24': 8, 'THR10': 10, 'VAL8': 10, 'VAL100': 10, 'ASP74': 10, 'ASP79': 10, 'SER16': 10, 'SER18': 10, 'GLY11': 2, 'ILE81': 10, 'ARG98': 10, 'TRP99': 10, 'GLY17': 10, 'LEU106': 1, 'LEU39': 3, 'ALA42': 4, 'ARG80': 10, 'CYS78': 10, 'LEU65': 10, 'ALA9': 10, 'GLN101': 10, 'ARG66': 1, 'LYS67': 10, 'VAL22': 10, 'PHE77': 10, 'PRO41': 10}, 'PHE77': {'ALA75': 10, 'ARG165': 2, 'PHE15': 9, 'HIE167': 1, 'THR10': 10, 'VAL100': 10, 'ASP79': 10, 'SER16': 10, 'SER18': 7, 'GLY11': 10, 'ARG98': 10, 'TRP13': 10, 'TRP99': 10, 'GLY17': 7, 'CYS78': 10, 'TYR395': 1, 'ALA9': 6, 'PRO162': 10, 'ASN163': 10}, 'CYS78': {'ARG80': 10, 'PHE94': 1, 'TRP76': 10, 'ALA9': 10, 'LEU65': 10, 'VAL8': 8, 'VAL100': 10, 'ILE81': 10, 'ASP79': 10, 'ARG98': 10, 'LEU106': 6, 'ASN163': 1, 'TRP99': 10, 'TYR97': 10, 'LEU108': 10, 'VAL22': 1, 'CYS96': 10, 'THR10': 9}, 'ASP79': {'ARG80': 10, 'PHE94': 9, 'TRP76': 10, 'ALA9': 10, 'CYS96': 10, 'VAL8': 8, 'GLY11': 10, 'ILE81': 10, 'ARG98': 10, 'GLU612': 3, 'TRP13': 2, 'TYR97': 10, 'PHE77': 10, 'PRO95': 10, 'THR10': 10}, 'ARG80': {'ARG7': 10, 'THR82': 10, 'CYS78': 10, 'PHE94': 10, 'TRP76': 10, 'ALA9': 10, 'CYS96': 10, 'VAL8': 10, 'ILE81': 10, 'SER616': 4, 'ALA93': 10, 'LEU108': 8, 'GLU46': 10, 'PRO95': 10, 'THR10': 4}, 'ILE81': {'LEU24': 10, 'ALA93': 10, 'PRO95': 10, 'THR82': 10, 'PHE94': 10, 'VAL8': 10, 'ASP79': 10, 'LEU26': 9, 'ILE6': 10, 'CYS96': 10, 'ARG7': 10, 'VAL92': 10, 'TRP76': 10, 'LEU108': 10, 'VAL83': 10, 'LEU39': 7, 'CYS78': 10, 'LEU65': 10, 'ALA9': 10, 'VAL63': 10, 'GLU46': 2, 'VAL22': 10}, 'THR82': {'ARG7': 10, 'GLN84': 10, 'PHE94': 10, 'VAL92': 10, 'ALA9': 7, 'VAL8': 10, 'ARG5': 10, 'ALA93': 10, 'ILE6': 10, 'GLU46': 10, 'GLU91': 10, 'VAL83': 10, 'PRO95': 1, 'ARG80': 10}, 'VAL83': {'ARG7': 10, 'GLN84': 10, 'PHE94': 10, 'VAL92': 10, 'GLY85': 10, 'ALA90': 4, 'TYR4': 10, 'VAL63': 8, 'ILE81': 10, 'ARG5': 10, 'LEU60': 10, 'LEU24': 1, 'LEU26': 10, 'ALA93': 10, 'ILE6': 10, 'LEU35': 9, 'GLU91': 10}, 'GLN84': {'PRO86': 10, 'THR82': 10, 'CYS89': 10, 'GLY85': 10, 'ALA90': 10, 'TYR4': 10, 'VAL92': 10, 'GLY87': 10, 'ARG3': 10, 'ALA88': 6, 'ARG5': 10, 'LEU60': 1, 'ILE6': 10, 'GLU91': 10}, 'GLY85': {'PRO86': 10, 'CYS89': 10, 'ALA90': 10, 'VAL92': 7, 'TYR4': 10, 'GLY87': 10, 'GLY2': 2, 'ARG3': 10, 'ARG5': 10, 'LEU60': 2, 'VAL83': 10, 'GLU91': 10}, 'PRO86': {'MET1': 10, 'GLN84': 10, 'VAL53': 3, 'CYS89': 10, 'TYR4': 10, 'GLY87': 10, 'GLY2': 10, 'ARG3': 10, 'ALA88': 10}, 'GLY87': {'GLN84': 10, 'CYS89': 10, 'GLY85': 10, 'ARG3': 5, 'ALA88': 10}, 'ALA88': {'PRO86': 10, 'GLN84': 6, 'CYS89': 10, 'ALA90': 10}, 'CYS89': {'PRO86': 10, 'GLN84': 10, 'ALA90': 10, 'GLY85': 10, 'TYR4': 1, 'GLY87': 10, 'GLU91': 10}, 'ALA90': {'GLN84': 10, 'VAL92': 10, 'GLY85': 10, 'ALA88': 10, 'VAL83': 4, 'GLU91': 10}, 'GLU91': {'THR82': 10, 'GLN84': 10, 'CYS89': 10, 'GLY85': 10, 'VAL92': 10, 'ALA93': 10, 'VAL83': 10}, 'VAL92': {'THR82': 10, 'GLN84': 10, 'PHE94': 10, 'GLU110': 10, 'ALA90': 10, 'GLY111': 9, 'TYR4': 8, 'GLY85': 7, 'ILE81': 10, 'LEU60': 10, 'GLN61': 10, 'LEU26': 2, 'LEU59': 4, 'ALA93': 10, 'PHE62': 3, 'VAL83': 10}, 'ALA93': {'ARG80': 10, 'THR82': 10, 'PHE94': 10, 'GLU110': 10, 'GLY111': 10, 'ILE81': 10, 'PRO109': 9, 'THR112': 4, 'VAL83': 10, 'PRO95': 10, 'GLU91': 10}, 'PHE94': {'GLU110': 10, 'LEU60': 10, 'PHE62': 10, 'PRO95': 10, 'THR82': 10, 'GLY111': 10, 'ASP79': 9, 'GLN61': 10, 'LEU26': 10, 'CYS96': 10, 'VAL92': 10, 'ILE81': 10, 'LEU108': 10, 'VAL83': 10, 'ARG80': 10, 'CYS78': 1, 'VAL27': 4, 'VAL63': 10, 'PRO109': 10}, 'PRO95': {'THR82': 1, 'GLU110': 10, 'GLY111': 10, 'ILE81': 10, 'PRO109': 9, 'ASP79': 10, 'LYS387': 10, 'ALA93': 10, 'THR112': 2, 'TYR97': 10, 'LEU108': 10, 'CYS96': 10, 'ARG80': 10}, 'CYS96': {'CYS78': 10, 'PHE94': 10, 'LEU65': 7, 'VAL100': 1, 'ILE81': 10, 'PRO109': 3, 'ASP79': 10, 'ARG98': 10, 'LEU106': 8, 'GLY381': 1, 'TYR97': 10, 'LEU108': 10, 'ARG80': 10}, 'TYR97': {'MET376': 9, 'CYS78': 10, 'TYR614': 6, 'ASP79': 10, 'ARG98': 10, 'GLY381': 9, 'PHE386': 10, 'ASN163': 10, 'TRP99': 10, 'LYS387': 10, 'ILE390': 10, 'GLU612': 10, 'LYS613': 10, 'ARG135': 1, 'PRO391': 4, 'PRO95': 10}, 'ARG98': {'CYS78': 10, 'GLU131': 10, 'ASP134': 10, 'TRP76': 10, 'LEU65': 8, 'GLN101': 10, 'ASP104': 10, 'VAL100': 10, 'TYR139': 1, 'ASP79': 10, 'ARG135': 10, 'ASN163': 10, 'TRP99': 10, 'LEU106': 10, 'LEU108': 10, 'ILE138': 10, 'PHE77': 10, 'CYS96': 10}, 'TRP99': {'ALA75': 10, 'CYS78': 10, 'TRP76': 10, 'GLN101': 10, 'ASP104': 1, 'VAL100': 10, 'PRO162': 10, 'LEU106': 10, 'ARG165': 10, 'ASN163': 10, 'TYR97': 10, 'ILE138': 3, 'PHE77': 10, 'GLY102': 4}, 'VAL100': {'ALA75': 10, 'CYS78': 10, 'LEU65': 10, 'TRP76': 10, 'ARG66': 10, 'GLN101': 10, 'ASP104': 8, 'LYS67': 10, 'HID69': 1, 'ASP74': 8, 'ARG98': 10, 'LEU106': 10, 'VAL22': 7, 'PHE77': 10, 'CYS96': 1, 'GLY102': 10}, 'GLN101': {'ALA75': 10, 'ASP73': 10, 'TRP76': 10, 'ARG66': 1, 'ASP104': 10, 'HID69': 10, 'LYS67': 10, 'ASP74': 10, 'ARG98': 10, 'GLU103': 10, 'TRP99': 10, 'LEU106': 9, 'GLY102': 10}, 'GLY102': {'ASP73': 5, 'ARG66': 10, 'ASP104': 10, 'VAL100': 10, 'HID69': 10, 'LYS67': 10, 'ASP74': 1, 'GLU103': 10, 'TRP99': 4}, 'GLU103': {'GLN101': 10, 'ARG66': 10, 'ASP104': 10, 'HID69': 10, 'ILE105': 10}, 'ASP104': {'LEU65': 10, 'GLN101': 10, 'ARG66': 10, 'VAL100': 8, 'LYS67': 1, 'ARG98': 10, 'ILE105': 10, 'TRP99': 1, 'LEU106': 10, 'ILE138': 1, 'GLY102': 10}, 'ILE105': {'GLN23': 8, 'LEU65': 10, 'ARG64': 10, 'ARG66': 10, 'GLU103': 10, 'SER107': 10, 'LEU106': 10, 'TRP25': 10}, 'LEU106': {'GLU131': 3, 'CYS78': 6, 'ASP134': 3, 'LEU65': 10, 'ARG64': 10, 'ARG66': 10, 'GLN101': 9, 'VAL63': 10, 'LYS67': 9, 'VAL100': 10, 'ASP104': 10, 'TRP76': 1, 'ARG98': 10, 'TRP99': 10, 'SER107': 10, 'LEU108': 10, 'TRP25': 1, 'CYS96': 8}, 'SER107': {'LEU108': 10, 'GLU131': 10, 'LEU65': 10, 'ARG64': 10, 'VAL27': 2, 'VAL63': 10, 'HID127': 10, 'PRO109': 10, 'ILE105': 10, 'LYS130': 2, 'PHE62': 10}, 'LEU108': {'CYS78': 10, 'PHE94': 10, 'GLU110': 10, 'LEU65': 10, 'ARG64': 10, 'CYS96': 10, 'VAL63': 10, 'ILE81': 10, 'PRO109': 10, 'LEU24': 1, 'ARG98': 10, 'GLN61': 10, 'LEU106': 10, 'PHE62': 10, 'PRO95': 10, 'ARG80': 8}, 'PRO109': {'LEU382': 10, 'GLU131': 3, 'PHE94': 10, 'GLU110': 10, 'GLY111': 10, 'CYS96': 3, 'VAL63': 10, 'HID127': 6, 'GLY381': 4, 'GLN61': 10, 'ARG114': 8, 'PHE124': 10, 'ALA93': 9, 'SER107': 10, 'THR112': 1, 'PHE62': 10, 'PRO95': 9}, 'GLU110': {'LEU108': 10, 'LEU382': 10, 'PHE94': 10, 'VAL92': 10, 'GLY111': 10, 'VAL63': 1, 'LEU60': 10, 'GLN61': 10, 'ARG114': 10, 'PHE124': 10, 'ALA93': 10, 'THR112': 10, 'PHE62': 10, 'PRO95': 10}, 'GLY111': {'PHE94': 10, 'VAL92': 9, 'PHE615': 8, 'PRO109': 10, 'ARG114': 10, 'LYS387': 10, 'ALA93': 10, 'THR112': 10, 'ALA113': 10, 'PRO95': 10}, 'THR112': {'LEU382': 10, 'GLU110': 10, 'PHE615': 8, 'PHE482': 7, 'PRO109': 1, 'ARG114': 10, 'LYS387': 2, 'ALA93': 4, 'PHE124': 1, 'ALA113': 10, 'PRO95': 2}, 'ALA113': {'ILE385': 1, 'LEU382': 10, 'GLY111': 10, 'PHE615': 10, 'PHE482': 10, 'HIP383': 10, 'GLY381': 10, 'ARG114': 10, 'LYS387': 10, 'PRO384': 10, 'LEU115': 10}, 'ARG114': {'LEU382': 10, 'GLU110': 10, 'GLY111': 10, 'PHE482': 4, 'HIP383': 10, 'PRO109': 8, 'ARG128': 10, 'GLN61': 10, 'PHE124': 10, 'LEU59': 4, 'THR112': 10, 'PRO384': 10, 'PRO116': 10, 'LEU115': 10}, 'LEU115': {'TYR483': 10, 'ILE488': 10, 'LEU382': 1, 'ASP492': 10, 'ALA113': 10, 'PHE482': 10, 'HIP383': 10, 'LEU495': 2, 'GLY491': 1, 'ARG128': 10, 'GLY117': 10, 'PRO384': 10, 'PRO116': 10}, 'PRO116': {'GLH494': 10, 'TYR483': 5, 'PRO493': 10, 'ASP492': 10, 'GLY491': 10, 'HIP383': 10, 'LEU495': 6, 'ARG128': 10, 'GLY117': 10, 'ARG114': 10, 'PRO384': 9, 'ASP118': 10}, 'GLY117': {'ASN119': 10, 'ARG128': 10, 'PHE124': 2, 'ASP122': 8, 'GLN125': 3, 'LEU115': 10, 'ASP118': 10}, 'ASP118': {'ASN119': 10, 'ARG128': 9, 'ALA120': 9, 'ASP122': 10, 'PRO116': 10}, 'ASN119': {'LEU121': 10, 'ARG128': 1, 'GLY117': 10, 'ALA120': 10, 'ASP122': 10, 'GLN125': 10}, 'ALA120': {'LYS126': 10, 'LEU121': 10, 'ASP122': 10, 'GLN125': 10, 'ASP118': 9}, 'LEU121': {'MET123': 10, 'ASN119': 10, 'LYS126': 5, 'THR29': 3, 'ASP122': 10, 'GLN125': 7}, 'ASP122': {'MET123': 10, 'ASN119': 10, 'LYS126': 10, 'HID127': 1, 'ARG128': 10, 'GLY117': 8, 'GLN61': 2, 'PHE124': 10, 'ALA120': 10, 'GLN125': 10, 'ASP118': 10}, 'MET123': {'LYS126': 10, 'VAL27': 10, 'GLY28': 10, 'HID127': 10, 'LEU121': 10, 'THR29': 10, 'ARG128': 6, 'LEU60': 5, 'GLN61': 10, 'PHE124': 10, 'LEU59': 10, 'PHE62': 10, 'GLY31': 2, 'GLN125': 10}, 'PHE124': {'GLU129': 7, 'LEU382': 10, 'GLU110': 10, 'LYS126': 10, 'HID127': 10, 'PRO109': 10, 'ARG128': 10, 'GLY117': 2, 'GLN61': 10, 'ARG114': 10, 'ASP122': 10, 'LEU59': 2, 'THR112': 1, 'PHE62': 10, 'GLN125': 10}, 'GLN125': {'MET123': 10, 'ASN119': 10, 'GLU129': 10, 'LYS126': 10, 'HID127': 10, 'LEU121': 7, 'ARG128': 10, 'GLY117': 3, 'ALA120': 10, 'ASP122': 10}, 'LYS126': {'MET123': 10, 'GLU129': 8, 'HID127': 10, 'LEU121': 5, 'ARG128': 10, 'LYS130': 10, 'PHE124': 10, 'ALA120': 10, 'ASP122': 10}, 'HID127': {'MET123': 10, 'GLU129': 10, 'GLU131': 10, 'LEU382': 6, 'ARG64': 10, 'VAL27': 7, 'PRO109': 6, 'ARG128': 10, 'LYS130': 10, 'PHE124': 10, 'SER107': 10, 'ASP122': 1, 'PHE62': 10, 'LEU132': 10, 'GLN125': 10}, 'ARG128': {'ASN119': 1, 'GLU129': 10, 'LYS126': 10, 'PRO380': 10, 'ARG114': 10, 'PHE62': 5, 'GLH494': 10, 'MET123': 6, 'GLU131': 10, 'LYS133': 10, 'LYS130': 10, 'LEU132': 10, 'PRO116': 10, 'LEU115': 10, 'ASP118': 9, 'LEU382': 10, 'HIP383': 2, 'GLY117': 10, 'ASP122': 10, 'GLN125': 10, 'PHE124': 10}, 'GLU129': {'GLU131': 10, 'LYS126': 8, 'LYS133': 10, 'ASP134': 10, 'HID127': 10, 'LYS130': 10, 'PHE124': 7, 'LEU132': 10, 'GLN125': 10}, 'LYS130': {'GLU131': 10, 'LYS126': 10, 'LYS133': 10, 'ASP134': 10, 'ARG64': 1, 'HID127': 10, 'ARG128': 10, 'ARG135': 5, 'SER107': 2, 'LEU132': 10}, 'GLU131': {'GLN136': 10, 'GLU129': 10, 'LEU382': 6, 'LYS133': 10, 'ASP134': 10, 'PRO380': 10, 'HID127': 10, 'PRO109': 3, 'ARG128': 10, 'GLY381': 10, 'ARG98': 10, 'ARG135': 10, 'SER107': 10, 'LEU106': 3, 'LEU132': 10}, 'LEU132': {'GLN136': 10, 'GLH494': 10, 'GLU129': 10, 'GLN137': 10, 'LYS133': 10, 'ASP134': 10, 'PRO380': 10, 'ALA497': 2, 'HID127': 10, 'ARG128': 10, 'ARG135': 10, 'LYS130': 10, 'LEU379': 8, 'ARG377': 10, 'CYS378': 10}, 'LYS133': {'GLN136': 10, 'GLU129': 10, 'GLU131': 10, 'GLN137': 10, 'ASP134': 10, 'ARG128': 10, 'ARG135': 10, 'LYS130': 10, 'ILE138': 10}, 'ASP134': {'GLN136': 10, 'GLU129': 10, 'GLU131': 10, 'GLN137': 10, 'TYR139': 10, 'ILE138': 10, 'ARG98': 10, 'ARG135': 10, 'LYS130': 10, 'LEU106': 3, 'LEU132': 10}, 'ARG135': {'MET376': 10, 'GLN136': 10, 'TYR97': 1, 'GLU131': 10, 'LYS133': 10, 'GLN137': 10, 'PRO380': 10, 'TYR139': 10, 'ILE138': 10, 'GLY381': 10, 'ARG98': 10, 'LYS130': 5, 'PHE386': 1, 'LEU379': 10, 'ARG377': 10, 'CYS140': 6, 'LEU132': 10, 'CYS378': 10}, 'GLN136': {'GLU501': 10, 'GLN137': 10, 'GLU131': 10, 'LYS133': 10, 'ASP134': 10, 'TYR139': 10, 'PRO380': 1, 'ILE138': 10, 'TRP141': 1, 'ARG377': 10, 'CYS140': 10, 'LEU132': 10, 'CYS378': 10}, 'GLN137': {'LYS133': 10, 'ASP134': 10, 'TYR139': 10, 'ILE138': 10, 'ARG135': 10, 'ARG377': 10, 'CYS140': 10, 'LEU132': 10}, 'ILE138': {'GLN136': 10, 'LYS133': 10, 'ASP134': 10, 'TYR139': 10, 'ASP104': 1, 'PRO162': 10, 'ARG98': 10, 'ARG135': 10, 'TRP99': 3, 'PRO161': 10, 'CYS140': 10, 'ILE152': 10}, 'TYR139': {'MET376': 10, 'GLN136': 10, 'MET164': 10, 'GLN137': 10, 'ASP134': 10, 'ALA372': 3, 'PRO162': 5, 'ARG135': 10, 'TRP141': 10, 'ARG98': 1, 'ALA153': 6, 'PRO161': 10, 'ASN163': 10, 'ARG377': 10, 'CYS140': 10, 'MET397': 10, 'ILE152': 10, 'VAL373': 10}, 'CYS140': {'GLN136': 10, 'THR151': 10, 'GLN137': 10, 'ARG135': 6, 'TRP141': 10, 'ALA142': 10, 'ALA153': 10, 'PRO161': 4, 'ARG377': 10, 'ILE138': 10, 'ILE152': 10, 'VAL373': 10}, 'TRP141': {'THR151': 10, 'GLU501': 10, 'GLN136': 1, 'TYR139': 10, 'ALA374': 3, 'THR143': 10, 'ALA142': 10, 'ALA153': 10, 'ARG500': 7, 'ARG377': 10, 'VAL505': 10, 'GLY506': 6, 'ILE152': 10, 'LEU150': 10, 'VAL373': 10}, 'ALA142': {'THR151': 10, 'LYS145': 2, 'ALA154': 8, 'THR143': 10, 'TRP144': 10, 'ALA153': 10, 'CYS140': 10, 'PRO149': 7, 'ILE152': 10, 'LEU150': 10, 'VAL373': 3}, 'THR143': {'GLU146': 10, 'THR151': 8, 'LYS145': 10, 'GLY147': 9, 'TRP144': 10, 'TRP141': 10, 'LEU148': 8, 'PRO149': 2, 'LEU150': 10}, 'TRP144': {'GLU146': 10, 'THR151': 10, 'ILE408': 10, 'LYS145': 10, 'ARG404': 10, 'LEU361': 8, 'VAL426': 2, 'LEU429': 10, 'LEU366': 10, 'HID425': 6, 'LEU362': 1, 'ALA142': 10, 'LEU148': 10, 'PRO149': 10, 'LEU150': 10}, 'LYS145': {'GLU146': 10, 'ARG430': 7, 'PRO149': 1, 'ILE408': 9, 'GLY147': 10, 'SER409': 7, 'ARG404': 3, 'THR143': 10, 'LEU429': 6, 'HID425': 10, 'ALA142': 2, 'LEU148': 10, 'VAL426': 10, 'LEU150': 7}, 'GLU146': {'ARG430': 10, 'CYS508': 4, 'GLY147': 10, 'THR143': 10, 'TRP144': 10, 'LEU148': 10, 'LEU150': 10}, 'GLY147': {'ARG430': 10, 'CYS508': 10, 'ALA433': 1, 'LYS145': 10, 'THR143': 9, 'LEU148': 10, 'PRO149': 10, 'LEU150': 6}, 'LEU148': {'GLU146': 10, 'ARG430': 10, 'CYS508': 10, 'THR151': 5, 'GLY506': 1, 'ILE408': 1, 'ALA433': 10, 'LYS145': 10, 'LEU362': 9, 'GLN427': 1, 'VAL426': 10, 'TRP144': 10, 'LEU366': 10, 'LEU429': 10, 'THR143': 8, 'HID425': 6, 'PRO149': 10, 'LEU150': 10}, 'PRO149': {'CYS508': 10, 'ARG513': 9, 'LYS145': 1, 'LEU429': 8, 'VAL505': 4, 'LEU362': 10, 'ARG404': 8, 'TRP144': 10, 'LEU507': 10, 'LEU436': 6, 'ALA433': 10, 'GLY147': 10, 'VAL370': 10, 'GLY506': 10, 'LEU150': 10, 'THR151': 10, 'VAL367': 6, 'THR143': 2, 'LEU366': 10, 'ALA142': 7}, 'LEU150': {'GLU146': 10, 'THR151': 10, 'CYS508': 10, 'LYS145': 7, 'GLY147': 6, 'THR143': 10, 'TRP144': 10, 'LEU507': 10, 'TRP141': 10, 'LEU366': 1, 'ALA142': 10, 'VAL370': 10, 'LEU148': 10, 'VAL505': 10, 'GLY506': 10, 'ILE152': 10, 'VAL373': 10}, 'THR151': {'ASN400': 10, 'GLY506': 9, 'LEU148': 5, 'ALA154': 6, 'ARG404': 10, 'THR143': 8, 'TRP144': 10, 'LEU366': 10, 'TRP141': 10, 'ALA142': 10, 'ALA153': 10, 'GLU369': 10, 'VAL370': 10, 'CYS140': 10, 'PRO149': 10, 'ILE152': 10, 'MET397': 4, 'VAL373': 10}, 'ILE152': {'ASP159': 6, 'ASP155': 2, 'TRP141': 10, 'ALA153': 10, 'PRO161': 10, 'GLU369': 10, 'CYS140': 10, 'ILE138': 10, 'ALA154': 10, 'ARG404': 10, 'MET397': 10, 'THR401': 10, 'LEU150': 10, 'LEU160': 10, 'MET164': 7, 'ASN400': 5, 'TYR139': 10, 'ALA142': 10, 'GLU398': 1, 'VAL373': 10}, 'ALA153': {'THR151': 10, 'TYR139': 6, 'ASP159': 10, 'ALA154': 10, 'ASP155': 10, 'TRP141': 10, 'ALA142': 10, 'PRO161': 2, 'LEU160': 2, 'CYS140': 10, 'VAL373': 5}, 'ALA154': {'THR151': 6, 'ASP159': 10, 'THR405': 10, 'ASP155': 10, 'ARG404': 10, 'ARG156': 10, 'ALA142': 8, 'PRO161': 10, 'THR401': 10, 'MET397': 3, 'ILE152': 10, 'LEU160': 10, 'LYS157': 1}, 'ASP155': {'GLN406': 5, 'ASP159': 10, 'THR405': 10, 'ARG404': 8, 'ARG156': 10, 'ALA153': 10, 'THR401': 10, 'ILE152': 2, 'LYS157': 10}, 'ARG156': {'ASP159': 10, 'THR405': 10, 'ALA154': 10, 'ARG402': 8, 'ARG165': 1, 'THR401': 10, 'ASP158': 10, 'LEU160': 10, 'LYS157': 10}, 'LYS157': {'PHE166': 10, 'ASP159': 10, 'ASP155': 10, 'ARG402': 8, 'ALA154': 1, 'ARG165': 10, 'GLU398': 10, 'ASP158': 10, 'LEU160': 10, 'ARG171': 10}, 'ASP158': {'ASP159': 10, 'ARG156': 10, 'PRO162': 2, 'ARG165': 10, 'LEU160': 10}, 'ASP159': {'ASP155': 10, 'ALA154': 10, 'ARG156': 10, 'ARG165': 10, 'ALA153': 10, 'PRO161': 10, 'ILE152': 6, 'LEU160': 10, 'LYS157': 10}, 'LEU160': {'MET164': 10, 'PHE166': 10, 'ALA154': 10, 'ARG156': 10, 'PRO162': 10, 'ARG165': 10, 'ALA153': 2, 'PRO161': 10, 'MET397': 10, 'THR401': 10, 'GLU398': 10, 'ASP158': 10, 'ILE152': 10, 'LYS157': 10}, 'PRO161': {'MET376': 1, 'MET164': 10, 'TYR139': 10, 'ASP159': 10, 'ALA154': 10, 'PRO162': 10, 'ARG165': 10, 'ALA153': 2, 'ASN163': 10, 'MET397': 10, 'CYS140': 4, 'ILE138': 10, 'ILE152': 10}, 'PRO162': {'MET164': 10, 'TYR395': 2, 'TYR139': 5, 'ARG165': 10, 'ASN163': 10, 'TRP99': 10, 'ILE138': 10, 'ASP158': 2, 'PHE77': 10, 'LEU160': 10}, 'ASN163': {'MET376': 10, 'MET164': 10, 'CYS78': 1, 'ARG394': 10, 'TYR395': 10, 'TYR139': 10, 'ARG98': 10, 'ARG165': 10, 'ILE390': 8, 'TRP99': 10, 'PRO161': 10, 'TYR97': 10, 'PHE77': 10}, 'MET164': {'MET376': 10, 'PHE166': 10, 'ARG394': 10, 'TYR395': 10, 'TYR139': 10, 'ALA372': 10, 'PRO162': 10, 'THR396': 7, 'ARG165': 10, 'ILE390': 9, 'PRO161': 10, 'ILE393': 10, 'MET397': 10, 'GLU398': 10, 'PHE386': 1, 'ILE152': 7, 'LEU160': 10}, 'ARG165': {'PHE166': 10, 'TYR395': 10, 'ASP159': 10, 'ARG156': 1, 'PRO162': 10, 'ASN163': 10, 'TRP99': 10, 'PRO161': 10, 'HIE167': 10, 'GLU398': 10, 'MET397': 1, 'ASP158': 10, 'PHE77': 2, 'LEU160': 10, 'LYS157': 10}, 'PHE166': {'MET164': 10, 'PHE174': 10, 'GLU175': 1, 'TYR395': 10, 'ILE399': 10, 'THR662': 10, 'ARG402': 8, 'LYS170': 10, 'GLU168': 10, 'LEU160': 10, 'GLU398': 10, 'HIE167': 10, 'LYS157': 10, 'ARG171': 10}, 'HIE167': {'ARG394': 10, 'TYR395': 10, 'GLU169': 10, 'LYS170': 10, 'TRP13': 10, 'GLU168': 10, 'ARG165': 10, 'LEU14': 10, 'ASP603': 10, 'LEU172': 10, 'PHE77': 1, 'ARG171': 10}, 'GLU168': {'ASP173': 10, 'PHE166': 10, 'GLU169': 10, 'LYS170': 10, 'TRP13': 10, 'LEU14': 10, 'PHE15': 10, 'LEU172': 10, 'ARG171': 10}, 'GLU169': {'ASP173': 10, 'ARG43': 10, 'PHE174': 10, 'LYS170': 10, 'TRP13': 5, 'LEU14': 10, 'LEU172': 10, 'HIE167': 10, 'ARG171': 10}, 'LYS170': {'ASP173': 10, 'PHE166': 10, 'PHE174': 10, 'GLU175': 10, 'ARG394': 10, 'TYR395': 10, 'THR662': 10, 'ILE399': 3, 'ARG599': 10, 'PRO602': 10, 'GLU168': 10, 'GLN601': 10, 'ASP603': 10, 'LEU172': 10, 'HIE167': 10, 'ARG171': 10}, 'ARG171': {'ASP173': 10, 'PHE166': 10, 'PHE174': 10, 'GLU175': 10, 'LYS179': 3, 'GLU169': 10, 'TRP176': 10, 'GLU398': 2, 'GLU168': 10, 'PHE15': 2, 'LEU172': 10, 'HIE167': 10, 'LYS157': 10}, 'LEU172': {'THR177': 10, 'ASP173': 10, 'PHE174': 10, 'GLU175': 10, 'GLU169': 10, 'LYS170': 10, 'TRP176': 10, 'GLU168': 10, 'LEU14': 10, 'PHE15': 7, 'HIE167': 10}, 'ASP173': {'THR177': 10, 'PHE174': 10, 'GLU175': 10, 'ARG599': 10, 'LYS170': 10, 'GLU169': 10, 'TRP176': 10, 'PRO602': 10, 'GLU168': 10, 'GLN601': 10, 'HIE596': 9, 'LEU178': 10, 'ARG171': 10}, 'PHE174': {'THR177': 10, 'LEU178': 10, 'PHE166': 10, 'GLU175': 10, 'ILE399': 10, 'THR662': 8, 'ARG599': 10, 'ARG402': 9, 'LYS170': 10, 'GLU169': 10, 'TRP176': 10, 'LEU597': 9, 'GLN601': 10, 'HIE596': 10, 'LEU172': 10, 'LYS179': 10, 'ARG171': 10}, 'GLU175': {'THR177': 10, 'ASP173': 10, 'LEU178': 10, 'PHE166': 1, 'LYS170': 10, 'ALA180': 10, 'TRP176': 10, 'HIE596': 8, 'LEU172': 10, 'LYS179': 10, 'ARG171': 10}, 'TRP176': {'THR177': 10, 'ASP173': 10, 'LEU178': 10, 'PHE174': 10, 'TRP595': 5, 'ARG599': 5, 'ALA180': 10, 'GLY181': 10, 'LEU183': 1, 'PRO602': 2, 'LEU172': 10, 'LYS179': 10, 'ARG171': 10}, 'THR177': {'ASP173': 10, 'ALA182': 10, 'LEU178': 10, 'PHE174': 10, 'GLU175': 10, 'TRP595': 10, 'ARG599': 10, 'ALA180': 10, 'GLY181': 10, 'HIE596': 10, 'LEU172': 10, 'LYS179': 10}, 'LEU178': {'ASP173': 10, 'ALA182': 10, 'ILE593': 10, 'PHE174': 10, 'GLU175': 10, 'ALA592': 4, 'TRP176': 10, 'GLY181': 10, 'LEU183': 10, 'ILE413': 5, 'HIE596': 10, 'ALA180': 10, 'LYS179': 10}, 'LYS179': {'THR177': 10, 'ALA182': 10, 'PHE174': 10, 'GLU175': 10, 'GLU184': 10, 'TRP176': 10, 'GLY181': 10, 'LEU183': 10, 'ALA180': 10, 'ARG171': 3}, 'ALA180': {'THR177': 10, 'ALA182': 10, 'TRP595': 3, 'GLU175': 10, 'GLU184': 10, 'TRP176': 10, 'GLY181': 10, 'LEU183': 10, 'LEU178': 10, 'MET185': 10}, 'GLY181': {'THR177': 10, 'ALA182': 10, 'TRP595': 8, 'GLU184': 10, 'ALA592': 10, 'TRP176': 10, 'LEU183': 10, 'ALA186': 10, 'HIE596': 5, 'LEU178': 10, 'MET185': 10, 'LYS179': 10}, 'ALA182': {'THR177': 10, 'LEU187': 10, 'ILE593': 1, 'ARG189': 2, 'GLU184': 10, 'ALA592': 10, 'LEU589': 6, 'ALA180': 10, 'LEU183': 10, 'ALA186': 10, 'LEU178': 10, 'MET185': 10, 'LYS179': 10}, 'LEU183': {'LEU187': 10, 'LYS188': 9, 'GLU184': 10, 'ALA180': 10, 'TRP176': 1, 'GLY181': 10, 'ALA186': 10, 'LEU178': 10, 'MET185': 10, 'LYS179': 10}, 'GLU184': {'LEU187': 10, 'ALA182': 10, 'ARG189': 10, 'LYS188': 10, 'ALA592': 9, 'GLN210': 9, 'MET591': 2, 'GLY209': 10, 'GLY181': 10, 'ALA186': 10, 'TRP208': 8, 'CYS588': 4, 'ALA180': 10, 'MET185': 10, 'LYS179': 10}, 'MET185': {'LEU187': 10, 'ALA182': 10, 'ARG189': 10, 'LYS188': 10, 'LYS416': 1, 'ALA592': 10, 'LEU589': 10, 'VAL190': 10, 'GLY181': 10, 'ALA186': 10, 'LEU183': 10, 'GLN586': 8, 'CYS588': 10, 'ALA180': 10, 'VAL584': 1, 'ARG585': 10}, 'ALA186': {'LEU187': 10, 'ALA182': 10, 'ARG189': 10, 'LYS188': 10, 'GLU184': 10, 'VAL190': 10, 'GLY181': 10, 'LEU183': 10, 'TYR191': 10}, 'LEU187': {'ALA182': 10, 'ARG189': 10, 'LYS188': 10, 'GLU184': 10, 'VAL190': 10, 'LEU183': 10, 'THR192': 10, 'TYR191': 10, 'MET185': 10}, 'LYS188': {'LEU193': 10, 'ARG189': 10, 'PHE207': 10, 'ALA592': 1, 'GLN210': 1, 'LEU183': 9, 'ALA186': 10, 'TRP208': 10, 'CYS588': 10, 'GLN205': 9, 'VAL584': 9, 'MET591': 10, 'VAL190': 10, 'MET185': 10, 'GLU184': 10, 'GLY209': 10, 'THR192': 10, 'TYR191': 10, 'ILE206': 10}, 'ARG189': {'LEU187': 10, 'LEU193': 10, 'ALA182': 2, 'LEU194': 3, 'GLU184': 10, 'VAL190': 10, 'THR192': 10, 'ALA186': 10, 'CYS588': 10, 'TYR191': 10, 'ILE206': 1, 'MET185': 10, 'VAL584': 10, 'ARG585': 10}, 'VAL190': {'LEU187': 10, 'LEU193': 10, 'LYS188': 10, 'LEU194': 10, 'THR192': 10, 'ALA186': 10, 'SER195': 1, 'TYR191': 10, 'MET185': 10}, 'TYR191': {'LEU187': 10, 'LEU193': 10, 'SER196': 2, 'ARG189': 10, 'LYS188': 10, 'LEU194': 10, 'THR192': 10, 'ALA186': 10, 'SER195': 10, 'GLN205': 10, 'ILE206': 10, 'ASP202': 1}, 'THR192': {'LEU187': 10, 'LEU193': 10, 'SER196': 7, 'ARG189': 10, 'TRP197': 10, 'LYS188': 10, 'LEU194': 10, 'VAL190': 10, 'GLN205': 9, 'CYS588': 7, 'SER195': 10, 'ILE206': 10, 'VAL584': 10, 'ASP202': 4}, 'LEU193': {'ARG189': 10, 'LYS188': 10, 'LEU194': 10, 'VAL190': 10, 'SER195': 10, 'TYR191': 10}, 'LEU194': {'SER196': 10, 'ARG189': 3, 'VAL190': 10, 'THR192': 10, 'SER195': 10, 'TYR191': 10}, 'SER195': {'LEU193': 10, 'SER196': 10, 'TRP197': 10, 'VAL190': 1, 'THR192': 10, 'GLN205': 9, 'TYR191': 10, 'ASP202': 6}, 'SER196': {'TRP197': 10, 'LEU194': 10, 'ASN198': 10, 'GLN205': 1, 'CYS199': 10, 'THR192': 7, 'TYR191': 2, 'ASP202': 10}, 'TRP197': {'LEU581': 10, 'GLN205': 9, 'PRO582': 10, 'CYS588': 3, 'VAL584': 10, 'VAL554': 4, 'MET591': 4, 'MET578': 10, 'ARG562': 10, 'CYS199': 10, 'ASP204': 1, 'ALA587': 10, 'ASP583': 10, 'PHE203': 10, 'ASN198': 10, 'THR192': 10, 'SER195': 10, 'ILE206': 10, 'ASP202': 10}, 'ASN198': {'SER196': 10, 'MET574': 10, 'ALA575': 10, 'CYS199': 10, 'MET578': 10, 'LEU200': 10, 'ASP202': 10}, 'CYS199': {'SER196': 10, 'MET574': 10, 'TRP197': 10, 'PHE203': 10, 'ALA575': 4, 'GLU201': 10, 'ASP204': 4, 'MET578': 10, 'LEU200': 10, 'ASP202': 10}, 'LEU200': {'PRO555': 10, 'TRP553': 10, 'VAL577': 10, 'MET574': 10, 'LEU581': 10, 'PHE203': 10, 'GLU201': 10, 'ASN198': 10, 'LEU338': 10, 'GLN223': 3, 'GLN205': 5, 'LEU341': 9, 'ASP204': 10, 'ARG219': 3, 'MET578': 10, 'TRP222': 10, 'ASP202': 10}, 'GLU201': {'TRP553': 4, 'MET574': 1, 'PHE203': 10, 'GLN205': 10, 'CYS199': 10, 'ASP204': 10, 'ARG219': 8, 'ILE206': 5, 'ASP202': 10}, 'ASP202': {'SER196': 10, 'TRP197': 10, 'PHE203': 10, 'PHE207': 6, 'ASN198': 10, 'GLN205': 10, 'CYS199': 10, 'SER195': 6, 'ASP204': 10, 'THR192': 4, 'TYR191': 1, 'MET578': 10, 'LEU200': 10, 'ILE206': 10}, 'PHE203': {'LEU581': 10, 'PHE207': 10, 'GLN205': 10, 'PRO582': 10, 'TRP208': 2, 'ARG219': 1, 'VAL584': 1, 'VAL554': 10, 'TRP197': 10, 'MET591': 10, 'MET578': 10, 'TRP553': 10, 'PRO558': 3, 'GLU201': 10, 'CYS199': 10, 'ASP204': 10, 'ALA587': 10, 'ASP583': 3, 'PRO555': 10, 'LEU200': 10, 'ILE206': 10}, 'ASP204': {'TRP553': 10, 'TRP197': 1, 'PHE207': 10, 'GLU201': 10, 'MET591': 1, 'GLN205': 10, 'TRP208': 10, 'CYS199': 4, 'ARG219': 10, 'LEU200': 10, 'ILE206': 10, 'ASP202': 10}, 'GLN205': {'SER196': 1, 'TRP197': 9, 'PHE203': 10, 'LYS188': 9, 'PHE207': 10, 'GLU201': 10, 'THR192': 9, 'TRP208': 10, 'SER195': 9, 'TYR191': 10, 'LEU200': 5, 'ILE206': 10, 'ASP202': 10}, 'ILE206': {'PRO555': 1, 'TRP553': 6, 'VAL554': 9, 'ARG189': 1, 'TRP197': 10, 'LYS188': 10, 'PHE203': 10, 'PHE207': 10, 'GLU201': 5, 'MET591': 10, 'THR192': 10, 'TRP208': 10, 'ASP204': 10, 'CYS588': 10, 'TYR191': 10, 'ALA587': 10, 'VAL584': 10, 'ASP202': 10}, 'PHE207': {'GLU216': 10, 'TRP553': 10, 'VAL554': 10, 'ALA215': 10, 'LEU548': 1, 'PHE203': 10, 'LYS188': 10, 'GLN210': 4, 'MET591': 10, 'GLY209': 10, 'LYS211': 10, 'GLN205': 10, 'TRP208': 10, 'ASP204': 10, 'ARG219': 10, 'ALA552': 10, 'VAL218': 1, 'ASP202': 6}, 'TRP208': {'PHE203': 2, 'LYS188': 10, 'GLU184': 8, 'GLN210': 10, 'GLY209': 10, 'LYS211': 10, 'GLN205': 10, 'ASP204': 10, 'ILE206': 10}, 'GLY209': {'LYS188': 10, 'GLU184': 10, 'PHE207': 10, 'GLN210': 10, 'LYS211': 10}, 'GLN210': {'LEU548': 10, 'TRP595': 10, 'SER212': 10, 'LYS188': 1, 'GLU184': 9, 'PHE207': 4, 'LYS211': 10, 'TRP208': 10, 'ALA552': 10}, 'LYS211': {'GLU216': 10, 'LEU548': 10, 'TRP553': 2, 'ALA213': 10, 'ALA215': 10, 'TRP595': 6, 'SER212': 10, 'PHE207': 10, 'GLY209': 10, 'TRP208': 10, 'ARG219': 7, 'ALA552': 10}, 'SER212': {'GLU216': 10, 'LEU548': 10, 'ALA215': 10, 'ALA213': 10, 'LEU214': 10, 'TRP595': 1, 'GLN210': 10, 'LYS217': 8, 'ALA552': 2, 'ASP549': 10}, 'ALA213': {'GLU216': 10, 'ALA215': 10, 'LEU214': 10, 'LYS217': 10, 'LYS211': 10, 'VAL218': 10, 'ASP549': 1}, 'LEU214': {'GLU216': 10, 'ALA215': 10, 'TRP550': 10, 'PRO648': 5, 'SER212': 10, 'LEU227': 10, 'TYR230': 10, 'GLY546': 3, 'LYS217': 10, 'GLY235': 10, 'TYR651': 10, 'ARG219': 10, 'VAL218': 10, 'ASP549': 10}, 'ALA215': {'GLU216': 10, 'LEU548': 10, 'TRP553': 10, 'ALA213': 10, 'TRP550': 8, 'SER212': 10, 'PHE207': 10, 'LYS217': 10, 'LYS211': 10, 'ARG219': 10, 'ALA552': 10, 'GLN220': 10, 'VAL218': 10, 'ASP549': 10}, 'GLU216': {'ALA213': 10, 'LEU214': 10, 'SER212': 10, 'PHE207': 10, 'CYS221': 9, 'LYS217': 10, 'LYS211': 10, 'ARG219': 10, 'GLN220': 10, 'VAL218': 10}, 'LYS217': {'ALA213': 10, 'ALA215': 10, 'LEU214': 10, 'TYR230': 10, 'LEU227': 10, 'SER212': 8, 'CYS221': 10, 'TRP222': 10, 'ARG219': 10, 'ASP646': 4, 'GLN220': 10, 'VAL218': 10, 'GLU226': 7, 'TRP647': 1}, 'VAL218': {'GLU216': 10, 'TRP553': 10, 'ALA213': 10, 'ALA215': 10, 'LEU214': 10, 'ARG348': 8, 'LEU227': 10, 'TRP550': 10, 'PHE207': 1, 'CYS221': 10, 'TRP222': 10, 'TYR551': 1, 'GLN223': 3, 'ASN556': 10, 'ARG219': 10, 'ALA552': 1, 'GLN220': 10, 'ASP549': 10}, 'ARG219': {'GLU216': 10, 'TRP553': 10, 'ALA215': 10, 'LEU214': 10, 'PHE203': 1, 'PHE207': 10, 'GLU201': 8, 'CYS221': 10, 'TRP222': 10, 'LEU200': 3, 'LYS217': 10, 'GLN223': 10, 'LYS211': 7, 'ASN556': 3, 'ASP204': 10, 'GLN220': 10}, 'GLN220': {'GLU216': 10, 'ALA215': 10, 'CYS221': 10, 'LYS217': 10, 'GLN223': 10, 'TRP222': 10, 'VAL218': 10}, 'CYS221': {'GLU216': 9, 'LEU227': 10, 'ASP225': 2, 'LYS217': 10, 'GLN223': 10, 'ASP224': 10, 'ARG219': 10, 'TRP222': 10, 'VAL218': 10, 'GLU226': 10}, 'TRP222': {'PRO555': 10, 'TRP553': 10, 'VAL577': 10, 'LEU581': 10, 'GLN220': 10, 'LEU227': 10, 'LYS217': 10, 'SER345': 9, 'GLN223': 10, 'PHE228': 7, 'ASP224': 10, 'LEU341': 10, 'ARG219': 10, 'ASN556': 10, 'LEU200': 10, 'VAL218': 10, 'LYS344': 10}, 'GLN223': {'TRP553': 2, 'ASP225': 10, 'CYS221': 10, 'LEU200': 3, 'ASP224': 10, 'LEU341': 10, 'ARG219': 10, 'PRO337': 3, 'TRP340': 8, 'GLN220': 10, 'VAL218': 3, 'LYS344': 10}, 'ASP224': {'GLN312': 1, 'LEU227': 10, 'ASP225': 10, 'CYS221': 10, 'PHE228': 10, 'LEU341': 5, 'TRP340': 4, 'TRP222': 10, 'GLU226': 10, 'LYS344': 10}, 'ASP225': {'GLN312': 10, 'TYR230': 9, 'LEU227': 10, 'SER229': 10, 'CYS221': 2, 'PRO313': 10, 'GLN223': 10, 'PHE228': 10, 'TRP340': 10, 'LEU311': 2, 'GLU226': 10, 'PRO333': 10, 'LYS344': 10}, 'GLU226': {'GLN312': 10, 'TYR230': 10, 'LEU227': 10, 'SER229': 10, 'CYS221': 10, 'ASP646': 4, 'LEU311': 6, 'LYS217': 7, 'PHE228': 10, 'ASP224': 10, 'GLN231': 10, 'LYS310': 9, 'TRP647': 5}, 'LEU227': {'GLN312': 2, 'PHE232': 6, 'TRP550': 10, 'LEU214': 10, 'TYR230': 10, 'SER229': 10, 'ARG348': 2, 'ASP225': 10, 'CYS221': 10, 'LYS217': 10, 'PHE228': 10, 'ASP224': 10, 'GLN231': 10, 'TRP222': 10, 'VAL218': 10}, 'PHE228': {'GLN312': 3, 'VAL301': 10, 'PHE232': 10, 'TYR230': 10, 'ASN556': 2, 'ASP225': 10, 'SER229': 10, 'PRO313': 10, 'ASP224': 10, 'GLN231': 10, 'LYS344': 10, 'VAL347': 10, 'TRP550': 10, 'ARG348': 10, 'SER345': 4, 'ILE316': 10, 'LEU303': 10, 'TRP222': 7, 'GLU226': 10, 'ALA343': 1, 'TRP340': 6}, 'SER229': {'GLN312': 10, 'PHE232': 10, 'TYR230': 10, 'LEU227': 10, 'LEU241': 9, 'PRO313': 10, 'GLN231': 10, 'LEU303': 10, 'LEU233': 10, 'ASP225': 10, 'LEU311': 10, 'GLU226': 10}, 'TYR230': {'GLN312': 1, 'PHE232': 10, 'LEU214': 10, 'ASP225': 9, 'ASN234': 10, 'ASP646': 10, 'GLN231': 10, 'PRO238': 4, 'LYS310': 2, 'TRP647': 10, 'TRP550': 10, 'PRO648': 8, 'GLY235': 10, 'LEU233': 10, 'GLU226': 10, 'LEU227': 10, 'LYS217': 10, 'GLY457': 5, 'PHE228': 10, 'LEU311': 8}, 'GLN231': {'PHE232': 10, 'ASP351': 10, 'TRP550': 10, 'ARG348': 10, 'LEU227': 10, 'SER229': 10, 'ASN237': 10, 'HIE355': 1, 'ALA236': 10, 'ASN234': 10, 'GLY235': 10, 'TYR551': 8, 'PHE228': 10, 'LEU240': 10, 'LEU233': 10, 'PRO238': 10, 'GLU226': 10}, 'PHE232': {'VAL301': 5, 'TYR230': 10, 'LEU241': 10, 'ALA276': 10, 'PHE278': 10, 'MET239': 10, 'SER229': 10, 'ASN234': 10, 'PRO313': 1, 'LEU452': 9, 'PRO238': 10, 'VAL347': 10, 'ARG348': 10, 'GLY235': 10, 'LEU240': 10, 'LEU303': 10, 'LEU233': 10, 'ASP351': 10, 'LEU227': 6, 'PHE228': 10, 'ASN237': 10, 'LEU311': 8}, 'LEU233': {'GLY309': 9, 'TYR230': 10, 'SER229': 10, 'LEU241': 10, 'LEU459': 10, 'ASN234': 10, 'GLY235': 10, 'LEU311': 10, 'GLY457': 10, 'LEU455': 10, 'LEU452': 10, 'GLN231': 10, 'LEU303': 9, 'TYR460': 10, 'PRO238': 10, 'LYS310': 10, 'ALA458': 10, 'ALA461': 10}, 'ASN234': {'PHE232': 10, 'TRP550': 10, 'PRO648': 10, 'TYR230': 10, 'LEU459': 10, 'ALA236': 10, 'TYR649': 10, 'GLY457': 10, 'GLY235': 10, 'TYR651': 9, 'GLN231': 10, 'TYR460': 10, 'PRO238': 10, 'ALA458': 10, 'TRP647': 10, 'ALA461': 4}, 'GLY235': {'PHE232': 10, 'TRP550': 10, 'LEU214': 10, 'TYR230': 10, 'ASN237': 10, 'TYR460': 2, 'ALA236': 10, 'GLY546': 7, 'ILE543': 1, 'TYR651': 10, 'ALA542': 10, 'GLN231': 10, 'LEU233': 10, 'PRO238': 10, 'TYR649': 10}, 'ALA236': {'TRP550': 10, 'MET239': 4, 'ARG348': 10, 'GLN547': 10, 'ASP351': 8, 'HIE355': 10, 'GLY546': 10, 'ASN234': 10, 'TYR551': 10, 'ILE543': 10, 'TYR651': 10, 'ALA542': 10, 'GLN231': 10, 'ASN237': 10, 'PRO238': 10, 'TYR359': 1, 'ASP549': 6}, 'ASN237': {'PHE232': 10, 'ASP351': 10, 'MET239': 10, 'ARG348': 10, 'LEU241': 5, 'LEU354': 8, 'HIE355': 10, 'TYR551': 10, 'GLY235': 10, 'LEU240': 10, 'PHE352': 9, 'ILE543': 10, 'ALA542': 1, 'GLN231': 10, 'PRO238': 10, 'TYR359': 10}, 'PRO238': {'PHE232': 10, 'TYR230': 4, 'LEU241': 10, 'ALA236': 10, 'TYR651': 3, 'TYR649': 10, 'ALA461': 3, 'MET239': 10, 'ASN234': 10, 'GLN231': 10, 'LEU452': 10, 'TYR460': 10, 'TYR359': 1, 'GLY235': 10, 'LEU447': 10, 'LEU240': 10, 'ALA542': 10, 'ILE543': 10, 'LEU233': 10, 'ALA458': 10}, 'MET239': {'PHE232': 10, 'ARG450': 10, 'TYR460': 10, 'LEU241': 10, 'ALA236': 4, 'LEU447': 10, 'ASN363': 1, 'LEU240': 10, 'THR364': 1, 'ASP446': 10, 'ALA542': 10, 'ILE543': 10, 'LEU452': 10, 'ASN237': 10, 'SER440': 1, 'TYR359': 10, 'GLN539': 10}, 'LEU240': {'ARG242': 10, 'PHE232': 10, 'VAL347': 2, 'ASP351': 10, 'GLU275': 10, 'ALA276': 10, 'LEU241': 10, 'ASP277': 10, 'LEU354': 10, 'ARG450': 10, 'GLN358': 3, 'GLN231': 10, 'LEU452': 7, 'ASN237': 10, 'PRO238': 10, 'TYR359': 10, 'PHE278': 10}, 'LEU241': {'ARG242': 10, 'VAL301': 5, 'PHE232': 10, 'ARG450': 10, 'ASP277': 10, 'ALA276': 10, 'PHE278': 1, 'MET239': 10, 'GLU275': 10, 'SER229': 9, 'LEU452': 10, 'PRO238': 10, 'PHE274': 10, 'ARG243': 10, 'LEU455': 10, 'LEU303': 10, 'LEU233': 10, 'ALA458': 4, 'MET302': 3, 'ASN237': 5, 'LEU311': 10}, 'ARG242': {'ASP449': 10, 'GLU275': 10, 'ARG450': 10, 'ASP277': 10, 'GLN435': 6, 'LEU273': 10, 'ARG243': 10, 'SER244': 10, 'LEU240': 10, 'ASP446': 3, 'GLY451': 7, 'LEU452': 10, 'ALA276': 10, 'PHE274': 10}, 'ARG243': {'GLY271': 10, 'ASP449': 7, 'MET305': 1, 'GLU275': 10, 'ARG450': 10, 'LEU241': 10, 'GLY454': 4, 'LEU268': 3, 'LEU273': 10, 'SER244': 10, 'SER272': 8, 'THR245': 10, 'GLY451': 10, 'LEU452': 9, 'LEU455': 7, 'ASN270': 4, 'PHE274': 10}, 'SER244': {'ARG242': 10, 'GLY271': 6, 'SER246': 10, 'MET302': 10, 'GLU275': 10, 'LEU268': 10, 'LEU273': 10, 'PRO248': 6, 'THR245': 10, 'LEU247': 5, 'PHE274': 10}, 'THR245': {'SER246': 10, 'GLU275': 10, 'LEU268': 10, 'ARG243': 10, 'LEU247': 8}, 'SER246': {'MET302': 10, 'GLU275': 10, 'LEU268': 10, 'LEU273': 9, 'PRO248': 10, 'SER244': 10, 'GLU265': 7, 'LEU247': 10}, 'LEU247': {'MET302': 10, 'SER249': 10, 'GLU275': 1, 'LEU253': 10, 'LEU251': 10, 'LEU268': 10, 'LEU273': 9, 'PRO248': 10, 'LEU260': 7, 'SER244': 5, 'VAL252': 2, 'LEU264': 10, 'THR245': 8, 'GLN261': 10, 'GLU265': 10}, 'PRO248': {'MET302': 10, 'SER246': 10, 'VAL301': 7, 'SER249': 10, 'GLU275': 10, 'LEU251': 10, 'VAL252': 2, 'LEU268': 2, 'LEU273': 5, 'SER244': 6, 'LEU264': 7, 'ARG250': 10, 'VAL315': 1, 'LEU300': 10}, 'SER249': {'LEU251': 10, 'VAL252': 10, 'ARG250': 10, 'LEU247': 10, 'LEU300': 1}, 'ARG250': {'ASN322': 10, 'ASP282': 10, 'PRO321': 10, 'VAL252': 10, 'LEU251': 10, 'PRO248': 10, 'ALA298': 6, 'ILE279': 10, 'GLN317': 10, 'LEU300': 10, 'PRO320': 10}, 'LEU251': {'MET302': 10, 'LEU330': 10, 'GLN261': 2, 'VAL301': 3, 'SER249': 10, 'GLN317': 10, 'LEU253': 10, 'VAL252': 10, 'PRO248': 10, 'LEU260': 1, 'LEU264': 10, 'ILE316': 3, 'VAL315': 10, 'PRO321': 10, 'LEU247': 10, 'LEU300': 10}, 'VAL252': {'LEU330': 10, 'SER249': 10, 'THR327': 5, 'LEU253': 10, 'PRO254': 10, 'PRO248': 2, 'ARG250': 10, 'GLN261': 5, 'PRO321': 10, 'GLN317': 10, 'LEU300': 5, 'LEU247': 2}, 'LEU253': {'LEU332': 5, 'LEU330': 10, 'GLU258': 10, 'PRO254': 10, 'LEU251': 10, 'SER255': 10, 'GLU259': 2, 'LEU260': 10, 'VAL315': 10, 'LEU264': 10, 'GLN261': 10, 'MET257': 10, 'LEU247': 10}, 'PRO254': {'LEU330': 10, 'THR327': 7, 'GLU258': 10, 'SER255': 10, 'VAL252': 10, 'GLY256': 10, 'MET257': 10}, 'SER255': {'MET257': 10, 'GLY256': 10, 'GLU258': 10, 'LEU253': 10}, 'GLY256': {'GLU258': 10, 'PRO254': 10, 'GLU259': 10, 'LEU260': 5, 'MET257': 10}, 'MET257': {'LEU332': 10, 'LEU330': 10, 'GLU258': 10, 'LEU253': 10, 'PRO254': 10, 'SER255': 10, 'LYS569': 10, 'THR329': 1, 'GLU259': 10, 'LEU260': 10, 'ALA262': 2, 'VAL315': 5, 'GLN261': 10, 'PHE331': 3}, 'GLU258': {'LEU253': 10, 'PRO254': 10, 'SER255': 10, 'GLY256': 10, 'GLU259': 10, 'LEU260': 10, 'ALA262': 10, 'GLN263': 8, 'GLN261': 10}, 'GLU259': {'LEU253': 2, 'GLY256': 10, 'LEU260': 10, 'ALA262': 10, 'GLN263': 10, 'LEU264': 9, 'GLN261': 10, 'MET257': 10}, 'LEU260': {'LEU332': 10, 'LEU330': 6, 'GLU258': 10, 'LEU253': 10, 'MET314': 10, 'LEU251': 1, 'GLY256': 5, 'GLN263': 10, 'ALA262': 10, 'VAL315': 10, 'LEU264': 10, 'GLN261': 10, 'GLU265': 10, 'MET257': 10, 'PRO333': 4, 'LEU247': 7}, 'GLN261': {'GLU258': 10, 'LEU253': 10, 'VAL252': 5, 'LEU251': 2, 'GLU259': 10, 'GLN263': 10, 'ALA262': 10, 'LEU264': 10, 'LYS266': 10, 'GLU265': 10, 'MET257': 10, 'LEU247': 10}, 'ALA262': {'GLU267': 10, 'GLU258': 10, 'GLU259': 10, 'GLN263': 10, 'LEU260': 10, 'LEU264': 10, 'LYS266': 10, 'GLU265': 10, 'MET257': 2}, 'GLN263': {'GLU267': 10, 'GLU258': 8, 'MET314': 10, 'LYS266': 10, 'LEU268': 10, 'GLU259': 10, 'LEU260': 10, 'LEU264': 10, 'GLN261': 10, 'GLU265': 10}, 'LEU264': {'MET302': 10, 'GLU267': 10, 'GLN269': 10, 'LEU253': 10, 'MET314': 10, 'LEU251': 10, 'LYS266': 10, 'LEU268': 10, 'PRO248': 7, 'GLU259': 9, 'LEU260': 10, 'ALA262': 10, 'LEU273': 10, 'VAL315': 10, 'GLN261': 10, 'LYS304': 4, 'GLU265': 10, 'LEU247': 10}, 'GLU265': {'SER246': 7, 'GLU267': 10, 'GLN269': 10, 'MET314': 1, 'LYS266': 10, 'LEU268': 10, 'LEU273': 5, 'GLN263': 10, 'ALA262': 10, 'LEU260': 10, 'GLN261': 10, 'LEU247': 10, 'ASN270': 9}, 'LYS266': {'GLY271': 10, 'GLU267': 10, 'GLN269': 10, 'MET314': 2, 'LEU268': 10, 'GLN263': 10, 'ALA262': 10, 'SER272': 1, 'LEU264': 10, 'GLN261': 10, 'ASN270': 10}, 'GLU267': {'GLY271': 10, 'GLN269': 10, 'MET305': 10, 'MET314': 10, 'LEU268': 10, 'LEU273': 10, 'GLN263': 10, 'ALA262': 10, 'SER272': 10, 'LEU264': 10, 'LEU303': 6, 'LYS304': 10, 'GLU265': 10, 'ASN270': 10, 'PHE274': 1}, 'LEU268': {'MET302': 10, 'GLY271': 10, 'SER246': 10, 'GLN269': 10, 'LEU273': 10, 'ARG243': 3, 'GLN263': 10, 'SER244': 10, 'SER272': 10, 'LEU264': 10, 'THR245': 10, 'PRO248': 2, 'LYS266': 10, 'GLU265': 10, 'LEU247': 10, 'ASN270': 10}, 'GLN269': {'GLY271': 10, 'GLU267': 10, 'SER272': 8, 'LEU264': 10, 'LYS266': 10, 'GLU265': 10, 'ASN270': 10}, 'ASN270': {'GLY271': 10, 'GLU267': 10, 'LEU268': 10, 'ARG243': 4, 'SER272': 10, 'LYS266': 10, 'GLU265': 9}, 'GLY271': {'GLU267': 10, 'GLN269': 10, 'MET305': 3, 'LEU268': 10, 'LEU273': 10, 'ARG243': 10, 'SER244': 6, 'SER272': 10, 'LYS266': 10, 'PHE274': 9}, 'SER272': {'GLU267': 10, 'GLN269': 8, 'MET305': 10, 'LEU268': 10, 'LEU273': 10, 'ARG243': 8, 'LEU455': 1, 'LYS266': 1, 'LEU303': 10, 'LYS304': 10, 'LEU311': 10, 'ASN270': 10, 'PHE274': 10}, 'LEU273': {'ARG242': 10, 'GLY271': 10, 'MET314': 10, 'PRO248': 5, 'VAL315': 5, 'LYS304': 10, 'MET305': 10, 'GLU275': 10, 'LEU268': 10, 'GLU265': 5, 'PHE274': 10, 'GLU267': 10, 'ARG243': 10, 'LEU264': 10, 'LEU303': 10, 'LEU247': 9, 'MET302': 10, 'SER246': 9, 'SER244': 10, 'LEU311': 6}, 'PHE274': {'ARG242': 10, 'GLY271': 9, 'VAL301': 10, 'ARG450': 10, 'LEU241': 10, 'GLY451': 10, 'ALA276': 10, 'LYS304': 10, 'MET305': 10, 'GLU275': 10, 'GLY454': 1, 'SER272': 10, 'LEU452': 10, 'GLU267': 1, 'ARG243': 10, 'LEU455': 10, 'LEU303': 10, 'MET302': 10, 'SER244': 10, 'LEU311': 10}, 'GLU275': {'ARG242': 10, 'MET302': 10, 'SER246': 10, 'VAL301': 10, 'LEU241': 10, 'ASP277': 10, 'LEU273': 10, 'ARG243': 10, 'PRO248': 10, 'SER244': 10, 'LEU240': 10, 'THR245': 10, 'LEU303': 10, 'ALA276': 10, 'LEU247': 1}, 'ALA276': {'ARG242': 10, 'MET302': 10, 'VAL301': 10, 'PHE232': 10, 'PRO299': 10, 'VAL347': 6, 'LEU241': 10, 'ASP277': 10, 'LEU240': 10, 'LEU303': 10, 'PHE278': 10, 'LEU300': 10, 'PHE274': 10}, 'ASP277': {'ARG242': 10, 'VAL301': 3, 'PRO299': 8, 'GLU275': 10, 'ARG450': 9, 'LEU241': 10, 'LEU354': 10, 'GLN435': 4, 'LEU280': 10, 'LEU240': 10, 'ILE279': 10, 'PHE278': 10, 'LEU300': 6}, 'PHE278': {'VAL301': 10, 'PHE232': 10, 'LEU241': 1, 'PHE352': 1, 'ILE279': 10, 'ALA276': 10, 'LEU300': 10, 'PRO299': 10, 'ALA298': 10, 'LEU281': 10, 'VAL347': 10, 'ARG348': 1, 'LEU280': 10, 'LEU240': 10, 'SER350': 10, 'ASP351': 10, 'ASP282': 10, 'LEU354': 10, 'ALA297': 2}, 'ILE279': {'ASP282': 10, 'LEU300': 5, 'LEU354': 5, 'ASP277': 10, 'GLN435': 7, 'LEU280': 10, 'ARG250': 10, 'ALA298': 10, 'ARG431': 10, 'LEU281': 10, 'PRO320': 4}, 'LEU280': {'LEU428': 10, 'ASP282': 10, 'LEU354': 10, 'ASP277': 10, 'GLN435': 10, 'GLN427': 3, 'ARG431': 10, 'ALA434': 4, 'GLN358': 10, 'ILE284': 10, 'ALA432': 10, 'PHE278': 10, 'LEU281': 10}, 'LEU281': {'LEU428': 10, 'GLY283': 10, 'THR420': 10, 'ASN349': 2, 'PRO299': 10, 'ASP351': 9, 'ASP282': 10, 'LEU354': 10, 'ILE357': 1, 'GLN353': 10, 'ALA298': 10, 'ALA297': 10, 'ARG431': 10, 'ILE279': 10, 'LEU296': 10, 'ILE284': 10, 'PHE278': 10, 'SER350': 10}, 'ASP282': {'GLY283': 10, 'ASN322': 2, 'LEU280': 10, 'ARG250': 10, 'ALA298': 10, 'ALA297': 8, 'ARG431': 10, 'ILE279': 10, 'ILE284': 10, 'PHE278': 10, 'PRO320': 10}, 'GLY283': {'GLN427': 6, 'PRO285': 10, 'ALA298': 10, 'ALA297': 5, 'ARG431': 10, 'ILE284': 10, 'LEU281': 10}, 'ILE284': {'ALA286': 10, 'LEU428': 10, 'THR420': 10, 'ASP282': 10, 'LEU280': 10, 'GLN427': 10, 'PRO285': 10, 'GLN353': 7, 'GLY424': 10, 'HID425': 10, 'ALA297': 2, 'ARG431': 10, 'SER419': 9, 'ASN287': 2, 'LEU296': 10, 'LEU281': 10}, 'PRO285': {'ALA286': 10, 'GLY283': 10, 'THR420': 9, 'LEU428': 3, 'GLY422': 2, 'GLN427': 10, 'GLY423': 10, 'HID425': 2, 'GLY421': 10, 'SER419': 7, 'ASN287': 10, 'LEU296': 2, 'GLY424': 10}, 'ALA286': {'VAL288': 10, 'THR420': 10, 'GLY422': 3, 'LYS293': 3, 'GLN294': 10, 'GLY421': 10, 'TYR295': 10, 'ASN287': 10, 'SER419': 10, 'LEU296': 10, 'ILE284': 10, 'GLY424': 9}, 'ASN287': {'THR420': 10, 'GLY422': 10, 'GLN294': 10, 'TYR295': 1, 'VAL288': 10, 'PHE414': 10, 'ALA417': 10, 'GLU292': 9, 'LYS293': 10, 'PRO285': 10, 'VAL418': 10, 'GLY421': 10, 'SER419': 10, 'ILE289': 10, 'LEU296': 8, 'LYS416': 10, 'ASP415': 10, 'ILE284': 2, 'THR560': 9, 'GLY424': 1}, 'VAL288': {'ALA286': 10, 'GLY291': 10, 'ILE289': 10, 'GLY422': 6, 'GLU292': 10, 'ASP415': 10, 'LYS293': 10, 'ARG290': 10, 'GLN294': 10, 'GLY421': 5}, 'ILE289': {'GLY291': 10, 'ASP583': 3, 'ALA417': 10, 'LYS416': 10, 'GLU292': 10, 'ASP415': 10, 'LYS293': 10, 'ARG290': 10, 'GLN294': 10, 'VAL418': 1, 'GLN586': 10, 'ASN287': 10, 'THR560': 10, 'ARG585': 7}, 'ARG290': {'GLY291': 10, 'VAL288': 10, 'ASP583': 10, 'ARG562': 1, 'LYS416': 10, 'GLU292': 10, 'GLN586': 10, 'VAL584': 5, 'ARG585': 10}, 'GLY291': {'VAL288': 10, 'ILE289': 10, 'GLU292': 10, 'LYS293': 10}, 'GLU292': {'ASP583': 10, 'VAL288': 10, 'ILE289': 10, 'ARG562': 10, 'LYS293': 10, 'ARG290': 10, 'GLN294': 10, 'GLN586': 2, 'MET563': 1, 'TYR295': 4, 'ASN287': 9}, 'LYS293': {'GLY291': 10, 'VAL288': 10, 'ILE289': 10, 'ARG562': 10, 'ALA286': 3, 'GLN294': 10, 'PRO564': 2, 'MET563': 10, 'TYR295': 10, 'ASN287': 10, 'MET561': 9}, 'GLN294': {'ALA286': 10, 'VAL288': 10, 'ILE289': 10, 'ARG562': 10, 'ASP583': 7, 'ALA417': 4, 'GLU292': 10, 'VAL418': 10, 'GLN586': 10, 'PRO582': 8, 'MET563': 10, 'TYR295': 10, 'ASN287': 10, 'SER419': 10, 'LEU296': 10, 'THR560': 10, 'MET561': 10}, 'TYR295': {'ALA286': 10, 'GLN319': 9, 'ASN349': 10, 'ARG562': 10, 'ASN287': 1, 'TRP346': 10, 'GLU292': 4, 'LYS293': 10, 'PRO565': 10, 'ALA297': 10, 'MET563': 10, 'SER419': 10, 'PRO564': 10, 'LEU296': 10, 'THR560': 10, 'MET561': 10}, 'LEU296': {'GLN319': 9, 'LEU428': 8, 'THR420': 10, 'GLN294': 10, 'MET561': 10, 'PRO299': 9, 'TRP346': 10, 'PRO285': 2, 'ALA298': 10, 'VAL418': 3, 'SER419': 10, 'LEU281': 10, 'ASN349': 10, 'PRO565': 10, 'GLN353': 10, 'ILE318': 10, 'ASN287': 8, 'SER350': 10, 'ALA286': 10, 'ALA297': 10, 'ILE284': 10, 'THR560': 5}, 'ALA297': {'GLN319': 10, 'GLY283': 5, 'ASN349': 8, 'PRO299': 10, 'VAL347': 5, 'ASP282': 8, 'TRP346': 10, 'PRO565': 10, 'ALA298': 10, 'ILE318': 10, 'TYR295': 10, 'SER419': 1, 'SER350': 10, 'ILE284': 2, 'PHE278': 2, 'LEU281': 10, 'PRO320': 8}, 'ALA298': {'GLN319': 10, 'GLY283': 10, 'PRO299': 10, 'ASP282': 10, 'LEU300': 10, 'GLN317': 5, 'ARG250': 6, 'ILE318': 10, 'ILE279': 10, 'LEU296': 10, 'PHE278': 10, 'SER350': 10, 'LEU281': 10, 'PRO320': 10}, 'PRO299': {'GLN319': 10, 'VAL301': 10, 'VAL347': 10, 'GLN317': 10, 'LEU300': 10, 'TRP346': 10, 'ASP277': 8, 'SER350': 10, 'ILE316': 10, 'ALA297': 10, 'ILE318': 10, 'ALA276': 10, 'PHE278': 10, 'LEU296': 9, 'LEU281': 10, 'PRO320': 10}, 'LEU300': {'GLN319': 7, 'VAL301': 10, 'ASP277': 6, 'PRO248': 10, 'VAL315': 10, 'ILE279': 5, 'ALA276': 10, 'PHE278': 10, 'PRO320': 10, 'LEU251': 10, 'ALA298': 10, 'GLN317': 10, 'LEU330': 3, 'VAL252': 5, 'ILE316': 10, 'ILE318': 10, 'MET302': 10, 'SER249': 1, 'ARG250': 10, 'PRO321': 10}, 'VAL301': {'PHE232': 5, 'LEU241': 5, 'ASP277': 3, 'MET314': 1, 'PRO248': 7, 'VAL315': 10, 'ALA276': 10, 'PHE278': 10, 'PRO299': 10, 'GLU275': 10, 'LEU251': 3, 'PRO313': 10, 'GLN317': 10, 'PHE274': 10, 'VAL347': 10, 'ILE316': 10, 'LEU303': 10, 'MET302': 10, 'PHE228': 10}, 'MET302': {'GLN312': 9, 'LEU241': 3, 'MET314': 10, 'PRO248': 10, 'VAL315': 10, 'ALA276': 10, 'LYS304': 10, 'LEU300': 10, 'GLU275': 10, 'LEU251': 10, 'LEU268': 10, 'PRO313': 10, 'GLN317': 1, 'PHE274': 10, 'LEU264': 10, 'ILE316': 10, 'LEU303': 10, 'LEU247': 10, 'SER246': 10, 'LEU273': 10, 'SER244': 10}, 'LEU303': {'GLN312': 10, 'VAL301': 10, 'PHE232': 10, 'LEU241': 10, 'MET314': 10, 'VAL315': 10, 'ALA276': 10, 'LYS304': 10, 'MET305': 10, 'GLU275': 10, 'SER229': 10, 'SER272': 10, 'PRO313': 10, 'PHE274': 10, 'GLU267': 6, 'VAL347': 1, 'ILE316': 3, 'LEU233': 9, 'LEU273': 10, 'PHE228': 10, 'LEU311': 10}, 'LYS304': {'MET302': 10, 'GLN312': 10, 'GLU267': 10, 'MET305': 10, 'MET314': 10, 'PRO307': 4, 'LEU273': 10, 'SER272': 10, 'LEU311': 10, 'PRO313': 10, 'GLU306': 10, 'LEU264': 4, 'LYS310': 10, 'PRO333': 3, 'PHE274': 10}, 'MET305': {'GLN312': 10, 'GLY271': 3, 'GLY309': 10, 'GLU267': 10, 'PRO456': 9, 'PRO307': 10, 'LEU273': 10, 'ARG243': 1, 'SER272': 10, 'LEU311': 10, 'GLY457': 1, 'GLU306': 10, 'LEU455': 10, 'LEU303': 10, 'LYS310': 10, 'PHE274': 10}, 'GLU306': {'GLN312': 10, 'GLY309': 10, 'PRO307': 10, 'LEU311': 10, 'LEU455': 7, 'ASN308': 10, 'LYS304': 10, 'LYS310': 10}, 'PRO307': {'GLY309': 10, 'MET305': 10, 'ASN308': 10, 'LYS304': 4, 'LYS310': 10}, 'ASN308': {'GLY309': 10, 'PRO456': 10, 'GLN644': 3, 'GLY457': 9, 'LEU645': 9, 'GLU306': 10, 'LYS310': 10, 'TRP647': 10}, 'GLY309': {'PRO456': 10, 'MET305': 10, 'PRO307': 10, 'LEU311': 10, 'GLY457': 10, 'GLU306': 10, 'LEU455': 10, 'LEU233': 9, 'LYS310': 10, 'ALA458': 6}, 'LYS310': {'GLN312': 10, 'PRO456': 8, 'MET305': 10, 'TYR230': 2, 'PRO307': 10, 'GLY457': 10, 'ASN308': 10, 'GLU306': 10, 'ASP646': 3, 'LEU455': 10, 'LEU233': 10, 'LYS304': 10, 'LEU311': 10, 'GLU226': 9, 'TRP647': 10}, 'LEU311': {'GLN312': 10, 'PHE232': 8, 'TYR230': 8, 'LEU241': 10, 'GLU306': 10, 'LYS304': 10, 'ASP225': 2, 'MET305': 10, 'SER229': 10, 'SER272': 10, 'PRO313': 10, 'LEU452': 3, 'PHE274': 10, 'GLY309': 10, 'LEU303': 10, 'LEU233': 10, 'LEU455': 10, 'GLU226': 6, 'LEU273': 6, 'GLY457': 9}, 'GLN312': {'MET302': 9, 'MET305': 10, 'TYR230': 1, 'LEU227': 2, 'SER229': 10, 'MET314': 10, 'PRO313': 10, 'PHE228': 3, 'GLU306': 10, 'ASP224': 1, 'LYS304': 10, 'LEU303': 10, 'TRP340': 1, 'ASP225': 10, 'LYS310': 10, 'GLU226': 10, 'PRO333': 5}, 'PRO313': {'MET302': 10, 'LEU332': 1, 'VAL301': 10, 'PHE232': 1, 'SER229': 10, 'MET314': 10, 'VAL315': 10, 'ILE316': 10, 'PHE228': 10, 'LYS304': 10, 'LEU303': 10, 'TRP340': 10, 'ASP225': 10, 'LEU311': 10, 'PRO333': 10, 'LYS344': 10}, 'MET314': {'MET302': 10, 'GLN312': 10, 'LEU332': 10, 'GLU267': 10, 'VAL301': 1, 'PHE331': 2, 'LEU273': 10, 'LEU260': 10, 'GLN263': 10, 'VAL315': 10, 'LEU264': 10, 'ILE316': 10, 'LYS266': 2, 'LEU303': 10, 'TRP340': 7, 'LYS304': 10, 'GLU265': 1, 'PRO333': 10}, 'VAL315': {'VAL301': 10, 'PRO248': 1, 'LEU260': 10, 'LEU300': 10, 'PHE331': 10, 'LEU251': 10, 'PRO313': 10, 'GLN317': 10, 'LEU332': 10, 'LEU330': 10, 'LEU253': 10, 'LEU264': 10, 'ILE316': 10, 'LEU303': 10, 'MET257': 5, 'PRO333': 10, 'MET302': 10, 'LEU273': 5, 'TRP340': 10}, 'ILE316': {'VAL301': 10, 'MET314': 10, 'THR329': 10, 'LEU300': 10, 'PRO299': 10, 'PHE331': 10, 'TRP346': 2, 'LEU251': 3, 'PRO313': 10, 'GLN317': 10, 'LYS344': 10, 'LEU332': 10, 'LEU330': 10, 'VAL347': 10, 'ILE318': 10, 'LEU303': 3, 'PRO333': 9, 'MET302': 10, 'PHE228': 10, 'ALA343': 10, 'TRP340': 10}, 'GLN317': {'GLN319': 10, 'VAL301': 10, 'THR329': 10, 'VAL315': 10, 'LEU300': 10, 'PRO320': 10, 'PRO299': 10, 'LEU251': 10, 'ALA298': 5, 'LEU330': 10, 'THR327': 10, 'VAL347': 5, 'VAL252': 10, 'ILE318': 10, 'PRO328': 10, 'MET302': 1, 'ARG250': 10, 'PRO321': 10, 'PHE331': 10}, 'ILE318': {'GLN319': 10, 'THR329': 10, 'LEU300': 10, 'PRO320': 10, 'PRO299': 10, 'TRP346': 10, 'PRO566': 10, 'ALA298': 10, 'LEU342': 10, 'LYS344': 1, 'LEU330': 10, 'VAL347': 10, 'PRO565': 10, 'ILE316': 10, 'LEU296': 10, 'SER350': 5, 'PRO328': 9, 'ALA297': 10, 'ALA343': 10, 'PHE331': 10}, 'GLN319': {'PRO299': 10, 'PHE331': 5, 'PRO328': 10, 'SER325': 10, 'THR329': 8, 'PRO565': 10, 'ALA298': 10, 'ALA297': 10, 'PRO564': 10, 'TYR295': 9, 'PRO321': 10, 'LEU296': 9, 'GLN317': 10, 'LEU300': 7, 'PRO320': 10}, 'PRO320': {'ASN322': 10, 'PRO299': 10, 'ASP282': 10, 'PRO328': 10, 'SER325': 10, 'ILE279': 4, 'ARG250': 10, 'ALA298': 10, 'ALA297': 8, 'ILE318': 10, 'PRO321': 10, 'GLN317': 10, 'LEU300': 10}, 'PRO321': {'GLN319': 10, 'LEU330': 1, 'ASN322': 10, 'THR327': 10, 'PRO326': 10, 'PRO328': 10, 'LEU251': 10, 'VAL252': 10, 'SER325': 10, 'PRO323': 10, 'ARG250': 10, 'GLN317': 10, 'LEU300': 10}, 'ASN322': {'PRO326': 10, 'ASP282': 2, 'SER325': 10, 'SER324': 10, 'PRO323': 10, 'ARG250': 10, 'PRO328': 10, 'PRO320': 10}, 'PRO323': {'PRO321': 10, 'SER325': 10, 'SER324': 10, 'PRO326': 4}, 'SER324': {'ASN322': 10, 'PRO326': 10, 'SER325': 10}, 'SER325': {'GLN319': 10, 'ASN322': 10, 'THR327': 10, 'PRO326': 10, 'PRO323': 10, 'PRO321': 10, 'PRO328': 10, 'PRO320': 10}, 'PRO326': {'ASN322': 10, 'THR327': 10, 'SER324': 10, 'PRO323': 4, 'PRO321': 10, 'PRO328': 10}, 'THR327': {'LEU330': 1, 'PRO328': 10, 'PRO254': 7, 'VAL252': 5, 'SER325': 10, 'THR329': 10, 'PRO321': 10, 'GLN317': 10}, 'PRO328': {'GLN319': 10, 'LEU330': 10, 'ASN322': 10, 'PRO326': 10, 'SER325': 10, 'THR329': 10, 'THR567': 7, 'ILE318': 9, 'PRO321': 10, 'PHE331': 1, 'GLN317': 10, 'PRO320': 10}, 'THR329': {'GLN319': 8, 'LEU330': 10, 'THR327': 10, 'LYS569': 10, 'THR567': 10, 'PRO566': 10, 'ILE316': 10, 'THR568': 10, 'ILE318': 10, 'PHE331': 10, 'MET257': 1, 'GLN317': 10}, 'LEU330': {'LEU332': 10, 'ASP335': 10, 'THR327': 1, 'GLN317': 10, 'LEU253': 10, 'PRO328': 10, 'PRO254': 10, 'LEU251': 10, 'VAL252': 10, 'LYS569': 10, 'LEU260': 6, 'ILE316': 10, 'ILE318': 10, 'VAL315': 10, 'PRO321': 1, 'PHE331': 10, 'MET257': 10, 'LEU300': 3}, 'PHE331': {'GLN319': 5, 'MET314': 2, 'THR329': 10, 'VAL315': 10, 'THR567': 3, 'PRO566': 10, 'ASP335': 10, 'GLN317': 10, 'LEU342': 8, 'LEU332': 10, 'VAL572': 10, 'LYS569': 10, 'PRO565': 9, 'ILE316': 10, 'ILE318': 10, 'MET257': 3, 'PRO333': 10, 'PRO328': 1, 'ALA339': 10, 'THR568': 4, 'ALA343': 10, 'TRP340': 10}, 'LEU332': {'LEU330': 10, 'ASP335': 10, 'LEU253': 5, 'MET314': 10, 'LYS569': 10, 'LEU260': 10, 'ILE316': 10, 'PRO313': 1, 'VAL315': 10, 'TRP340': 10, 'MET257': 10, 'PRO333': 10, 'SER334': 10}, 'PRO333': {'GLN312': 5, 'MET314': 10, 'LEU260': 4, 'LYS304': 3, 'PRO313': 10, 'ILE316': 9, 'TRP340': 10, 'VAL315': 10, 'PHE331': 10, 'ASP225': 10, 'ASP335': 10, 'SER334': 10}, 'SER334': {'LEU332': 10, 'PRO337': 1, 'ASP335': 10, 'PRO336': 10}, 'ASP335': {'LEU332': 10, 'LEU330': 10, 'LYS569': 10, 'ALA339': 10, 'TRP340': 10, 'PRO337': 10, 'PHE331': 10, 'PRO333': 10, 'PRO336': 10}, 'PRO336': {'VAL572': 9, 'LYS569': 10, 'ALA339': 10, 'LEU338': 10, 'LEU341': 10, 'GLU570': 10, 'PRO337': 10, 'TRP340': 10, 'SER334': 10}, 'PRO337': {'SER334': 1, 'ALA339': 10, 'LEU338': 10, 'GLN223': 3, 'LEU341': 10, 'TRP340': 10, 'ASP335': 10, 'LEU342': 10, 'LYS344': 3}, 'LEU338': {'THR573': 10, 'VAL577': 10, 'MET574': 10, 'VAL572': 10, 'ALA339': 10, 'LEU341': 10, 'ALA343': 10, 'TRP340': 10, 'LEU200': 10, 'LEU342': 10, 'PRO336': 10}, 'ALA339': {'VAL577': 5, 'VAL572': 10, 'LYS569': 10, 'PRO566': 10, 'LYS344': 10, 'LEU341': 10, 'TRP340': 10, 'ALA343': 10, 'GLU570': 2, 'PRO337': 10, 'PHE331': 10, 'ASP335': 10, 'LEU342': 10, 'PRO336': 10}, 'TRP340': {'GLN312': 1, 'MET314': 7, 'GLN223': 8, 'LEU341': 10, 'VAL315': 10, 'ASP225': 10, 'PRO313': 10, 'ASP224': 4, 'ASP335': 10, 'LEU342': 10, 'LYS344': 10, 'LEU332': 10, 'PRO336': 10, 'LYS569': 3, 'SER345': 10, 'ILE316': 10, 'PRO337': 10, 'PRO333': 10, 'LEU338': 10, 'PHE228': 6, 'ALA343': 10, 'PHE331': 10}, 'LEU341': {'PRO555': 5, 'VAL577': 10, 'LEU581': 2, 'TRP346': 9, 'TRP222': 10, 'ALA339': 10, 'LEU338': 10, 'LYS344': 10, 'GLN223': 10, 'SER345': 10, 'ASP224': 5, 'ASN556': 6, 'ALA343': 10, 'PRO337': 10, 'LEU200': 9, 'LEU342': 10, 'PRO336': 10}, 'LEU342': {'VAL577': 10, 'LEU581': 10, 'MET561': 10, 'THR573': 3, 'THR576': 10, 'PHE331': 8, 'TRP346': 10, 'PRO566': 10, 'MET578': 1, 'LYS344': 10, 'VAL347': 10, 'VAL572': 10, 'PRO565': 9, 'SER345': 10, 'ILE318': 10, 'PRO337': 10, 'SER580': 10, 'ALA339': 10, 'LEU338': 10, 'ALA343': 10, 'TRP340': 10}, 'ALA343': {'VAL347': 10, 'TRP346': 10, 'ARG348': 10, 'LYS569': 1, 'ALA339': 10, 'LEU338': 10, 'ILE316': 10, 'SER345': 10, 'PRO566': 5, 'ILE318': 10, 'TRP340': 10, 'LEU341': 10, 'PHE228': 1, 'PHE331': 10, 'LYS344': 10}, 'LYS344': {'GLN223': 10, 'ASN556': 2, 'LEU341': 10, 'ASP225': 10, 'TRP346': 10, 'PRO313': 10, 'ASP224': 10, 'LEU342': 10, 'ASN349': 10, 'VAL347': 10, 'ARG348': 10, 'SER345': 10, 'ILE316': 10, 'ILE318': 1, 'PRO337': 3, 'TRP222': 10, 'ALA339': 10, 'PHE228': 10, 'TRP340': 10}, 'SER345': {'PRO555': 10, 'ASN349': 10, 'VAL577': 9, 'VAL347': 10, 'PRO558': 10, 'LEU581': 10, 'TRP346': 10, 'ARG348': 10, 'ALA557': 10, 'PHE228': 4, 'ASN556': 10, 'LEU341': 10, 'ALA343': 10, 'TRP340': 10, 'TRP222': 9, 'SER350': 10, 'LEU342': 10, 'MET561': 10}, 'TRP346': {'VAL577': 1, 'LEU341': 9, 'TYR295': 10, 'MET561': 10, 'PRO299': 10, 'THR576': 3, 'PRO566': 10, 'MET563': 10, 'LEU342': 10, 'LYS344': 10, 'ASN349': 10, 'ARG562': 6, 'VAL347': 10, 'PRO558': 7, 'ARG348': 10, 'PRO565': 10, 'ILE316': 2, 'ILE318': 10, 'LEU296': 10, 'SER350': 10, 'SER580': 10, 'ASP351': 10, 'ALA297': 10, 'PRO564': 10, 'ALA343': 10}, 'VAL347': {'VAL301': 10, 'PHE232': 10, 'PHE352': 10, 'ALA276': 6, 'PHE278': 10, 'PRO299': 10, 'GLN317': 5, 'LEU342': 10, 'LYS344': 10, 'ASN349': 10, 'ARG348': 10, 'SER345': 10, 'ILE316': 10, 'LEU240': 2, 'ILE318': 10, 'LEU303': 1, 'SER350': 10, 'ASP351': 10, 'PHE228': 10, 'ALA297': 5, 'ALA343': 10}, 'ARG348': {'PHE232': 10, 'ALA236': 10, 'PHE352': 10, 'ASN556': 10, 'PHE278': 1, 'ALA557': 10, 'TRP346': 10, 'TYR551': 10, 'GLN231': 10, 'LYS344': 10, 'ASN349': 10, 'PRO558': 10, 'TRP550': 10, 'SER345': 10, 'GLN353': 8, 'SER350': 10, 'ASP351': 10, 'LEU227': 2, 'PHE228': 10, 'ALA343': 10, 'ASN237': 10, 'VAL218': 8}, 'ASN349': {'THR420': 4, 'CYS559': 10, 'PHE352': 10, 'PRO582': 1, 'TYR295': 10, 'ALA557': 10, 'MET561': 10, 'TRP346': 10, 'TYR551': 2, 'VAL418': 7, 'SER419': 10, 'LEU281': 2, 'LYS344': 10, 'VAL347': 10, 'PRO558': 10, 'SER345': 10, 'GLN353': 10, 'LEU296': 10, 'SER350': 10, 'ASP351': 10, 'LEU354': 8, 'ALA297': 8, 'THR560': 10}, 'SER350': {'PRO299': 10, 'VAL347': 10, 'ASP351': 10, 'TRP346': 10, 'ARG348': 10, 'LEU354': 10, 'PRO558': 1, 'HIE355': 10, 'SER345': 10, 'GLN353': 10, 'ALA298': 10, 'PHE352': 10, 'ALA297': 10, 'ILE318': 5, 'SER419': 2, 'LEU296': 10, 'PHE278': 10, 'LEU281': 10}, 'ASP351': {'PHE232': 10, 'GLH356': 10, 'ALA236': 8, 'PHE352': 10, 'PHE278': 10, 'ALA557': 9, 'TRP346': 10, 'TYR551': 10, 'GLN231': 10, 'TYR359': 6, 'LEU281': 9, 'ASN349': 10, 'VAL347': 10, 'ARG348': 10, 'HIE355': 10, 'GLN353': 10, 'LEU240': 10, 'LEU354': 10, 'ASN237': 10}, 'PHE352': {'CYS559': 10, 'THR420': 6, 'GLN590': 10, 'GLN547': 3, 'GLH356': 10, 'ILE357': 10, 'PHE278': 1, 'ALA557': 10, 'PHE414': 10, 'SER594': 4, 'TYR551': 10, 'VAL418': 10, 'SER419': 4, 'ASN349': 10, 'VAL347': 10, 'PRO558': 10, 'ARG348': 10, 'HIE355': 10, 'GLN353': 10, 'SER350': 10, 'LEU354': 10, 'ASN237': 9, 'THR560': 1}, 'GLN353': {'LEU428': 10, 'THR420': 10, 'CYS559': 10, 'GLH356': 10, 'ILE357': 10, 'PHE414': 10, 'VAL418': 10, 'GLY421': 6, 'SER419': 10, 'LEU281': 10, 'ASN349': 10, 'PRO558': 10, 'ARG348': 8, 'HIE355': 10, 'GLN358': 10, 'LEU296': 10, 'SER350': 10, 'ASP351': 10, 'LEU354': 10, 'ILE284': 7, 'THR560': 10}, 'LEU354': {'LEU428': 10, 'ASP277': 10, 'GLH356': 10, 'ILE357': 10, 'PHE352': 10, 'ILE279': 5, 'ALA432': 2, 'PHE278': 10, 'TYR359': 10, 'LEU281': 10, 'ASN349': 8, 'GLN435': 1, 'HIE355': 10, 'LEU280': 10, 'LEU240': 10, 'GLN358': 10, 'SER350': 10, 'ASP351': 10, 'ASN237': 8}, 'HIE355': {'ASP351': 10, 'GLN358': 10, 'GLH356': 10, 'GLN547': 10, 'ALA236': 10, 'ILE357': 10, 'LEU361': 10, 'GLY546': 10, 'GLN353': 10, 'TYR551': 10, 'LEU597': 1, 'PHE352': 10, 'ALA542': 7, 'ILE543': 10, 'GLN231': 1, 'ASN237': 10, 'TYR359': 10, 'SER350': 10}, 'GLH356': {'THR420': 1, 'ILE593': 6, 'ASP351': 10, 'LEU354': 10, 'PHE414': 10, 'GLN547': 10, 'ILE357': 10, 'LEU362': 9, 'LEU361': 10, 'TYR551': 4, 'GLN353': 10, 'LEU597': 10, 'VAL418': 8, 'PHE352': 10, 'SER594': 1, 'ILE543': 1, 'GLN358': 10, 'TYR359': 7}, 'ILE357': {'LEU428': 10, 'THR420': 10, 'ILE408': 10, 'LEU361': 10, 'LEU429': 10, 'ASN363': 10, 'PHE352': 10, 'ALA432': 10, 'PHE414': 10, 'LEU362': 10, 'VAL418': 6, 'HID425': 10, 'TYR359': 10, 'LEU281': 1, 'HIE355': 10, 'GLN353': 10, 'GLN358': 10, 'LEU354': 10, 'LEU366': 1}, 'GLN358': {'LEU428': 10, 'ALA432': 10, 'LEU354': 10, 'GLH356': 10, 'GLN435': 10, 'HIE355': 10, 'ALA433': 1, 'LEU280': 10, 'LEU361': 10, 'LEU362': 10, 'GLN353': 10, 'ASN363': 10, 'THR364': 10, 'LEU429': 10, 'LEU240': 3, 'SER440': 1, 'TYR359': 10, 'LEU436': 9}, 'TYR359': {'ARG450': 10, 'GLH356': 7, 'ILE357': 10, 'ALA236': 1, 'LEU361': 10, 'ASN363': 10, 'MET239': 10, 'LEU362': 10, 'THR364': 10, 'PRO238': 1, 'GLN435': 2, 'HIE355': 10, 'LEU240': 10, 'ILE543': 10, 'ASP351': 6, 'LEU354': 10, 'ASN237': 10, 'SER440': 2, 'GLN539': 3}, 'LEU361': {'ILE408': 10, 'GLH356': 10, 'ILE399': 1, 'ILE357': 10, 'LEU429': 10, 'ASN363': 10, 'PHE414': 7, 'LEU362': 10, 'ARG404': 3, 'TRP144': 8, 'THR364': 10, 'HID425': 9, 'TYR359': 10, 'HIE355': 10, 'LEU407': 10, 'GLN358': 10, 'ASN400': 9, 'ALA403': 10, 'VAL367': 10, 'LEU366': 10}, 'LEU362': {'ALA368': 10, 'ARG430': 1, 'PRO149': 10, 'ALA432': 10, 'GLH356': 9, 'ALA433': 10, 'ILE357': 10, 'VAL367': 10, 'THR364': 10, 'ASN363': 10, 'LEU366': 10, 'LEU429': 10, 'TRP144': 1, 'LEU148': 9, 'GLN358': 10, 'LEU507': 6, 'TYR359': 10, 'LEU436': 10}, 'ASN363': {'ALA368': 7, 'ALA432': 10, 'MET239': 1, 'THR535': 10, 'GLN435': 7, 'ILE357': 10, 'VAL367': 10, 'LEU361': 10, 'THR364': 10, 'LEU366': 10, 'THR437': 3, 'LEU441': 10, 'ILE543': 2, 'GLN358': 10, 'SER440': 9, 'TYR359': 10, 'LEU436': 10, 'GLN539': 10}, 'THR364': {'ALA368': 10, 'CYS536': 10, 'ASN400': 1, 'VAL661': 10, 'MET239': 1, 'THR535': 10, 'VAL367': 10, 'LEU362': 10, 'LEU361': 10, 'LEU441': 8, 'LEU366': 10, 'THR396': 4, 'GLU369': 5, 'ALA542': 1, 'ILE543': 10, 'GLN358': 10, 'TYR359': 10, 'GLN539': 10}, 'LEU366': {'ILE408': 6, 'ILE357': 1, 'LEU361': 10, 'LEU429': 10, 'ASN363': 10, 'GLU369': 10, 'ALA368': 10, 'LEU362': 10, 'ARG404': 10, 'TRP144': 10, 'THR364': 10, 'LEU148': 10, 'LEU507': 1, 'VAL370': 10, 'PRO149': 10, 'LEU150': 1, 'THR151': 10, 'ASN400': 10, 'ALA403': 4, 'VAL367': 10, 'ILE371': 10}, 'VAL367': {'MET531': 6, 'LEU361': 10, 'LEU441': 10, 'ASN363': 10, 'GLU369': 10, 'ALA368': 10, 'CYS536': 2, 'THR535': 10, 'LEU362': 10, 'ILE502': 10, 'THR364': 10, 'CYS532': 2, 'LEU507': 10, 'LEU436': 10, 'ALA372': 10, 'VAL370': 10, 'PRO149': 6, 'ASN400': 7, 'PHE515': 10, 'ILE371': 10}, 'ALA368': {'CYS536': 10, 'ASN400': 10, 'VAL661': 10, 'THR535': 10, 'LEU362': 10, 'ALA372': 10, 'THR364': 10, 'ASN363': 7, 'LEU366': 10, 'ILE371': 10, 'THR396': 10, 'GLU369': 10, 'ILE393': 10, 'VAL370': 10, 'HID392': 6, 'VAL373': 10}, 'GLU369': {'THR151': 10, 'ASN400': 10, 'VAL367': 10, 'ARG404': 10, 'ALA374': 10, 'ALA372': 10, 'THR364': 5, 'LEU366': 10, 'ILE371': 10, 'THR396': 10, 'MET397': 10, 'ILE393': 3, 'VAL370': 10, 'THR401': 9, 'ILE152': 10, 'VAL373': 10}, 'VAL370': {'ALA368': 10, 'THR151': 10, 'GLU501': 10, 'THR375': 10, 'GLY506': 10, 'ASN400': 1, 'THR503': 1, 'VAL367': 10, 'ILE502': 10, 'ALA374': 10, 'ALA372': 10, 'LEU366': 10, 'ILE371': 10, 'PRO149': 10, 'LEU507': 10, 'LEU150': 10, 'VAL373': 10}, 'ILE371': {'MET531': 10, 'ALA374': 10, 'LEU441': 5, 'GLU369': 10, 'ILE393': 10, 'ALA368': 10, 'CYS536': 10, 'THR535': 10, 'ILE502': 10, 'PHE528': 10, 'CYS532': 10, 'TRP498': 10, 'LEU507': 2, 'LEU389': 10, 'MET376': 10, 'THR375': 10, 'ALA372': 10, 'PHE515': 3, 'VAL367': 10, 'VAL533': 5, 'LEU366': 10, 'VAL373': 10}, 'ALA372': {'ALA368': 10, 'MET376': 10, 'MET164': 10, 'THR375': 10, 'LEU389': 2, 'ARG377': 10, 'TYR139': 3, 'VAL367': 10, 'ALA374': 10, 'THR396': 10, 'GLU369': 10, 'ILE393': 10, 'VAL370': 10, 'MET397': 10, 'VAL373': 10}, 'VAL373': {'GLU501': 10, 'ALA374': 10, 'TRP141': 10, 'ALA153': 5, 'GLU369': 10, 'VAL505': 1, 'CYS140': 10, 'ALA368': 10, 'MET397': 10, 'CYS378': 10, 'MET376': 10, 'THR375': 10, 'VAL370': 10, 'GLY506': 4, 'ILE152': 10, 'LEU150': 10, 'THR151': 10, 'TYR139': 10, 'ILE371': 10, 'ALA142': 3, 'ARG377': 10}, 'ALA374': {'MET376': 10, 'GLU501': 10, 'CYS499': 3, 'THR375': 10, 'ARG377': 10, 'ILE502': 10, 'ALA372': 10, 'ILE371': 10, 'TRP141': 3, 'GLU369': 10, 'LEU379': 10, 'VAL370': 10, 'TRP498': 10, 'GLY506': 5, 'CYS378': 10}, 'THR375': {'MET376': 10, 'ILE385': 10, 'LEU389': 10, 'ARG377': 10, 'ALA372': 10, 'ILE371': 10, 'PHE386': 10, 'ILE390': 10, 'LEU379': 10, 'ILE393': 10, 'VAL370': 10, 'TRP498': 10, 'CYS378': 10, 'VAL373': 10}, 'MET376': {'PRO380': 3, 'ALA374': 10, 'PRO161': 1, 'ILE393': 10, 'TYR97': 9, 'ARG135': 10, 'PHE386': 10, 'ILE390': 10, 'LEU379': 10, 'MET397': 10, 'TRP498': 2, 'CYS378': 10, 'ALA372': 10, 'MET164': 10, 'TYR139': 10, 'ILE371': 10, 'ASN163': 10, 'ARG377': 10, 'VAL373': 10}, 'ARG377': {'GLN136': 10, 'GLU501': 10, 'GLN137': 10, 'THR375': 10, 'PRO380': 9, 'TYR139': 10, 'ALA374': 10, 'ALA372': 10, 'ARG135': 10, 'TRP141': 10, 'TRP498': 2, 'LEU379': 10, 'VAL505': 6, 'CYS140': 10, 'LEU132': 10, 'CYS378': 10, 'ARG500': 6, 'VAL373': 10}, 'CYS378': {'MET376': 10, 'GLN136': 10, 'GLH494': 10, 'ALA497': 10, 'THR375': 10, 'PRO380': 10, 'GLU501': 10, 'ILE502': 1, 'ALA374': 10, 'ARG135': 10, 'LEU379': 10, 'TRP498': 10, 'LEU132': 10, 'ARG500': 3, 'VAL373': 10}, 'LEU379': {'MET376': 10, 'GLH494': 10, 'ILE385': 10, 'ALA497': 3, 'THR375': 10, 'PRO380': 10, 'HIP383': 10, 'LEU495': 10, 'ALA374': 10, 'GLY381': 10, 'ARG135': 10, 'PHE386': 10, 'PRO384': 10, 'ARG377': 10, 'TRP498': 10, 'LEU132': 8, 'LEU389': 10}, 'PRO380': {'MET376': 3, 'GLH494': 10, 'GLN136': 1, 'LEU382': 10, 'GLU131': 10, 'HIP383': 10, 'ARG128': 10, 'GLY381': 10, 'ARG135': 10, 'PHE386': 10, 'ARG377': 9, 'LEU132': 10, 'CYS378': 10}, 'GLY381': {'LEU382': 10, 'GLU131': 10, 'HIP383': 10, 'PRO109': 4, 'ARG135': 10, 'PHE386': 10, 'LYS387': 10, 'LEU379': 10, 'PRO384': 1, 'TYR97': 9, 'ALA113': 10, 'CYS96': 1}, 'LEU382': {'GLU131': 6, 'GLU110': 10, 'PRO380': 10, 'HIP383': 10, 'HID127': 6, 'PRO109': 10, 'ARG128': 10, 'ARG114': 10, 'PHE124': 10, 'THR112': 10, 'PRO384': 10, 'PHE62': 10, 'ALA113': 10, 'LEU115': 1}, 'HIP383': {'GLH494': 10, 'ILE385': 10, 'TYR483': 10, 'PRO380': 10, 'ASP492': 10, 'ALA113': 10, 'PRO493': 6, 'PHE615': 4, 'LEU495': 10, 'ARG128': 2, 'GLY381': 10, 'ARG114': 10, 'PHE386': 10, 'LEU379': 10, 'PRO384': 10, 'LYS387': 10, 'PRO116': 10, 'LEU115': 10}, 'PRO384': {'ILE385': 10, 'LEU382': 10, 'TYR483': 10, 'ASP492': 9, 'ALA113': 10, 'PHE615': 10, 'PHE482': 10, 'LEU495': 8, 'PHE388': 10, 'GLY381': 1, 'ARG114': 10, 'PHE386': 10, 'LEU379': 10, 'LYS387': 10, 'PRO116': 9, 'LEU115': 10}, 'ILE385': {'TYR483': 10, 'THR375': 10, 'PHE615': 10, 'PHE482': 7, 'HIP383': 10, 'LEU495': 10, 'PHE388': 10, 'VAL479': 10, 'PHE386': 10, 'LYS387': 10, 'LEU379': 10, 'LEU529': 10, 'ILE390': 7, 'TRP498': 10, 'ALA113': 1, 'LEU389': 10, 'LEU525': 10}, 'PHE386': {'MET376': 10, 'MET164': 1, 'THR375': 10, 'PRO380': 10, 'PHE615': 1, 'HIP383': 10, 'PHE388': 10, 'GLY381': 10, 'ARG135': 1, 'PRO391': 8, 'ILE390': 10, 'LYS387': 10, 'LEU379': 10, 'PRO384': 10, 'TYR97': 10, 'ILE393': 10, 'LEU389': 10}, 'LYS387': {'ILE385': 10, 'LEU389': 10, 'TYR614': 10, 'GLY111': 10, 'PHE615': 10, 'HIP383': 10, 'SER616': 6, 'PHE388': 10, 'GLY381': 10, 'PRO391': 10, 'ILE390': 10, 'GLU612': 10, 'PRO384': 10, 'TYR97': 10, 'THR112': 2, 'LYS613': 10, 'ALA113': 10, 'PRO95': 10}, 'PHE388': {'TYR483': 4, 'PHE615': 10, 'ILE385': 10, 'TYR614': 10, 'VAL479': 10, 'ILE390': 10, 'PHE386': 10, 'PRO384': 10, 'LEU389': 10, 'VAL475': 10, 'PHE482': 10, 'PRO391': 10, 'HID392': 10, 'TYR474': 8, 'GLU476': 5, 'LEU481': 5, 'VAL533': 10, 'ILE478': 10, 'LEU529': 10}, 'LEU389': {'CYS536': 7, 'ILE385': 10, 'THR375': 10, 'TYR614': 10, 'ARG394': 7, 'ALA372': 2, 'VAL533': 10, 'ILE371': 10, 'PHE386': 10, 'LYS387': 10, 'LEU379': 10, 'ILE390': 10, 'PRO391': 10, 'HID392': 10, 'ILE393': 10, 'TRP498': 10, 'LEU529': 10, 'CYS532': 10}, 'ILE390': {'MET376': 10, 'ILE385': 7, 'MET164': 9, 'THR375': 10, 'TYR614': 8, 'ARG394': 10, 'PHE388': 10, 'PRO391': 10, 'PHE386': 10, 'ASN163': 8, 'LYS387': 10, 'HID392': 10, 'TYR97': 10, 'ILE393': 10, 'GLU612': 10}, 'PRO391': {'TYR97': 4, 'TYR614': 10, 'ARG394': 10, 'VAL661': 1, 'LYS611': 10, 'SER660': 10, 'ASN659': 10, 'PHE388': 10, 'HID392': 10, 'GLU612': 10, 'LYS387': 10, 'PHE386': 8, 'ILE393': 10, 'HID610': 10, 'LYS613': 10, 'LEU389': 10, 'LEU607': 1}, 'HID392': {'ASN659': 10, 'THR396': 10, 'PHE627': 5, 'ILE393': 10, 'ALA368': 6, 'CYS536': 10, 'TYR614': 10, 'ILE390': 10, 'CYS532': 5, 'LEU389': 10, 'LEU607': 10, 'SER660': 10, 'PHE388': 10, 'TYR474': 8, 'HID610': 10, 'VAL661': 10, 'ARG394': 10, 'TYR395': 1, 'VAL533': 10, 'LEU529': 1, 'THR537': 10}, 'ILE393': {'ALA368': 10, 'MET376': 10, 'CYS536': 10, 'MET164': 10, 'THR375': 10, 'VAL661': 10, 'ARG394': 10, 'TYR395': 10, 'SER660': 10, 'ALA372': 10, 'THR396': 10, 'ILE371': 10, 'PHE386': 10, 'ILE390': 10, 'GLU369': 3, 'PRO391': 10, 'MET397': 10, 'LEU389': 10}, 'ARG394': {'ASN659': 10, 'THR396': 10, 'HIE167': 10, 'VAL605': 10, 'ILE390': 10, 'MET397': 3, 'MET604': 10, 'LEU389': 7, 'SER660': 10, 'GLH658': 10, 'PRO391': 10, 'HID392': 10, 'ASP603': 10, 'MET164': 10, 'VAL661': 10, 'TYR395': 10, 'THR662': 10, 'LYS170': 10, 'ASN163': 10, 'GLN601': 10}, 'TYR395': {'ILE399': 10, 'THR396': 10, 'ARG165': 10, 'ILE393': 10, 'HIE167': 10, 'MET397': 10, 'PHE166': 10, 'SER660': 10, 'HID392': 1, 'ASP603': 7, 'MET164': 10, 'ASN400': 10, 'VAL661': 10, 'THR662': 10, 'LYS170': 10, 'PRO162': 2, 'ASN163': 10, 'GLU398': 10, 'PHE77': 1}, 'THR396': {'ALA368': 10, 'CYS536': 3, 'MET164': 7, 'ASN400': 10, 'VAL661': 10, 'ARG394': 10, 'SER660': 5, 'ILE399': 10, 'THR662': 9, 'ALA372': 10, 'THR364': 4, 'HID392': 10, 'GLU369': 10, 'ILE393': 10, 'MET397': 10, 'GLU398': 10, 'THR401': 10}, 'MET397': {'ILE399': 10, 'ARG165': 1, 'PRO161': 10, 'GLU369': 10, 'ILE393': 10, 'ALA154': 3, 'ARG404': 10, 'THR401': 10, 'MET376': 10, 'ARG402': 10, 'ALA372': 10, 'ILE152': 10, 'LEU160': 10, 'THR151': 4, 'MET164': 10, 'ASN400': 10, 'ARG394': 3, 'TYR395': 10, 'TYR139': 10, 'GLU398': 10, 'VAL373': 10}, 'GLU398': {'MET164': 10, 'ASN400': 10, 'PHE166': 10, 'TYR395': 10, 'ALA403': 9, 'ILE399': 10, 'ARG402': 10, 'THR396': 10, 'ARG165': 10, 'THR401': 10, 'ILE152': 1, 'LEU160': 10, 'LYS157': 10, 'ARG171': 2}, 'ILE399': {'ASN400': 10, 'PHE174': 10, 'PHE166': 10, 'TYR395': 10, 'VAL661': 10, 'ALA403': 10, 'THR662': 10, 'ARG402': 10, 'ARG404': 10, 'LYS170': 3, 'LEU361': 1, 'THR396': 10, 'LEU597': 2, 'MET397': 10, 'THR401': 10, 'HIE596': 1}, 'ASN400': {'LEU361': 9, 'THR396': 10, 'GLU369': 10, 'ALA368': 10, 'THR405': 10, 'ARG404': 10, 'THR364': 1, 'MET397': 10, 'THR401': 10, 'ARG402': 10, 'VAL370': 1, 'ILE152': 5, 'THR151': 10, 'VAL661': 9, 'TYR395': 10, 'ALA403': 10, 'VAL367': 7, 'LEU366': 10, 'GLU398': 10}, 'THR401': {'GLN406': 10, 'ALA403': 10, 'ILE399': 10, 'THR405': 10, 'ASP155': 10, 'ALA154': 10, 'ARG402': 10, 'ARG156': 10, 'ARG404': 10, 'THR396': 10, 'GLU369': 9, 'MET397': 10, 'GLU398': 10, 'ILE152': 10, 'LEU160': 10}, 'ARG402': {'ASN400': 10, 'PHE174': 9, 'PHE166': 8, 'GLN406': 10, 'ILE408': 10, 'ALA403': 10, 'ILE399': 10, 'THR405': 10, 'ARG404': 10, 'ARG156': 8, 'LEU407': 10, 'MET397': 10, 'GLU398': 10, 'LYS157': 8}, 'ALA403': {'ASN400': 10, 'GLN406': 10, 'ILE408': 10, 'ILE399': 10, 'THR405': 10, 'SER409': 10, 'ARG404': 10, 'LEU361': 10, 'LEU407': 10, 'LEU366': 4, 'THR401': 10, 'GLU398': 9}, 'ARG404': {'ILE408': 10, 'ILE399': 10, 'LYS145': 3, 'ASP155': 8, 'LEU361': 3, 'GLU369': 10, 'THR405': 10, 'ALA154': 10, 'TRP144': 10, 'MET397': 10, 'THR401': 10, 'GLN406': 10, 'ARG402': 10, 'LEU407': 10, 'PRO149': 8, 'ILE152': 10, 'THR151': 10, 'ASN400': 10, 'SER409': 10, 'LEU366': 10}, 'THR405': {'ASN400': 10, 'GLN406': 10, 'ILE408': 9, 'ALA403': 10, 'ASP155': 10, 'ALA154': 10, 'ARG402': 10, 'ARG156': 10, 'LEU407': 10, 'SER409': 10, 'THR401': 10}, 'GLN406': {'GLY412': 10, 'ILE408': 10, 'ALA403': 10, 'PHE414': 2, 'SER409': 10, 'ARG404': 10, 'ARG402': 10, 'LEU407': 10, 'ASP155': 5, 'ILE413': 10, 'THR401': 10}, 'LEU407': {'ILE593': 10, 'GLY412': 10, 'ILE408': 10, 'ALA403': 10, 'THR405': 10, 'PHE414': 10, 'SER409': 10, 'ARG404': 10, 'ARG402': 10, 'LEU361': 10, 'ASP415': 10, 'LEU597': 1, 'ILE413': 10}, 'ILE408': {'LYS145': 9, 'ILE357': 10, 'LEU361': 10, 'LEU429': 10, 'PHE414': 10, 'THR405': 9, 'ARG404': 10, 'TRP144': 10, 'HID425': 10, 'GLY421': 1, 'LEU148': 1, 'VAL426': 2, 'GLN406': 10, 'GLY412': 10, 'ARG402': 10, 'ILE413': 10, 'ASP410': 10, 'ALA403': 10, 'SER409': 10, 'ASP415': 10, 'LEU366': 6}, 'SER409': {'ASP410': 10, 'GLN406': 10, 'GLY412': 10, 'ALA403': 10, 'THR405': 10, 'LYS145': 7, 'ASP415': 10, 'ARG404': 10, 'PHE414': 6, 'LEU407': 10, 'HID425': 7, 'ILE413': 9, 'GLY411': 10}, 'ASP410': {'GLY412': 10, 'ILE408': 10, 'GLY422': 9, 'ASP415': 10, 'HID425': 3, 'GLY411': 10}, 'GLY411': {'GLY412': 10, 'LYS416': 10, 'GLY422': 4, 'ASP415': 10, 'SER409': 10, 'ILE413': 10}, 'GLY412': {'ASP410': 10, 'GLN406': 10, 'ILE408': 10, 'PHE414': 10, 'LYS416': 10, 'ALA417': 10, 'SER409': 10, 'ASP415': 10, 'LEU407': 10, 'ILE413': 10}, 'ILE413': {'ILE593': 10, 'GLN406': 10, 'ILE408': 10, 'PHE414': 10, 'LYS416': 10, 'ALA417': 10, 'SER409': 9, 'ASP415': 10, 'LEU589': 10, 'LEU407': 10, 'VAL418': 10, 'GLY411': 10, 'LEU178': 5}, 'PHE414': {'THR420': 10, 'ILE408': 10, 'GLH356': 10, 'GLY422': 10, 'ILE357': 10, 'LEU361': 7, 'PHE352': 10, 'ALA417': 10, 'VAL418': 10, 'HID425': 10, 'GLY421': 10, 'SER419': 1, 'GLY412': 10, 'GLN406': 2, 'LEU407': 10, 'GLN353': 10, 'ASN287': 10, 'LYS416': 10, 'ASP415': 10, 'SER409': 6}, 'ASP415': {'VAL288': 10, 'ILE289': 10, 'ASP410': 10, 'GLY412': 10, 'ILE408': 10, 'ALA417': 10, 'LYS416': 10, 'GLY422': 10, 'SER409': 10, 'LEU407': 10, 'VAL418': 10, 'ILE413': 10, 'GLY411': 10, 'GLY421': 10, 'HID425': 9, 'ASN287': 10}, 'LYS416': {'ILE289': 10, 'GLY412': 10, 'PHE414': 10, 'ALA417': 10, 'LEU589': 10, 'ARG290': 10, 'VAL418': 10, 'ILE413': 10, 'GLY411': 10, 'GLN586': 10, 'GLY421': 1, 'ASN287': 10, 'MET185': 1, 'ARG585': 10}, 'ALA417': {'ILE289': 10, 'CYS559': 10, 'GLN590': 8, 'GLY412': 10, 'PHE414': 10, 'ASP415': 10, 'LEU589': 10, 'GLN294': 4, 'VAL418': 10, 'ILE413': 10, 'GLN586': 10, 'GLY421': 6, 'SER419': 10, 'ASN287': 10, 'PRO582': 1, 'THR560': 10}, 'VAL418': {'THR420': 10, 'CYS559': 10, 'GLH356': 8, 'ILE357': 6, 'GLN294': 10, 'PHE352': 10, 'PHE414': 10, 'GLN586': 6, 'GLY421': 10, 'SER419': 10, 'ASN349': 7, 'ILE289': 1, 'GLN353': 10, 'ILE413': 10, 'ASN287': 10, 'LEU296': 3, 'LYS416': 10, 'ASP415': 10, 'LEU589': 2, 'THR560': 10}, 'SER419': {'THR420': 10, 'CYS559': 10, 'GLN294': 10, 'PHE352': 4, 'TYR295': 10, 'MET561': 10, 'ALA417': 10, 'PHE414': 1, 'PRO285': 7, 'GLY421': 10, 'ASN349': 10, 'GLN353': 10, 'ASN287': 10, 'LEU296': 10, 'SER350': 2, 'ALA286': 10, 'ALA297': 1, 'ILE284': 9, 'THR560': 10}, 'THR420': {'LEU428': 10, 'CYS559': 10, 'GLH356': 1, 'GLY422': 10, 'ILE357': 10, 'GLY423': 3, 'PHE352': 6, 'PHE414': 10, 'PRO285': 9, 'VAL418': 10, 'HID425': 10, 'GLY421': 10, 'VAL426': 1, 'LEU281': 10, 'ASN349': 4, 'GLN353': 10, 'ASN287': 10, 'LEU296': 10, 'ALA286': 10, 'ILE284': 10, 'THR560': 2, 'GLY424': 10}, 'GLY421': {'ALA286': 10, 'VAL288': 5, 'PHE414': 10, 'ALA417': 6, 'GLY422': 10, 'LYS416': 1, 'ASP415': 10, 'ILE408': 1, 'PRO285': 10, 'GLN353': 6, 'GLY423': 10, 'VAL418': 10, 'HID425': 10, 'SER419': 10, 'ASN287': 10, 'GLY424': 10}, 'GLY422': {'ALA286': 3, 'VAL288': 6, 'THR420': 10, 'ASP410': 9, 'PHE414': 10, 'ASP415': 10, 'PRO285': 2, 'GLY423': 10, 'HID425': 10, 'GLY411': 4, 'ASN287': 10, 'VAL426': 10, 'GLY424': 10}, 'GLY423': {'LEU428': 8, 'THR420': 3, 'ARG430': 1, 'GLN427': 10, 'PRO285': 10, 'HID425': 10, 'GLY421': 10, 'VAL426': 10, 'GLY424': 10}, 'GLY424': {'ALA286': 9, 'LEU428': 10, 'THR420': 10, 'GLY422': 10, 'GLN427': 10, 'PRO285': 10, 'LEU429': 10, 'HID425': 10, 'GLY421': 10, 'ASN287': 1, 'VAL426': 10, 'ILE284': 10}, 'HID425': {'LEU428': 10, 'THR420': 10, 'ARG430': 10, 'ILE408': 10, 'GLY422': 10, 'LYS145': 10, 'ILE357': 10, 'LEU361': 9, 'GLY423': 10, 'LEU429': 10, 'PHE414': 10, 'PRO285': 2, 'TRP144': 6, 'GLY421': 10, 'LEU148': 6, 'VAL426': 10, 'ASP410': 3, 'SER409': 7, 'ASP415': 9, 'GLN427': 10, 'ILE284': 10}, 'VAL426': {'LEU428': 10, 'ARG430': 10, 'THR420': 1, 'ILE408': 2, 'GLY422': 10, 'LYS145': 10, 'GLN427': 10, 'GLY423': 10, 'LEU429': 10, 'TRP144': 2, 'ARG431': 10, 'LEU148': 10, 'GLY424': 10}, 'GLN427': {'LEU428': 10, 'GLY283': 6, 'ARG430': 10, 'LEU280': 3, 'PRO285': 10, 'GLY423': 10, 'LEU429': 10, 'HID425': 10, 'ARG431': 10, 'LEU148': 1, 'ILE284': 10, 'ALA432': 10, 'GLY424': 10}, 'LEU428': {'THR420': 10, 'ARG430': 10, 'ILE357': 10, 'GLY423': 8, 'LEU429': 10, 'ALA432': 10, 'PRO285': 3, 'HID425': 10, 'VAL426': 10, 'LEU281': 10, 'ALA433': 10, 'LEU280': 10, 'GLN353': 10, 'GLN358': 10, 'LEU296': 8, 'LEU354': 10, 'ARG431': 10, 'ILE284': 10, 'GLY424': 10}, 'LEU429': {'ARG430': 10, 'ILE408': 10, 'LYS145': 6, 'ILE357': 10, 'LEU361': 10, 'ALA432': 10, 'LEU362': 10, 'TRP144': 10, 'HID425': 10, 'LEU148': 10, 'VAL426': 10, 'ALA433': 10, 'ALA434': 10, 'GLN358': 10, 'PRO149': 8, 'GLN427': 10, 'LEU366': 10, 'ARG431': 10, 'GLY424': 10}, 'ARG430': {'LEU428': 10, 'GLU146': 10, 'ALA433': 10, 'ALA434': 10, 'LYS145': 7, 'GLY147': 10, 'GLN435': 7, 'GLN427': 10, 'LEU362': 1, 'GLY423': 1, 'HID425': 10, 'ARG431': 10, 'LEU148': 10, 'VAL426': 10, 'ALA432': 10}, 'ARG431': {'LEU428': 10, 'GLY283': 10, 'ASP282': 10, 'ALA433': 10, 'GLN435': 10, 'LEU280': 10, 'GLN427': 10, 'LEU429': 10, 'ALA434': 10, 'ILE279': 10, 'VAL426': 10, 'ILE284': 10, 'ALA432': 10, 'LEU436': 8, 'LEU281': 10}, 'ALA432': {'LEU428': 10, 'ARG430': 10, 'ARG513': 4, 'LEU354': 2, 'ALA433': 10, 'GLN435': 10, 'ILE357': 10, 'LEU362': 10, 'LEU280': 10, 'GLN427': 10, 'LEU429': 10, 'ASN363': 10, 'ALA434': 10, 'GLN358': 10, 'LEU436': 10}, 'ALA433': {'LEU428': 10, 'ARG430': 10, 'ARG513': 10, 'GLN358': 1, 'GLN435': 10, 'ALA434': 10, 'GLY147': 1, 'LEU362': 10, 'LEU429': 10, 'THR437': 1, 'ARG431': 10, 'LEU148': 10, 'PRO149': 10, 'LEU507': 1, 'LEU436': 10}, 'ALA434': {'ARG430': 10, 'ARG513': 10, 'GLN435': 10, 'LEU280': 4, 'LEU429': 10, 'THR437': 10, 'ARG431': 10, 'ALA432': 10, 'LEU436': 10}, 'GLN435': {'ARG242': 6, 'ARG430': 7, 'ARG513': 10, 'ARG450': 7, 'LEU354': 1, 'ALA433': 10, 'ASP277': 4, 'TYR359': 2, 'LEU280': 10, 'ASN363': 7, 'THR437': 10, 'ARG431': 10, 'ILE279': 7, 'GLN358': 10, 'SER440': 10, 'ALA432': 10, 'LEU436': 10}, 'LEU436': {'PRO149': 6, 'ARG513': 10, 'ALA433': 10, 'TYR438': 10, 'PHE515': 10, 'VAL367': 10, 'LEU362': 10, 'LEU441': 10, 'ASN363': 10, 'THR437': 10, 'ALA510': 8, 'ARG431': 8, 'ALA434': 10, 'LEU507': 10, 'GLN358': 9, 'SER440': 10, 'ALA432': 10}, 'THR437': {'ARG513': 10, 'ARG450': 1, 'GLN435': 10, 'TYR438': 10, 'PHE515': 10, 'ASP512': 4, 'ALA433': 1, 'CYS439': 10, 'LEU441': 10, 'ASN363': 3, 'ALA510': 1, 'GLY514': 9, 'ASP446': 2, 'ALA434': 10, 'SER440': 10}, 'TYR438': {'ARG513': 10, 'MET531': 10, 'PHE515': 10, 'CYS439': 10, 'LEU441': 10, 'GLY514': 10, 'PRO516': 10, 'PHE519': 6, 'GLN524': 10, 'PHE528': 10, 'CYS442': 10, 'TRP468': 1, 'ASP445': 1, 'ALA510': 4, 'SER440': 10, 'LEU436': 10, 'HIE527': 10, 'CYS532': 1}, 'CYS439': {'ASP449': 6, 'ARG513': 10, 'SER440': 10, 'LEU441': 10, 'GLY514': 10, 'THR437': 10, 'ASP445': 10, 'ASP446': 10, 'CYS442': 10, 'HIE527': 7}, 'SER440': {'ARG450': 9, 'MET239': 1, 'PRO443': 1, 'GLN435': 10, 'TYR438': 10, 'PHE515': 5, 'TYR359': 2, 'LEU441': 10, 'ASN363': 9, 'THR437': 10, 'ASP445': 1, 'ASP446': 10, 'GLN358': 1, 'CYS442': 10, 'LEU436': 10, 'GLN539': 10}, 'LEU441': {'MET531': 10, 'PRO443': 10, 'PHE534': 10, 'THR535': 10, 'PHE515': 10, 'TYR438': 10, 'VAL367': 10, 'CYS439': 10, 'THR364': 8, 'ASN363': 10, 'ILE371': 5, 'THR437': 10, 'TRP468': 10, 'ASP446': 10, 'CYS442': 10, 'LEU436': 10, 'GLN539': 10}, 'CYS442': {'MET531': 10, 'PRO443': 10, 'PHE534': 6, 'TYR438': 10, 'CYS439': 10, 'LEU447': 10, 'ASP445': 10, 'TRP468': 10, 'ASP446': 10, 'PRO444': 10, 'SER440': 10, 'HIE527': 10, 'GLN539': 9, 'THR530': 6}, 'PRO443': {'LEU453': 10, 'GLN539': 10, 'PHE534': 10, 'ALA448': 10, 'SER440': 1, 'LEU441': 10, 'LEU447': 10, 'ASP445': 10, 'TRP468': 10, 'ASP446': 10, 'PRO444': 10, 'TYR460': 10, 'LEU465': 10, 'ALA464': 10, 'ALA461': 10}, 'PRO444': {'LEU453': 10, 'ASP449': 3, 'ALA448': 10, 'LEU447': 10, 'ASP445': 10, 'LEU465': 10, 'TRP468': 10, 'ASP446': 10, 'HIE527': 8, 'CYS442': 10, 'ALA464': 10, 'GLU469': 8}, 'ASP445': {'LEU453': 1, 'ASP449': 10, 'PRO443': 10, 'ARG450': 10, 'ALA448': 10, 'TYR438': 1, 'SER440': 1, 'CYS439': 10, 'LEU447': 10, 'ASP446': 10, 'CYS442': 10, 'HIE527': 10}, 'ASP446': {'ARG242': 3, 'ASP449': 10, 'MET239': 10, 'PRO443': 10, 'ARG450': 10, 'ALA448': 10, 'CYS439': 10, 'LEU441': 10, 'LEU447': 10, 'THR437': 2, 'GLY451': 8, 'CYS442': 10, 'PRO444': 10, 'LEU452': 3, 'SER440': 10}, 'LEU447': {'ARG450': 10, 'GLY451': 10, 'PRO444': 10, 'ALA461': 10, 'MET239': 10, 'PRO443': 10, 'GLY454': 4, 'LEU465': 10, 'LEU452': 10, 'TYR460': 10, 'PRO238': 10, 'LEU453': 10, 'ASP449': 10, 'ASP445': 10, 'CYS442': 10, 'ALA458': 5, 'ALA464': 10, 'ALA448': 10, 'HID462': 3, 'GLN539': 3}, 'ALA448': {'LEU453': 10, 'ASP449': 10, 'PRO443': 10, 'ARG450': 10, 'GLY454': 7, 'ASP445': 10, 'ASP446': 10, 'PRO444': 10, 'GLY451': 10, 'LEU452': 10, 'LEU465': 3}, 'ASP449': {'ARG242': 10, 'ARG450': 10, 'ARG243': 7, 'CYS439': 6, 'LEU447': 10, 'ASP445': 10, 'ASP446': 10, 'GLY451': 10, 'PRO444': 3, 'LEU452': 6}, 'ARG450': {'ARG242': 10, 'LEU453': 1, 'MET239': 10, 'LEU241': 10, 'ASP277': 9, 'GLN435': 7, 'ALA448': 10, 'ARG243': 10, 'LEU447': 10, 'LEU240': 10, 'ASP445': 10, 'THR437': 1, 'ASP446': 10, 'GLY451': 10, 'LEU452': 10, 'SER440': 9, 'TYR359': 10, 'PHE274': 10}, 'GLY451': {'ARG242': 7, 'LEU453': 10, 'ASP449': 10, 'ALA448': 10, 'GLY454': 10, 'ARG243': 10, 'LEU447': 10, 'ASP446': 8, 'LEU452': 10, 'LEU455': 10, 'PHE274': 10}, 'LEU452': {'ARG242': 10, 'PHE232': 9, 'ARG450': 10, 'LEU241': 10, 'ALA461': 10, 'MET239': 10, 'GLY454': 10, 'TYR460': 10, 'PRO238': 10, 'PHE274': 10, 'LEU453': 10, 'ASP449': 6, 'ARG243': 9, 'LEU447': 10, 'LEU240': 7, 'LEU455': 10, 'LEU233': 10, 'ALA458': 10, 'ALA448': 10, 'ASP446': 3, 'HID462': 9, 'LEU311': 3}, 'LEU453': {'PRO456': 8, 'PRO443': 10, 'ARG450': 1, 'ALA448': 10, 'GLY454': 10, 'LEU447': 10, 'LEU465': 10, 'ASP445': 1, 'GLY451': 10, 'PRO444': 10, 'LEU455': 10, 'HID462': 10, 'ALA458': 10, 'ALA461': 10}, 'GLY454': {'PRO456': 10, 'ALA448': 7, 'ARG243': 4, 'LEU447': 4, 'GLY451': 10, 'LEU452': 10, 'HID462': 10, 'LEU455': 10, 'PHE274': 1}, 'LEU455': {'GLY309': 10, 'LEU453': 10, 'PRO456': 10, 'MET305': 10, 'LEU241': 10, 'ARG243': 7, 'LEU311': 10, 'GLY457': 10, 'SER272': 1, 'GLU306': 7, 'GLY451': 10, 'LEU452': 10, 'LEU233': 10, 'HID462': 4, 'LYS310': 10, 'ALA458': 10, 'PHE274': 10}, 'PRO456': {'GLY309': 10, 'LEU453': 8, 'MET305': 9, 'GLY454': 10, 'LEU459': 10, 'GLY457': 10, 'ASN308': 10, 'HID462': 8, 'LYS310': 8, 'ALA458': 10, 'TRP647': 8}, 'GLY457': {'GLY309': 10, 'LEU233': 10, 'MET305': 1, 'TYR460': 7, 'TYR230': 5, 'LEU459': 10, 'ASN234': 10, 'LEU311': 9, 'LEU455': 10, 'ASN308': 9, 'LYS310': 10, 'ALA458': 10, 'TRP647': 10}, 'ALA458': {'GLY309': 6, 'LEU453': 10, 'PRO456': 10, 'ASP463': 2, 'TYR460': 10, 'LEU241': 4, 'LEU459': 10, 'ASN234': 10, 'HID462': 10, 'LEU447': 5, 'LEU452': 10, 'LEU233': 10, 'PRO238': 10, 'LEU455': 10, 'ALA461': 10}, 'LEU459': {'PRO456': 10, 'ARG641': 4, 'LEU233': 10, 'ASP463': 10, 'PRO648': 10, 'ASN642': 10, 'TYR649': 10, 'GLY457': 10, 'ASN234': 10, 'LEU645': 10, 'TYR460': 10, 'HID462': 10, 'ALA464': 10, 'TRP647': 10, 'ALA461': 10}, 'TYR460': {'ASP463': 10, 'TYR651': 1, 'TYR649': 10, 'ALA461': 10, 'PHE534': 10, 'MET239': 10, 'PRO443': 10, 'LEU652': 10, 'ALA538': 3, 'ASN234': 10, 'LEU465': 10, 'LEU452': 10, 'PRO238': 10, 'GLY235': 2, 'LEU447': 10, 'ALA542': 10, 'LEU233': 10, 'ALA458': 10, 'ALA464': 10, 'GLY457': 7, 'HID462': 10, 'GLN539': 10}, 'ALA461': {'LEU453': 10, 'ASP463': 10, 'PRO443': 10, 'LEU459': 10, 'ASN234': 4, 'LEU447': 10, 'ARG466': 10, 'LEU465': 10, 'PRO238': 3, 'LEU452': 10, 'LEU233': 10, 'HID462': 10, 'ALA458': 10, 'ALA464': 10}, 'HID462': {'LEU453': 10, 'PRO456': 8, 'ARG641': 6, 'ASP463': 10, 'LEU634': 5, 'GLY454': 10, 'LEU459': 10, 'LEU447': 3, 'ARG466': 10, 'LEU467': 10, 'LEU452': 9, 'TYR460': 10, 'LEU465': 10, 'LEU455': 4, 'ALA458': 10, 'ALA464': 10}, 'ASP463': {'LYS653': 2, 'ARG641': 9, 'PHE534': 5, 'LEU634': 10, 'LEU652': 10, 'LEU459': 10, 'ILE638': 10, 'ARG466': 10, 'TRP468': 10, 'GLU650': 4, 'LEU467': 10, 'TYR460': 10, 'LEU465': 10, 'ALA458': 2, 'TYR649': 10, 'ALA461': 10, 'ALA464': 10}, 'ALA464': {'GLN539': 6, 'PHE534': 10, 'PRO443': 10, 'LEU652': 10, 'LEU459': 10, 'ALA538': 8, 'LEU447': 10, 'ARG466': 10, 'LEU465': 10, 'TRP468': 10, 'PRO444': 10, 'LEU467': 10, 'TYR460': 10, 'HID462': 10, 'TYR649': 9, 'ALA461': 10, 'GLU469': 10}, 'LEU465': {'LEU453': 10, 'ASP463': 10, 'PRO443': 10, 'PHE534': 6, 'ALA448': 3, 'LEU447': 10, 'ARG466': 10, 'TRP468': 10, 'PRO444': 10, 'LEU467': 10, 'ILE470': 9, 'TYR460': 10, 'HID462': 10, 'ALA461': 10, 'GLU469': 10}, 'ARG466': {'ARG641': 10, 'ILE471': 5, 'ASP463': 10, 'LEU634': 10, 'TRP468': 10, 'GLU637': 10, 'LEU467': 10, 'ILE470': 10, 'ILE638': 9, 'HID462': 10, 'ALA464': 10, 'ALA461': 10, 'GLU469': 10}, 'LEU467': {'ASP463': 10, 'PHE627': 9, 'HID462': 10, 'ILE657': 10, 'PHE534': 10, 'LEU652': 10, 'ALA538': 10, 'TRP468': 10, 'LEU465': 10, 'LEU607': 10, 'PRO654': 10, 'ILE470': 10, 'ILE638': 9, 'LEU634': 10, 'ALA464': 10, 'GLU469': 10, 'ALA472': 10, 'LEU631': 10, 'ILE471': 10}, 'TRP468': {'ASP463': 10, 'MET531': 10, 'TYR438': 1, 'LEU441': 10, 'PRO444': 10, 'HIE527': 10, 'ILE471': 10, 'THR530': 10, 'PHE534': 10, 'PRO443': 10, 'THR535': 10, 'ARG466': 10, 'PHE528': 2, 'CYS442': 10, 'ILE470': 10, 'ALA464': 10, 'GLU469': 10, 'ALA472': 10, 'LEU465': 10, 'ARG473': 10, 'GLN539': 1}, 'GLU469': {'ARG473': 10, 'ARG466': 10, 'TYR474': 10, 'ALA472': 10, 'PRO444': 8, 'LEU467': 10, 'ILE470': 10, 'LEU465': 10, 'ALA464': 10, 'ILE471': 10, 'THR530': 5}, 'ILE470': {'VAL475': 9, 'LYS633': 4, 'LEU634': 10, 'ASP630': 10, 'ARG466': 10, 'TRP468': 10, 'PHE627': 10, 'TYR474': 10, 'ALA472': 10, 'GLN626': 9, 'LEU467': 10, 'LEU631': 10, 'PRO654': 2, 'LEU465': 9, 'ARG473': 10, 'ILE471': 10}, 'ILE471': {'PHE627': 10, 'THR530': 10, 'ILE657': 5, 'PHE534': 10, 'ALA538': 10, 'ARG466': 5, 'TRP468': 10, 'LEU607': 10, 'VAL475': 10, 'GLU476': 10, 'TYR474': 10, 'LEU631': 10, 'GLU469': 10, 'VAL533': 10, 'LEU529': 3, 'ALA472': 10, 'LEU467': 10, 'THR537': 10, 'ARG473': 10}, 'ALA472': {'THR530': 10, 'VAL475': 10, 'GLY477': 10, 'CYS526': 10, 'TRP468': 10, 'GLU476': 10, 'TYR474': 10, 'HIE527': 4, 'LEU467': 10, 'ILE470': 10, 'ARG473': 10, 'GLU469': 10}, 'ARG473': {'VAL475': 10, 'GLY477': 10, 'ASP630': 10, 'VAL623': 10, 'ILE478': 10, 'TRP468': 10, 'GLU476': 10, 'TYR474': 10, 'GLN626': 10, 'PHE627': 2, 'ILE470': 10, 'ILE471': 10, 'GLU469': 10}, 'TYR474': {'GLY477': 10, 'PHE627': 10, 'TYR614': 10, 'VAL623': 10, 'PRO620': 9, 'VAL479': 10, 'GLN626': 5, 'LEU607': 4, 'VAL475': 10, 'PHE388': 8, 'LEU624': 10, 'HID392': 8, 'GLU476': 10, 'HID610': 10, 'ILE470': 10, 'GLU469': 10, 'ILE478': 10, 'VAL533': 10, 'ALA472': 10, 'ILE471': 10}, 'VAL475': {'HID480': 10, 'TYR614': 10, 'GLY477': 10, 'CYS526': 10, 'VAL623': 7, 'PHE388': 10, 'ILE478': 10, 'VAL479': 10, 'VAL533': 10, 'GLU476': 10, 'LEU529': 10, 'ALA472': 10, 'ILE470': 9, 'ILE471': 10, 'ARG473': 10, 'THR530': 10}, 'GLU476': {'HID480': 10, 'GLN522': 10, 'SER523': 10, 'GLY477': 10, 'CYS526': 10, 'VAL623': 1, 'PHE388': 5, 'ILE478': 10, 'VAL479': 10, 'LEU481': 9, 'LEU529': 7, 'TYR474': 10, 'ALA472': 10, 'ILE471': 10, 'ARG473': 10, 'THR530': 3}, 'GLY477': {'VAL475': 10, 'HID480': 10, 'VAL623': 10, 'LEU481': 10, 'ILE478': 10, 'VAL479': 10, 'TYR474': 10, 'ALA472': 10, 'ARG473': 10}, 'ILE478': {'VAL475': 10, 'HID480': 10, 'TYR483': 5, 'LYS619': 7, 'TYR614': 10, 'PHE615': 10, 'PHE482': 10, 'VAL623': 10, 'PHE388': 10, 'VAL479': 10, 'LEU481': 10, 'PRO620': 10, 'GLU476': 10, 'TYR474': 10, 'LEU624': 10, 'LYS621': 1, 'ARG473': 10}, 'VAL479': {'VAL475': 10, 'ILE385': 10, 'HID480': 10, 'TYR483': 10, 'GLN522': 10, 'PHE482': 10, 'GLY477': 10, 'CYS526': 10, 'PHE388': 10, 'LEU481': 10, 'VAL489': 5, 'GLU476': 10, 'TYR474': 10, 'LEU529': 10, 'GLN484': 9, 'LEU525': 10}, 'HID480': {'VAL475': 10, 'TYR483': 10, 'GLN522': 10, 'PHE482': 10, 'GLY477': 10, 'ARG485': 5, 'CYS526': 6, 'LEU481': 10, 'ILE478': 10, 'GLU476': 10, 'GLN484': 10}, 'LEU481': {'TYR483': 10, 'LYS619': 10, 'PHE615': 8, 'PHE482': 10, 'GLY477': 10, 'VAL623': 10, 'PRO620': 10, 'ILE478': 10, 'VAL479': 10, 'PHE388': 5, 'GLU476': 9, 'GLN484': 7}, 'PHE482': {'ILE385': 7, 'HID480': 10, 'TYR483': 10, 'ILE488': 10, 'PHE615': 10, 'PHE388': 10, 'ILE478': 10, 'VAL479': 10, 'PRO620': 10, 'ARG114': 4, 'PRO384': 10, 'THR112': 7, 'GLN484': 10, 'ALA113': 10, 'LEU115': 10}, 'TYR483': {'HID480': 10, 'GLN522': 10, 'LEU495': 10, 'ILE385': 10, 'VAL479': 10, 'PRO384': 10, 'PRO116': 5, 'ASP486': 4, 'LEU115': 10, 'ILE488': 10, 'ARG485': 10, 'HIP383': 10, 'PHE388': 4, 'LEU525': 10, 'ASP492': 10, 'LEU481': 10, 'ILE478': 5, 'VAL489': 10, 'GLN484': 10}, 'GLN484': {'HID480': 10, 'ILE488': 10, 'GLN522': 10, 'PHE482': 10, 'ARG485': 10, 'LEU481': 7, 'VAL479': 9, 'ASP486': 10}, 'ARG485': {'HID480': 5, 'ILE488': 10, 'TYR483': 10, 'GLN522': 10, 'LYS490': 5, 'VAL489': 10, 'ASP487': 10, 'ASP486': 10}, 'ASP486': {'TYR483': 4, 'ILE488': 10, 'LYS490': 10, 'GLN522': 10, 'GLY491': 9, 'SER523': 10, 'VAL489': 10, 'GLN520': 10, 'ASP487': 10, 'GLN484': 10, 'SER521': 10}, 'ASP487': {'ILE488': 10, 'LYS490': 10, 'GLN522': 1, 'ARG485': 10, 'GLY491': 10, 'VAL489': 10}, 'ILE488': {'TYR483': 10, 'LYS490': 10, 'ASP492': 10, 'GLN522': 4, 'PHE482': 10, 'ARG485': 10, 'GLY491': 10, 'VAL489': 10, 'GLN484': 10, 'ASP486': 10, 'LEU115': 10}, 'VAL489': {'TYR483': 10, 'LYS490': 10, 'ASP492': 10, 'GLN522': 10, 'ARG485': 10, 'GLY491': 10, 'LEU495': 10, 'VAL479': 5, 'PHE519': 10, 'GLN520': 7, 'ASP487': 10, 'ASP486': 10, 'SER521': 10, 'LEU525': 10}, 'LYS490': {'ILE488': 10, 'PRO493': 3, 'ASP492': 10, 'GLY491': 10, 'ARG485': 5, 'LEU495': 1, 'PHE519': 10, 'SER518': 1, 'GLN520': 10, 'ASP487': 10, 'ASP486': 10, 'SER521': 10}, 'GLY491': {'ILE488': 10, 'PRO493': 10, 'ASP492': 10, 'VAL489': 10, 'ASP487': 10, 'PRO116': 10, 'ASP486': 9, 'LEU115': 1}, 'ASP492': {'GLH494': 10, 'TYR483': 10, 'ILE488': 10, 'ALA497': 7, 'LYS490': 10, 'PRO493': 10, 'GLN496': 10, 'HIP383': 10, 'LEU495': 10, 'VAL489': 10, 'PRO384': 9, 'PRO116': 10, 'LEU115': 10}, 'PRO493': {'GLH494': 10, 'ALA497': 10, 'GLN496': 10, 'LYS490': 3, 'GLY491': 10, 'HIP383': 6, 'LEU495': 10, 'TRP498': 10, 'PRO116': 10}, 'GLH494': {'CYS499': 9, 'ALA497': 10, 'PRO380': 10, 'ASP492': 10, 'GLN496': 10, 'HIP383': 10, 'LEU495': 10, 'ARG128': 10, 'LEU379': 10, 'TRP498': 10, 'LEU132': 10, 'PRO116': 10, 'CYS378': 10}, 'LEU495': {'CYS499': 10, 'ILE385': 10, 'TYR483': 10, 'ALA497': 10, 'PRO493': 10, 'ASP492': 10, 'GLN496': 10, 'LYS490': 1, 'HIP383': 10, 'VAL489': 10, 'PHE519': 10, 'LEU379': 10, 'PRO384': 8, 'TRP498': 10, 'PRO116': 6, 'ARG500': 10, 'LEU115': 2, 'LEU525': 10}, 'GLN496': {'GLH494': 10, 'CYS499': 10, 'ALA497': 10, 'GLU501': 10, 'PRO493': 10, 'ASP492': 10, 'VAL517': 10, 'SER518': 10, 'PHE519': 10, 'TRP498': 10, 'ARG500': 10}, 'ALA497': {'GLH494': 10, 'CYS499': 10, 'GLU501': 10, 'PRO493': 10, 'ASP492': 7, 'LEU495': 10, 'ILE502': 8, 'LEU379': 3, 'TRP498': 10, 'LEU132': 2, 'CYS378': 10, 'ARG500': 10}, 'TRP498': {'CYS499': 10, 'GLU501': 10, 'PRO493': 10, 'GLN496': 10, 'THR503': 10, 'LEU495': 10, 'ALA374': 10, 'ARG500': 10, 'GLH494': 10, 'ILE385': 10, 'ILE502': 10, 'PHE528': 10, 'LEU379': 10, 'CYS532': 10, 'CYS378': 10, 'LEU389': 10, 'MET376': 2, 'THR375': 10, 'PHE519': 10, 'LEU525': 10, 'ILE371': 10, 'LEU529': 10, 'ARG377': 2}, 'CYS499': {'GLH494': 9, 'GLU501': 10, 'ALA497': 10, 'VAL517': 10, 'GLN496': 10, 'PHE515': 1, 'THR503': 10, 'LEU495': 10, 'ILE502': 10, 'ALA374': 3, 'PRO516': 10, 'SER518': 10, 'PHE519': 10, 'PHE528': 9, 'GLU504': 10, 'ARG500': 10}, 'ARG500': {'GLU501': 10, 'VAL517': 10, 'ALA497': 10, 'GLN496': 10, 'THR503': 10, 'LEU495': 10, 'ILE502': 10, 'TRP141': 7, 'SER518': 2, 'GLU504': 10, 'VAL505': 10, 'ARG377': 6, 'TRP498': 10, 'GLY506': 10, 'CYS378': 3}, 'GLU501': {'GLN136': 10, 'CYS499': 10, 'ALA497': 10, 'GLN496': 10, 'ARG377': 10, 'THR503': 10, 'ILE502': 10, 'ALA374': 10, 'TRP141': 10, 'GLU504': 10, 'VAL505': 10, 'VAL370': 10, 'TRP498': 10, 'GLY506': 10, 'LEU507': 10, 'CYS378': 10, 'VAL373': 10}, 'ILE502': {'CYS499': 10, 'ALA497': 8, 'CYS508': 10, 'MET531': 10, 'THR503': 10, 'ALA374': 10, 'VAL505': 10, 'ARG500': 10, 'PRO516': 6, 'PHE528': 10, 'CYS532': 10, 'TRP498': 10, 'LEU507': 10, 'GLU504': 10, 'CYS378': 1, 'PHE519': 2, 'VAL370': 10, 'GLY506': 10, 'GLN509': 6, 'VAL517': 2, 'PHE515': 10, 'VAL367': 10, 'ILE371': 10, 'ALA510': 5}, 'THR503': {'CYS499': 10, 'GLU501': 10, 'CYS508': 10, 'VAL517': 10, 'PHE515': 10, 'GLN511': 10, 'ALA510': 10, 'PRO516': 10, 'PHE528': 10, 'SER518': 1, 'VAL505': 10, 'VAL370': 1, 'TRP498': 10, 'GLY506': 10, 'LEU507': 10, 'GLU504': 10, 'ARG500': 10, 'GLN509': 10}, 'GLU504': {'CYS499': 10, 'GLU501': 10, 'CYS508': 10, 'VAL517': 10, 'ILE502': 10, 'GLN511': 10, 'ALA510': 10, 'VAL505': 10, 'GLY506': 10, 'LEU507': 10, 'ARG500': 10, 'GLN509': 10}, 'VAL505': {'GLU501': 10, 'VAL517': 1, 'CYS508': 10, 'GLY506': 10, 'THR503': 10, 'ILE502': 10, 'TRP141': 10, 'ARG500': 10, 'ARG377': 6, 'PRO149': 4, 'LEU507': 10, 'LEU150': 10, 'VAL373': 1, 'GLN509': 10}, 'GLY506': {'THR151': 9, 'GLU501': 10, 'CYS508': 10, 'LEU148': 1, 'VAL373': 4, 'THR503': 10, 'ILE502': 10, 'ALA374': 5, 'TRP141': 6, 'VAL370': 10, 'PRO149': 10, 'LEU507': 10, 'GLU504': 10, 'LEU150': 10, 'ARG500': 10, 'GLN509': 3}, 'LEU507': {'GLU501': 10, 'CYS508': 10, 'ARG513': 10, 'MET531': 1, 'THR503': 10, 'VAL505': 10, 'LEU362': 6, 'ILE502': 10, 'PHE528': 4, 'GLU504': 10, 'LEU436': 10, 'ALA433': 1, 'VAL370': 10, 'PRO149': 10, 'LEU150': 10, 'GLN509': 10, 'PHE515': 10, 'VAL367': 10, 'LEU366': 1, 'ALA510': 10, 'ILE371': 2}, 'CYS508': {'GLU146': 4, 'GLY506': 10, 'ARG513': 10, 'THR503': 10, 'GLY147': 10, 'ILE502': 10, 'ALA510': 10, 'VAL505': 10, 'LEU148': 10, 'PRO149': 10, 'GLU504': 10, 'LEU150': 10, 'GLN509': 10}, 'GLN509': {'ARG513': 10, 'THR503': 10, 'ASP512': 10, 'ILE502': 6, 'GLN511': 10, 'ALA510': 10, 'VAL505': 10, 'GLY506': 3, 'LEU507': 10, 'GLU504': 10}, 'ALA510': {'CYS508': 10, 'ARG513': 10, 'PHE515': 10, 'THR503': 10, 'ASP512': 10, 'TYR438': 4, 'ILE502': 5, 'GLN511': 10, 'GLY514': 10, 'THR437': 1, 'GLU504': 10, 'LEU507': 10, 'LEU436': 8}, 'GLN511': {'VAL517': 10, 'ARG513': 10, 'PHE515': 10, 'THR503': 10, 'ASP512': 10, 'GLY514': 10, 'PRO516': 10, 'GLU504': 10, 'GLN509': 10}, 'ASP512': {'ARG513': 10, 'PHE515': 6, 'GLY514': 10, 'ALA510': 10, 'THR437': 4, 'GLN509': 10}, 'ARG513': {'CYS508': 10, 'ALA432': 4, 'ALA433': 10, 'GLN435': 10, 'TYR438': 10, 'PHE515': 10, 'CYS439': 10, 'GLN511': 10, 'GLY514': 10, 'THR437': 10, 'ALA510': 10, 'ALA434': 10, 'PRO149': 9, 'LEU507': 10, 'LEU436': 10, 'GLN509': 10}, 'GLY514': {'PHE515': 10, 'TYR438': 10, 'ASP512': 10, 'GLN511': 10, 'CYS439': 10, 'THR437': 9, 'ALA510': 10, 'PRO516': 10}, 'PHE515': {'CYS499': 1, 'ARG513': 10, 'MET531': 10, 'TYR438': 10, 'THR503': 10, 'ASP512': 6, 'GLN511': 10, 'LEU441': 10, 'ILE502': 10, 'THR437': 10, 'PRO516': 10, 'PHE528': 10, 'LEU507': 10, 'LEU436': 10, 'VAL517': 10, 'VAL367': 10, 'ALA510': 10, 'ILE371': 3, 'SER440': 5}, 'PRO516': {'CYS499': 10, 'VAL517': 10, 'MET531': 3, 'TYR438': 10, 'THR503': 10, 'ILE502': 6, 'GLN511': 10, 'GLY514': 10, 'GLN524': 10, 'PHE528': 10, 'SER518': 10, 'PHE519': 10, 'GLN520': 4}, 'VAL517': {'CYS499': 10, 'GLN496': 10, 'PHE515': 10, 'THR503': 10, 'ILE502': 2, 'GLN511': 10, 'GLN524': 1, 'SER518': 10, 'PHE519': 10, 'VAL505': 1, 'GLU504': 10, 'ARG500': 10}, 'SER518': {'CYS499': 10, 'GLN496': 10, 'LYS490': 1, 'THR503': 1, 'GLN524': 10, 'PRO516': 10, 'PHE519': 10, 'GLN520': 10, 'ARG500': 2}, 'PHE519': {'CYS499': 10, 'VAL517': 10, 'LYS490': 10, 'GLN496': 10, 'GLN522': 7, 'TYR438': 6, 'LEU495': 10, 'ILE502': 2, 'GLN524': 10, 'VAL489': 10, 'PRO516': 10, 'PHE528': 10, 'LEU529': 3, 'GLN520': 10, 'TRP498': 10, 'SER521': 10, 'LEU525': 10}, 'GLN520': {'LYS490': 10, 'GLN522': 10, 'GLN524': 10, 'VAL489': 7, 'PRO516': 4, 'SER518': 10, 'ASP486': 10, 'SER521': 10}, 'SER521': {'GLN522': 10, 'LYS490': 10, 'SER523': 10, 'CYS526': 7, 'GLN524': 10, 'VAL489': 10, 'PHE519': 10, 'ASP486': 10, 'LEU525': 10}, 'GLN522': {'HID480': 10, 'TYR483': 10, 'ILE488': 4, 'ARG485': 10, 'SER523': 10, 'CYS526': 10, 'GLN524': 10, 'VAL479': 10, 'VAL489': 10, 'GLU476': 10, 'PHE519': 7, 'GLN520': 10, 'HIE527': 10, 'ASP487': 1, 'GLN484': 10, 'ASP486': 10, 'LEU525': 10}, 'SER523': {'CYS526': 10, 'GLN524': 10, 'PHE528': 10, 'GLU476': 10, 'HIE527': 10, 'ASP486': 10, 'SER521': 10, 'LEU525': 10}, 'GLN524': {'VAL517': 1, 'GLN522': 10, 'TYR438': 10, 'CYS526': 10, 'PRO516': 10, 'SER518': 10, 'PHE519': 10, 'GLN520': 10, 'PHE528': 10, 'LEU529': 10, 'HIE527': 10, 'SER521': 10, 'LEU525': 10}, 'LEU525': {'ILE385': 10, 'TYR483': 10, 'GLN522': 10, 'SER523': 10, 'CYS526': 10, 'LEU495': 10, 'VAL479': 10, 'VAL489': 10, 'PHE528': 10, 'PHE519': 10, 'LEU529': 10, 'TRP498': 10, 'HIE527': 10, 'SER521': 10, 'THR530': 10}, 'CYS526': {'VAL475': 10, 'HID480': 6, 'GLN522': 10, 'MET531': 10, 'SER523': 10, 'GLN524': 10, 'VAL479': 10, 'PHE528': 10, 'GLU476': 10, 'LEU529': 10, 'ALA472': 10, 'HIE527': 10, 'SER521': 7, 'THR530': 10}, 'HIE527': {'THR530': 10, 'GLN522': 10, 'MET531': 10, 'SER523': 10, 'TYR438': 10, 'CYS439': 7, 'GLN524': 10, 'ASP445': 10, 'TRP468': 10, 'PHE528': 10, 'LEU529': 10, 'PRO444': 8, 'CYS532': 10, 'ALA472': 4, 'CYS442': 10, 'LEU525': 10}, 'PHE528': {'CYS499': 9, 'MET531': 10, 'TYR438': 10, 'THR503': 10, 'CYS526': 10, 'THR530': 10, 'ILE502': 10, 'PRO516': 10, 'TRP468': 2, 'CYS532': 10, 'TRP498': 10, 'LEU507': 4, 'PHE519': 10, 'LEU525': 10, 'PHE515': 10, 'SER523': 10, 'GLN524': 10, 'VAL533': 10, 'ILE371': 10, 'LEU529': 10}, 'LEU529': {'MET531': 10, 'CYS526': 10, 'HIE527': 10, 'THR530': 10, 'ILE385': 10, 'PHE534': 10, 'TYR614': 4, 'VAL479': 10, 'CYS532': 10, 'TRP498': 10, 'LEU389': 10, 'VAL475': 10, 'PHE388': 10, 'HID392': 1, 'GLU476': 7, 'PHE519': 3, 'LEU525': 10, 'GLN524': 10, 'VAL533': 10, 'ILE471': 3}, 'THR530': {'VAL475': 10, 'MET531': 10, 'PHE534': 10, 'THR535': 10, 'CYS526': 10, 'VAL533': 10, 'TRP468': 10, 'GLU476': 3, 'PHE528': 10, 'ALA472': 10, 'CYS532': 10, 'CYS442': 6, 'GLU469': 5, 'ILE471': 10, 'HIE527': 10, 'LEU525': 10}, 'MET531': {'TYR438': 10, 'CYS526': 10, 'LEU441': 10, 'HIE527': 10, 'CYS536': 10, 'PHE534': 10, 'THR535': 10, 'ILE502': 10, 'TRP468': 10, 'PHE528': 10, 'PRO516': 3, 'CYS532': 10, 'LEU507': 1, 'PHE515': 10, 'VAL367': 6, 'VAL533': 10, 'ILE371': 10, 'LEU529': 10, 'CYS442': 10}, 'CYS532': {'CYS536': 10, 'PHE534': 10, 'THR535': 10, 'TYR438': 1, 'VAL367': 2, 'ILE502': 10, 'VAL533': 10, 'ILE371': 10, 'HID392': 5, 'PHE528': 10, 'LEU529': 10, 'TRP498': 10, 'THR537': 8, 'HIE527': 10, 'LEU389': 10, 'THR530': 10}, 'VAL533': {'MET531': 10, 'PHE627': 8, 'THR530': 10, 'CYS536': 10, 'PHE534': 10, 'TYR614': 10, 'THR535': 10, 'ALA538': 10, 'PHE528': 10, 'LEU389': 10, 'LEU607': 10, 'VAL475': 10, 'PHE388': 10, 'HID392': 10, 'TYR474': 10, 'ILE371': 5, 'LEU529': 10, 'ILE471': 10, 'THR537': 10, 'GLN539': 1}, 'PHE534': {'MET531': 10, 'ASP463': 5, 'LEU441': 10, 'ILE471': 10, 'THR530': 10, 'CYS536': 10, 'PRO443': 10, 'THR535': 10, 'LEU652': 10, 'ALA538': 10, 'TRP468': 10, 'CYS532': 10, 'TYR460': 10, 'LEU607': 4, 'CYS442': 6, 'ALA542': 2, 'ALA464': 10, 'LEU529': 10, 'LEU467': 10, 'LEU465': 6, 'THR537': 10, 'GLN539': 10}, 'THR535': {'ALA368': 10, 'CYS536': 10, 'ALA541': 2, 'MET531': 10, 'VAL661': 10, 'VAL367': 10, 'ALA538': 8, 'THR364': 10, 'ASN363': 10, 'ILE371': 10, 'LEU441': 10, 'TRP468': 10, 'VAL533': 10, 'CYS532': 10, 'THR537': 10, 'GLN539': 10, 'THR530': 10}, 'CYS536': {'MET531': 10, 'ASN659': 8, 'THR396': 3, 'ILE393': 10, 'ALA368': 10, 'PHE534': 10, 'ALA538': 10, 'THR364': 10, 'CYS532': 10, 'LEU389': 7, 'ALA541': 1, 'SER660': 10, 'GLH658': 6, 'HID392': 10, 'VAL661': 10, 'VAL367': 2, 'VAL533': 10, 'ILE371': 10, 'THR537': 10, 'GLN539': 9}, 'THR537': {'ILE657': 10, 'ALA541': 10, 'PHE534': 10, 'VAL661': 10, 'THR535': 10, 'LEU607': 10, 'SER660': 10, 'ALA538': 10, 'ASN659': 10, 'VAL533': 10, 'GLH658': 10, 'HID392': 10, 'CYS532': 8, 'ALA542': 10, 'ILE471': 10, 'GLN539': 10}, 'ALA538': {'CYS536': 10, 'ALA541': 10, 'ILE657': 10, 'PHE534': 10, 'THR535': 8, 'LEU607': 10, 'LEU652': 10, 'VAL533': 10, 'GLH658': 3, 'ILE543': 10, 'ALA542': 10, 'LEU467': 10, 'TYR460': 3, 'ILE471': 10, 'ALA464': 8, 'GLN539': 10}, 'GLN539': {'LEU441': 10, 'ASN363': 10, 'CYS536': 9, 'PHE534': 10, 'MET239': 10, 'PRO443': 10, 'THR535': 10, 'LEU652': 10, 'SER440': 10, 'THR364': 10, 'TRP468': 1, 'TYR460': 10, 'TYR359': 3, 'ALA541': 10, 'LEU447': 3, 'ALA542': 10, 'ILE543': 10, 'ALA464': 6, 'VAL533': 1, 'CYS442': 9, 'THR537': 10}, 'ALA541': {'ILE657': 10, 'CYS536': 1, 'CYS656': 9, 'LEU652': 10, 'THR535': 2, 'ALA538': 10, 'GLY546': 10, 'GLN545': 10, 'GLH658': 10, 'TYR651': 10, 'ALA542': 10, 'ILE543': 10, 'THR537': 10, 'GLN539': 10}, 'ALA542': {'PHE534': 2, 'MET239': 10, 'LEU652': 10, 'ASN237': 1, 'HIE355': 7, 'ALA236': 10, 'ALA538': 10, 'GLY546': 9, 'GLN545': 10, 'GLY235': 10, 'THR364': 1, 'TYR651': 10, 'ILE543': 10, 'TYR460': 10, 'PRO238': 10, 'THR537': 10, 'TYR649': 10, 'GLN539': 10}, 'ILE543': {'GLN547': 10, 'GLH356': 1, 'ALA236': 10, 'ASN363': 2, 'TYR651': 3, 'TYR649': 1, 'MET239': 10, 'ALA538': 10, 'GLY546': 10, 'TYR551': 1, 'THR364': 10, 'PRO238': 10, 'TYR359': 10, 'ALA541': 10, 'HIE355': 10, 'GLY235': 1, 'ASN237': 10, 'GLN545': 10, 'GLN539': 10}, 'GLN545': {'LEU548': 10, 'ALA541': 10, 'ARG600': 8, 'CYS656': 10, 'GLN547': 10, 'GLY546': 10, 'LEU597': 4, 'SER598': 10, 'GLH658': 2, 'TYR651': 10, 'ALA542': 10, 'ILE543': 10, 'ASP549': 10}, 'GLY546': {'LEU548': 10, 'ALA541': 10, 'TRP550': 10, 'LEU214': 3, 'GLN547': 10, 'HIE355': 10, 'ALA236': 10, 'TYR551': 10, 'GLY235': 7, 'SER598': 7, 'TYR651': 10, 'ALA542': 9, 'ILE543': 10, 'ASP549': 10}, 'GLN547': {'LEU548': 10, 'ILE593': 4, 'TRP550': 10, 'GLH356': 10, 'HIE355': 10, 'ALA236': 10, 'SER594': 10, 'TYR551': 10, 'LEU597': 10, 'SER598': 10, 'PHE352': 3, 'ILE543': 10, 'ALA552': 10, 'GLN545': 10, 'ASP549': 10}, 'LEU548': {'TRP553': 10, 'ALA215': 10, 'TRP550': 10, 'TRP595': 10, 'SER212': 10, 'PHE207': 1, 'ARG599': 9, 'GLN210': 10, 'GLY546': 10, 'TYR551': 10, 'MET591': 8, 'LYS211': 10, 'SER594': 10, 'SER598': 10, 'ALA552': 10, 'GLN545': 10, 'ASP549': 10}, 'ASP549': {'TRP553': 9, 'ALA215': 10, 'TRP550': 10, 'LEU214': 10, 'SER212': 10, 'GLN547': 10, 'ALA213': 1, 'ALA236': 6, 'GLY546': 10, 'GLN545': 10, 'TYR551': 10, 'SER598': 10, 'ASN556': 6, 'TYR651': 10, 'ALA552': 10, 'VAL218': 10}, 'TRP550': {'LEU214': 10, 'TYR230': 10, 'GLN547': 10, 'ALA236': 10, 'ASN556': 10, 'TYR651': 10, 'ALA552': 10, 'TYR649': 3, 'ALA557': 10, 'GLY546': 10, 'ASN234': 10, 'TYR551': 10, 'GLN231': 10, 'LEU548': 10, 'TRP553': 9, 'ALA215': 8, 'ARG348': 10, 'GLY235': 10, 'LEU227': 10, 'PHE228': 10, 'VAL218': 10}, 'TYR551': {'GLN590': 10, 'GLN547': 10, 'GLH356': 4, 'ALA236': 10, 'PHE352': 10, 'ASN556': 10, 'ALA552': 10, 'ALA557': 10, 'VAL554': 10, 'GLY546': 10, 'SER594': 10, 'MET591': 6, 'GLN231': 8, 'LEU548': 10, 'TRP553': 10, 'ASN349': 2, 'PRO558': 8, 'ARG348': 10, 'HIE355': 10, 'ILE543': 1, 'ASP351': 10, 'ASN237': 10, 'VAL218': 1, 'ASP549': 10}, 'ALA552': {'LEU548': 10, 'TRP553': 10, 'VAL554': 10, 'ALA215': 10, 'TRP550': 10, 'GLN590': 10, 'SER212': 2, 'GLN547': 10, 'TRP595': 3, 'PHE207': 10, 'GLN210': 10, 'MET591': 10, 'SER594': 10, 'LYS211': 10, 'ASN556': 9, 'VAL218': 1, 'ASP549': 10}, 'TRP553': {'GLN590': 6, 'PHE207': 10, 'GLN223': 2, 'ASN556': 10, 'ARG219': 10, 'ALA557': 10, 'VAL554': 10, 'MET591': 10, 'TYR551': 10, 'ILE206': 6, 'LEU548': 10, 'ALA215': 10, 'TRP550': 9, 'GLU201': 4, 'ASP204': 10, 'TRP222': 10, 'PRO555': 10, 'PHE203': 10, 'LYS211': 2, 'LEU200': 10, 'VAL218': 10, 'ASP549': 9}, 'VAL554': {'CYS559': 10, 'LEU581': 10, 'GLN590': 10, 'PHE207': 10, 'ASN556': 10, 'PRO582': 10, 'CYS588': 10, 'ALA552': 10, 'ALA557': 10, 'TRP197': 4, 'MET591': 10, 'TYR551': 10, 'GLN586': 7, 'PRO558': 10, 'ALA587': 10, 'PRO555': 10, 'PHE203': 10, 'ILE206': 9, 'THR560': 3}, 'PRO555': {'TRP553': 10, 'LEU581': 10, 'PRO558': 10, 'VAL577': 3, 'PHE203': 10, 'TRP222': 10, 'MET591': 6, 'SER345': 10, 'ASN556': 10, 'LEU341': 5, 'PRO582': 10, 'LEU200': 10, 'ILE206': 1, 'ALA587': 1, 'ALA557': 10}, 'ASN556': {'TRP553': 10, 'VAL554': 10, 'PRO558': 10, 'TRP550': 10, 'LEU581': 4, 'ARG348': 10, 'ALA557': 10, 'GLN590': 2, 'TYR551': 10, 'SER345': 10, 'PHE228': 2, 'LEU341': 6, 'ARG219': 3, 'ALA552': 9, 'TRP222': 10, 'VAL218': 10, 'ASP549': 6, 'LYS344': 2}, 'ALA557': {'PRO555': 10, 'ASN349': 10, 'TRP553': 10, 'VAL554': 10, 'ASP351': 9, 'TRP550': 10, 'ARG348': 10, 'PRO558': 10, 'CYS559': 10, 'GLN590': 10, 'TYR551': 10, 'SER345': 10, 'PHE352': 10}, 'PRO558': {'PRO555': 10, 'ASN349': 10, 'VAL554': 10, 'CYS559': 10, 'LEU581': 10, 'GLN590': 10, 'ARG348': 10, 'TRP346': 7, 'PHE203': 3, 'TYR551': 8, 'SER345': 10, 'GLN353': 10, 'PHE352': 10, 'ASN556': 10, 'PRO582': 10, 'SER350': 1, 'THR560': 10, 'MET561': 10}, 'CYS559': {'ASN349': 10, 'THR420': 10, 'VAL554': 10, 'GLN590': 10, 'ALA417': 10, 'GLN353': 10, 'VAL418': 10, 'PHE352': 10, 'PRO582': 10, 'GLN586': 10, 'SER419': 10, 'ALA587': 10, 'THR560': 10, 'ALA557': 10, 'MET561': 10}, 'THR560': {'THR420': 2, 'LEU581': 10, 'GLN590': 10, 'GLN294': 10, 'PHE352': 1, 'PRO582': 10, 'TYR295': 10, 'MET561': 10, 'VAL554': 3, 'ALA417': 10, 'VAL418': 10, 'GLN586': 10, 'SER419': 10, 'ASN349': 10, 'ILE289': 10, 'PRO558': 10, 'ARG562': 10, 'GLN353': 10, 'ASN287': 9, 'LEU296': 5, 'SER580': 9, 'ASP583': 10}, 'MET561': {'SER580': 10, 'ASN349': 10, 'CYS559': 10, 'PRO558': 10, 'ARG562': 10, 'LEU581': 10, 'TRP346': 10, 'VAL577': 7, 'LYS293': 9, 'GLN294': 10, 'SER345': 10, 'PRO565': 1, 'PRO582': 10, 'MET563': 10, 'TYR295': 10, 'SER419': 10, 'LEU296': 10, 'LEU342': 10}, 'ARG562': {'SER580': 10, 'GLY579': 10, 'LEU581': 10, 'ASP583': 10, 'TRP197': 10, 'TRP346': 6, 'GLU292': 10, 'LYS293': 10, 'GLN294': 10, 'ARG290': 1, 'VAL584': 10, 'GLN586': 10, 'PRO564': 9, 'MET563': 10, 'TYR295': 10, 'PRO582': 10, 'MET578': 10, 'THR560': 10}, 'MET563': {'SER580': 10, 'GLY579': 10, 'THR576': 10, 'TRP346': 10, 'ALA575': 5, 'GLU292': 1, 'LYS293': 10, 'GLN294': 10, 'PRO565': 10, 'PRO564': 10, 'TYR295': 10, 'MET561': 10}, 'PRO564': {'GLN319': 10, 'SER580': 9, 'ARG562': 9, 'TRP346': 10, 'LYS293': 2, 'PRO565': 10, 'PRO566': 10, 'TYR295': 10}, 'PRO565': {'GLN319': 10, 'SER580': 4, 'PHE331': 9, 'TRP346': 10, 'THR567': 10, 'PRO566': 10, 'ALA297': 10, 'ILE318': 10, 'TYR295': 10, 'MET563': 10, 'LEU296': 10, 'LEU342': 9, 'MET561': 1}, 'PRO566': {'SER580': 3, 'THR576': 10, 'TRP346': 10, 'VAL572': 10, 'LYS569': 10, 'THR329': 10, 'THR567': 10, 'ALA339': 10, 'THR568': 10, 'ILE318': 10, 'PRO564': 10, 'ALA343': 5, 'PHE331': 10, 'ASP571': 5, 'LEU342': 10}, 'THR567': {'LYS569': 10, 'THR329': 10, 'ASP571': 3, 'PRO565': 10, 'THR568': 10, 'PHE331': 3, 'PRO328': 7}, 'THR568': {'VAL572': 10, 'LYS569': 10, 'THR329': 10, 'PRO566': 10, 'GLU570': 10, 'PHE331': 4, 'ASP571': 10}, 'LYS569': {'LEU332': 10, 'LEU330': 10, 'ASP335': 10, 'VAL572': 10, 'ASP571': 10, 'THR329': 10, 'THR567': 10, 'ALA339': 10, 'PRO566': 10, 'TRP340': 3, 'ALA343': 1, 'GLU570': 10, 'PHE331': 10, 'MET257': 10, 'PRO336': 10}, 'GLU570': {'VAL572': 10, 'ALA339': 2, 'THR568': 10, 'ASP571': 10, 'PRO336': 10}, 'ASP571': {'THR573': 10, 'THR576': 10, 'VAL572': 10, 'LYS569': 10, 'THR567': 3, 'THR568': 10, 'PRO566': 5}, 'VAL572': {'THR573': 10, 'VAL577': 10, 'MET574': 10, 'THR576': 10, 'LYS569': 10, 'ALA339': 10, 'LEU338': 10, 'PRO566': 10, 'THR568': 10, 'GLU570': 10, 'PHE331': 10, 'LEU342': 10, 'PRO336': 9}, 'THR573': {'VAL577': 10, 'MET574': 10, 'THR576': 10, 'ALA575': 10, 'LEU338': 10, 'MET578': 10, 'ASP571': 10, 'LEU342': 3}, 'MET574': {'GLY579': 10, 'VAL577': 10, 'THR576': 10, 'VAL572': 10, 'ALA575': 10, 'GLU201': 1, 'ASN198': 10, 'LEU338': 10, 'CYS199': 10, 'MET578': 10, 'LEU200': 10}, 'ALA575': {'THR573': 10, 'GLY579': 10, 'VAL577': 10, 'SER580': 10, 'THR576': 10, 'ASN198': 10, 'CYS199': 4, 'MET563': 5, 'MET578': 10}, 'THR576': {'THR573': 10, 'GLY579': 10, 'VAL577': 10, 'SER580': 10, 'MET574': 10, 'LEU581': 10, 'TRP346': 3, 'VAL572': 10, 'PRO566': 10, 'MET563': 10, 'MET578': 10, 'ASP571': 10, 'LEU342': 10}, 'VAL577': {'THR573': 10, 'GLY579': 10, 'SER580': 10, 'LEU581': 10, 'MET574': 10, 'PRO555': 3, 'VAL572': 10, 'TRP346': 1, 'ALA575': 10, 'TRP222': 10, 'ALA339': 5, 'LEU338': 10, 'SER345': 9, 'LEU341': 10, 'MET578': 10, 'LEU200': 10, 'LEU342': 10, 'MET561': 7}, 'MET578': {'THR573': 10, 'GLY579': 10, 'ARG562': 10, 'SER580': 10, 'MET574': 10, 'THR576': 10, 'LEU581': 10, 'TRP197': 10, 'PHE203': 10, 'ALA575': 10, 'ASN198': 10, 'CYS199': 10, 'PRO582': 8, 'LEU200': 10, 'LEU342': 1, 'ASP202': 10}, 'GLY579': {'SER580': 10, 'ARG562': 10, 'VAL577': 10, 'MET574': 10, 'THR576': 10, 'LEU581': 10, 'ALA575': 10, 'MET563': 10}, 'SER580': {'ARG562': 10, 'VAL577': 10, 'LEU581': 10, 'THR576': 10, 'TRP346': 10, 'ALA575': 10, 'PRO565': 4, 'PRO566': 3, 'PRO564': 9, 'MET563': 10, 'PRO582': 10, 'MET578': 10, 'THR560': 9, 'LEU342': 10, 'MET561': 10}, 'LEU581': {'GLY579': 10, 'VAL577': 10, 'ASN556': 4, 'PRO582': 10, 'LEU341': 2, 'MET561': 10, 'VAL554': 10, 'THR576': 10, 'TRP197': 10, 'MET578': 10, 'LEU342': 10, 'ARG562': 10, 'PRO558': 10, 'SER345': 10, 'TRP222': 10, 'ASP583': 10, 'PRO555': 10, 'PHE203': 10, 'LEU200': 10, 'THR560': 10}, 'PRO582': {'CYS559': 10, 'GLN590': 10, 'GLN294': 8, 'VAL584': 10, 'MET561': 10, 'VAL554': 10, 'TRP197': 10, 'ALA417': 1, 'GLN586': 10, 'MET578': 8, 'ARG562': 10, 'ASN349': 1, 'PRO558': 10, 'ALA587': 10, 'SER580': 10, 'PRO555': 10, 'ASP583': 10, 'PHE203': 10, 'THR560': 10}, 'ASP583': {'ARG562': 10, 'LEU581': 10, 'ILE289': 3, 'PHE203': 3, 'TRP197': 10, 'GLU292': 10, 'ARG290': 10, 'GLN294': 7, 'VAL584': 10, 'GLN586': 10, 'CYS588': 9, 'ALA587': 10, 'THR560': 10, 'ARG585': 10}, 'VAL584': {'ARG562': 10, 'ARG189': 10, 'PHE203': 1, 'LYS188': 9, 'TRP197': 10, 'MET185': 1, 'LEU589': 10, 'ARG290': 5, 'THR192': 10, 'PRO582': 10, 'GLN586': 10, 'CYS588': 10, 'ILE206': 10, 'ALA587': 10, 'ARG585': 10}, 'ARG585': {'ASP583': 10, 'ILE289': 7, 'ARG189': 10, 'GLN590': 9, 'LYS416': 10, 'LEU589': 10, 'ARG290': 10, 'GLN586': 10, 'CYS588': 10, 'MET185': 10, 'ALA587': 10}, 'GLN586': {'CYS559': 10, 'GLN590': 10, 'GLN294': 10, 'CYS588': 10, 'PRO582': 10, 'VAL584': 10, 'VAL554': 7, 'ALA417': 10, 'GLU292': 2, 'MET591': 9, 'VAL418': 6, 'ILE289': 10, 'ARG562': 10, 'MET185': 8, 'ALA587': 10, 'ASP583': 10, 'LYS416': 10, 'LEU589': 10, 'ARG290': 10, 'THR560': 10}, 'ALA587': {'ASP583': 10, 'PRO555': 1, 'VAL554': 10, 'CYS559': 10, 'GLN590': 10, 'PHE203': 10, 'TRP197': 10, 'ALA592': 8, 'LEU589': 10, 'MET591': 10, 'CYS588': 10, 'PRO582': 10, 'ILE206': 10, 'VAL584': 10, 'ARG585': 10}, 'CYS588': {'ASP583': 9, 'VAL554': 10, 'ILE593': 10, 'MET185': 10, 'ARG189': 10, 'LYS188': 10, 'GLN590': 10, 'GLU184': 4, 'ALA592': 10, 'TRP197': 3, 'LEU589': 10, 'MET591': 10, 'THR192': 7, 'GLN586': 10, 'ILE206': 10, 'VAL584': 10, 'ARG585': 10}, 'LEU589': {'ALA182': 6, 'ILE593': 10, 'GLN590': 10, 'ALA417': 10, 'LYS416': 10, 'ALA592': 10, 'MET591': 10, 'SER594': 10, 'VAL418': 2, 'ILE413': 10, 'GLN586': 10, 'MET185': 10, 'ALA587': 10, 'VAL584': 10, 'ARG585': 10}, 'GLN590': {'CYS559': 10, 'ALA592': 10, 'PHE352': 10, 'PRO582': 10, 'CYS588': 10, 'ASN556': 2, 'ALA552': 10, 'ALA557': 10, 'VAL554': 10, 'ILE593': 10, 'ALA417': 8, 'MET591': 10, 'TYR551': 10, 'SER594': 10, 'GLN586': 10, 'TRP553': 6, 'PRO558': 10, 'TRP595': 10, 'ALA587': 10, 'THR560': 10, 'ARG585': 9}, 'MET591': {'PHE207': 10, 'ALA592': 10, 'CYS588': 10, 'ALA552': 10, 'VAL554': 10, 'ILE593': 10, 'TRP197': 4, 'SER594': 10, 'TYR551': 6, 'GLN586': 9, 'HIE596': 9, 'LEU548': 8, 'TRP553': 10, 'TRP595': 10, 'LYS188': 10, 'ASP204': 1, 'ALA587': 10, 'PRO555': 6, 'PHE203': 10, 'GLU184': 2, 'LEU589': 10, 'ILE206': 10}, 'ALA592': {'ALA182': 10, 'ILE593': 10, 'TRP595': 10, 'GLN590': 10, 'LYS188': 1, 'GLU184': 9, 'LEU589': 10, 'SER594': 10, 'LEU597': 8, 'GLY181': 10, 'CYS588': 10, 'HIE596': 10, 'LEU178': 4, 'MET185': 10, 'ALA587': 8}, 'ILE593': {'ALA182': 1, 'TRP595': 10, 'GLN590': 10, 'GLN547': 4, 'GLH356': 6, 'LEU589': 10, 'MET591': 10, 'LEU407': 10, 'SER594': 10, 'LEU597': 10, 'SER598': 9, 'ILE413': 10, 'CYS588': 10, 'HIE596': 10, 'LEU178': 10}, 'SER594': {'LEU548': 10, 'TRP595': 10, 'GLN590': 10, 'GLN547': 10, 'GLH356': 1, 'ALA592': 10, 'ARG599': 10, 'LEU589': 10, 'MET591': 10, 'TYR551': 10, 'LEU597': 10, 'SER598': 10, 'PHE352': 4, 'HIE596': 10, 'ALA552': 10}, 'TRP595': {'THR177': 10, 'LEU548': 10, 'ARG600': 6, 'ILE593': 10, 'GLN590': 10, 'SER212': 1, 'ALA592': 10, 'ARG599': 10, 'GLN210': 10, 'MET591': 10, 'TRP176': 5, 'LEU597': 10, 'LYS211': 6, 'GLY181': 8, 'SER598': 10, 'HIE596': 10, 'ALA552': 3, 'ALA180': 3}, 'HIE596': {'THR177': 10, 'ASP173': 9, 'ILE593': 10, 'PHE174': 10, 'GLU175': 8, 'ILE399': 1, 'ALA592': 10, 'ARG599': 10, 'MET591': 9, 'SER594': 10, 'LEU597': 10, 'SER598': 10, 'GLY181': 5, 'GLN601': 7, 'LEU178': 10}, 'LEU597': {'ILE593': 10, 'PHE174': 9, 'TRP595': 10, 'GLN547': 10, 'GLH356': 10, 'ILE399': 2, 'ALA592': 8, 'ARG599': 10, 'HIE355': 1, 'SER594': 10, 'LEU407': 1, 'SER598': 10, 'GLN601': 5, 'GLN545': 4}, 'SER598': {'LEU548': 10, 'ARG600': 10, 'ILE593': 9, 'TRP595': 10, 'GLN547': 10, 'ARG599': 10, 'GLY546': 7, 'SER594': 10, 'HIE596': 10, 'GLN545': 10, 'ASP549': 10}, 'ARG599': {'THR177': 10, 'LEU548': 9, 'ASP173': 10, 'ARG600': 10, 'PHE174': 10, 'TRP595': 10, 'SER594': 10, 'LYS170': 10, 'TRP176': 5, 'LEU597': 10, 'PRO602': 10, 'GLN601': 10, 'HIE596': 10}, 'ARG600': {'SER655': 6, 'TRP595': 6, 'SER598': 10, 'PRO602': 10, 'PRO606': 6, 'GLN601': 10, 'MET604': 10, 'CYS656': 6, 'GLN545': 8}, 'GLN601': {'ASP173': 10, 'PHE174': 10, 'ARG394': 10, 'THR662': 10, 'ARG599': 10, 'LYS170': 10, 'LEU597': 5, 'GLH658': 10, 'PRO602': 10, 'MET604': 10, 'HIE596': 7, 'VAL605': 9, 'ASP603': 10}, 'PRO602': {'ASP173': 10, 'ARG600': 10, 'ARG599': 10, 'LYS170': 10, 'TRP176': 2, 'MET604': 10, 'ASP603': 10}, 'ASP603': {'ARG394': 10, 'TYR395': 7, 'THR662': 9, 'LYS170': 10, 'GLH658': 10, 'TRP13': 10, 'GLN601': 10, 'MET604': 10, 'HIE167': 10, 'VAL605': 10}, 'MET604': {'SER655': 10, 'ILE657': 10, 'ARG600': 10, 'ARG394': 10, 'LEU607': 10, 'LYS611': 1, 'THR662': 7, 'ASN659': 10, 'GLH658': 10, 'PRO606': 10, 'PRO602': 10, 'GLN601': 10, 'PRO654': 3, 'CYS656': 10, 'VAL605': 10}, 'VAL605': {'ILE657': 10, 'ARG394': 10, 'LYS611': 10, 'SER660': 10, 'THR662': 3, 'ASN659': 10, 'GLH658': 10, 'PRO606': 10, 'TRP13': 2, 'GLU612': 1, 'GLN601': 9, 'HID609': 10, 'HID610': 10, 'LEU607': 10, 'ASP603': 10}, 'PRO606': {'SER655': 10, 'ILE657': 10, 'ARG600': 6, 'CYS656': 9, 'LYS611': 10, 'ASN659': 10, 'GLY608': 10, 'ARG628': 9, 'GLH658': 10, 'HID609': 10, 'MET604': 10, 'HID610': 10, 'PRO654': 10, 'LEU607': 10}, 'LEU607': {'ASN659': 10, 'PHE627': 10, 'VAL605': 10, 'ILE657': 10, 'TYR614': 2, 'PHE534': 4, 'ALA538': 10, 'MET604': 10, 'SER655': 5, 'SER660': 2, 'GLH658': 10, 'HID392': 10, 'LEU631': 10, 'TYR474': 4, 'HID609': 10, 'HID610': 10, 'PRO654': 10, 'PRO391': 1, 'GLY608': 10, 'VAL533': 10, 'LEU467': 10, 'ILE471': 10, 'THR537': 10}, 'GLY608': {'SER655': 4, 'ILE657': 7, 'ASN659': 1, 'ARG628': 10, 'LEU624': 10, 'PRO606': 10, 'PHE627': 10, 'HID609': 10, 'HID610': 10, 'LEU631': 10, 'PRO654': 10}, 'HID609': {'LYS611': 10, 'ASN659': 8, 'ARG628': 10, 'LEU624': 4, 'PRO606': 10, 'LEU631': 1, 'PHE627': 2, 'HID610': 10, 'VAL605': 10, 'LYS613': 4, 'LEU607': 10}, 'HID610': {'TYR614': 10, 'LEU607': 10, 'LYS611': 10, 'ASN659': 10, 'GLY608': 10, 'LEU624': 10, 'PRO606': 10, 'PRO391': 10, 'TYR474': 10, 'HID392': 10, 'GLU612': 10, 'PHE627': 10, 'LYS613': 10, 'VAL605': 10}, 'LYS611': {'ASN659': 10, 'PRO606': 10, 'PRO391': 10, 'TRP13': 8, 'HID609': 10, 'GLU612': 10, 'MET604': 1, 'LYS613': 10, 'VAL605': 10}, 'GLU612': {'TYR614': 10, 'ASN659': 10, 'ASP79': 3, 'PRO391': 10, 'ILE390': 10, 'TRP13': 10, 'LYS387': 10, 'HID610': 10, 'TYR97': 10, 'LYS613': 10, 'VAL605': 1}, 'LYS613': {'TYR614': 10, 'LYS611': 10, 'PHE615': 10, 'SER616': 6, 'LEU624': 10, 'PRO391': 10, 'LYS387': 10, 'HID609': 4, 'HID610': 10, 'TYR97': 10, 'LYS621': 9}, 'TYR614': {'PHE615': 10, 'GLU612': 10, 'PHE627': 1, 'TYR97': 6, 'PRO620': 10, 'ILE390': 8, 'LEU389': 10, 'LEU607': 2, 'VAL475': 10, 'PHE388': 10, 'LEU624': 10, 'PRO391': 10, 'HID392': 10, 'LYS387': 10, 'TYR474': 10, 'HID610': 10, 'SER616': 10, 'ILE478': 10, 'VAL533': 10, 'LEU529': 4}, 'PHE615': {'LEU624': 1, 'ILE385': 10, 'GLY111': 8, 'PHE482': 10, 'HIP383': 4, 'SER616': 10, 'PHE388': 10, 'ILE478': 10, 'LEU481': 8, 'PRO620': 10, 'PHE386': 1, 'LYS387': 10, 'GLY617': 10, 'PRO384': 10, 'LYS621': 10, 'THR112': 8, 'LYS613': 10, 'ALA113': 10}, 'SER616': {'TYR614': 10, 'PRO620': 10, 'GLY617': 10, 'LYS387': 6, 'LYS621': 10, 'LYS613': 6, 'PRO618': 10, 'ARG80': 4}, 'GLY617': {'LYS619': 10, 'ALA622': 10, 'PHE615': 10, 'PRO620': 10, 'LYS621': 10, 'PRO618': 10}, 'PRO618': {'LYS619': 10, 'ALA622': 10, 'SER616': 10, 'PRO620': 10, 'VAL623': 10, 'LYS621': 10}, 'LYS619': {'ALA622': 10, 'VAL623': 10, 'PRO620': 10, 'LEU481': 10, 'LEU624': 10, 'ILE478': 7, 'GLY617': 10, 'LYS621': 10}, 'PRO620': {'TYR614': 10, 'ALA622': 10, 'PHE615': 10, 'PHE482': 10, 'ASN625': 10, 'SER616': 10, 'LEU481': 10, 'ILE478': 10, 'VAL623': 10, 'LEU624': 10, 'TYR474': 9, 'GLY617': 10, 'LYS621': 10, 'PRO618': 10}, 'LYS621': {'LYS619': 10, 'ALA622': 10, 'PHE615': 10, 'ASN625': 10, 'SER616': 10, 'VAL623': 10, 'LEU624': 10, 'ILE478': 1, 'GLY617': 10, 'GLN626': 10, 'LYS613': 9, 'PRO618': 10}, 'ALA622': {'LYS619': 10, 'ASN625': 10, 'VAL623': 10, 'PRO620': 10, 'LEU624': 10, 'PHE627': 10, 'GLY617': 10, 'GLN626': 10, 'PRO618': 10}, 'VAL623': {'VAL475': 7, 'LYS619': 10, 'GLY477': 10, 'ASN625': 10, 'LEU481': 10, 'ILE478': 10, 'PRO620': 10, 'LEU624': 10, 'PHE627': 10, 'TYR474': 10, 'GLN626': 10, 'LYS621': 10, 'ARG628': 10, 'GLU476': 1, 'PRO618': 10, 'ARG473': 10}, 'LEU624': {'LYS619': 10, 'THR629': 10, 'TYR614': 10, 'ALA622': 10, 'PHE615': 1, 'ASN625': 10, 'GLY608': 10, 'ILE478': 10, 'PRO620': 10, 'PHE627': 10, 'ARG628': 10, 'TYR474': 10, 'HID609': 4, 'HID610': 10, 'LYS621': 10, 'GLN626': 10, 'LYS613': 10}, 'ASN625': {'THR629': 10, 'ALA622': 10, 'ASP630': 10, 'VAL623': 10, 'PRO620': 10, 'ARG628': 10, 'PHE627': 10, 'GLN626': 10, 'LYS621': 10}, 'GLN626': {'THR629': 10, 'ALA622': 10, 'ASP630': 10, 'VAL623': 10, 'ARG628': 10, 'LEU624': 10, 'PHE627': 10, 'TYR474': 5, 'LYS621': 10, 'LEU631': 10, 'ILE470': 9, 'ARG473': 10}, 'PHE627': {'GLU632': 10, 'ALA622': 10, 'ASN625': 10, 'ASP630': 10, 'ARG628': 10, 'THR629': 10, 'TYR614': 1, 'VAL623': 10, 'LEU607': 10, 'LEU624': 10, 'HID392': 5, 'TYR474': 10, 'HID609': 2, 'HID610': 10, 'ILE470': 10, 'GLY608': 10, 'VAL533': 8, 'LEU631': 10, 'ILE471': 10, 'LEU467': 9, 'ARG473': 2}, 'ARG628': {'SER655': 3, 'THR629': 10, 'LYS633': 10, 'GLU632': 10, 'ASN625': 10, 'VAL623': 10, 'GLY608': 10, 'LEU624': 10, 'ASP630': 10, 'PRO606': 9, 'GLN626': 10, 'HID609': 10, 'LEU631': 10}, 'THR629': {'LYS633': 10, 'GLU632': 10, 'ASN625': 10, 'ASP630': 10, 'LEU624': 10, 'PHE627': 10, 'GLN626': 10, 'LEU631': 10, 'LEU634': 10}, 'ASP630': {'LYS633': 10, 'GLU632': 10, 'LEU634': 10, 'ASN625': 10, 'GLU635': 10, 'ARG628': 10, 'PHE627': 10, 'GLN626': 10, 'PRO654': 2, 'LEU631': 10, 'ILE470': 10, 'ARG473': 10}, 'LEU631': {'ILE657': 10, 'SER655': 1, 'THR629': 10, 'LYS633': 10, 'GLU632': 10, 'GLU635': 10, 'GLY608': 10, 'ARG628': 10, 'PHE627': 10, 'GLN626': 10, 'PRO654': 10, 'HID609': 1, 'LEU467': 10, 'ILE470': 10, 'ILE471': 10, 'LEU634': 10, 'LEU607': 10, 'LYS636': 10}, 'GLU632': {'SER655': 7, 'THR629': 10, 'LYS633': 10, 'ASP630': 10, 'GLU635': 10, 'ARG628': 10, 'PHE627': 10, 'PRO654': 10, 'GLU637': 10, 'LEU634': 10, 'LYS636': 10}, 'LYS633': {'THR629': 10, 'ASP630': 10, 'GLU635': 10, 'ARG628': 10, 'GLU637': 10, 'LEU631': 10, 'ILE470': 4, 'ILE638': 10, 'LEU634': 10, 'LYS636': 10}, 'LEU634': {'LYS653': 9, 'THR639': 10, 'THR629': 10, 'ILE657': 3, 'ASP463': 10, 'GLU632': 10, 'LEU652': 10, 'ASP630': 10, 'GLU635': 10, 'ARG466': 10, 'LEU631': 10, 'PRO654': 10, 'GLU637': 10, 'LEU467': 10, 'ILE470': 10, 'ILE638': 10, 'HID462': 5, 'LYS636': 10}, 'GLU635': {'LYS653': 10, 'THR639': 10, 'SER655': 10, 'CYS656': 9, 'LYS633': 10, 'GLU632': 10, 'ASP630': 10, 'ALA640': 9, 'PRO654': 10, 'GLU637': 10, 'LEU631': 10, 'ILE638': 10, 'LYS636': 10}, 'LYS636': {'LYS653': 1, 'THR639': 10, 'ARG641': 9, 'LYS633': 10, 'GLU632': 10, 'ALA640': 10, 'GLU637': 10, 'LEU631': 10, 'ILE638': 10, 'LEU634': 10}, 'GLU637': {'THR639': 10, 'ARG641': 10, 'LYS633': 10, 'GLU632': 10, 'ASN642': 10, 'GLU635': 10, 'ALA640': 10, 'ARG466': 10, 'ILE638': 10, 'LEU634': 10}, 'ILE638': {'GLU643': 10, 'THR639': 10, 'ARG641': 10, 'LYS653': 10, 'LYS633': 10, 'ASP463': 10, 'ASN642': 10, 'LEU652': 10, 'GLU635': 10, 'ALA640': 10, 'GLU650': 10, 'ARG466': 9, 'PRO654': 10, 'LEU467': 9, 'LEU634': 10, 'TYR649': 10, 'LYS636': 10}, 'THR639': {'GLU643': 10, 'LYS653': 10, 'ARG641': 10, 'ASN642': 10, 'ALA640': 10, 'GLU635': 10, 'GLU650': 4, 'GLU637': 10, 'LEU634': 10, 'LYS636': 10}, 'ALA640': {'GLU643': 10, 'GLN644': 10, 'ARG641': 10, 'GLU635': 9, 'GLU637': 10, 'ILE638': 10, 'ASN642': 10, 'LYS636': 10}, 'ARG641': {'GLU643': 10, 'THR639': 10, 'GLN644': 10, 'ASP463': 9, 'LEU459': 4, 'LEU645': 10, 'ARG466': 10, 'GLU637': 10, 'ILE638': 10, 'HID462': 6, 'ASN642': 10, 'LYS636': 9}, 'ASN642': {'GLU643': 10, 'THR639': 10, 'GLN644': 10, 'LYS653': 10, 'PRO648': 10, 'TRP647': 1, 'LEU459': 10, 'ALA640': 10, 'LEU645': 10, 'GLU650': 10, 'GLU637': 10, 'ILE638': 10, 'TYR649': 10}, 'GLU643': {'THR639': 10, 'ARG641': 10, 'GLN644': 10, 'PRO648': 9, 'ALA640': 10, 'LEU645': 10, 'ILE638': 10}, 'GLN644': {'ARG641': 10, 'ALA640': 10, 'LEU645': 10, 'ASP646': 10, 'ASN308': 3, 'ASN642': 10, 'TRP647': 1}, 'LEU645': {'GLU643': 10, 'ARG641': 10, 'PRO648': 10, 'LEU459': 10, 'TYR649': 10, 'ASP646': 10, 'ASN308': 9, 'ASN642': 10, 'TRP647': 10}, 'ASP646': {'GLN644': 10, 'PRO648': 10, 'TYR230': 10, 'LYS217': 4, 'LYS310': 3, 'GLU226': 4, 'TRP647': 10}, 'TRP647': {'PRO456': 8, 'GLN644': 1, 'PRO648': 10, 'TYR230': 10, 'ASN642': 1, 'LEU459': 10, 'ASN234': 10, 'GLY457': 10, 'LEU645': 10, 'LYS217': 1, 'ASN308': 10, 'LYS310': 10, 'GLU226': 5, 'TYR649': 10}, 'PRO648': {'GLU643': 9, 'LEU214': 5, 'TYR230': 8, 'LEU459': 10, 'ASN234': 10, 'LEU645': 10, 'GLU650': 10, 'ASP646': 10, 'ASN642': 10, 'TYR649': 10}, 'TYR649': {'LYS653': 10, 'ASP463': 10, 'TRP550': 3, 'ASN642': 10, 'LEU652': 10, 'LEU459': 10, 'ASN234': 10, 'GLY235': 10, 'ILE638': 10, 'LEU645': 10, 'GLU650': 10, 'TYR651': 10, 'ALA542': 10, 'ILE543': 1, 'TYR460': 10, 'PRO238': 10, 'ALA464': 9, 'TRP647': 10}, 'GLU650': {'LYS653': 10, 'THR639': 4, 'PRO648': 10, 'ASP463': 4, 'LEU652': 10, 'TYR651': 10, 'CYS656': 10, 'ILE638': 10, 'ASN642': 10}, 'TYR651': {'LYS653': 10, 'ALA541': 10, 'ILE657': 10, 'CYS656': 10, 'TRP550': 10, 'LEU214': 10, 'LEU652': 10, 'ALA236': 10, 'GLY546': 10, 'ASN234': 9, 'GLY235': 10, 'ALA542': 10, 'ILE543': 3, 'TYR460': 1, 'PRO238': 3, 'GLN545': 10, 'TYR649': 10, 'ASP549': 10}, 'LEU652': {'LYS653': 10, 'ALA541': 10, 'ILE657': 10, 'CYS656': 10, 'PHE534': 10, 'ASP463': 10, 'LEU634': 10, 'ALA538': 10, 'ILE638': 10, 'GLU650': 10, 'PRO654': 10, 'ALA542': 10, 'LEU467': 10, 'TYR460': 10, 'ALA464': 10, 'TYR649': 10, 'GLN539': 10}, 'LYS653': {'SER655': 10, 'THR639': 10, 'ILE657': 10, 'ASP463': 2, 'ASN642': 10, 'GLU635': 10, 'GLU650': 10, 'TYR651': 10, 'PRO654': 10, 'CYS656': 10, 'ILE638': 10, 'LEU634': 9, 'TYR649': 10, 'LYS636': 1}, 'PRO654': {'SER655': 10, 'ILE657': 10, 'CYS656': 10, 'GLU632': 10, 'LEU652': 10, 'GLU635': 10, 'ASP630': 2, 'GLY608': 10, 'PRO606': 10, 'MET604': 3, 'LEU631': 10, 'ILE470': 2, 'ILE638': 10, 'LEU467': 10, 'LEU634': 10, 'LEU607': 10}, 'SER655': {'LYS653': 10, 'ILE657': 10, 'ARG600': 6, 'GLU632': 7, 'GLU635': 10, 'GLY608': 4, 'ARG628': 3, 'PRO606': 10, 'LEU631': 1, 'MET604': 10, 'CYS656': 10, 'LEU607': 5}, 'CYS656': {'LYS653': 10, 'ALA541': 9, 'ARG600': 6, 'ILE657': 10, 'LEU652': 10, 'GLU635': 9, 'GLU650': 10, 'GLH658': 10, 'PRO606': 9, 'TYR651': 10, 'PRO654': 10, 'MET604': 10, 'GLN545': 10}, 'ILE657': {'ASN659': 10, 'TYR651': 10, 'VAL605': 10, 'LEU652': 10, 'ALA538': 10, 'MET604': 10, 'LEU607': 10, 'SER655': 10, 'ALA541': 10, 'SER660': 3, 'GLH658': 10, 'PRO654': 10, 'LEU634': 3, 'LYS653': 10, 'GLY608': 7, 'THR537': 10, 'PRO606': 10, 'LEU631': 10, 'ILE471': 5, 'LEU467': 10}, 'GLH658': {'CYS536': 6, 'ALA541': 10, 'VAL661': 10, 'ARG394': 10, 'LEU607': 10, 'SER660': 10, 'THR662': 10, 'ALA538': 3, 'ASN659': 10, 'GLN545': 2, 'PRO606': 10, 'GLN601': 10, 'MET604': 10, 'CYS656': 10, 'THR537': 10, 'VAL605': 10, 'ASP603': 10}, 'ASN659': {'CYS536': 8, 'ILE657': 10, 'VAL661': 10, 'ARG394': 10, 'LEU607': 10, 'LYS611': 10, 'SER660': 10, 'THR662': 6, 'GLY608': 1, 'PRO391': 10, 'HID392': 10, 'PRO606': 10, 'HID609': 8, 'MET604': 10, 'HID610': 10, 'GLU612': 10, 'THR537': 10, 'VAL605': 10}, 'SER660': {'CYS536': 10, 'ILE657': 3, 'VAL661': 10, 'ARG394': 10, 'TYR395': 10, 'LEU607': 2, 'THR662': 10, 'GLH658': 10, 'PRO391': 10, 'HID392': 10, 'THR396': 5, 'ILE393': 10, 'THR537': 10, 'VAL605': 10}, 'VAL661': {'ALA368': 10, 'CYS536': 10, 'ASN400': 9, 'ARG394': 10, 'TYR395': 10, 'THR535': 10, 'ILE399': 10, 'THR662': 10, 'ASN659': 10, 'THR364': 10, 'THR396': 10, 'HID392': 10, 'PRO391': 1, 'GLH658': 10, 'ILE393': 10, 'THR537': 10}, 'THR662': {'PHE174': 8, 'ARG394': 10, 'PHE166': 10, 'TYR395': 10, 'SER660': 10, 'ILE399': 10, 'LYS170': 10, 'ASN659': 6, 'THR396': 9, 'GLH658': 10, 'GLN601': 10, 'MET604': 7, 'VAL605': 3, 'ASP603': 9}}


In [2]:
b = {'MET1': {'PRO86': 10, 'VAL53': 10, 'ALA54': 10, 'ASP56': 10, 'TYR4': 9, 'LEU57': 7, 'GLY2': 10, 'ARG3': 10, 'GLU55': 10}, 'GLY2': {'PRO86': 10, 'VAL53': 10, 'ASP52': 10, 'ALA54': 10, 'GLY85': 2, 'TYR4': 10, 'ARG3': 10, 'HID51': 8, 'GLU55': 2}, 'ARG3': {'MET1': 10, 'PRO86': 10, 'GLN84': 10, 'ASP52': 0, 'VAL53': 10, 'GLY85': 10, 'TYR4': 10, 'GLY87': 5, 'ARG5': 10, 'HID51': 10, 'ASP50': 10}, 'TYR4': {'PRO86': 10, 'GLN84': 10, 'ASP52': 10, 'VAL53': 10, 'ASP56': 8, 'ARG5': 10, 'LEU60': 10, 'MET1': 9, 'CYS89': 1, 'GLY85': 10, 'LEU57': 10, 'LEU26': 10, 'ILE6': 10, 'PHE49': 10, 'ASP50': 10, 'HID51': 10, 'GLY58': 5, 'VAL92': 8, 'VAL83': 10, 'GLU55': 9, 'GLY2': 10, 'LEU35': 10}, 'ARG5': {'ARG7': 10, 'THR82': 10, 'GLN84': 10, 'GLU48': 7, 'GLY85': 10, 'ARG3': 10, 'PHE49': 10, 'ILE6': 10, 'ASP50': 10, 'HID51': 10, 'VAL83': 10}, 'ILE6': {'LEU37': 10, 'ARG7': 10, 'GLU47': 10, 'THR82': 10, 'GLN84': 10, 'VAL83': 10, 'TYR4': 10, 'VAL8': 10, 'VAL63': 9, 'ILE81': 10, 'LEU24': 10, 'LEU26': 10, 'PHE49': 10, 'GLU48': 10, 'LEU35': 10, 'ASP50': 10, 'HID51': 10, 'LEU39': 10}, 'ARG7': {'LEU39': 7, 'GLU47': 10, 'THR82': 10, 'ALA9': 10, 'VAL8': 10, 'ILE81': 10, 'ARG5': 10, 'PHE49': 10, 'GLU48': 10, 'GLU46': 10, 'VAL83': 10, 'ARG80': 10}, 'VAL8': {'LEU24': 10, 'THR10': 10, 'THR82': 10, 'ASP79': 8, 'ILE6': 10, 'GLU48': 10, 'PHE49': 3, 'TRP76': 10, 'ILE81': 10, 'LEU39': 10, 'ARG80': 10, 'VAL22': 10, 'GLU47': 10, 'CYS78': 8, 'LEU65': 1, 'ALA9': 10, 'GLU46': 10, 'GLU45': 10, 'PRO41': 10}, 'ALA9': {'ARG7': 10, 'ARG80': 10, 'GLU47': 10, 'CYS78': 10, 'THR82': 7, 'TRP76': 10, 'GLY11': 10, 'ILE81': 10, 'ASP79': 10, 'GLU46': 10, 'GLU45': 10, 'PHE77': 6, 'THR10': 10, 'PRO41': 9}, 'THR10': {'VAL8': 10, 'ASP79': 10, 'GLU45': 10, 'SER16': 10, 'ARG43': 10, 'TRP76': 10, 'SER18': 10, 'GLY11': 10, 'GLY44': 10, 'TRP13': 1, 'GLY17': 10, 'ALA42': 10, 'ARG80': 4, 'GLU47': 9, 'CYS78': 9, 'ALA12': 10, 'GLU46': 10, 'PHE77': 10, 'PRO41': 10}, 'GLY11': {'SER16': 10, 'ARG43': 10, 'TRP76': 2, 'ALA9': 10, 'GLY44': 10, 'ALA12': 10, 'ASP79': 10, 'TRP13': 10, 'GLY17': 1, 'GLU45': 10, 'PHE77': 10, 'ALA42': 4}, 'ALA12': {'SER16': 10, 'ARG43': 10, 'GLY44': 10, 'TRP13': 10, 'LEU14': 10, 'GLU45': 3, 'THR10': 10}, 'TRP13': {'SER16': 10, 'ARG43': 10, 'LYS611': 8, 'GLY11': 10, 'GLY44': 8, 'GLU169': 5, 'ASP79': 2, 'GLU612': 10, 'THR10': 1, 'GLU168': 10, 'LEU14': 10, 'GLY17': 9, 'PHE15': 10, 'PHE77': 10, 'HIE167': 10, 'VAL605': 2, 'ASP603': 10}, 'LEU14': {'SER16': 10, 'ARG43': 10, 'GLU169': 10, 'ALA12': 10, 'GLU168': 10, 'PHE15': 10, 'LEU172': 10, 'HIE167': 10}, 'PHE15': {'SER16': 10, 'ARG43': 10, 'ALA75': 2, 'TYR19': 6, 'TRP13': 10, 'GLU168': 10, 'GLY17': 10, 'LEU172': 7, 'PHE77': 9, 'ARG171': 2}, 'SER16': {'ARG43': 10, 'ALA75': 9, 'TRP76': 10, 'SER18': 10, 'GLY11': 10, 'GLY44': 5, 'ALA12': 10, 'TYR19': 2, 'TRP13': 10, 'LEU14': 10, 'GLY17': 10, 'PHE77': 10, 'THR10': 10}, 'GLY17': {'ARG43': 10, 'ALA75': 10, 'TRP76': 10, 'SER18': 10, 'ASN20': 1, 'LYS67': 8, 'TYR19': 10, 'GLY44': 2, 'GLY11': 1, 'TRP13': 9, 'PHE15': 10, 'PHE77': 7, 'ALA42': 10, 'THR10': 10, 'PRO41': 10}, 'SER18': {'ARG21': 10, 'SER16': 10, 'ARG43': 10, 'ALA75': 10, 'HIE68': 1, 'TRP76': 10, 'ASN20': 10, 'LYS67': 10, 'TYR19': 10, 'ARG40': 10, 'ASP74': 3, 'VAL22': 10, 'LEU39': 6, 'PHE77': 7, 'ALA42': 10, 'THR10': 10, 'PRO41': 10}, 'TYR19': {'ARG21': 10, 'SER16': 2, 'ARG43': 1, 'ASN20': 10, 'LYS67': 10, 'ARG40': 10, 'ASP74': 9, 'GLY17': 10, 'PHE15': 6, 'PRO41': 5}, 'ASN20': {'ARG21': 10, 'TRP70': 0, 'HIE68': 10, 'ASP73': 2, 'SER18': 10, 'LYS67': 10, 'HID69': 10, 'ARG40': 10, 'ASP74': 10, 'GLY17': 1, 'VAL22': 10, 'LEU39': 10, 'PRO41': 10}, 'ARG21': {'LEU37': 10, 'GLN38': 10, 'GLN23': 10, 'HIE68': 10, 'SER18': 10, 'ARG66': 10, 'GLU36': 10, 'LYS67': 10, 'TYR19': 10, 'ARG40': 10, 'LEU24': 5, 'GLU34': 5, 'LEU35': 10, 'VAL22': 10, 'LEU39': 10, 'PRO41': 10}, 'VAL22': {'LEU37': 6, 'GLN38': 3, 'CYS78': 1, 'GLN23': 10, 'LEU65': 10, 'SER18': 10, 'ARG66': 10, 'ASN20': 10, 'VAL8': 10, 'LYS67': 10, 'HIE68': 10, 'TRP76': 10, 'ARG40': 10, 'LEU24': 10, 'ILE81': 10, 'VAL100': 7, 'LEU39': 10, 'PRO41': 10}, 'GLN23': {'ARG21': 10, 'HIE68': 10, 'LEU65': 10, 'ARG64': 10, 'ARG66': 10, 'LYS67': 10, 'HID69': 3, 'LEU24': 10, 'GLU34': 4, 'ILE105': 8, 'LEU35': 3, 'TRP25': 10, 'LEU39': 3}, 'LEU24': {'TRP25': 10, 'LEU37': 10, 'VAL8': 10, 'LEU26': 10, 'ILE6': 10, 'ARG21': 5, 'ALA33': 10, 'TRP76': 8, 'ARG64': 10, 'ILE81': 10, 'LEU108': 1, 'VAL83': 1, 'LEU39': 10, 'LEU65': 10, 'ARG66': 10, 'VAL63': 10, 'GLU34': 10, 'LEU35': 10, 'VAL22': 10}, 'TRP25': {'ALA33': 10, 'PHE62': 10, 'GLN23': 10, 'LEU65': 10, 'ARG64': 10, 'VAL27': 10, 'ARG66': 10, 'VAL63': 10, 'GLU34': 10, 'ILE105': 10, 'LEU26': 10, 'LEU106': 1, 'LEU35': 10, 'GLU32': 10}, 'LEU26': {'VAL53': 10, 'LEU24': 10, 'LEU60': 10, 'PHE62': 10, 'PHE94': 10, 'LEU57': 10, 'GLY28': 10, 'ILE6': 10, 'HID51': 2, 'ALA33': 10, 'VAL92': 2, 'ARG64': 10, 'ILE81': 9, 'GLY31': 10, 'VAL83': 10, 'GLU32': 10, 'TYR4': 10, 'VAL27': 10, 'VAL63': 10, 'GLU34': 10, 'LEU35': 10}, 'VAL27': {'MET123': 10, 'ALA33': 10, 'PHE94': 4, 'ARG64': 10, 'LEU57': 10, 'GLY28': 10, 'VAL63': 10, 'HID127': 7, 'THR29': 10, 'LEU60': 10, 'GLN61': 10, 'LEU59': 7, 'SER107': 2, 'PHE62': 10, 'TRP25': 10, 'GLY31': 10, 'GLU32': 10}, 'GLY28': {'MET123': 10, 'ARG30': 10, 'LEU57': 10, 'THR29': 10, 'LEU60': 10, 'GLN61': 10, 'LEU26': 10, 'LEU59': 10, 'PHE62': 10, 'GLY31': 10, 'GLY58': 10, 'GLU32': 10}, 'THR29': {'MET123': 10, 'ARG30': 10, 'VAL27': 10, 'LEU57': 10, 'LEU121': 3, 'LEU60': 10, 'GLN61': 10, 'LEU59': 10, 'GLY31': 10, 'GLY58': 10}, 'ARG30': {'ASP56': 10, 'GLY58': 10, 'LEU57': 10, 'GLY28': 10, 'LEU59': 10, 'GLY31': 10, 'GLU32': 10, 'GLU55': 10}, 'GLY31': {'MET123': 2, 'ALA33': 10, 'ASP56': 10, 'VAL27': 10, 'LEU57': 10, 'GLY28': 10, 'THR29': 10, 'LEU26': 10, 'GLU55': 10, 'GLY58': 7, 'GLU32': 10}, 'GLU32': {'ARG30': 10, 'ALA33': 10, 'ARG64': 10, 'VAL27': 10, 'LEU57': 10, 'GLY28': 10, 'GLU34': 10, 'LEU26': 10, 'TRP25': 10, 'GLU55': 10}, 'ALA33': {'VAL53': 10, 'ALA54': 2, 'VAL27': 10, 'LEU57': 10, 'LEU24': 10, 'GLU34': 10, 'LEU26': 10, 'HID51': 2, 'LEU35': 10, 'TRP25': 10, 'GLY31': 10, 'GLU55': 10}, 'GLU34': {'ARG21': 5, 'VAL53': 7, 'GLN23': 4, 'ASP52': 1, 'GLU36': 10, 'LEU24': 10, 'LEU26': 10, 'HID51': 10, 'LEU35': 10, 'TRP25': 10, 'GLU32': 10}, 'LEU35': {'ARG21': 10, 'LEU37': 10, 'ALA33': 10, 'ASP52': 8, 'VAL53': 10, 'VAL83': 9, 'GLN23': 3, 'TYR4': 10, 'LEU57': 1, 'GLU36': 10, 'VAL63': 8, 'LEU24': 10, 'LEU26': 10, 'ILE6': 10, 'HID51': 10, 'TRP25': 10, 'LEU39': 8}, 'GLU36': {'ARG21': 10, 'LEU37': 10, 'GLN38': 10, 'GLU34': 10, 'HID51': 10}, 'LEU37': {'ARG21': 10, 'GLN38': 10, 'LEU24': 10, 'ILE6': 10, 'PHE49': 10, 'LEU35': 10, 'VAL22': 6, 'ASP50': 10, 'LEU39': 10, 'HID51': 10}, 'GLN38': {'ARG21': 10, 'GLU36': 10, 'ARG40': 10, 'PHE49': 10, 'VAL22': 3, 'LEU39': 10}, 'LEU39': {'ARG21': 10, 'LEU37': 10, 'ARG7': 7, 'GLU47': 10, 'GLN23': 3, 'TRP76': 3, 'SER18': 6, 'ASN20': 10, 'VAL8': 10, 'ILE81': 7, 'ARG40': 10, 'LEU24': 10, 'ILE6': 10, 'GLU48': 8, 'LEU35': 8, 'VAL22': 10, 'PHE49': 10, 'PRO41': 10}, 'ARG40': {'ARG21': 10, 'GLN38': 10, 'GLU47': 10, 'SER18': 10, 'ASN20': 10, 'TYR19': 10, 'PHE49': 9, 'VAL22': 10, 'ALA42': 10, 'PRO41': 10}, 'PRO41': {'ARG21': 10, 'GLU45': 7, 'ARG43': 10, 'GLU47': 10, 'TRP76': 10, 'ALA9': 9, 'SER18': 10, 'ASN20': 10, 'VAL8': 10, 'TYR19': 5, 'GLU46': 3, 'PHE49': 1, 'GLY17': 10, 'VAL22': 10, 'LEU39': 10, 'ALA42': 10, 'THR10': 10}, 'ALA42': {'ARG43': 10, 'GLU47': 10, 'TRP76': 4, 'SER18': 10, 'GLY44': 10, 'ARG40': 10, 'GLY11': 4, 'GLY17': 10, 'GLU45': 10, 'THR10': 10}, 'ARG43': {'SER16': 10, 'SER18': 10, 'GLY11': 10, 'GLY44': 10, 'ALA12': 10, 'GLU169': 10, 'TYR19': 1, 'TRP13': 10, 'LEU14': 10, 'GLY17': 10, 'PHE15': 10, 'GLU45': 10, 'THR10': 10, 'PRO41': 10}, 'GLY44': {'SER16': 5, 'GLY11': 10, 'ALA12': 10, 'TRP13': 8, 'GLU46': 10, 'GLY17': 2, 'GLU45': 10, 'ALA42': 10, 'THR10': 10}, 'GLU45': {'ARG43': 10, 'GLU47': 10, 'ALA9': 10, 'VAL8': 10, 'GLY11': 10, 'ALA12': 3, 'GLU46': 10, 'ALA42': 10, 'THR10': 10, 'PRO41': 7}, 'GLU46': {'ARG7': 10, 'ARG80': 10, 'GLU47': 10, 'THR82': 10, 'ALA9': 10, 'VAL8': 10, 'GLY44': 10, 'ILE81': 2, 'GLU48': 10, 'THR10': 10, 'PRO41': 3}, 'GLU47': {'ARG7': 10, 'ALA9': 10, 'VAL8': 10, 'ARG40': 10, 'ILE6': 10, 'GLU48': 10, 'PHE49': 10, 'GLU45': 10, 'LEU39': 10, 'ALA42': 10, 'THR10': 9, 'PRO41': 10}, 'GLU48': {'ARG7': 10, 'VAL8': 10, 'ARG5': 7, 'ILE6': 10, 'PHE49': 10, 'ASP50': 10, 'GLU46': 10, 'LEU39': 8}, 'PHE49': {'LEU37': 10, 'ARG7': 10, 'GLN38': 10, 'GLU47': 10, 'TYR4': 10, 'VAL8': 3, 'ARG5': 10, 'ARG40': 9, 'ILE6': 10, 'ASP50': 10, 'HID51': 10, 'LEU39': 10, 'PRO41': 1}, 'ASP50': {'LEU37': 10, 'ASP52': 10, 'TYR4': 10, 'ARG3': 10, 'ARG5': 10, 'ILE6': 10, 'GLU48': 10, 'HID51': 10}, 'HID51': {'LEU37': 10, 'ALA33': 2, 'ASP52': 10, 'VAL53': 10, 'TYR4': 10, 'GLU36': 10, 'GLY2': 8, 'ARG3': 10, 'ARG5': 10, 'GLU34': 10, 'LEU26': 2, 'ILE6': 10, 'PHE49': 10, 'LEU35': 10}, 'ASP52': {'VAL53': 10, 'ALA54': 10, 'TYR4': 10, 'GLY2': 10, 'ARG3': 10, 'GLU34': 1, 'ASP50': 10, 'LEU35': 8}, 'VAL53': {'MET1': 10, 'PRO86': 3, 'ALA33': 10, 'ALA54': 10, 'ASP56': 10, 'TYR4': 10, 'LEU57': 10, 'GLY2': 10, 'ARG3': 10, 'GLU34': 7, 'LEU26': 10, 'HID51': 10, 'LEU35': 10, 'GLU55': 10}, 'ALA54': {'MET1': 10, 'ALA33': 2, 'ASP52': 10, 'ASP56': 10, 'GLY2': 10, 'GLU55': 10}, 'GLU55': {'MET1': 10, 'ARG30': 10, 'ALA33': 10, 'VAL53': 10, 'ASP56': 10, 'TYR4': 9, 'LEU57': 10, 'GLY2': 2, 'GLY31': 10, 'GLU32': 10}, 'ASP56': {'MET1': 10, 'ARG30': 10, 'VAL53': 10, 'ALA54': 10, 'TYR4': 8, 'LEU57': 10, 'GLY31': 10, 'GLY58': 10}, 'LEU57': {'MET1': 7, 'ARG30': 10, 'ALA33': 10, 'VAL53': 10, 'TYR4': 10, 'VAL27': 10, 'GLY28': 10, 'THR29': 10, 'LEU60': 10, 'GLN61': 2, 'LEU26': 10, 'LEU59': 10, 'LEU35': 1, 'GLY31': 10, 'GLU55': 10, 'GLY58': 10, 'GLU32': 10}, 'GLY58': {'ARG30': 10, 'ASP56': 10, 'TYR4': 5, 'GLY28': 10, 'THR29': 10, 'LEU60': 10, 'GLN61': 3, 'LEU59': 10, 'GLY31': 7}, 'LEU59': {'MET123': 10, 'ARG30': 10, 'VAL92': 4, 'VAL27': 7, 'LEU57': 10, 'GLY28': 10, 'THR29': 10, 'LEU60': 10, 'GLN61': 10, 'ARG114': 4, 'PHE124': 2, 'PHE62': 1}, 'LEU60': {'MET123': 5, 'GLN84': 1, 'PHE94': 10, 'VAL92': 10, 'GLU110': 10, 'GLY85': 2, 'TYR4': 10, 'VAL27': 10, 'GLY28': 10, 'LEU57': 10, 'VAL63': 10, 'THR29': 10, 'GLN61': 10, 'LEU26': 10, 'PHE62': 10, 'VAL83': 10, 'GLY58': 10}, 'GLN61': {'MET123': 10, 'LEU108': 10, 'PHE94': 10, 'VAL92': 10, 'GLU110': 10, 'VAL27': 10, 'LEU57': 2, 'GLY28': 10, 'VAL63': 10, 'THR29': 10, 'PRO109': 10, 'ARG114': 10, 'PHE124': 10, 'LEU59': 10, 'ASP122': 2, 'PHE62': 10, 'GLY58': 3}, 'PHE62': {'GLU110': 10, 'ARG128': 5, 'LEU60': 10, 'LEU59': 1, 'TRP25': 10, 'MET123': 10, 'PHE94': 10, 'GLY28': 10, 'LEU26': 10, 'LEU382': 10, 'VAL92': 3, 'ARG64': 10, 'SER107': 10, 'LEU108': 10, 'VAL27': 10, 'VAL63': 10, 'HID127': 10, 'PRO109': 10, 'PHE124': 10}, 'VAL63': {'LEU108': 10, 'PHE94': 10, 'GLU110': 1, 'LEU65': 10, 'ARG64': 10, 'VAL27': 10, 'ILE81': 10, 'PRO109': 10, 'LEU24': 10, 'LEU60': 10, 'GLN61': 10, 'LEU26': 10, 'SER107': 10, 'ILE6': 9, 'LEU106': 10, 'LEU35': 8, 'TRP25': 10, 'VAL83': 8}, 'ARG64': {'LEU108': 10, 'GLN23': 10, 'LEU65': 10, 'VAL27': 10, 'ARG66': 10, 'HID127': 10, 'LEU24': 10, 'ILE105': 10, 'LYS130': 1, 'LEU26': 10, 'SER107': 10, 'LEU106': 10, 'PHE62': 10, 'TRP25': 10, 'GLU32': 10}, 'LEU65': {'GLN23': 10, 'LEU24': 10, 'TRP25': 10, 'ASP104': 10, 'VAL100': 10, 'VAL8': 1, 'CYS96': 7, 'TRP76': 10, 'ILE81': 10, 'ARG98': 8, 'ILE105': 10, 'SER107': 10, 'LEU106': 10, 'LEU108': 10, 'CYS78': 10, 'ARG66': 10, 'VAL63': 10, 'LYS67': 10, 'VAL22': 10}, 'ARG66': {'ARG21': 10, 'GLN23': 10, 'HIE68': 10, 'ARG64': 10, 'GLN101': 1, 'ASP104': 10, 'VAL100': 10, 'LYS67': 10, 'HID69': 2, 'TRP76': 1, 'LEU24': 10, 'ILE105': 10, 'GLU103': 10, 'LEU106': 10, 'VAL22': 10, 'TRP25': 10, 'GLY102': 10}, 'LYS67': {'ALA75': 10, 'GLN23': 10, 'HIE68': 10, 'HID69': 10, 'ASP104': 1, 'VAL100': 10, 'TYR19': 10, 'ASP74': 10, 'ARG21': 10, 'TRP70': 1, 'TRP76': 10, 'SER18': 10, 'ASN20': 10, 'GLY17': 8, 'LEU106': 9, 'GLY102': 10, 'ASP73': 6, 'LEU65': 10, 'GLN101': 10, 'VAL22': 10}, 'HIE68': {'ARG21': 10, 'TRP70': 10, 'GLN23': 10, 'SER18': 1, 'ARG66': 10, 'ASN20': 10, 'HID69': 10, 'ASP74': 9, 'VAL22': 10}, 'HID69': {'ALA75': 7, 'VAL72': 10, 'TRP70': 10, 'ASP73': 10, 'GLN23': 3, 'GLN101': 10, 'ARG66': 2, 'ASN20': 10, 'VAL100': 1, 'LYS67': 10, 'ASP74': 10, 'GLU103': 10, 'LEU71': 10, 'GLY102': 10}, 'TRP70': {'VAL72': 10, 'ASP73': 10, 'HIE68': 10, 'ASN20': 10, 'LYS67': 1, 'ASP74': 4, 'LEU71': 10}, 'LEU71': {'VAL72': 10, 'HID69': 10, 'ASP73': 10}, 'VAL72': {'ASP74': 10, 'HID69': 10, 'ASP73': 10, 'TRP70': 10}, 'ASP73': {'ALA75': 10, 'TRP70': 10, 'GLN101': 10, 'ASN20': 2, 'HID69': 10, 'LYS67': 6, 'ASP74': 10, 'LEU71': 10, 'GLY102': 5}, 'ASP74': {'ALA75': 10, 'VAL72': 10, 'TRP70': 4, 'HIE68': 9, 'TRP76': 10, 'GLN101': 10, 'SER18': 3, 'ASN20': 10, 'VAL100': 8, 'HID69': 10, 'LYS67': 10, 'TYR19': 9, 'GLY102': 1}, 'ALA75': {'SER16': 9, 'ASP73': 10, 'TRP76': 10, 'SER18': 10, 'GLN101': 10, 'VAL100': 10, 'HID69': 7, 'LYS67': 10, 'TRP99': 10, 'GLY17': 10, 'PHE15': 2, 'PHE77': 10}, 'TRP76': {'LEU24': 8, 'THR10': 10, 'VAL8': 10, 'VAL100': 10, 'ASP74': 10, 'ASP79': 10, 'SER16': 10, 'SER18': 10, 'GLY11': 2, 'ILE81': 10, 'ARG98': 10, 'TRP99': 10, 'GLY17': 10, 'LEU106': 1, 'LEU39': 3, 'ALA42': 4, 'ARG80': 10, 'CYS78': 10, 'LEU65': 10, 'ALA9': 10, 'GLN101': 10, 'ARG66': 1, 'LYS67': 10, 'VAL22': 10, 'PHE77': 10, 'PRO41': 10}, 'PHE77': {'ALA75': 10, 'ARG165': 2, 'PHE15': 9, 'HIE167': 1, 'THR10': 10, 'VAL100': 10, 'ASP79': 10, 'SER16': 10, 'SER18': 7, 'GLY11': 10, 'ARG98': 10, 'TRP13': 10, 'TRP99': 10, 'GLY17': 7, 'CYS78': 10, 'TYR395': 1, 'ALA9': 6, 'PRO162': 10, 'ASN163': 10}, 'CYS78': {'ARG80': 10, 'PHE94': 1, 'TRP76': 10, 'ALA9': 10, 'LEU65': 10, 'VAL8': 8, 'VAL100': 0, 'ILE81': 10, 'ASP79': 10, 'ARG98': 10, 'LEU106': 6, 'ASN163': 1, 'TRP99': 10, 'TYR97': 10, 'LEU108': 10, 'VAL22': 1, 'CYS96': 10, 'THR10': 9}, 'ASP79': {'ARG80': 10, 'PHE94': 9, 'TRP76': 10, 'ALA9': 10, 'CYS96': 10, 'VAL8': 8, 'GLY11': 10, 'ILE81': 10, 'ARG98': 10, 'GLU612': 3, 'TRP13': 2, 'TYR97': 10, 'PHE77': 10, 'PRO95': 10, 'THR10': 10}, 'ARG80': {'ARG7': 10, 'THR82': 10, 'CYS78': 10, 'PHE94': 10, 'TRP76': 10, 'ALA9': 10, 'CYS96': 10, 'VAL8': 10, 'ILE81': 10, 'SER616': 4, 'ALA93': 10, 'LEU108': 8, 'GLU46': 10, 'PRO95': 10, 'THR10': 4}, 'ILE81': {'LEU24': 10, 'ALA93': 10, 'PRO95': 10, 'THR82': 10, 'PHE94': 10, 'VAL8': 10, 'ASP79': 10, 'LEU26': 9, 'ILE6': 10, 'CYS96': 10, 'ARG7': 10, 'VAL92': 10, 'TRP76': 10, 'LEU108': 10, 'VAL83': 10, 'LEU39': 7, 'CYS78': 10, 'LEU65': 10, 'ALA9': 10, 'VAL63': 10, 'GLU46': 2, 'VAL22': 10}, 'THR82': {'ARG7': 10, 'GLN84': 10, 'PHE94': 10, 'VAL92': 10, 'ALA9': 7, 'VAL8': 10, 'ARG5': 10, 'ALA93': 10, 'ILE6': 10, 'GLU46': 10, 'GLU91': 10, 'VAL83': 10, 'PRO95': 1, 'ARG80': 10}, 'VAL83': {'ARG7': 10, 'GLN84': 10, 'PHE94': 10, 'VAL92': 10, 'GLY85': 10, 'ALA90': 4, 'TYR4': 10, 'VAL63': 8, 'ILE81': 10, 'ARG5': 10, 'LEU60': 10, 'LEU24': 1, 'LEU26': 10, 'ALA93': 10, 'ILE6': 10, 'LEU35': 9, 'GLU91': 10}, 'GLN84': {'PRO86': 10, 'THR82': 10, 'CYS89': 10, 'GLY85': 10, 'ALA90': 10, 'TYR4': 10, 'VAL92': 10, 'GLY87': 10, 'ARG3': 10, 'ALA88': 6, 'ARG5': 10, 'LEU60': 1, 'ILE6': 10, 'GLU91': 10}, 'GLY85': {'PRO86': 10, 'CYS89': 10, 'ALA90': 10, 'VAL92': 7, 'TYR4': 10, 'GLY87': 10, 'GLY2': 2, 'ARG3': 10, 'ARG5': 10, 'LEU60': 2, 'VAL83': 10, 'GLU91': 10}, 'PRO86': {'MET1': 10, 'GLN84': 10, 'VAL53': 3, 'CYS89': 10, 'TYR4': 10, 'GLY87': 10, 'GLY2': 10, 'ARG3': 10, 'ALA88': 10}, 'GLY87': {'GLN84': 10, 'CYS89': 10, 'GLY85': 10, 'ARG3': 5, 'ALA88': 10}, 'ALA88': {'PRO86': 10, 'GLN84': 6, 'CYS89': 10, 'ALA90': 10}, 'CYS89': {'PRO86': 10, 'GLN84': 10, 'ALA90': 10, 'GLY85': 10, 'TYR4': 1, 'GLY87': 10, 'GLU91': 10}, 'ALA90': {'GLN84': 10, 'VAL92': 10, 'GLY85': 10, 'ALA88': 10, 'VAL83': 4, 'GLU91': 10}, 'GLU91': {'THR82': 10, 'GLN84': 10, 'CYS89': 10, 'GLY85': 10, 'VAL92': 10, 'ALA93': 10, 'VAL83': 10}, 'VAL92': {'THR82': 10, 'GLN84': 10, 'PHE94': 10, 'GLU110': 10, 'ALA90': 10, 'GLY111': 9, 'TYR4': 8, 'GLY85': 7, 'ILE81': 10, 'LEU60': 10, 'GLN61': 10, 'LEU26': 2, 'LEU59': 4, 'ALA93': 10, 'PHE62': 3, 'VAL83': 10}, 'ALA93': {'ARG80': 10, 'THR82': 10, 'PHE94': 10, 'GLU110': 10, 'GLY111': 10, 'ILE81': 10, 'PRO109': 9, 'THR112': 4, 'VAL83': 10, 'PRO95': 10, 'GLU91': 10}, 'PHE94': {'GLU110': 10, 'LEU60': 10, 'PHE62': 10, 'PRO95': 10, 'THR82': 10, 'GLY111': 10, 'ASP79': 9, 'GLN61': 10, 'LEU26': 10, 'CYS96': 10, 'VAL92': 10, 'ILE81': 10, 'LEU108': 10, 'VAL83': 10, 'ARG80': 10, 'CYS78': 1, 'VAL27': 4, 'VAL63': 10, 'PRO109': 10}, 'PRO95': {'THR82': 1, 'GLU110': 10, 'GLY111': 10, 'ILE81': 10, 'PRO109': 9, 'ASP79': 10, 'LYS387': 10, 'ALA93': 10, 'THR112': 2, 'TYR97': 10, 'LEU108': 10, 'CYS96': 10, 'ARG80': 10}, 'CYS96': {'CYS78': 10, 'PHE94': 10, 'LEU65': 7, 'VAL100': 1, 'ILE81': 10, 'PRO109': 3, 'ASP79': 10, 'ARG98': 10, 'LEU106': 8, 'GLY381': 1, 'TYR97': 10, 'LEU108': 10, 'ARG80': 10}, 'TYR97': {'MET376': 9, 'CYS78': 10, 'TYR614': 6, 'ASP79': 10, 'ARG98': 10, 'GLY381': 9, 'PHE386': 10, 'ASN163': 10, 'TRP99': 10, 'LYS387': 10, 'ILE390': 10, 'GLU612': 10, 'LYS613': 10, 'ARG135': 1, 'PRO391': 4, 'PRO95': 10}, 'ARG98': {'CYS78': 10, 'GLU131': 10, 'ASP134': 10, 'TRP76': 10, 'LEU65': 8, 'GLN101': 10, 'ASP104': 10, 'VAL100': 10, 'TYR139': 1, 'ASP79': 10, 'ARG135': 10, 'ASN163': 10, 'TRP99': 10, 'LEU106': 10, 'LEU108': 10, 'ILE138': 10, 'PHE77': 10, 'CYS96': 10}, 'TRP99': {'ALA75': 10, 'CYS78': 10, 'TRP76': 10, 'GLN101': 10, 'ASP104': 1, 'VAL100': 10, 'PRO162': 10, 'LEU106': 10, 'ARG165': 10, 'ASN163': 10, 'TYR97': 10, 'ILE138': 3, 'PHE77': 10, 'GLY102': 4}, 'VAL100': {'ALA75': 10, 'CYS78': 10, 'LEU65': 10, 'TRP76': 10, 'ARG66': 10, 'GLN101': 10, 'ASP104': 8, 'LYS67': 10, 'HID69': 1, 'ASP74': 8, 'ARG98': 10, 'LEU106': 10, 'VAL22': 7, 'PHE77': 10, 'CYS96': 1, 'GLY102': 10}, 'GLN101': {'ALA75': 10, 'ASP73': 10, 'TRP76': 10, 'ARG66': 1, 'ASP104': 10, 'HID69': 10, 'LYS67': 10, 'ASP74': 10, 'ARG98': 10, 'GLU103': 10, 'TRP99': 10, 'LEU106': 9, 'GLY102': 10}, 'GLY102': {'ASP73': 5, 'ARG66': 10, 'ASP104': 10, 'VAL100': 10, 'HID69': 10, 'LYS67': 10, 'ASP74': 1, 'GLU103': 10, 'TRP99': 4}, 'GLU103': {'GLN101': 10, 'ARG66': 10, 'ASP104': 10, 'HID69': 10, 'ILE105': 10}, 'ASP104': {'LEU65': 10, 'GLN101': 10, 'ARG66': 10, 'VAL100': 8, 'LYS67': 1, 'ARG98': 10, 'ILE105': 10, 'TRP99': 1, 'LEU106': 10, 'ILE138': 1, 'GLY102': 10}, 'ILE105': {'GLN23': 8, 'LEU65': 10, 'ARG64': 10, 'ARG66': 10, 'GLU103': 10, 'SER107': 10, 'LEU106': 10, 'TRP25': 10}, 'LEU106': {'GLU131': 3, 'CYS78': 6, 'ASP134': 3, 'LEU65': 10, 'ARG64': 10, 'ARG66': 10, 'GLN101': 9, 'VAL63': 10, 'LYS67': 9, 'VAL100': 10, 'ASP104': 10, 'TRP76': 1, 'ARG98': 10, 'TRP99': 10, 'SER107': 10, 'LEU108': 10, 'TRP25': 1, 'CYS96': 8}, 'SER107': {'LEU108': 10, 'GLU131': 10, 'LEU65': 10, 'ARG64': 10, 'VAL27': 2, 'VAL63': 10, 'HID127': 10, 'PRO109': 10, 'ILE105': 10, 'LYS130': 2, 'PHE62': 10}, 'LEU108': {'CYS78': 10, 'PHE94': 10, 'GLU110': 10, 'LEU65': 10, 'ARG64': 10, 'CYS96': 10, 'VAL63': 10, 'ILE81': 10, 'PRO109': 10, 'LEU24': 1, 'ARG98': 10, 'GLN61': 10, 'LEU106': 10, 'PHE62': 10, 'PRO95': 10, 'ARG80': 8}, 'PRO109': {'LEU382': 10, 'GLU131': 3, 'PHE94': 10, 'GLU110': 10, 'GLY111': 10, 'CYS96': 3, 'VAL63': 10, 'HID127': 6, 'GLY381': 4, 'GLN61': 10, 'ARG114': 8, 'PHE124': 10, 'ALA93': 9, 'SER107': 10, 'THR112': 1, 'PHE62': 10, 'PRO95': 9}, 'GLU110': {'LEU108': 10, 'LEU382': 10, 'PHE94': 10, 'VAL92': 10, 'GLY111': 10, 'VAL63': 1, 'LEU60': 10, 'GLN61': 10, 'ARG114': 10, 'PHE124': 10, 'ALA93': 10, 'THR112': 10, 'PHE62': 10, 'PRO95': 10}, 'GLY111': {'PHE94': 10, 'VAL92': 9, 'PHE615': 8, 'PRO109': 10, 'ARG114': 10, 'LYS387': 10, 'ALA93': 10, 'THR112': 10, 'ALA113': 10, 'PRO95': 10}, 'THR112': {'LEU382': 10, 'GLU110': 10, 'PHE615': 8, 'PHE482': 7, 'PRO109': 1, 'ARG114': 10, 'LYS387': 2, 'ALA93': 4, 'PHE124': 1, 'ALA113': 10, 'PRO95': 2}, 'ALA113': {'ILE385': 1, 'LEU382': 10, 'GLY111': 10, 'PHE615': 10, 'PHE482': 10, 'HIP383': 10, 'GLY381': 10, 'ARG114': 10, 'LYS387': 10, 'PRO384': 10, 'LEU115': 10}, 'ARG114': {'LEU382': 10, 'GLU110': 10, 'GLY111': 10, 'PHE482': 4, 'HIP383': 10, 'PRO109': 8, 'ARG128': 10, 'GLN61': 10, 'PHE124': 10, 'LEU59': 4, 'THR112': 10, 'PRO384': 10, 'PRO116': 10, 'LEU115': 10}, 'LEU115': {'TYR483': 10, 'ILE488': 10, 'LEU382': 1, 'ASP492': 10, 'ALA113': 10, 'PHE482': 10, 'HIP383': 10, 'LEU495': 2, 'GLY491': 1, 'ARG128': 10, 'GLY117': 10, 'PRO384': 10, 'PRO116': 10}, 'PRO116': {'GLH494': 10, 'TYR483': 5, 'PRO493': 10, 'ASP492': 10, 'GLY491': 10, 'HIP383': 10, 'LEU495': 6, 'ARG128': 10, 'GLY117': 10, 'ARG114': 10, 'PRO384': 9, 'ASP118': 10}, 'GLY117': {'ASN119': 10, 'ARG128': 10, 'PHE124': 2, 'ASP122': 8, 'GLN125': 3, 'LEU115': 10, 'ASP118': 10}, 'ASP118': {'ASN119': 10, 'ARG128': 9, 'ALA120': 9, 'ASP122': 10, 'PRO116': 10}, 'ASN119': {'LEU121': 10, 'ARG128': 1, 'GLY117': 10, 'ALA120': 10, 'ASP122': 10, 'GLN125': 10}, 'ALA120': {'LYS126': 10, 'LEU121': 10, 'ASP122': 10, 'GLN125': 10, 'ASP118': 9}, 'LEU121': {'MET123': 10, 'ASN119': 10, 'LYS126': 5, 'THR29': 3, 'ASP122': 10, 'GLN125': 7}, 'ASP122': {'MET123': 10, 'ASN119': 10, 'LYS126': 10, 'HID127': 1, 'ARG128': 10, 'GLY117': 8, 'GLN61': 2, 'PHE124': 10, 'ALA120': 10, 'GLN125': 10, 'ASP118': 10}, 'MET123': {'LYS126': 10, 'VAL27': 10, 'GLY28': 10, 'HID127': 10, 'LEU121': 10, 'THR29': 10, 'ARG128': 6, 'LEU60': 5, 'GLN61': 10, 'PHE124': 10, 'LEU59': 10, 'PHE62': 10, 'GLY31': 2, 'GLN125': 10}, 'PHE124': {'GLU129': 7, 'LEU382': 10, 'GLU110': 10, 'LYS126': 10, 'HID127': 10, 'PRO109': 10, 'ARG128': 10, 'GLY117': 2, 'GLN61': 10, 'ARG114': 10, 'ASP122': 10, 'LEU59': 2, 'THR112': 1, 'PHE62': 10, 'GLN125': 10}, 'GLN125': {'MET123': 10, 'ASN119': 10, 'GLU129': 10, 'LYS126': 10, 'HID127': 10, 'LEU121': 7, 'ARG128': 10, 'GLY117': 3, 'ALA120': 10, 'ASP122': 10}, 'LYS126': {'MET123': 10, 'GLU129': 8, 'HID127': 10, 'LEU121': 5, 'ARG128': 10, 'LYS130': 10, 'PHE124': 10, 'ALA120': 10, 'ASP122': 10}, 'HID127': {'MET123': 10, 'GLU129': 10, 'GLU131': 10, 'LEU382': 6, 'ARG64': 10, 'VAL27': 7, 'PRO109': 6, 'ARG128': 10, 'LYS130': 10, 'PHE124': 10, 'SER107': 10, 'ASP122': 1, 'PHE62': 10, 'LEU132': 10, 'GLN125': 10}, 'ARG128': {'ASN119': 1, 'GLU129': 10, 'LYS126': 10, 'PRO380': 10, 'ARG114': 10, 'PHE62': 5, 'GLH494': 10, 'MET123': 6, 'GLU131': 10, 'LYS133': 10, 'LYS130': 10, 'LEU132': 10, 'PRO116': 10, 'LEU115': 10, 'ASP118': 9, 'LEU382': 10, 'HIP383': 2, 'GLY117': 10, 'ASP122': 10, 'GLN125': 10, 'PHE124': 10}, 'GLU129': {'GLU131': 10, 'LYS126': 8, 'LYS133': 10, 'ASP134': 10, 'HID127': 10, 'LYS130': 10, 'PHE124': 7, 'LEU132': 10, 'GLN125': 10}, 'LYS130': {'GLU131': 10, 'LYS126': 10, 'LYS133': 10, 'ASP134': 10, 'ARG64': 1, 'HID127': 10, 'ARG128': 10, 'ARG135': 5, 'SER107': 2, 'LEU132': 10}, 'GLU131': {'GLN136': 10, 'GLU129': 10, 'LEU382': 6, 'LYS133': 10, 'ASP134': 10, 'PRO380': 10, 'HID127': 10, 'PRO109': 3, 'ARG128': 10, 'GLY381': 10, 'ARG98': 10, 'ARG135': 10, 'SER107': 10, 'LEU106': 3, 'LEU132': 10}, 'LEU132': {'GLN136': 10, 'GLH494': 10, 'GLU129': 10, 'GLN137': 10, 'LYS133': 10, 'ASP134': 10, 'PRO380': 10, 'ALA497': 2, 'HID127': 10, 'ARG128': 10, 'ARG135': 10, 'LYS130': 10, 'LEU379': 8, 'ARG377': 10, 'CYS378': 10}, 'LYS133': {'GLN136': 10, 'GLU129': 10, 'GLU131': 10, 'GLN137': 10, 'ASP134': 10, 'ARG128': 10, 'ARG135': 10, 'LYS130': 10, 'ILE138': 10}, 'ASP134': {'GLN136': 10, 'GLU129': 10, 'GLU131': 10, 'GLN137': 10, 'TYR139': 10, 'ILE138': 10, 'ARG98': 10, 'ARG135': 10, 'LYS130': 10, 'LEU106': 3, 'LEU132': 10}, 'ARG135': {'MET376': 10, 'GLN136': 10, 'TYR97': 1, 'GLU131': 10, 'LYS133': 10, 'GLN137': 10, 'PRO380': 10, 'TYR139': 10, 'ILE138': 10, 'GLY381': 10, 'ARG98': 10, 'LYS130': 5, 'PHE386': 1, 'LEU379': 10, 'ARG377': 10, 'CYS140': 6, 'LEU132': 10, 'CYS378': 10}, 'GLN136': {'GLU501': 10, 'GLN137': 10, 'GLU131': 10, 'LYS133': 10, 'ASP134': 10, 'TYR139': 10, 'PRO380': 1, 'ILE138': 10, 'TRP141': 1, 'ARG377': 10, 'CYS140': 10, 'LEU132': 10, 'CYS378': 10}, 'GLN137': {'LYS133': 10, 'ASP134': 10, 'TYR139': 10, 'ILE138': 10, 'ARG135': 10, 'ARG377': 10, 'CYS140': 10, 'LEU132': 10}, 'ILE138': {'GLN136': 10, 'LYS133': 10, 'ASP134': 10, 'TYR139': 10, 'ASP104': 1, 'PRO162': 10, 'ARG98': 10, 'ARG135': 10, 'TRP99': 3, 'PRO161': 10, 'CYS140': 10, 'ILE152': 10}, 'TYR139': {'MET376': 10, 'GLN136': 10, 'MET164': 10, 'GLN137': 10, 'ASP134': 10, 'ALA372': 3, 'PRO162': 5, 'ARG135': 10, 'TRP141': 10, 'ARG98': 1, 'ALA153': 6, 'PRO161': 10, 'ASN163': 10, 'ARG377': 10, 'CYS140': 10, 'MET397': 10, 'ILE152': 10, 'VAL373': 10}, 'CYS140': {'GLN136': 10, 'THR151': 10, 'GLN137': 10, 'ARG135': 6, 'TRP141': 10, 'ALA142': 10, 'ALA153': 10, 'PRO161': 4, 'ARG377': 10, 'ILE138': 10, 'ILE152': 10, 'VAL373': 10}, 'TRP141': {'THR151': 10, 'GLU501': 10, 'GLN136': 1, 'TYR139': 10, 'ALA374': 3, 'THR143': 10, 'ALA142': 10, 'ALA153': 10, 'ARG500': 7, 'ARG377': 10, 'VAL505': 10, 'GLY506': 6, 'ILE152': 10, 'LEU150': 10, 'VAL373': 10}, 'ALA142': {'THR151': 10, 'LYS145': 2, 'ALA154': 8, 'THR143': 10, 'TRP144': 10, 'ALA153': 10, 'CYS140': 10, 'PRO149': 7, 'ILE152': 10, 'LEU150': 10, 'VAL373': 3}, 'THR143': {'GLU146': 10, 'THR151': 8, 'LYS145': 10, 'GLY147': 9, 'TRP144': 10, 'TRP141': 10, 'LEU148': 8, 'PRO149': 2, 'LEU150': 10}, 'TRP144': {'GLU146': 10, 'THR151': 10, 'ILE408': 10, 'LYS145': 10, 'ARG404': 10, 'LEU361': 8, 'VAL426': 2, 'LEU429': 10, 'LEU366': 10, 'HID425': 6, 'LEU362': 1, 'ALA142': 10, 'LEU148': 10, 'PRO149': 10, 'LEU150': 10}, 'LYS145': {'GLU146': 10, 'ARG430': 7, 'PRO149': 1, 'ILE408': 9, 'GLY147': 10, 'SER409': 7, 'ARG404': 3, 'THR143': 10, 'LEU429': 6, 'HID425': 10, 'ALA142': 2, 'LEU148': 10, 'VAL426': 10, 'LEU150': 7}, 'GLU146': {'ARG430': 10, 'CYS508': 4, 'GLY147': 10, 'THR143': 10, 'TRP144': 10, 'LEU148': 10, 'LEU150': 10}, 'GLY147': {'ARG430': 10, 'CYS508': 10, 'ALA433': 1, 'LYS145': 10, 'THR143': 9, 'LEU148': 10, 'PRO149': 10, 'LEU150': 6}, 'LEU148': {'GLU146': 10, 'ARG430': 10, 'CYS508': 10, 'THR151': 5, 'GLY506': 1, 'ILE408': 1, 'ALA433': 10, 'LYS145': 10, 'LEU362': 9, 'GLN427': 1, 'VAL426': 10, 'TRP144': 10, 'LEU366': 10, 'LEU429': 10, 'THR143': 8, 'HID425': 6, 'PRO149': 10, 'LEU150': 10}, 'PRO149': {'CYS508': 10, 'ARG513': 9, 'LYS145': 1, 'LEU429': 8, 'VAL505': 4, 'LEU362': 10, 'ARG404': 8, 'TRP144': 10, 'LEU507': 10, 'LEU436': 6, 'ALA433': 10, 'GLY147': 10, 'VAL370': 10, 'GLY506': 10, 'LEU150': 10, 'THR151': 10, 'VAL367': 6, 'THR143': 2, 'LEU366': 10, 'ALA142': 7}, 'LEU150': {'GLU146': 10, 'THR151': 10, 'CYS508': 10, 'LYS145': 7, 'GLY147': 6, 'THR143': 10, 'TRP144': 10, 'LEU507': 10, 'TRP141': 10, 'LEU366': 1, 'ALA142': 10, 'VAL370': 10, 'LEU148': 10, 'VAL505': 10, 'GLY506': 10, 'ILE152': 10, 'VAL373': 10}, 'THR151': {'ASN400': 10, 'GLY506': 9, 'LEU148': 5, 'ALA154': 6, 'ARG404': 10, 'THR143': 8, 'TRP144': 10, 'LEU366': 10, 'TRP141': 10, 'ALA142': 10, 'ALA153': 10, 'GLU369': 10, 'VAL370': 10, 'CYS140': 10, 'PRO149': 10, 'ILE152': 10, 'MET397': 4, 'VAL373': 10}, 'ILE152': {'ASP159': 6, 'ASP155': 2, 'TRP141': 10, 'ALA153': 10, 'PRO161': 10, 'GLU369': 10, 'CYS140': 10, 'ILE138': 10, 'ALA154': 10, 'ARG404': 10, 'MET397': 10, 'THR401': 10, 'LEU150': 10, 'LEU160': 10, 'MET164': 7, 'ASN400': 5, 'TYR139': 10, 'ALA142': 10, 'GLU398': 1, 'VAL373': 10}, 'ALA153': {'THR151': 10, 'TYR139': 6, 'ASP159': 10, 'ALA154': 10, 'ASP155': 10, 'TRP141': 10, 'ALA142': 10, 'PRO161': 2, 'LEU160': 2, 'CYS140': 10, 'VAL373': 5}, 'ALA154': {'THR151': 6, 'ASP159': 10, 'THR405': 10, 'ASP155': 10, 'ARG404': 10, 'ARG156': 10, 'ALA142': 8, 'PRO161': 10, 'THR401': 10, 'MET397': 3, 'ILE152': 10, 'LEU160': 10, 'LYS157': 1}, 'ASP155': {'GLN406': 5, 'ASP159': 10, 'THR405': 10, 'ARG404': 8, 'ARG156': 10, 'ALA153': 10, 'THR401': 10, 'ILE152': 2, 'LYS157': 10}, 'ARG156': {'ASP159': 10, 'THR405': 10, 'ALA154': 10, 'ARG402': 8, 'ARG165': 1, 'THR401': 10, 'ASP158': 10, 'LEU160': 10, 'LYS157': 0}, 'LYS157': {'PHE166': 10, 'ASP159': 10, 'ASP155': 10, 'ARG402': 8, 'ALA154': 1, 'ARG165': 10, 'GLU398': 10, 'ASP158': 10, 'LEU160': 10, 'ARG171': 10}, 'ASP158': {'ASP159': 10, 'ARG156': 10, 'PRO162': 2, 'ARG165': 10, 'LEU160': 10}, 'ASP159': {'ASP155': 10, 'ALA154': 10, 'ARG156': 10, 'ARG165': 10, 'ALA153': 10, 'PRO161': 10, 'ILE152': 6, 'LEU160': 10, 'LYS157': 10}, 'LEU160': {'MET164': 10, 'PHE166': 10, 'ALA154': 10, 'ARG156': 10, 'PRO162': 10, 'ARG165': 10, 'ALA153': 2, 'PRO161': 10, 'MET397': 10, 'THR401': 10, 'GLU398': 10, 'ASP158': 10, 'ILE152': 10, 'LYS157': 10}, 'PRO161': {'MET376': 1, 'MET164': 10, 'TYR139': 10, 'ASP159': 10, 'ALA154': 10, 'PRO162': 10, 'ARG165': 10, 'ALA153': 2, 'ASN163': 10, 'MET397': 10, 'CYS140': 4, 'ILE138': 10, 'ILE152': 10}, 'PRO162': {'MET164': 10, 'TYR395': 2, 'TYR139': 5, 'ARG165': 10, 'ASN163': 10, 'TRP99': 10, 'ILE138': 10, 'ASP158': 2, 'PHE77': 10, 'LEU160': 10}, 'ASN163': {'MET376': 10, 'MET164': 10, 'CYS78': 1, 'ARG394': 10, 'TYR395': 10, 'TYR139': 10, 'ARG98': 10, 'ARG165': 10, 'ILE390': 8, 'TRP99': 10, 'PRO161': 10, 'TYR97': 10, 'PHE77': 10}, 'MET164': {'MET376': 10, 'PHE166': 10, 'ARG394': 10, 'TYR395': 10, 'TYR139': 10, 'ALA372': 10, 'PRO162': 10, 'THR396': 7, 'ARG165': 10, 'ILE390': 9, 'PRO161': 10, 'ILE393': 10, 'MET397': 10, 'GLU398': 10, 'PHE386': 1, 'ILE152': 7, 'LEU160': 10}, 'ARG165': {'PHE166': 10, 'TYR395': 10, 'ASP159': 10, 'ARG156': 1, 'PRO162': 10, 'ASN163': 10, 'TRP99': 10, 'PRO161': 10, 'HIE167': 10, 'GLU398': 10, 'MET397': 1, 'ASP158': 10, 'PHE77': 2, 'LEU160': 10, 'LYS157': 10}, 'PHE166': {'MET164': 10, 'PHE174': 10, 'GLU175': 1, 'TYR395': 10, 'ILE399': 10, 'THR662': 10, 'ARG402': 8, 'LYS170': 10, 'GLU168': 10, 'LEU160': 10, 'GLU398': 10, 'HIE167': 10, 'LYS157': 10, 'ARG171': 10}, 'HIE167': {'ARG394': 10, 'TYR395': 10, 'GLU169': 10, 'LYS170': 10, 'TRP13': 10, 'GLU168': 10, 'ARG165': 10, 'LEU14': 10, 'ASP603': 10, 'LEU172': 10, 'PHE77': 1, 'ARG171': 10}, 'GLU168': {'ASP173': 10, 'PHE166': 10, 'GLU169': 10, 'LYS170': 10, 'TRP13': 10, 'LEU14': 10, 'PHE15': 10, 'LEU172': 10, 'ARG171': 10}, 'GLU169': {'ASP173': 10, 'ARG43': 10, 'PHE174': 10, 'LYS170': 10, 'TRP13': 5, 'LEU14': 10, 'LEU172': 10, 'HIE167': 10, 'ARG171': 10}, 'LYS170': {'ASP173': 10, 'PHE166': 10, 'PHE174': 10, 'GLU175': 10, 'ARG394': 10, 'TYR395': 10, 'THR662': 10, 'ILE399': 3, 'ARG599': 10, 'PRO602': 10, 'GLU168': 10, 'GLN601': 10, 'ASP603': 10, 'LEU172': 10, 'HIE167': 10, 'ARG171': 10}, 'ARG171': {'ASP173': 10, 'PHE166': 10, 'PHE174': 10, 'GLU175': 10, 'LYS179': 3, 'GLU169': 10, 'TRP176': 10, 'GLU398': 2, 'GLU168': 10, 'PHE15': 2, 'LEU172': 10, 'HIE167': 10, 'LYS157': 10}, 'LEU172': {'THR177': 10, 'ASP173': 10, 'PHE174': 10, 'GLU175': 10, 'GLU169': 10, 'LYS170': 10, 'TRP176': 10, 'GLU168': 10, 'LEU14': 10, 'PHE15': 7, 'HIE167': 10}, 'ASP173': {'THR177': 10, 'PHE174': 10, 'GLU175': 10, 'ARG599': 10, 'LYS170': 10, 'GLU169': 10, 'TRP176': 10, 'PRO602': 10, 'GLU168': 10, 'GLN601': 10, 'HIE596': 9, 'LEU178': 10, 'ARG171': 10}, 'PHE174': {'THR177': 10, 'LEU178': 10, 'PHE166': 10, 'GLU175': 10, 'ILE399': 10, 'THR662': 8, 'ARG599': 10, 'ARG402': 9, 'LYS170': 10, 'GLU169': 10, 'TRP176': 10, 'LEU597': 9, 'GLN601': 10, 'HIE596': 10, 'LEU172': 10, 'LYS179': 10, 'ARG171': 10}, 'GLU175': {'THR177': 10, 'ASP173': 10, 'LEU178': 10, 'PHE166': 1, 'LYS170': 10, 'ALA180': 10, 'TRP176': 10, 'HIE596': 8, 'LEU172': 10, 'LYS179': 10, 'ARG171': 10}, 'TRP176': {'THR177': 10, 'ASP173': 10, 'LEU178': 10, 'PHE174': 10, 'TRP595': 5, 'ARG599': 5, 'ALA180': 10, 'GLY181': 10, 'LEU183': 1, 'PRO602': 2, 'LEU172': 10, 'LYS179': 10, 'ARG171': 10}, 'THR177': {'ASP173': 10, 'ALA182': 10, 'LEU178': 10, 'PHE174': 10, 'GLU175': 10, 'TRP595': 10, 'ARG599': 10, 'ALA180': 10, 'GLY181': 10, 'HIE596': 10, 'LEU172': 10, 'LYS179': 10}, 'LEU178': {'ASP173': 10, 'ALA182': 10, 'ILE593': 10, 'PHE174': 10, 'GLU175': 10, 'ALA592': 4, 'TRP176': 10, 'GLY181': 10, 'LEU183': 10, 'ILE413': 5, 'HIE596': 10, 'ALA180': 10, 'LYS179': 10}, 'LYS179': {'THR177': 10, 'ALA182': 10, 'PHE174': 10, 'GLU175': 10, 'GLU184': 10, 'TRP176': 0, 'GLY181': 10, 'LEU183': 10, 'ALA180': 10, 'ARG171': 3}, 'ALA180': {'THR177': 10, 'ALA182': 10, 'TRP595': 3, 'GLU175': 10, 'GLU184': 10, 'TRP176': 10, 'GLY181': 10, 'LEU183': 10, 'LEU178': 10, 'MET185': 10}, 'GLY181': {'THR177': 10, 'ALA182': 10, 'TRP595': 8, 'GLU184': 10, 'ALA592': 10, 'TRP176': 10, 'LEU183': 10, 'ALA186': 10, 'HIE596': 5, 'LEU178': 10, 'MET185': 10, 'LYS179': 10}, 'ALA182': {'THR177': 10, 'LEU187': 10, 'ILE593': 1, 'ARG189': 2, 'GLU184': 10, 'ALA592': 10, 'LEU589': 6, 'ALA180': 10, 'LEU183': 10, 'ALA186': 10, 'LEU178': 10, 'MET185': 10, 'LYS179': 10}, 'LEU183': {'LEU187': 10, 'LYS188': 9, 'GLU184': 10, 'ALA180': 10, 'TRP176': 1, 'GLY181': 10, 'ALA186': 10, 'LEU178': 10, 'MET185': 10, 'LYS179': 10}, 'GLU184': {'LEU187': 10, 'ALA182': 10, 'ARG189': 10, 'LYS188': 10, 'ALA592': 9, 'GLN210': 9, 'MET591': 2, 'GLY209': 10, 'GLY181': 10, 'ALA186': 10, 'TRP208': 8, 'CYS588': 4, 'ALA180': 10, 'MET185': 10, 'LYS179': 10}, 'MET185': {'LEU187': 10, 'ALA182': 10, 'ARG189': 10, 'LYS188': 10, 'LYS416': 1, 'ALA592': 10, 'LEU589': 10, 'VAL190': 10, 'GLY181': 10, 'ALA186': 10, 'LEU183': 10, 'GLN586': 8, 'CYS588': 10, 'ALA180': 10, 'VAL584': 1, 'ARG585': 10}, 'ALA186': {'LEU187': 10, 'ALA182': 10, 'ARG189': 10, 'LYS188': 10, 'GLU184': 10, 'VAL190': 10, 'GLY181': 10, 'LEU183': 10, 'TYR191': 10}, 'LEU187': {'ALA182': 10, 'ARG189': 10, 'LYS188': 10, 'GLU184': 10, 'VAL190': 10, 'LEU183': 10, 'THR192': 10, 'TYR191': 10, 'MET185': 10}, 'LYS188': {'LEU193': 10, 'ARG189': 10, 'PHE207': 10, 'ALA592': 1, 'GLN210': 1, 'LEU183': 9, 'ALA186': 10, 'TRP208': 10, 'CYS588': 10, 'GLN205': 9, 'VAL584': 9, 'MET591': 10, 'VAL190': 10, 'MET185': 10, 'GLU184': 10, 'GLY209': 10, 'THR192': 10, 'TYR191': 10, 'ILE206': 10}, 'ARG189': {'LEU187': 10, 'LEU193': 10, 'ALA182': 2, 'LEU194': 3, 'GLU184': 10, 'VAL190': 10, 'THR192': 10, 'ALA186': 10, 'CYS588': 10, 'TYR191': 10, 'ILE206': 1, 'MET185': 10, 'VAL584': 10, 'ARG585': 10}, 'VAL190': {'LEU187': 10, 'LEU193': 10, 'LYS188': 10, 'LEU194': 10, 'THR192': 10, 'ALA186': 10, 'SER195': 1, 'TYR191': 10, 'MET185': 10}, 'TYR191': {'LEU187': 10, 'LEU193': 10, 'SER196': 2, 'ARG189': 10, 'LYS188': 10, 'LEU194': 10, 'THR192': 10, 'ALA186': 10, 'SER195': 10, 'GLN205': 10, 'ILE206': 10, 'ASP202': 1}, 'THR192': {'LEU187': 10, 'LEU193': 10, 'SER196': 7, 'ARG189': 10, 'TRP197': 10, 'LYS188': 10, 'LEU194': 10, 'VAL190': 10, 'GLN205': 9, 'CYS588': 7, 'SER195': 10, 'ILE206': 10, 'VAL584': 10, 'ASP202': 4}, 'LEU193': {'ARG189': 10, 'LYS188': 10, 'LEU194': 10, 'VAL190': 10, 'SER195': 10, 'TYR191': 10}, 'LEU194': {'SER196': 10, 'ARG189': 3, 'VAL190': 10, 'THR192': 10, 'SER195': 10, 'TYR191': 10}, 'SER195': {'LEU193': 10, 'SER196': 10, 'TRP197': 10, 'VAL190': 1, 'THR192': 10, 'GLN205': 9, 'TYR191': 10, 'ASP202': 6}, 'SER196': {'TRP197': 10, 'LEU194': 10, 'ASN198': 10, 'GLN205': 1, 'CYS199': 10, 'THR192': 7, 'TYR191': 2, 'ASP202': 10}, 'TRP197': {'LEU581': 10, 'GLN205': 9, 'PRO582': 10, 'CYS588': 3, 'VAL584': 10, 'VAL554': 4, 'MET591': 4, 'MET578': 10, 'ARG562': 10, 'CYS199': 10, 'ASP204': 1, 'ALA587': 10, 'ASP583': 10, 'PHE203': 10, 'ASN198': 10, 'THR192': 10, 'SER195': 10, 'ILE206': 10, 'ASP202': 10}, 'ASN198': {'SER196': 10, 'MET574': 10, 'ALA575': 10, 'CYS199': 10, 'MET578': 10, 'LEU200': 10, 'ASP202': 10}, 'CYS199': {'SER196': 10, 'MET574': 10, 'TRP197': 10, 'PHE203': 10, 'ALA575': 4, 'GLU201': 10, 'ASP204': 4, 'MET578': 10, 'LEU200': 10, 'ASP202': 10}, 'LEU200': {'PRO555': 10, 'TRP553': 10, 'VAL577': 10, 'MET574': 10, 'LEU581': 10, 'PHE203': 10, 'GLU201': 10, 'ASN198': 10, 'LEU338': 10, 'GLN223': 3, 'GLN205': 5, 'LEU341': 9, 'ASP204': 10, 'ARG219': 3, 'MET578': 10, 'TRP222': 10, 'ASP202': 10}, 'GLU201': {'TRP553': 4, 'MET574': 1, 'PHE203': 10, 'GLN205': 10, 'CYS199': 10, 'ASP204': 10, 'ARG219': 8, 'ILE206': 5, 'ASP202': 10}, 'ASP202': {'SER196': 10, 'TRP197': 10, 'PHE203': 10, 'PHE207': 6, 'ASN198': 10, 'GLN205': 10, 'CYS199': 10, 'SER195': 6, 'ASP204': 10, 'THR192': 4, 'TYR191': 1, 'MET578': 10, 'LEU200': 10, 'ILE206': 10}, 'PHE203': {'LEU581': 10, 'PHE207': 10, 'GLN205': 10, 'PRO582': 10, 'TRP208': 2, 'ARG219': 1, 'VAL584': 1, 'VAL554': 10, 'TRP197': 10, 'MET591': 10, 'MET578': 10, 'TRP553': 10, 'PRO558': 3, 'GLU201': 10, 'CYS199': 10, 'ASP204': 10, 'ALA587': 10, 'ASP583': 3, 'PRO555': 10, 'LEU200': 10, 'ILE206': 10}, 'ASP204': {'TRP553': 10, 'TRP197': 1, 'PHE207': 10, 'GLU201': 10, 'MET591': 1, 'GLN205': 10, 'TRP208': 10, 'CYS199': 4, 'ARG219': 10, 'LEU200': 10, 'ILE206': 10, 'ASP202': 10}, 'GLN205': {'SER196': 1, 'TRP197': 9, 'PHE203': 10, 'LYS188': 9, 'PHE207': 10, 'GLU201': 10, 'THR192': 9, 'TRP208': 10, 'SER195': 9, 'TYR191': 10, 'LEU200': 5, 'ILE206': 10, 'ASP202': 10}, 'ILE206': {'PRO555': 1, 'TRP553': 6, 'VAL554': 9, 'ARG189': 1, 'TRP197': 10, 'LYS188': 10, 'PHE203': 10, 'PHE207': 10, 'GLU201': 5, 'MET591': 10, 'THR192': 10, 'TRP208': 10, 'ASP204': 10, 'CYS588': 10, 'TYR191': 10, 'ALA587': 10, 'VAL584': 10, 'ASP202': 10}, 'PHE207': {'GLU216': 10, 'TRP553': 10, 'VAL554': 10, 'ALA215': 10, 'LEU548': 1, 'PHE203': 10, 'LYS188': 10, 'GLN210': 4, 'MET591': 10, 'GLY209': 10, 'LYS211': 10, 'GLN205': 10, 'TRP208': 10, 'ASP204': 10, 'ARG219': 10, 'ALA552': 10, 'VAL218': 1, 'ASP202': 6}, 'TRP208': {'PHE203': 2, 'LYS188': 10, 'GLU184': 8, 'GLN210': 10, 'GLY209': 10, 'LYS211': 10, 'GLN205': 10, 'ASP204': 10, 'ILE206': 10}, 'GLY209': {'LYS188': 10, 'GLU184': 10, 'PHE207': 10, 'GLN210': 10, 'LYS211': 10}, 'GLN210': {'LEU548': 10, 'TRP595': 10, 'SER212': 10, 'LYS188': 1, 'GLU184': 9, 'PHE207': 4, 'LYS211': 10, 'TRP208': 10, 'ALA552': 10}, 'LYS211': {'GLU216': 10, 'LEU548': 10, 'TRP553': 2, 'ALA213': 10, 'ALA215': 10, 'TRP595': 6, 'SER212': 10, 'PHE207': 10, 'GLY209': 10, 'TRP208': 10, 'ARG219': 7, 'ALA552': 10}, 'SER212': {'GLU216': 10, 'LEU548': 10, 'ALA215': 10, 'ALA213': 10, 'LEU214': 10, 'TRP595': 1, 'GLN210': 10, 'LYS217': 8, 'ALA552': 2, 'ASP549': 10}, 'ALA213': {'GLU216': 10, 'ALA215': 10, 'LEU214': 10, 'LYS217': 10, 'LYS211': 10, 'VAL218': 10, 'ASP549': 1}, 'LEU214': {'GLU216': 10, 'ALA215': 10, 'TRP550': 10, 'PRO648': 5, 'SER212': 10, 'LEU227': 10, 'TYR230': 10, 'GLY546': 3, 'LYS217': 10, 'GLY235': 10, 'TYR651': 10, 'ARG219': 10, 'VAL218': 10, 'ASP549': 10}, 'ALA215': {'GLU216': 10, 'LEU548': 10, 'TRP553': 10, 'ALA213': 10, 'TRP550': 8, 'SER212': 10, 'PHE207': 10, 'LYS217': 10, 'LYS211': 10, 'ARG219': 10, 'ALA552': 10, 'GLN220': 10, 'VAL218': 10, 'ASP549': 10}, 'GLU216': {'ALA213': 10, 'LEU214': 10, 'SER212': 10, 'PHE207': 10, 'CYS221': 9, 'LYS217': 10, 'LYS211': 10, 'ARG219': 10, 'GLN220': 10, 'VAL218': 10}, 'LYS217': {'ALA213': 10, 'ALA215': 10, 'LEU214': 10, 'TYR230': 10, 'LEU227': 10, 'SER212': 8, 'CYS221': 10, 'TRP222': 10, 'ARG219': 10, 'ASP646': 4, 'GLN220': 10, 'VAL218': 10, 'GLU226': 7, 'TRP647': 1}, 'VAL218': {'GLU216': 10, 'TRP553': 10, 'ALA213': 10, 'ALA215': 10, 'LEU214': 10, 'ARG348': 8, 'LEU227': 10, 'TRP550': 10, 'PHE207': 1, 'CYS221': 10, 'TRP222': 10, 'TYR551': 1, 'GLN223': 3, 'ASN556': 10, 'ARG219': 10, 'ALA552': 1, 'GLN220': 10, 'ASP549': 10}, 'ARG219': {'GLU216': 10, 'TRP553': 10, 'ALA215': 10, 'LEU214': 10, 'PHE203': 1, 'PHE207': 10, 'GLU201': 8, 'CYS221': 10, 'TRP222': 10, 'LEU200': 3, 'LYS217': 10, 'GLN223': 10, 'LYS211': 7, 'ASN556': 3, 'ASP204': 10, 'GLN220': 10}, 'GLN220': {'GLU216': 10, 'ALA215': 10, 'CYS221': 10, 'LYS217': 10, 'GLN223': 10, 'TRP222': 10, 'VAL218': 10}, 'CYS221': {'GLU216': 9, 'LEU227': 10, 'ASP225': 2, 'LYS217': 10, 'GLN223': 10, 'ASP224': 10, 'ARG219': 10, 'TRP222': 10, 'VAL218': 10, 'GLU226': 10}, 'TRP222': {'PRO555': 10, 'TRP553': 10, 'VAL577': 10, 'LEU581': 10, 'GLN220': 10, 'LEU227': 10, 'LYS217': 10, 'SER345': 9, 'GLN223': 10, 'PHE228': 7, 'ASP224': 10, 'LEU341': 10, 'ARG219': 10, 'ASN556': 10, 'LEU200': 10, 'VAL218': 10, 'LYS344': 10}, 'GLN223': {'TRP553': 2, 'ASP225': 10, 'CYS221': 10, 'LEU200': 3, 'ASP224': 10, 'LEU341': 10, 'ARG219': 10, 'PRO337': 3, 'TRP340': 8, 'GLN220': 10, 'VAL218': 3, 'LYS344': 10}, 'ASP224': {'GLN312': 1, 'LEU227': 10, 'ASP225': 10, 'CYS221': 10, 'PHE228': 10, 'LEU341': 5, 'TRP340': 4, 'TRP222': 10, 'GLU226': 10, 'LYS344': 10}, 'ASP225': {'GLN312': 10, 'TYR230': 9, 'LEU227': 10, 'SER229': 10, 'CYS221': 2, 'PRO313': 10, 'GLN223': 10, 'PHE228': 10, 'TRP340': 10, 'LEU311': 2, 'GLU226': 10, 'PRO333': 10, 'LYS344': 10}, 'GLU226': {'GLN312': 10, 'TYR230': 10, 'LEU227': 10, 'SER229': 10, 'CYS221': 10, 'ASP646': 4, 'LEU311': 6, 'LYS217': 7, 'PHE228': 10, 'ASP224': 10, 'GLN231': 10, 'LYS310': 9, 'TRP647': 5}, 'LEU227': {'GLN312': 2, 'PHE232': 6, 'TRP550': 10, 'LEU214': 10, 'TYR230': 10, 'SER229': 10, 'ARG348': 2, 'ASP225': 10, 'CYS221': 10, 'LYS217': 10, 'PHE228': 10, 'ASP224': 10, 'GLN231': 10, 'TRP222': 10, 'VAL218': 10}, 'PHE228': {'GLN312': 3, 'VAL301': 10, 'PHE232': 10, 'TYR230': 10, 'ASN556': 2, 'ASP225': 10, 'SER229': 10, 'PRO313': 10, 'ASP224': 10, 'GLN231': 10, 'LYS344': 10, 'VAL347': 10, 'TRP550': 10, 'ARG348': 10, 'SER345': 4, 'ILE316': 10, 'LEU303': 10, 'TRP222': 7, 'GLU226': 10, 'ALA343': 1, 'TRP340': 6}, 'SER229': {'GLN312': 10, 'PHE232': 10, 'TYR230': 10, 'LEU227': 10, 'LEU241': 9, 'PRO313': 10, 'GLN231': 10, 'LEU303': 10, 'LEU233': 10, 'ASP225': 10, 'LEU311': 10, 'GLU226': 10}, 'TYR230': {'GLN312': 1, 'PHE232': 10, 'LEU214': 10, 'ASP225': 9, 'ASN234': 10, 'ASP646': 10, 'GLN231': 10, 'PRO238': 4, 'LYS310': 2, 'TRP647': 10, 'TRP550': 10, 'PRO648': 8, 'GLY235': 10, 'LEU233': 10, 'GLU226': 10, 'LEU227': 10, 'LYS217': 10, 'GLY457': 5, 'PHE228': 10, 'LEU311': 8}, 'GLN231': {'PHE232': 10, 'ASP351': 10, 'TRP550': 10, 'ARG348': 10, 'LEU227': 10, 'SER229': 10, 'ASN237': 10, 'HIE355': 1, 'ALA236': 10, 'ASN234': 10, 'GLY235': 10, 'TYR551': 8, 'PHE228': 10, 'LEU240': 10, 'LEU233': 10, 'PRO238': 10, 'GLU226': 10}, 'PHE232': {'VAL301': 5, 'TYR230': 10, 'LEU241': 10, 'ALA276': 10, 'PHE278': 10, 'MET239': 10, 'SER229': 10, 'ASN234': 10, 'PRO313': 1, 'LEU452': 9, 'PRO238': 10, 'VAL347': 10, 'ARG348': 10, 'GLY235': 10, 'LEU240': 10, 'LEU303': 10, 'LEU233': 10, 'ASP351': 10, 'LEU227': 6, 'PHE228': 10, 'ASN237': 10, 'LEU311': 8}, 'LEU233': {'GLY309': 9, 'TYR230': 10, 'SER229': 10, 'LEU241': 10, 'LEU459': 10, 'ASN234': 10, 'GLY235': 10, 'LEU311': 10, 'GLY457': 10, 'LEU455': 10, 'LEU452': 10, 'GLN231': 10, 'LEU303': 9, 'TYR460': 10, 'PRO238': 10, 'LYS310': 10, 'ALA458': 10, 'ALA461': 10}, 'ASN234': {'PHE232': 10, 'TRP550': 10, 'PRO648': 10, 'TYR230': 10, 'LEU459': 10, 'ALA236': 10, 'TYR649': 10, 'GLY457': 10, 'GLY235': 10, 'TYR651': 9, 'GLN231': 10, 'TYR460': 10, 'PRO238': 10, 'ALA458': 10, 'TRP647': 10, 'ALA461': 4}, 'GLY235': {'PHE232': 10, 'TRP550': 10, 'LEU214': 10, 'TYR230': 10, 'ASN237': 10, 'TYR460': 2, 'ALA236': 10, 'GLY546': 7, 'ILE543': 1, 'TYR651': 10, 'ALA542': 10, 'GLN231': 10, 'LEU233': 10, 'PRO238': 10, 'TYR649': 10}, 'ALA236': {'TRP550': 10, 'MET239': 4, 'ARG348': 10, 'GLN547': 10, 'ASP351': 8, 'HIE355': 10, 'GLY546': 10, 'ASN234': 10, 'TYR551': 10, 'ILE543': 10, 'TYR651': 10, 'ALA542': 10, 'GLN231': 10, 'ASN237': 10, 'PRO238': 10, 'TYR359': 1, 'ASP549': 6}, 'ASN237': {'PHE232': 10, 'ASP351': 10, 'MET239': 10, 'ARG348': 10, 'LEU241': 5, 'LEU354': 8, 'HIE355': 10, 'TYR551': 10, 'GLY235': 10, 'LEU240': 10, 'PHE352': 9, 'ILE543': 10, 'ALA542': 1, 'GLN231': 10, 'PRO238': 10, 'TYR359': 10}, 'PRO238': {'PHE232': 10, 'TYR230': 4, 'LEU241': 10, 'ALA236': 10, 'TYR651': 3, 'TYR649': 10, 'ALA461': 3, 'MET239': 10, 'ASN234': 10, 'GLN231': 10, 'LEU452': 10, 'TYR460': 10, 'TYR359': 1, 'GLY235': 10, 'LEU447': 10, 'LEU240': 10, 'ALA542': 10, 'ILE543': 10, 'LEU233': 10, 'ALA458': 10}, 'MET239': {'PHE232': 10, 'ARG450': 10, 'TYR460': 10, 'LEU241': 10, 'ALA236': 4, 'LEU447': 10, 'ASN363': 1, 'LEU240': 10, 'THR364': 1, 'ASP446': 10, 'ALA542': 10, 'ILE543': 10, 'LEU452': 10, 'ASN237': 10, 'SER440': 1, 'TYR359': 10, 'GLN539': 10}, 'LEU240': {'ARG242': 10, 'PHE232': 10, 'VAL347': 2, 'ASP351': 10, 'GLU275': 10, 'ALA276': 10, 'LEU241': 10, 'ASP277': 10, 'LEU354': 10, 'ARG450': 10, 'GLN358': 3, 'GLN231': 10, 'LEU452': 7, 'ASN237': 10, 'PRO238': 10, 'TYR359': 10, 'PHE278': 10}, 'LEU241': {'ARG242': 10, 'VAL301': 5, 'PHE232': 10, 'ARG450': 10, 'ASP277': 10, 'ALA276': 10, 'PHE278': 1, 'MET239': 10, 'GLU275': 10, 'SER229': 9, 'LEU452': 10, 'PRO238': 10, 'PHE274': 10, 'ARG243': 10, 'LEU455': 10, 'LEU303': 10, 'LEU233': 10, 'ALA458': 4, 'MET302': 3, 'ASN237': 5, 'LEU311': 10}, 'ARG242': {'ASP449': 10, 'GLU275': 10, 'ARG450': 10, 'ASP277': 10, 'GLN435': 6, 'LEU273': 10, 'ARG243': 10, 'SER244': 10, 'LEU240': 10, 'ASP446': 3, 'GLY451': 7, 'LEU452': 10, 'ALA276': 10, 'PHE274': 10}, 'ARG243': {'GLY271': 10, 'ASP449': 7, 'MET305': 1, 'GLU275': 10, 'ARG450': 10, 'LEU241': 10, 'GLY454': 4, 'LEU268': 3, 'LEU273': 10, 'SER244': 10, 'SER272': 8, 'THR245': 10, 'GLY451': 10, 'LEU452': 9, 'LEU455': 7, 'ASN270': 4, 'PHE274': 10}, 'SER244': {'ARG242': 10, 'GLY271': 6, 'SER246': 10, 'MET302': 10, 'GLU275': 10, 'LEU268': 10, 'LEU273': 10, 'PRO248': 6, 'THR245': 10, 'LEU247': 5, 'PHE274': 10}, 'THR245': {'SER246': 10, 'GLU275': 10, 'LEU268': 10, 'ARG243': 10, 'LEU247': 8}, 'SER246': {'MET302': 10, 'GLU275': 10, 'LEU268': 10, 'LEU273': 9, 'PRO248': 10, 'SER244': 10, 'GLU265': 7, 'LEU247': 10}, 'LEU247': {'MET302': 10, 'SER249': 10, 'GLU275': 1, 'LEU253': 10, 'LEU251': 10, 'LEU268': 10, 'LEU273': 9, 'PRO248': 10, 'LEU260': 7, 'SER244': 5, 'VAL252': 2, 'LEU264': 10, 'THR245': 8, 'GLN261': 10, 'GLU265': 10}, 'PRO248': {'MET302': 10, 'SER246': 10, 'VAL301': 7, 'SER249': 10, 'GLU275': 10, 'LEU251': 10, 'VAL252': 2, 'LEU268': 2, 'LEU273': 5, 'SER244': 6, 'LEU264': 7, 'ARG250': 10, 'VAL315': 1, 'LEU300': 10}, 'SER249': {'LEU251': 10, 'VAL252': 10, 'ARG250': 10, 'LEU247': 10, 'LEU300': 1}, 'ARG250': {'ASN322': 10, 'ASP282': 10, 'PRO321': 10, 'VAL252': 10, 'LEU251': 10, 'PRO248': 10, 'ALA298': 6, 'ILE279': 10, 'GLN317': 10, 'LEU300': 10, 'PRO320': 10}, 'LEU251': {'MET302': 10, 'LEU330': 10, 'GLN261': 2, 'VAL301': 3, 'SER249': 10, 'GLN317': 10, 'LEU253': 10, 'VAL252': 10, 'PRO248': 10, 'LEU260': 1, 'LEU264': 10, 'ILE316': 3, 'VAL315': 10, 'PRO321': 10, 'LEU247': 10, 'LEU300': 10}, 'VAL252': {'LEU330': 10, 'SER249': 10, 'THR327': 5, 'LEU253': 10, 'PRO254': 10, 'PRO248': 2, 'ARG250': 10, 'GLN261': 5, 'PRO321': 10, 'GLN317': 10, 'LEU300': 5, 'LEU247': 2}, 'LEU253': {'LEU332': 5, 'LEU330': 10, 'GLU258': 10, 'PRO254': 10, 'LEU251': 10, 'SER255': 10, 'GLU259': 2, 'LEU260': 10, 'VAL315': 10, 'LEU264': 10, 'GLN261': 10, 'MET257': 10, 'LEU247': 10}, 'PRO254': {'LEU330': 10, 'THR327': 7, 'GLU258': 10, 'SER255': 10, 'VAL252': 10, 'GLY256': 10, 'MET257': 10}, 'SER255': {'MET257': 10, 'GLY256': 10, 'GLU258': 10, 'LEU253': 10}, 'GLY256': {'GLU258': 10, 'PRO254': 10, 'GLU259': 10, 'LEU260': 5, 'MET257': 10}, 'MET257': {'LEU332': 10, 'LEU330': 10, 'GLU258': 10, 'LEU253': 10, 'PRO254': 10, 'SER255': 10, 'LYS569': 10, 'THR329': 1, 'GLU259': 10, 'LEU260': 10, 'ALA262': 2, 'VAL315': 5, 'GLN261': 10, 'PHE331': 3}, 'GLU258': {'LEU253': 10, 'PRO254': 10, 'SER255': 10, 'GLY256': 10, 'GLU259': 10, 'LEU260': 10, 'ALA262': 10, 'GLN263': 8, 'GLN261': 10}, 'GLU259': {'LEU253': 2, 'GLY256': 10, 'LEU260': 10, 'ALA262': 10, 'GLN263': 10, 'LEU264': 9, 'GLN261': 10, 'MET257': 10}, 'LEU260': {'LEU332': 10, 'LEU330': 6, 'GLU258': 10, 'LEU253': 10, 'MET314': 10, 'LEU251': 1, 'GLY256': 5, 'GLN263': 10, 'ALA262': 10, 'VAL315': 10, 'LEU264': 10, 'GLN261': 10, 'GLU265': 10, 'MET257': 10, 'PRO333': 4, 'LEU247': 7}, 'GLN261': {'GLU258': 10, 'LEU253': 10, 'VAL252': 5, 'LEU251': 2, 'GLU259': 10, 'GLN263': 10, 'ALA262': 10, 'LEU264': 10, 'LYS266': 10, 'GLU265': 10, 'MET257': 10, 'LEU247': 10}, 'ALA262': {'GLU267': 10, 'GLU258': 10, 'GLU259': 10, 'GLN263': 10, 'LEU260': 10, 'LEU264': 0, 'LYS266': 10, 'GLU265': 10, 'MET257': 2}, 'GLN263': {'GLU267': 10, 'GLU258': 8, 'MET314': 10, 'LYS266': 10, 'LEU268': 10, 'GLU259': 10, 'LEU260': 10, 'LEU264': 10, 'GLN261': 10, 'GLU265': 10}, 'LEU264': {'MET302': 10, 'GLU267': 10, 'GLN269': 10, 'LEU253': 10, 'MET314': 10, 'LEU251': 10, 'LYS266': 10, 'LEU268': 10, 'PRO248': 7, 'GLU259': 9, 'LEU260': 10, 'ALA262': 10, 'LEU273': 10, 'VAL315': 10, 'GLN261': 10, 'LYS304': 4, 'GLU265': 10, 'LEU247': 10}, 'GLU265': {'SER246': 7, 'GLU267': 10, 'GLN269': 10, 'MET314': 1, 'LYS266': 10, 'LEU268': 10, 'LEU273': 5, 'GLN263': 10, 'ALA262': 10, 'LEU260': 10, 'GLN261': 10, 'LEU247': 10, 'ASN270': 9}, 'LYS266': {'GLY271': 10, 'GLU267': 10, 'GLN269': 10, 'MET314': 2, 'LEU268': 10, 'GLN263': 10, 'ALA262': 10, 'SER272': 1, 'LEU264': 10, 'GLN261': 10, 'ASN270': 10}, 'GLU267': {'GLY271': 10, 'GLN269': 10, 'MET305': 10, 'MET314': 10, 'LEU268': 10, 'LEU273': 10, 'GLN263': 10, 'ALA262': 10, 'SER272': 10, 'LEU264': 10, 'LEU303': 6, 'LYS304': 10, 'GLU265': 10, 'ASN270': 10, 'PHE274': 1}, 'LEU268': {'MET302': 10, 'GLY271': 10, 'SER246': 10, 'GLN269': 10, 'LEU273': 10, 'ARG243': 3, 'GLN263': 10, 'SER244': 10, 'SER272': 10, 'LEU264': 10, 'THR245': 10, 'PRO248': 2, 'LYS266': 10, 'GLU265': 10, 'LEU247': 10, 'ASN270': 10}, 'GLN269': {'GLY271': 10, 'GLU267': 10, 'SER272': 8, 'LEU264': 10, 'LYS266': 10, 'GLU265': 10, 'ASN270': 10}, 'ASN270': {'GLY271': 10, 'GLU267': 10, 'LEU268': 10, 'ARG243': 4, 'SER272': 10, 'LYS266': 10, 'GLU265': 9}, 'GLY271': {'GLU267': 10, 'GLN269': 10, 'MET305': 3, 'LEU268': 10, 'LEU273': 10, 'ARG243': 10, 'SER244': 6, 'SER272': 10, 'LYS266': 10, 'PHE274': 9}, 'SER272': {'GLU267': 10, 'GLN269': 8, 'MET305': 10, 'LEU268': 10, 'LEU273': 10, 'ARG243': 8, 'LEU455': 1, 'LYS266': 1, 'LEU303': 10, 'LYS304': 10, 'LEU311': 10, 'ASN270': 10, 'PHE274': 10}, 'LEU273': {'ARG242': 10, 'GLY271': 10, 'MET314': 10, 'PRO248': 5, 'VAL315': 5, 'LYS304': 10, 'MET305': 10, 'GLU275': 10, 'LEU268': 10, 'GLU265': 5, 'PHE274': 10, 'GLU267': 10, 'ARG243': 10, 'LEU264': 10, 'LEU303': 10, 'LEU247': 9, 'MET302': 10, 'SER246': 9, 'SER244': 10, 'LEU311': 6}, 'PHE274': {'ARG242': 10, 'GLY271': 9, 'VAL301': 10, 'ARG450': 10, 'LEU241': 10, 'GLY451': 10, 'ALA276': 10, 'LYS304': 10, 'MET305': 10, 'GLU275': 10, 'GLY454': 1, 'SER272': 10, 'LEU452': 10, 'GLU267': 1, 'ARG243': 10, 'LEU455': 10, 'LEU303': 10, 'MET302': 10, 'SER244': 10, 'LEU311': 10}, 'GLU275': {'ARG242': 10, 'MET302': 10, 'SER246': 10, 'VAL301': 10, 'LEU241': 10, 'ASP277': 10, 'LEU273': 10, 'ARG243': 10, 'PRO248': 10, 'SER244': 10, 'LEU240': 10, 'THR245': 10, 'LEU303': 10, 'ALA276': 10, 'LEU247': 1}, 'ALA276': {'ARG242': 10, 'MET302': 10, 'VAL301': 10, 'PHE232': 10, 'PRO299': 10, 'VAL347': 6, 'LEU241': 10, 'ASP277': 10, 'LEU240': 10, 'LEU303': 10, 'PHE278': 10, 'LEU300': 10, 'PHE274': 10}, 'ASP277': {'ARG242': 10, 'VAL301': 3, 'PRO299': 8, 'GLU275': 10, 'ARG450': 9, 'LEU241': 10, 'LEU354': 10, 'GLN435': 4, 'LEU280': 10, 'LEU240': 10, 'ILE279': 10, 'PHE278': 10, 'LEU300': 6}, 'PHE278': {'VAL301': 10, 'PHE232': 10, 'LEU241': 1, 'PHE352': 1, 'ILE279': 10, 'ALA276': 10, 'LEU300': 10, 'PRO299': 10, 'ALA298': 10, 'LEU281': 10, 'VAL347': 10, 'ARG348': 1, 'LEU280': 10, 'LEU240': 10, 'SER350': 10, 'ASP351': 10, 'ASP282': 10, 'LEU354': 10, 'ALA297': 2}, 'ILE279': {'ASP282': 10, 'LEU300': 5, 'LEU354': 5, 'ASP277': 10, 'GLN435': 7, 'LEU280': 10, 'ARG250': 10, 'ALA298': 10, 'ARG431': 10, 'LEU281': 10, 'PRO320': 4}, 'LEU280': {'LEU428': 10, 'ASP282': 10, 'LEU354': 10, 'ASP277': 10, 'GLN435': 10, 'GLN427': 3, 'ARG431': 10, 'ALA434': 4, 'GLN358': 10, 'ILE284': 10, 'ALA432': 10, 'PHE278': 10, 'LEU281': 10}, 'LEU281': {'LEU428': 10, 'GLY283': 10, 'THR420': 10, 'ASN349': 2, 'PRO299': 10, 'ASP351': 9, 'ASP282': 10, 'LEU354': 10, 'ILE357': 1, 'GLN353': 10, 'ALA298': 10, 'ALA297': 10, 'ARG431': 10, 'ILE279': 10, 'LEU296': 10, 'ILE284': 10, 'PHE278': 10, 'SER350': 10}, 'ASP282': {'GLY283': 10, 'ASN322': 2, 'LEU280': 10, 'ARG250': 10, 'ALA298': 10, 'ALA297': 8, 'ARG431': 10, 'ILE279': 10, 'ILE284': 10, 'PHE278': 10, 'PRO320': 10}, 'GLY283': {'GLN427': 6, 'PRO285': 10, 'ALA298': 10, 'ALA297': 5, 'ARG431': 10, 'ILE284': 10, 'LEU281': 10}, 'ILE284': {'ALA286': 10, 'LEU428': 10, 'THR420': 10, 'ASP282': 10, 'LEU280': 10, 'GLN427': 10, 'PRO285': 10, 'GLN353': 7, 'GLY424': 10, 'HID425': 10, 'ALA297': 2, 'ARG431': 10, 'SER419': 9, 'ASN287': 2, 'LEU296': 10, 'LEU281': 10}, 'PRO285': {'ALA286': 10, 'GLY283': 10, 'THR420': 9, 'LEU428': 3, 'GLY422': 2, 'GLN427': 10, 'GLY423': 10, 'HID425': 2, 'GLY421': 10, 'SER419': 7, 'ASN287': 10, 'LEU296': 2, 'GLY424': 10}, 'ALA286': {'VAL288': 10, 'THR420': 10, 'GLY422': 3, 'LYS293': 3, 'GLN294': 10, 'GLY421': 10, 'TYR295': 10, 'ASN287': 10, 'SER419': 10, 'LEU296': 10, 'ILE284': 10, 'GLY424': 9}, 'ASN287': {'THR420': 10, 'GLY422': 10, 'GLN294': 10, 'TYR295': 1, 'VAL288': 10, 'PHE414': 10, 'ALA417': 10, 'GLU292': 9, 'LYS293': 10, 'PRO285': 10, 'VAL418': 10, 'GLY421': 10, 'SER419': 10, 'ILE289': 10, 'LEU296': 8, 'LYS416': 10, 'ASP415': 10, 'ILE284': 2, 'THR560': 9, 'GLY424': 1}, 'VAL288': {'ALA286': 10, 'GLY291': 10, 'ILE289': 10, 'GLY422': 6, 'GLU292': 10, 'ASP415': 10, 'LYS293': 10, 'ARG290': 10, 'GLN294': 10, 'GLY421': 5}, 'ILE289': {'GLY291': 10, 'ASP583': 3, 'ALA417': 10, 'LYS416': 10, 'GLU292': 10, 'ASP415': 10, 'LYS293': 10, 'ARG290': 10, 'GLN294': 10, 'VAL418': 1, 'GLN586': 10, 'ASN287': 10, 'THR560': 10, 'ARG585': 7}, 'ARG290': {'GLY291': 10, 'VAL288': 10, 'ASP583': 10, 'ARG562': 1, 'LYS416': 10, 'GLU292': 10, 'GLN586': 10, 'VAL584': 5, 'ARG585': 10}, 'GLY291': {'VAL288': 10, 'ILE289': 10, 'GLU292': 10, 'LYS293': 10}, 'GLU292': {'ASP583': 10, 'VAL288': 10, 'ILE289': 10, 'ARG562': 10, 'LYS293': 10, 'ARG290': 10, 'GLN294': 10, 'GLN586': 2, 'MET563': 1, 'TYR295': 4, 'ASN287': 9}, 'LYS293': {'GLY291': 10, 'VAL288': 10, 'ILE289': 10, 'ARG562': 10, 'ALA286': 3, 'GLN294': 10, 'PRO564': 2, 'MET563': 10, 'TYR295': 10, 'ASN287': 10, 'MET561': 9}, 'GLN294': {'ALA286': 10, 'VAL288': 10, 'ILE289': 10, 'ARG562': 10, 'ASP583': 7, 'ALA417': 4, 'GLU292': 10, 'VAL418': 10, 'GLN586': 10, 'PRO582': 8, 'MET563': 10, 'TYR295': 10, 'ASN287': 10, 'SER419': 10, 'LEU296': 10, 'THR560': 10, 'MET561': 10}, 'TYR295': {'ALA286': 10, 'GLN319': 9, 'ASN349': 10, 'ARG562': 10, 'ASN287': 1, 'TRP346': 10, 'GLU292': 4, 'LYS293': 10, 'PRO565': 10, 'ALA297': 10, 'MET563': 10, 'SER419': 10, 'PRO564': 10, 'LEU296': 10, 'THR560': 10, 'MET561': 10}, 'LEU296': {'GLN319': 9, 'LEU428': 8, 'THR420': 10, 'GLN294': 10, 'MET561': 10, 'PRO299': 9, 'TRP346': 10, 'PRO285': 2, 'ALA298': 10, 'VAL418': 3, 'SER419': 10, 'LEU281': 10, 'ASN349': 10, 'PRO565': 10, 'GLN353': 10, 'ILE318': 10, 'ASN287': 8, 'SER350': 10, 'ALA286': 10, 'ALA297': 10, 'ILE284': 10, 'THR560': 5}, 'ALA297': {'GLN319': 10, 'GLY283': 5, 'ASN349': 8, 'PRO299': 10, 'VAL347': 5, 'ASP282': 8, 'TRP346': 10, 'PRO565': 10, 'ALA298': 10, 'ILE318': 10, 'TYR295': 10, 'SER419': 1, 'SER350': 10, 'ILE284': 2, 'PHE278': 2, 'LEU281': 10, 'PRO320': 8}, 'ALA298': {'GLN319': 10, 'GLY283': 10, 'PRO299': 10, 'ASP282': 10, 'LEU300': 10, 'GLN317': 5, 'ARG250': 6, 'ILE318': 10, 'ILE279': 10, 'LEU296': 10, 'PHE278': 10, 'SER350': 10, 'LEU281': 10, 'PRO320': 10}, 'PRO299': {'GLN319': 10, 'VAL301': 10, 'VAL347': 10, 'GLN317': 10, 'LEU300': 10, 'TRP346': 10, 'ASP277': 8, 'SER350': 10, 'ILE316': 10, 'ALA297': 10, 'ILE318': 10, 'ALA276': 10, 'PHE278': 10, 'LEU296': 9, 'LEU281': 10, 'PRO320': 10}, 'LEU300': {'GLN319': 7, 'VAL301': 10, 'ASP277': 6, 'PRO248': 10, 'VAL315': 10, 'ILE279': 5, 'ALA276': 10, 'PHE278': 10, 'PRO320': 10, 'LEU251': 10, 'ALA298': 10, 'GLN317': 10, 'LEU330': 3, 'VAL252': 5, 'ILE316': 10, 'ILE318': 10, 'MET302': 10, 'SER249': 1, 'ARG250': 10, 'PRO321': 10}, 'VAL301': {'PHE232': 5, 'LEU241': 5, 'ASP277': 3, 'MET314': 1, 'PRO248': 7, 'VAL315': 10, 'ALA276': 10, 'PHE278': 10, 'PRO299': 10, 'GLU275': 10, 'LEU251': 3, 'PRO313': 10, 'GLN317': 10, 'PHE274': 10, 'VAL347': 10, 'ILE316': 10, 'LEU303': 10, 'MET302': 10, 'PHE228': 10}, 'MET302': {'GLN312': 9, 'LEU241': 3, 'MET314': 10, 'PRO248': 10, 'VAL315': 10, 'ALA276': 10, 'LYS304': 10, 'LEU300': 10, 'GLU275': 10, 'LEU251': 10, 'LEU268': 10, 'PRO313': 10, 'GLN317': 1, 'PHE274': 10, 'LEU264': 10, 'ILE316': 10, 'LEU303': 10, 'LEU247': 10, 'SER246': 10, 'LEU273': 10, 'SER244': 10}, 'LEU303': {'GLN312': 10, 'VAL301': 10, 'PHE232': 10, 'LEU241': 10, 'MET314': 10, 'VAL315': 10, 'ALA276': 10, 'LYS304': 10, 'MET305': 10, 'GLU275': 10, 'SER229': 10, 'SER272': 10, 'PRO313': 10, 'PHE274': 10, 'GLU267': 6, 'VAL347': 1, 'ILE316': 3, 'LEU233': 9, 'LEU273': 10, 'PHE228': 10, 'LEU311': 10}, 'LYS304': {'MET302': 10, 'GLN312': 10, 'GLU267': 10, 'MET305': 10, 'MET314': 10, 'PRO307': 4, 'LEU273': 10, 'SER272': 10, 'LEU311': 10, 'PRO313': 10, 'GLU306': 10, 'LEU264': 4, 'LYS310': 10, 'PRO333': 3, 'PHE274': 10}, 'MET305': {'GLN312': 10, 'GLY271': 3, 'GLY309': 10, 'GLU267': 10, 'PRO456': 9, 'PRO307': 10, 'LEU273': 10, 'ARG243': 1, 'SER272': 10, 'LEU311': 10, 'GLY457': 1, 'GLU306': 10, 'LEU455': 10, 'LEU303': 10, 'LYS310': 10, 'PHE274': 10}, 'GLU306': {'GLN312': 10, 'GLY309': 10, 'PRO307': 10, 'LEU311': 10, 'LEU455': 7, 'ASN308': 10, 'LYS304': 10, 'LYS310': 10}, 'PRO307': {'GLY309': 10, 'MET305': 10, 'ASN308': 10, 'LYS304': 4, 'LYS310': 10}, 'ASN308': {'GLY309': 10, 'PRO456': 10, 'GLN644': 3, 'GLY457': 9, 'LEU645': 9, 'GLU306': 10, 'LYS310': 10, 'TRP647': 10}, 'GLY309': {'PRO456': 10, 'MET305': 10, 'PRO307': 10, 'LEU311': 10, 'GLY457': 10, 'GLU306': 10, 'LEU455': 10, 'LEU233': 9, 'LYS310': 10, 'ALA458': 6}, 'LYS310': {'GLN312': 10, 'PRO456': 8, 'MET305': 10, 'TYR230': 2, 'PRO307': 10, 'GLY457': 10, 'ASN308': 10, 'GLU306': 10, 'ASP646': 3, 'LEU455': 10, 'LEU233': 10, 'LYS304': 10, 'LEU311': 10, 'GLU226': 9, 'TRP647': 10}, 'LEU311': {'GLN312': 10, 'PHE232': 8, 'TYR230': 8, 'LEU241': 10, 'GLU306': 10, 'LYS304': 10, 'ASP225': 2, 'MET305': 10, 'SER229': 10, 'SER272': 10, 'PRO313': 10, 'LEU452': 3, 'PHE274': 10, 'GLY309': 10, 'LEU303': 10, 'LEU233': 10, 'LEU455': 10, 'GLU226': 6, 'LEU273': 6, 'GLY457': 9}, 'GLN312': {'MET302': 9, 'MET305': 10, 'TYR230': 1, 'LEU227': 2, 'SER229': 10, 'MET314': 10, 'PRO313': 10, 'PHE228': 3, 'GLU306': 10, 'ASP224': 1, 'LYS304': 10, 'LEU303': 10, 'TRP340': 1, 'ASP225': 10, 'LYS310': 10, 'GLU226': 10, 'PRO333': 5}, 'PRO313': {'MET302': 10, 'LEU332': 1, 'VAL301': 10, 'PHE232': 1, 'SER229': 10, 'MET314': 10, 'VAL315': 10, 'ILE316': 10, 'PHE228': 10, 'LYS304': 10, 'LEU303': 10, 'TRP340': 10, 'ASP225': 10, 'LEU311': 10, 'PRO333': 10, 'LYS344': 10}, 'MET314': {'MET302': 10, 'GLN312': 10, 'LEU332': 10, 'GLU267': 10, 'VAL301': 1, 'PHE331': 2, 'LEU273': 10, 'LEU260': 10, 'GLN263': 10, 'VAL315': 10, 'LEU264': 10, 'ILE316': 10, 'LYS266': 2, 'LEU303': 10, 'TRP340': 7, 'LYS304': 10, 'GLU265': 1, 'PRO333': 10}, 'VAL315': {'VAL301': 10, 'PRO248': 1, 'LEU260': 10, 'LEU300': 10, 'PHE331': 10, 'LEU251': 10, 'PRO313': 10, 'GLN317': 10, 'LEU332': 10, 'LEU330': 10, 'LEU253': 10, 'LEU264': 10, 'ILE316': 10, 'LEU303': 10, 'MET257': 5, 'PRO333': 10, 'MET302': 10, 'LEU273': 5, 'TRP340': 10}, 'ILE316': {'VAL301': 10, 'MET314': 10, 'THR329': 10, 'LEU300': 10, 'PRO299': 0, 'PHE331': 10, 'TRP346': 2, 'LEU251': 3, 'PRO313': 10, 'GLN317': 10, 'LYS344': 10, 'LEU332': 10, 'LEU330': 10, 'VAL347': 10, 'ILE318': 10, 'LEU303': 3, 'PRO333': 9, 'MET302': 10, 'PHE228': 10, 'ALA343': 10, 'TRP340': 10}, 'GLN317': {'GLN319': 10, 'VAL301': 10, 'THR329': 10, 'VAL315': 10, 'LEU300': 10, 'PRO320': 10, 'PRO299': 10, 'LEU251': 10, 'ALA298': 5, 'LEU330': 10, 'THR327': 10, 'VAL347': 5, 'VAL252': 10, 'ILE318': 10, 'PRO328': 10, 'MET302': 1, 'ARG250': 10, 'PRO321': 10, 'PHE331': 10}, 'ILE318': {'GLN319': 10, 'THR329': 10, 'LEU300': 10, 'PRO320': 10, 'PRO299': 10, 'TRP346': 10, 'PRO566': 10, 'ALA298': 10, 'LEU342': 10, 'LYS344': 1, 'LEU330': 10, 'VAL347': 10, 'PRO565': 10, 'ILE316': 10, 'LEU296': 10, 'SER350': 5, 'PRO328': 9, 'ALA297': 10, 'ALA343': 10, 'PHE331': 10}, 'GLN319': {'PRO299': 10, 'PHE331': 5, 'PRO328': 10, 'SER325': 10, 'THR329': 8, 'PRO565': 10, 'ALA298': 10, 'ALA297': 10, 'PRO564': 10, 'TYR295': 9, 'PRO321': 10, 'LEU296': 9, 'GLN317': 10, 'LEU300': 7, 'PRO320': 10}, 'PRO320': {'ASN322': 10, 'PRO299': 10, 'ASP282': 10, 'PRO328': 10, 'SER325': 10, 'ILE279': 4, 'ARG250': 10, 'ALA298': 10, 'ALA297': 8, 'ILE318': 10, 'PRO321': 10, 'GLN317': 10, 'LEU300': 10}, 'PRO321': {'GLN319': 10, 'LEU330': 1, 'ASN322': 10, 'THR327': 10, 'PRO326': 10, 'PRO328': 10, 'LEU251': 10, 'VAL252': 10, 'SER325': 10, 'PRO323': 10, 'ARG250': 10, 'GLN317': 10, 'LEU300': 10}, 'ASN322': {'PRO326': 10, 'ASP282': 2, 'SER325': 10, 'SER324': 10, 'PRO323': 10, 'ARG250': 10, 'PRO328': 10, 'PRO320': 10}, 'PRO323': {'PRO321': 10, 'SER325': 10, 'SER324': 10, 'PRO326': 4}, 'SER324': {'ASN322': 10, 'PRO326': 10, 'SER325': 10}, 'SER325': {'GLN319': 10, 'ASN322': 10, 'THR327': 10, 'PRO326': 10, 'PRO323': 10, 'PRO321': 10, 'PRO328': 10, 'PRO320': 10}, 'PRO326': {'ASN322': 10, 'THR327': 10, 'SER324': 10, 'PRO323': 4, 'PRO321': 10, 'PRO328': 10}, 'THR327': {'LEU330': 1, 'PRO328': 10, 'PRO254': 7, 'VAL252': 5, 'SER325': 10, 'THR329': 10, 'PRO321': 10, 'GLN317': 10}, 'PRO328': {'GLN319': 10, 'LEU330': 10, 'ASN322': 10, 'PRO326': 10, 'SER325': 10, 'THR329': 10, 'THR567': 7, 'ILE318': 9, 'PRO321': 10, 'PHE331': 1, 'GLN317': 10, 'PRO320': 10}, 'THR329': {'GLN319': 8, 'LEU330': 10, 'THR327': 10, 'LYS569': 10, 'THR567': 10, 'PRO566': 10, 'ILE316': 10, 'THR568': 10, 'ILE318': 10, 'PHE331': 10, 'MET257': 1, 'GLN317': 10}, 'LEU330': {'LEU332': 10, 'ASP335': 10, 'THR327': 1, 'GLN317': 10, 'LEU253': 10, 'PRO328': 10, 'PRO254': 10, 'LEU251': 10, 'VAL252': 10, 'LYS569': 10, 'LEU260': 6, 'ILE316': 10, 'ILE318': 10, 'VAL315': 10, 'PRO321': 1, 'PHE331': 10, 'MET257': 10, 'LEU300': 3}, 'PHE331': {'GLN319': 5, 'MET314': 2, 'THR329': 10, 'VAL315': 10, 'THR567': 3, 'PRO566': 10, 'ASP335': 10, 'GLN317': 10, 'LEU342': 8, 'LEU332': 10, 'VAL572': 10, 'LYS569': 10, 'PRO565': 9, 'ILE316': 10, 'ILE318': 10, 'MET257': 3, 'PRO333': 10, 'PRO328': 1, 'ALA339': 10, 'THR568': 4, 'ALA343': 10, 'TRP340': 10}, 'LEU332': {'LEU330': 10, 'ASP335': 10, 'LEU253': 5, 'MET314': 10, 'LYS569': 10, 'LEU260': 10, 'ILE316': 10, 'PRO313': 1, 'VAL315': 10, 'TRP340': 10, 'MET257': 10, 'PRO333': 10, 'SER334': 10}, 'PRO333': {'GLN312': 5, 'MET314': 10, 'LEU260': 4, 'LYS304': 3, 'PRO313': 10, 'ILE316': 9, 'TRP340': 10, 'VAL315': 10, 'PHE331': 10, 'ASP225': 10, 'ASP335': 10, 'SER334': 10}, 'SER334': {'LEU332': 10, 'PRO337': 1, 'ASP335': 10, 'PRO336': 10}, 'ASP335': {'LEU332': 10, 'LEU330': 10, 'LYS569': 10, 'ALA339': 10, 'TRP340': 10, 'PRO337': 10, 'PHE331': 10, 'PRO333': 10, 'PRO336': 10}, 'PRO336': {'VAL572': 9, 'LYS569': 10, 'ALA339': 10, 'LEU338': 10, 'LEU341': 10, 'GLU570': 10, 'PRO337': 10, 'TRP340': 10, 'SER334': 10}, 'PRO337': {'SER334': 1, 'ALA339': 10, 'LEU338': 10, 'GLN223': 3, 'LEU341': 10, 'TRP340': 10, 'ASP335': 10, 'LEU342': 10, 'LYS344': 3}, 'LEU338': {'THR573': 10, 'VAL577': 10, 'MET574': 10, 'VAL572': 10, 'ALA339': 10, 'LEU341': 10, 'ALA343': 10, 'TRP340': 10, 'LEU200': 10, 'LEU342': 10, 'PRO336': 10}, 'ALA339': {'VAL577': 5, 'VAL572': 10, 'LYS569': 10, 'PRO566': 10, 'LYS344': 10, 'LEU341': 10, 'TRP340': 10, 'ALA343': 10, 'GLU570': 2, 'PRO337': 10, 'PHE331': 10, 'ASP335': 10, 'LEU342': 10, 'PRO336': 10}, 'TRP340': {'GLN312': 1, 'MET314': 7, 'GLN223': 8, 'LEU341': 10, 'VAL315': 10, 'ASP225': 10, 'PRO313': 10, 'ASP224': 4, 'ASP335': 10, 'LEU342': 10, 'LYS344': 10, 'LEU332': 10, 'PRO336': 10, 'LYS569': 3, 'SER345': 10, 'ILE316': 10, 'PRO337': 10, 'PRO333': 10, 'LEU338': 10, 'PHE228': 6, 'ALA343': 10, 'PHE331': 10}, 'LEU341': {'PRO555': 5, 'VAL577': 10, 'LEU581': 2, 'TRP346': 9, 'TRP222': 10, 'ALA339': 10, 'LEU338': 10, 'LYS344': 10, 'GLN223': 10, 'SER345': 10, 'ASP224': 5, 'ASN556': 6, 'ALA343': 10, 'PRO337': 10, 'LEU200': 9, 'LEU342': 10, 'PRO336': 10}, 'LEU342': {'VAL577': 10, 'LEU581': 10, 'MET561': 10, 'THR573': 3, 'THR576': 10, 'PHE331': 8, 'TRP346': 10, 'PRO566': 10, 'MET578': 1, 'LYS344': 10, 'VAL347': 10, 'VAL572': 10, 'PRO565': 9, 'SER345': 10, 'ILE318': 10, 'PRO337': 10, 'SER580': 10, 'ALA339': 10, 'LEU338': 10, 'ALA343': 10, 'TRP340': 10}, 'ALA343': {'VAL347': 10, 'TRP346': 10, 'ARG348': 10, 'LYS569': 1, 'ALA339': 10, 'LEU338': 10, 'ILE316': 10, 'SER345': 10, 'PRO566': 5, 'ILE318': 10, 'TRP340': 10, 'LEU341': 10, 'PHE228': 1, 'PHE331': 10, 'LYS344': 10}, 'LYS344': {'GLN223': 10, 'ASN556': 2, 'LEU341': 10, 'ASP225': 10, 'TRP346': 10, 'PRO313': 10, 'ASP224': 10, 'LEU342': 10, 'ASN349': 10, 'VAL347': 10, 'ARG348': 10, 'SER345': 10, 'ILE316': 10, 'ILE318': 1, 'PRO337': 3, 'TRP222': 10, 'ALA339': 10, 'PHE228': 10, 'TRP340': 10}, 'SER345': {'PRO555': 10, 'ASN349': 10, 'VAL577': 9, 'VAL347': 10, 'PRO558': 10, 'LEU581': 10, 'TRP346': 10, 'ARG348': 10, 'ALA557': 10, 'PHE228': 4, 'ASN556': 10, 'LEU341': 10, 'ALA343': 10, 'TRP340': 10, 'TRP222': 9, 'SER350': 10, 'LEU342': 10, 'MET561': 10}, 'TRP346': {'VAL577': 1, 'LEU341': 9, 'TYR295': 10, 'MET561': 10, 'PRO299': 10, 'THR576': 3, 'PRO566': 10, 'MET563': 10, 'LEU342': 10, 'LYS344': 10, 'ASN349': 10, 'ARG562': 6, 'VAL347': 10, 'PRO558': 7, 'ARG348': 10, 'PRO565': 10, 'ILE316': 2, 'ILE318': 10, 'LEU296': 10, 'SER350': 10, 'SER580': 10, 'ASP351': 10, 'ALA297': 10, 'PRO564': 10, 'ALA343': 10}, 'VAL347': {'VAL301': 10, 'PHE232': 10, 'PHE352': 10, 'ALA276': 6, 'PHE278': 10, 'PRO299': 10, 'GLN317': 5, 'LEU342': 10, 'LYS344': 10, 'ASN349': 10, 'ARG348': 10, 'SER345': 10, 'ILE316': 10, 'LEU240': 2, 'ILE318': 10, 'LEU303': 1, 'SER350': 10, 'ASP351': 10, 'PHE228': 10, 'ALA297': 5, 'ALA343': 10}, 'ARG348': {'PHE232': 10, 'ALA236': 10, 'PHE352': 10, 'ASN556': 10, 'PHE278': 1, 'ALA557': 10, 'TRP346': 10, 'TYR551': 10, 'GLN231': 10, 'LYS344': 10, 'ASN349': 10, 'PRO558': 10, 'TRP550': 10, 'SER345': 10, 'GLN353': 8, 'SER350': 10, 'ASP351': 10, 'LEU227': 2, 'PHE228': 10, 'ALA343': 10, 'ASN237': 10, 'VAL218': 8}, 'ASN349': {'THR420': 4, 'CYS559': 10, 'PHE352': 10, 'PRO582': 1, 'TYR295': 10, 'ALA557': 10, 'MET561': 10, 'TRP346': 10, 'TYR551': 2, 'VAL418': 7, 'SER419': 10, 'LEU281': 2, 'LYS344': 10, 'VAL347': 10, 'PRO558': 10, 'SER345': 10, 'GLN353': 10, 'LEU296': 10, 'SER350': 10, 'ASP351': 10, 'LEU354': 8, 'ALA297': 8, 'THR560': 10}, 'SER350': {'PRO299': 10, 'VAL347': 10, 'ASP351': 10, 'TRP346': 10, 'ARG348': 10, 'LEU354': 10, 'PRO558': 1, 'HIE355': 10, 'SER345': 10, 'GLN353': 10, 'ALA298': 10, 'PHE352': 10, 'ALA297': 10, 'ILE318': 5, 'SER419': 2, 'LEU296': 10, 'PHE278': 10, 'LEU281': 10}, 'ASP351': {'PHE232': 10, 'GLH356': 10, 'ALA236': 8, 'PHE352': 10, 'PHE278': 10, 'ALA557': 9, 'TRP346': 10, 'TYR551': 10, 'GLN231': 10, 'TYR359': 6, 'LEU281': 9, 'ASN349': 10, 'VAL347': 10, 'ARG348': 10, 'HIE355': 10, 'GLN353': 10, 'LEU240': 10, 'LEU354': 10, 'ASN237': 10}, 'PHE352': {'CYS559': 10, 'THR420': 6, 'GLN590': 10, 'GLN547': 3, 'GLH356': 10, 'ILE357': 10, 'PHE278': 1, 'ALA557': 10, 'PHE414': 10, 'SER594': 4, 'TYR551': 10, 'VAL418': 10, 'SER419': 4, 'ASN349': 10, 'VAL347': 10, 'PRO558': 10, 'ARG348': 10, 'HIE355': 10, 'GLN353': 10, 'SER350': 10, 'LEU354': 10, 'ASN237': 9, 'THR560': 1}, 'GLN353': {'LEU428': 10, 'THR420': 10, 'CYS559': 10, 'GLH356': 10, 'ILE357': 10, 'PHE414': 10, 'VAL418': 10, 'GLY421': 6, 'SER419': 10, 'LEU281': 10, 'ASN349': 10, 'PRO558': 10, 'ARG348': 8, 'HIE355': 10, 'GLN358': 10, 'LEU296': 10, 'SER350': 10, 'ASP351': 10, 'LEU354': 10, 'ILE284': 7, 'THR560': 10}, 'LEU354': {'LEU428': 10, 'ASP277': 10, 'GLH356': 10, 'ILE357': 10, 'PHE352': 10, 'ILE279': 5, 'ALA432': 2, 'PHE278': 10, 'TYR359': 10, 'LEU281': 10, 'ASN349': 8, 'GLN435': 1, 'HIE355': 10, 'LEU280': 10, 'LEU240': 10, 'GLN358': 10, 'SER350': 10, 'ASP351': 10, 'ASN237': 8}, 'HIE355': {'ASP351': 10, 'GLN358': 10, 'GLH356': 10, 'GLN547': 10, 'ALA236': 10, 'ILE357': 10, 'LEU361': 10, 'GLY546': 10, 'GLN353': 10, 'TYR551': 10, 'LEU597': 1, 'PHE352': 10, 'ALA542': 7, 'ILE543': 10, 'GLN231': 1, 'ASN237': 10, 'TYR359': 10, 'SER350': 10}, 'GLH356': {'THR420': 1, 'ILE593': 6, 'ASP351': 10, 'LEU354': 10, 'PHE414': 10, 'GLN547': 10, 'ILE357': 10, 'LEU362': 9, 'LEU361': 10, 'TYR551': 4, 'GLN353': 10, 'LEU597': 10, 'VAL418': 8, 'PHE352': 10, 'SER594': 1, 'ILE543': 1, 'GLN358': 10, 'TYR359': 7}, 'ILE357': {'LEU428': 10, 'THR420': 10, 'ILE408': 10, 'LEU361': 10, 'LEU429': 10, 'ASN363': 10, 'PHE352': 10, 'ALA432': 10, 'PHE414': 10, 'LEU362': 10, 'VAL418': 6, 'HID425': 10, 'TYR359': 10, 'LEU281': 1, 'HIE355': 10, 'GLN353': 10, 'GLN358': 10, 'LEU354': 10, 'LEU366': 1}, 'GLN358': {'LEU428': 10, 'ALA432': 10, 'LEU354': 10, 'GLH356': 10, 'GLN435': 10, 'HIE355': 10, 'ALA433': 1, 'LEU280': 10, 'LEU361': 10, 'LEU362': 10, 'GLN353': 10, 'ASN363': 10, 'THR364': 10, 'LEU429': 10, 'LEU240': 3, 'SER440': 1, 'TYR359': 10, 'LEU436': 9}, 'TYR359': {'ARG450': 10, 'GLH356': 7, 'ILE357': 10, 'ALA236': 1, 'LEU361': 10, 'ASN363': 10, 'MET239': 10, 'LEU362': 10, 'THR364': 10, 'PRO238': 1, 'GLN435': 2, 'HIE355': 10, 'LEU240': 10, 'ILE543': 10, 'ASP351': 6, 'LEU354': 10, 'ASN237': 10, 'SER440': 2, 'GLN539': 3}, 'LEU361': {'ILE408': 10, 'GLH356': 10, 'ILE399': 1, 'ILE357': 10, 'LEU429': 10, 'ASN363': 10, 'PHE414': 7, 'LEU362': 10, 'ARG404': 3, 'TRP144': 8, 'THR364': 10, 'HID425': 9, 'TYR359': 10, 'HIE355': 10, 'LEU407': 10, 'GLN358': 10, 'ASN400': 9, 'ALA403': 10, 'VAL367': 10, 'LEU366': 10}, 'LEU362': {'ALA368': 10, 'ARG430': 1, 'PRO149': 10, 'ALA432': 10, 'GLH356': 9, 'ALA433': 10, 'ILE357': 10, 'VAL367': 10, 'THR364': 10, 'ASN363': 10, 'LEU366': 10, 'LEU429': 10, 'TRP144': 1, 'LEU148': 9, 'GLN358': 10, 'LEU507': 6, 'TYR359': 10, 'LEU436': 10}, 'ASN363': {'ALA368': 7, 'ALA432': 10, 'MET239': 1, 'THR535': 10, 'GLN435': 7, 'ILE357': 10, 'VAL367': 10, 'LEU361': 10, 'THR364': 10, 'LEU366': 10, 'THR437': 3, 'LEU441': 10, 'ILE543': 2, 'GLN358': 10, 'SER440': 9, 'TYR359': 10, 'LEU436': 10, 'GLN539': 10}, 'THR364': {'ALA368': 10, 'CYS536': 10, 'ASN400': 1, 'VAL661': 10, 'MET239': 1, 'THR535': 10, 'VAL367': 10, 'LEU362': 10, 'LEU361': 10, 'LEU441': 8, 'LEU366': 10, 'THR396': 4, 'GLU369': 5, 'ALA542': 1, 'ILE543': 10, 'GLN358': 10, 'TYR359': 10, 'GLN539': 10}, 'LEU366': {'ILE408': 6, 'ILE357': 1, 'LEU361': 10, 'LEU429': 10, 'ASN363': 10, 'GLU369': 10, 'ALA368': 10, 'LEU362': 10, 'ARG404': 10, 'TRP144': 10, 'THR364': 10, 'LEU148': 10, 'LEU507': 1, 'VAL370': 10, 'PRO149': 10, 'LEU150': 1, 'THR151': 10, 'ASN400': 10, 'ALA403': 4, 'VAL367': 10, 'ILE371': 10}, 'VAL367': {'MET531': 6, 'LEU361': 10, 'LEU441': 10, 'ASN363': 10, 'GLU369': 10, 'ALA368': 10, 'CYS536': 2, 'THR535': 10, 'LEU362': 10, 'ILE502': 10, 'THR364': 10, 'CYS532': 2, 'LEU507': 10, 'LEU436': 10, 'ALA372': 10, 'VAL370': 10, 'PRO149': 6, 'ASN400': 7, 'PHE515': 10, 'ILE371': 10}, 'ALA368': {'CYS536': 10, 'ASN400': 10, 'VAL661': 10, 'THR535': 10, 'LEU362': 10, 'ALA372': 10, 'THR364': 10, 'ASN363': 7, 'LEU366': 10, 'ILE371': 10, 'THR396': 10, 'GLU369': 10, 'ILE393': 10, 'VAL370': 10, 'HID392': 6, 'VAL373': 10}, 'GLU369': {'THR151': 10, 'ASN400': 10, 'VAL367': 10, 'ARG404': 10, 'ALA374': 10, 'ALA372': 10, 'THR364': 5, 'LEU366': 10, 'ILE371': 10, 'THR396': 10, 'MET397': 10, 'ILE393': 3, 'VAL370': 10, 'THR401': 9, 'ILE152': 10, 'VAL373': 10}, 'VAL370': {'ALA368': 10, 'THR151': 10, 'GLU501': 10, 'THR375': 10, 'GLY506': 10, 'ASN400': 1, 'THR503': 1, 'VAL367': 10, 'ILE502': 10, 'ALA374': 10, 'ALA372': 10, 'LEU366': 10, 'ILE371': 10, 'PRO149': 10, 'LEU507': 10, 'LEU150': 10, 'VAL373': 10}, 'ILE371': {'MET531': 10, 'ALA374': 10, 'LEU441': 5, 'GLU369': 10, 'ILE393': 10, 'ALA368': 10, 'CYS536': 10, 'THR535': 10, 'ILE502': 10, 'PHE528': 10, 'CYS532': 10, 'TRP498': 10, 'LEU507': 2, 'LEU389': 10, 'MET376': 10, 'THR375': 10, 'ALA372': 10, 'PHE515': 3, 'VAL367': 10, 'VAL533': 5, 'LEU366': 10, 'VAL373': 10}, 'ALA372': {'ALA368': 10, 'MET376': 10, 'MET164': 10, 'THR375': 10, 'LEU389': 2, 'ARG377': 10, 'TYR139': 3, 'VAL367': 10, 'ALA374': 10, 'THR396': 10, 'GLU369': 10, 'ILE393': 10, 'VAL370': 10, 'MET397': 10, 'VAL373': 10}, 'VAL373': {'GLU501': 10, 'ALA374': 10, 'TRP141': 10, 'ALA153': 5, 'GLU369': 10, 'VAL505': 1, 'CYS140': 10, 'ALA368': 10, 'MET397': 10, 'CYS378': 10, 'MET376': 10, 'THR375': 10, 'VAL370': 10, 'GLY506': 4, 'ILE152': 10, 'LEU150': 10, 'THR151': 10, 'TYR139': 10, 'ILE371': 10, 'ALA142': 3, 'ARG377': 10}, 'ALA374': {'MET376': 10, 'GLU501': 10, 'CYS499': 3, 'THR375': 10, 'ARG377': 10, 'ILE502': 10, 'ALA372': 10, 'ILE371': 10, 'TRP141': 3, 'GLU369': 10, 'LEU379': 10, 'VAL370': 10, 'TRP498': 10, 'GLY506': 5, 'CYS378': 10}, 'THR375': {'MET376': 10, 'ILE385': 10, 'LEU389': 10, 'ARG377': 10, 'ALA372': 10, 'ILE371': 10, 'PHE386': 10, 'ILE390': 10, 'LEU379': 10, 'ILE393': 10, 'VAL370': 10, 'TRP498': 10, 'CYS378': 10, 'VAL373': 10}, 'MET376': {'PRO380': 3, 'ALA374': 10, 'PRO161': 1, 'ILE393': 10, 'TYR97': 9, 'ARG135': 10, 'PHE386': 10, 'ILE390': 10, 'LEU379': 10, 'MET397': 10, 'TRP498': 2, 'CYS378': 10, 'ALA372': 10, 'MET164': 10, 'TYR139': 10, 'ILE371': 10, 'ASN163': 10, 'ARG377': 10, 'VAL373': 10}, 'ARG377': {'GLN136': 0, 'GLU501': 10, 'GLN137': 10, 'THR375': 10, 'PRO380': 9, 'TYR139': 10, 'ALA374': 10, 'ALA372': 10, 'ARG135': 10, 'TRP141': 10, 'TRP498': 2, 'LEU379': 10, 'VAL505': 6, 'CYS140': 10, 'LEU132': 10, 'CYS378': 10, 'ARG500': 6, 'VAL373': 10}, 'CYS378': {'MET376': 10, 'GLN136': 10, 'GLH494': 10, 'ALA497': 10, 'THR375': 10, 'PRO380': 10, 'GLU501': 10, 'ILE502': 1, 'ALA374': 10, 'ARG135': 10, 'LEU379': 10, 'TRP498': 10, 'LEU132': 10, 'ARG500': 3, 'VAL373': 10}, 'LEU379': {'MET376': 10, 'GLH494': 10, 'ILE385': 10, 'ALA497': 3, 'THR375': 10, 'PRO380': 10, 'HIP383': 10, 'LEU495': 10, 'ALA374': 10, 'GLY381': 10, 'ARG135': 10, 'PHE386': 10, 'PRO384': 10, 'ARG377': 10, 'TRP498': 10, 'LEU132': 8, 'LEU389': 10}, 'PRO380': {'MET376': 3, 'GLH494': 10, 'GLN136': 1, 'LEU382': 10, 'GLU131': 10, 'HIP383': 10, 'ARG128': 10, 'GLY381': 10, 'ARG135': 10, 'PHE386': 10, 'ARG377': 9, 'LEU132': 10, 'CYS378': 10}, 'GLY381': {'LEU382': 10, 'GLU131': 10, 'HIP383': 10, 'PRO109': 4, 'ARG135': 10, 'PHE386': 10, 'LYS387': 10, 'LEU379': 10, 'PRO384': 1, 'TYR97': 9, 'ALA113': 10, 'CYS96': 1}, 'LEU382': {'GLU131': 6, 'GLU110': 10, 'PRO380': 10, 'HIP383': 10, 'HID127': 6, 'PRO109': 10, 'ARG128': 10, 'ARG114': 10, 'PHE124': 10, 'THR112': 10, 'PRO384': 10, 'PHE62': 10, 'ALA113': 10, 'LEU115': 1}, 'HIP383': {'GLH494': 10, 'ILE385': 10, 'TYR483': 10, 'PRO380': 10, 'ASP492': 10, 'ALA113': 10, 'PRO493': 6, 'PHE615': 4, 'LEU495': 10, 'ARG128': 2, 'GLY381': 10, 'ARG114': 10, 'PHE386': 10, 'LEU379': 10, 'PRO384': 10, 'LYS387': 10, 'PRO116': 10, 'LEU115': 10}, 'PRO384': {'ILE385': 10, 'LEU382': 10, 'TYR483': 10, 'ASP492': 9, 'ALA113': 10, 'PHE615': 10, 'PHE482': 10, 'LEU495': 8, 'PHE388': 10, 'GLY381': 1, 'ARG114': 10, 'PHE386': 10, 'LEU379': 10, 'LYS387': 10, 'PRO116': 9, 'LEU115': 10}, 'ILE385': {'TYR483': 10, 'THR375': 10, 'PHE615': 10, 'PHE482': 7, 'HIP383': 10, 'LEU495': 10, 'PHE388': 10, 'VAL479': 10, 'PHE386': 10, 'LYS387': 10, 'LEU379': 10, 'LEU529': 10, 'ILE390': 7, 'TRP498': 10, 'ALA113': 1, 'LEU389': 10, 'LEU525': 10}, 'PHE386': {'MET376': 10, 'MET164': 1, 'THR375': 10, 'PRO380': 10, 'PHE615': 1, 'HIP383': 10, 'PHE388': 10, 'GLY381': 10, 'ARG135': 1, 'PRO391': 8, 'ILE390': 10, 'LYS387': 10, 'LEU379': 10, 'PRO384': 10, 'TYR97': 10, 'ILE393': 10, 'LEU389': 10}, 'LYS387': {'ILE385': 10, 'LEU389': 10, 'TYR614': 10, 'GLY111': 10, 'PHE615': 10, 'HIP383': 10, 'SER616': 6, 'PHE388': 10, 'GLY381': 10, 'PRO391': 10, 'ILE390': 10, 'GLU612': 10, 'PRO384': 10, 'TYR97': 10, 'THR112': 2, 'LYS613': 10, 'ALA113': 10, 'PRO95': 10}, 'PHE388': {'TYR483': 4, 'PHE615': 10, 'ILE385': 10, 'TYR614': 10, 'VAL479': 10, 'ILE390': 10, 'PHE386': 10, 'PRO384': 10, 'LEU389': 10, 'VAL475': 10, 'PHE482': 10, 'PRO391': 10, 'HID392': 10, 'TYR474': 8, 'GLU476': 5, 'LEU481': 5, 'VAL533': 10, 'ILE478': 10, 'LEU529': 10}, 'LEU389': {'CYS536': 7, 'ILE385': 10, 'THR375': 10, 'TYR614': 10, 'ARG394': 7, 'ALA372': 2, 'VAL533': 10, 'ILE371': 10, 'PHE386': 10, 'LYS387': 10, 'LEU379': 10, 'ILE390': 10, 'PRO391': 10, 'HID392': 10, 'ILE393': 10, 'TRP498': 10, 'LEU529': 10, 'CYS532': 10}, 'ILE390': {'MET376': 10, 'ILE385': 7, 'MET164': 9, 'THR375': 10, 'TYR614': 8, 'ARG394': 10, 'PHE388': 10, 'PRO391': 10, 'PHE386': 10, 'ASN163': 8, 'LYS387': 10, 'HID392': 10, 'TYR97': 10, 'ILE393': 10, 'GLU612': 10}, 'PRO391': {'TYR97': 4, 'TYR614': 10, 'ARG394': 10, 'VAL661': 1, 'LYS611': 10, 'SER660': 10, 'ASN659': 10, 'PHE388': 10, 'HID392': 10, 'GLU612': 10, 'LYS387': 10, 'PHE386': 8, 'ILE393': 10, 'HID610': 10, 'LYS613': 10, 'LEU389': 10, 'LEU607': 1}, 'HID392': {'ASN659': 10, 'THR396': 10, 'PHE627': 5, 'ILE393': 10, 'ALA368': 6, 'CYS536': 10, 'TYR614': 10, 'ILE390': 10, 'CYS532': 5, 'LEU389': 10, 'LEU607': 10, 'SER660': 10, 'PHE388': 10, 'TYR474': 8, 'HID610': 10, 'VAL661': 10, 'ARG394': 10, 'TYR395': 1, 'VAL533': 10, 'LEU529': 1, 'THR537': 10}, 'ILE393': {'ALA368': 10, 'MET376': 10, 'CYS536': 10, 'MET164': 10, 'THR375': 10, 'VAL661': 10, 'ARG394': 10, 'TYR395': 10, 'SER660': 10, 'ALA372': 10, 'THR396': 10, 'ILE371': 10, 'PHE386': 10, 'ILE390': 10, 'GLU369': 3, 'PRO391': 10, 'MET397': 10, 'LEU389': 10}, 'ARG394': {'ASN659': 10, 'THR396': 10, 'HIE167': 10, 'VAL605': 10, 'ILE390': 10, 'MET397': 3, 'MET604': 10, 'LEU389': 7, 'SER660': 10, 'GLH658': 10, 'PRO391': 10, 'HID392': 10, 'ASP603': 10, 'MET164': 10, 'VAL661': 10, 'TYR395': 10, 'THR662': 10, 'LYS170': 10, 'ASN163': 10, 'GLN601': 10}, 'TYR395': {'ILE399': 10, 'THR396': 10, 'ARG165': 10, 'ILE393': 10, 'HIE167': 10, 'MET397': 10, 'PHE166': 10, 'SER660': 10, 'HID392': 1, 'ASP603': 7, 'MET164': 10, 'ASN400': 10, 'VAL661': 10, 'THR662': 10, 'LYS170': 10, 'PRO162': 2, 'ASN163': 10, 'GLU398': 10, 'PHE77': 1}, 'THR396': {'ALA368': 10, 'CYS536': 3, 'MET164': 7, 'ASN400': 10, 'VAL661': 10, 'ARG394': 10, 'SER660': 5, 'ILE399': 10, 'THR662': 9, 'ALA372': 10, 'THR364': 4, 'HID392': 10, 'GLU369': 10, 'ILE393': 10, 'MET397': 10, 'GLU398': 10, 'THR401': 10}, 'MET397': {'ILE399': 10, 'ARG165': 1, 'PRO161': 10, 'GLU369': 10, 'ILE393': 10, 'ALA154': 3, 'ARG404': 10, 'THR401': 10, 'MET376': 10, 'ARG402': 10, 'ALA372': 10, 'ILE152': 10, 'LEU160': 10, 'THR151': 4, 'MET164': 10, 'ASN400': 10, 'ARG394': 3, 'TYR395': 10, 'TYR139': 10, 'GLU398': 10, 'VAL373': 10}, 'GLU398': {'MET164': 10, 'ASN400': 10, 'PHE166': 10, 'TYR395': 10, 'ALA403': 9, 'ILE399': 10, 'ARG402': 10, 'THR396': 10, 'ARG165': 10, 'THR401': 10, 'ILE152': 1, 'LEU160': 10, 'LYS157': 10, 'ARG171': 2}, 'ILE399': {'ASN400': 10, 'PHE174': 10, 'PHE166': 10, 'TYR395': 10, 'VAL661': 10, 'ALA403': 10, 'THR662': 10, 'ARG402': 10, 'ARG404': 10, 'LYS170': 3, 'LEU361': 1, 'THR396': 10, 'LEU597': 2, 'MET397': 10, 'THR401': 10, 'HIE596': 1}, 'ASN400': {'LEU361': 9, 'THR396': 10, 'GLU369': 10, 'ALA368': 10, 'THR405': 10, 'ARG404': 10, 'THR364': 1, 'MET397': 10, 'THR401': 10, 'ARG402': 10, 'VAL370': 1, 'ILE152': 5, 'THR151': 10, 'VAL661': 9, 'TYR395': 10, 'ALA403': 10, 'VAL367': 7, 'LEU366': 10, 'GLU398': 10}, 'THR401': {'GLN406': 10, 'ALA403': 10, 'ILE399': 10, 'THR405': 10, 'ASP155': 10, 'ALA154': 10, 'ARG402': 10, 'ARG156': 10, 'ARG404': 10, 'THR396': 10, 'GLU369': 9, 'MET397': 10, 'GLU398': 10, 'ILE152': 10, 'LEU160': 10}, 'ARG402': {'ASN400': 10, 'PHE174': 9, 'PHE166': 8, 'GLN406': 10, 'ILE408': 10, 'ALA403': 10, 'ILE399': 10, 'THR405': 10, 'ARG404': 10, 'ARG156': 8, 'LEU407': 10, 'MET397': 10, 'GLU398': 10, 'LYS157': 8}, 'ALA403': {'ASN400': 10, 'GLN406': 10, 'ILE408': 10, 'ILE399': 10, 'THR405': 10, 'SER409': 10, 'ARG404': 10, 'LEU361': 10, 'LEU407': 10, 'LEU366': 4, 'THR401': 10, 'GLU398': 9}, 'ARG404': {'ILE408': 10, 'ILE399': 10, 'LYS145': 3, 'ASP155': 8, 'LEU361': 3, 'GLU369': 10, 'THR405': 10, 'ALA154': 10, 'TRP144': 10, 'MET397': 10, 'THR401': 10, 'GLN406': 10, 'ARG402': 10, 'LEU407': 10, 'PRO149': 8, 'ILE152': 10, 'THR151': 10, 'ASN400': 10, 'SER409': 10, 'LEU366': 10}, 'THR405': {'ASN400': 10, 'GLN406': 10, 'ILE408': 9, 'ALA403': 10, 'ASP155': 10, 'ALA154': 10, 'ARG402': 10, 'ARG156': 10, 'LEU407': 10, 'SER409': 10, 'THR401': 10}, 'GLN406': {'GLY412': 10, 'ILE408': 10, 'ALA403': 10, 'PHE414': 2, 'SER409': 10, 'ARG404': 10, 'ARG402': 10, 'LEU407': 10, 'ASP155': 5, 'ILE413': 10, 'THR401': 10}, 'LEU407': {'ILE593': 10, 'GLY412': 10, 'ILE408': 10, 'ALA403': 10, 'THR405': 10, 'PHE414': 10, 'SER409': 10, 'ARG404': 10, 'ARG402': 10, 'LEU361': 10, 'ASP415': 10, 'LEU597': 1, 'ILE413': 10}, 'ILE408': {'LYS145': 9, 'ILE357': 10, 'LEU361': 10, 'LEU429': 10, 'PHE414': 10, 'THR405': 9, 'ARG404': 10, 'TRP144': 10, 'HID425': 10, 'GLY421': 1, 'LEU148': 1, 'VAL426': 2, 'GLN406': 10, 'GLY412': 10, 'ARG402': 10, 'ILE413': 10, 'ASP410': 10, 'ALA403': 10, 'SER409': 10, 'ASP415': 10, 'LEU366': 6}, 'SER409': {'ASP410': 10, 'GLN406': 10, 'GLY412': 10, 'ALA403': 10, 'THR405': 10, 'LYS145': 7, 'ASP415': 10, 'ARG404': 10, 'PHE414': 6, 'LEU407': 10, 'HID425': 7, 'ILE413': 9, 'GLY411': 10}, 'ASP410': {'GLY412': 10, 'ILE408': 10, 'GLY422': 9, 'ASP415': 10, 'HID425': 3, 'GLY411': 10}, 'GLY411': {'GLY412': 10, 'LYS416': 10, 'GLY422': 4, 'ASP415': 10, 'SER409': 10, 'ILE413': 10}, 'GLY412': {'ASP410': 10, 'GLN406': 10, 'ILE408': 10, 'PHE414': 10, 'LYS416': 10, 'ALA417': 10, 'SER409': 10, 'ASP415': 10, 'LEU407': 10, 'ILE413': 10}, 'ILE413': {'ILE593': 10, 'GLN406': 10, 'ILE408': 10, 'PHE414': 10, 'LYS416': 10, 'ALA417': 10, 'SER409': 9, 'ASP415': 10, 'LEU589': 10, 'LEU407': 10, 'VAL418': 10, 'GLY411': 10, 'LEU178': 5}, 'PHE414': {'THR420': 10, 'ILE408': 10, 'GLH356': 10, 'GLY422': 10, 'ILE357': 10, 'LEU361': 7, 'PHE352': 10, 'ALA417': 10, 'VAL418': 10, 'HID425': 10, 'GLY421': 10, 'SER419': 1, 'GLY412': 10, 'GLN406': 2, 'LEU407': 10, 'GLN353': 10, 'ASN287': 10, 'LYS416': 10, 'ASP415': 10, 'SER409': 6}, 'ASP415': {'VAL288': 10, 'ILE289': 10, 'ASP410': 10, 'GLY412': 10, 'ILE408': 10, 'ALA417': 10, 'LYS416': 10, 'GLY422': 10, 'SER409': 10, 'LEU407': 10, 'VAL418': 10, 'ILE413': 10, 'GLY411': 10, 'GLY421': 10, 'HID425': 9, 'ASN287': 10}, 'LYS416': {'ILE289': 10, 'GLY412': 10, 'PHE414': 10, 'ALA417': 10, 'LEU589': 10, 'ARG290': 10, 'VAL418': 10, 'ILE413': 10, 'GLY411': 10, 'GLN586': 10, 'GLY421': 1, 'ASN287': 10, 'MET185': 1, 'ARG585': 10}, 'ALA417': {'ILE289': 10, 'CYS559': 10, 'GLN590': 8, 'GLY412': 10, 'PHE414': 10, 'ASP415': 10, 'LEU589': 10, 'GLN294': 4, 'VAL418': 10, 'ILE413': 10, 'GLN586': 10, 'GLY421': 6, 'SER419': 10, 'ASN287': 10, 'PRO582': 1, 'THR560': 10}, 'VAL418': {'THR420': 10, 'CYS559': 10, 'GLH356': 8, 'ILE357': 6, 'GLN294': 10, 'PHE352': 10, 'PHE414': 10, 'GLN586': 6, 'GLY421': 10, 'SER419': 10, 'ASN349': 7, 'ILE289': 1, 'GLN353': 10, 'ILE413': 10, 'ASN287': 10, 'LEU296': 3, 'LYS416': 10, 'ASP415': 10, 'LEU589': 2, 'THR560': 10}, 'SER419': {'THR420': 10, 'CYS559': 10, 'GLN294': 10, 'PHE352': 4, 'TYR295': 10, 'MET561': 10, 'ALA417': 10, 'PHE414': 1, 'PRO285': 7, 'GLY421': 10, 'ASN349': 10, 'GLN353': 10, 'ASN287': 10, 'LEU296': 10, 'SER350': 2, 'ALA286': 10, 'ALA297': 1, 'ILE284': 9, 'THR560': 10}, 'THR420': {'LEU428': 10, 'CYS559': 10, 'GLH356': 1, 'GLY422': 10, 'ILE357': 10, 'GLY423': 3, 'PHE352': 6, 'PHE414': 10, 'PRO285': 9, 'VAL418': 10, 'HID425': 10, 'GLY421': 10, 'VAL426': 1, 'LEU281': 10, 'ASN349': 4, 'GLN353': 10, 'ASN287': 10, 'LEU296': 10, 'ALA286': 10, 'ILE284': 10, 'THR560': 2, 'GLY424': 10}, 'GLY421': {'ALA286': 10, 'VAL288': 5, 'PHE414': 10, 'ALA417': 6, 'GLY422': 10, 'LYS416': 1, 'ASP415': 10, 'ILE408': 1, 'PRO285': 10, 'GLN353': 6, 'GLY423': 10, 'VAL418': 10, 'HID425': 10, 'SER419': 10, 'ASN287': 10, 'GLY424': 10}, 'GLY422': {'ALA286': 3, 'VAL288': 6, 'THR420': 10, 'ASP410': 9, 'PHE414': 10, 'ASP415': 10, 'PRO285': 2, 'GLY423': 10, 'HID425': 10, 'GLY411': 4, 'ASN287': 10, 'VAL426': 10, 'GLY424': 10}, 'GLY423': {'LEU428': 8, 'THR420': 3, 'ARG430': 1, 'GLN427': 10, 'PRO285': 10, 'HID425': 10, 'GLY421': 10, 'VAL426': 10, 'GLY424': 10}, 'GLY424': {'ALA286': 9, 'LEU428': 10, 'THR420': 10, 'GLY422': 10, 'GLN427': 10, 'PRO285': 10, 'LEU429': 10, 'HID425': 10, 'GLY421': 10, 'ASN287': 1, 'VAL426': 10, 'ILE284': 10}, 'HID425': {'LEU428': 10, 'THR420': 10, 'ARG430': 10, 'ILE408': 10, 'GLY422': 10, 'LYS145': 10, 'ILE357': 10, 'LEU361': 9, 'GLY423': 10, 'LEU429': 10, 'PHE414': 10, 'PRO285': 2, 'TRP144': 6, 'GLY421': 10, 'LEU148': 6, 'VAL426': 10, 'ASP410': 3, 'SER409': 7, 'ASP415': 9, 'GLN427': 10, 'ILE284': 10}, 'VAL426': {'LEU428': 10, 'ARG430': 10, 'THR420': 1, 'ILE408': 2, 'GLY422': 10, 'LYS145': 10, 'GLN427': 10, 'GLY423': 10, 'LEU429': 10, 'TRP144': 2, 'ARG431': 10, 'LEU148': 10, 'GLY424': 10}, 'GLN427': {'LEU428': 10, 'GLY283': 6, 'ARG430': 10, 'LEU280': 3, 'PRO285': 10, 'GLY423': 10, 'LEU429': 10, 'HID425': 10, 'ARG431': 10, 'LEU148': 1, 'ILE284': 10, 'ALA432': 10, 'GLY424': 10}, 'LEU428': {'THR420': 10, 'ARG430': 10, 'ILE357': 10, 'GLY423': 8, 'LEU429': 10, 'ALA432': 10, 'PRO285': 3, 'HID425': 10, 'VAL426': 10, 'LEU281': 10, 'ALA433': 10, 'LEU280': 10, 'GLN353': 10, 'GLN358': 10, 'LEU296': 8, 'LEU354': 10, 'ARG431': 10, 'ILE284': 10, 'GLY424': 10}, 'LEU429': {'ARG430': 10, 'ILE408': 10, 'LYS145': 6, 'ILE357': 10, 'LEU361': 10, 'ALA432': 10, 'LEU362': 10, 'TRP144': 10, 'HID425': 10, 'LEU148': 10, 'VAL426': 10, 'ALA433': 10, 'ALA434': 10, 'GLN358': 10, 'PRO149': 8, 'GLN427': 10, 'LEU366': 10, 'ARG431': 10, 'GLY424': 10}, 'ARG430': {'LEU428': 10, 'GLU146': 10, 'ALA433': 10, 'ALA434': 10, 'LYS145': 7, 'GLY147': 10, 'GLN435': 7, 'GLN427': 10, 'LEU362': 1, 'GLY423': 1, 'HID425': 10, 'ARG431': 10, 'LEU148': 10, 'VAL426': 10, 'ALA432': 10}, 'ARG431': {'LEU428': 10, 'GLY283': 10, 'ASP282': 10, 'ALA433': 10, 'GLN435': 10, 'LEU280': 10, 'GLN427': 10, 'LEU429': 10, 'ALA434': 10, 'ILE279': 10, 'VAL426': 10, 'ILE284': 10, 'ALA432': 10, 'LEU436': 8, 'LEU281': 10}, 'ALA432': {'LEU428': 10, 'ARG430': 10, 'ARG513': 4, 'LEU354': 2, 'ALA433': 10, 'GLN435': 10, 'ILE357': 10, 'LEU362': 10, 'LEU280': 10, 'GLN427': 10, 'LEU429': 10, 'ASN363': 10, 'ALA434': 10, 'GLN358': 10, 'LEU436': 10}, 'ALA433': {'LEU428': 10, 'ARG430': 10, 'ARG513': 10, 'GLN358': 1, 'GLN435': 10, 'ALA434': 10, 'GLY147': 1, 'LEU362': 10, 'LEU429': 10, 'THR437': 1, 'ARG431': 10, 'LEU148': 10, 'PRO149': 10, 'LEU507': 1, 'LEU436': 10}, 'ALA434': {'ARG430': 10, 'ARG513': 10, 'GLN435': 10, 'LEU280': 4, 'LEU429': 10, 'THR437': 10, 'ARG431': 10, 'ALA432': 10, 'LEU436': 10}, 'GLN435': {'ARG242': 6, 'ARG430': 7, 'ARG513': 10, 'ARG450': 7, 'LEU354': 1, 'ALA433': 10, 'ASP277': 4, 'TYR359': 2, 'LEU280': 10, 'ASN363': 7, 'THR437': 10, 'ARG431': 10, 'ILE279': 7, 'GLN358': 10, 'SER440': 10, 'ALA432': 10, 'LEU436': 10}, 'LEU436': {'PRO149': 6, 'ARG513': 10, 'ALA433': 10, 'TYR438': 10, 'PHE515': 10, 'VAL367': 10, 'LEU362': 10, 'LEU441': 10, 'ASN363': 10, 'THR437': 10, 'ALA510': 8, 'ARG431': 8, 'ALA434': 10, 'LEU507': 10, 'GLN358': 9, 'SER440': 10, 'ALA432': 10}, 'THR437': {'ARG513': 10, 'ARG450': 1, 'GLN435': 10, 'TYR438': 10, 'PHE515': 10, 'ASP512': 4, 'ALA433': 1, 'CYS439': 10, 'LEU441': 10, 'ASN363': 3, 'ALA510': 1, 'GLY514': 9, 'ASP446': 2, 'ALA434': 10, 'SER440': 10}, 'TYR438': {'ARG513': 10, 'MET531': 10, 'PHE515': 10, 'CYS439': 10, 'LEU441': 10, 'GLY514': 10, 'PRO516': 10, 'PHE519': 6, 'GLN524': 10, 'PHE528': 10, 'CYS442': 10, 'TRP468': 1, 'ASP445': 1, 'ALA510': 4, 'SER440': 10, 'LEU436': 10, 'HIE527': 10, 'CYS532': 1}, 'CYS439': {'ASP449': 6, 'ARG513': 10, 'SER440': 10, 'LEU441': 10, 'GLY514': 10, 'THR437': 10, 'ASP445': 10, 'ASP446': 10, 'CYS442': 10, 'HIE527': 7}, 'SER440': {'ARG450': 9, 'MET239': 1, 'PRO443': 1, 'GLN435': 10, 'TYR438': 10, 'PHE515': 5, 'TYR359': 2, 'LEU441': 10, 'ASN363': 9, 'THR437': 10, 'ASP445': 1, 'ASP446': 10, 'GLN358': 1, 'CYS442': 10, 'LEU436': 10, 'GLN539': 10}, 'LEU441': {'MET531': 10, 'PRO443': 10, 'PHE534': 10, 'THR535': 10, 'PHE515': 10, 'TYR438': 10, 'VAL367': 10, 'CYS439': 10, 'THR364': 8, 'ASN363': 10, 'ILE371': 5, 'THR437': 10, 'TRP468': 10, 'ASP446': 10, 'CYS442': 10, 'LEU436': 0, 'GLN539': 10}, 'CYS442': {'MET531': 10, 'PRO443': 10, 'PHE534': 6, 'TYR438': 10, 'CYS439': 10, 'LEU447': 10, 'ASP445': 10, 'TRP468': 10, 'ASP446': 10, 'PRO444': 10, 'SER440': 10, 'HIE527': 10, 'GLN539': 9, 'THR530': 6}, 'PRO443': {'LEU453': 10, 'GLN539': 10, 'PHE534': 10, 'ALA448': 10, 'SER440': 1, 'LEU441': 10, 'LEU447': 10, 'ASP445': 10, 'TRP468': 10, 'ASP446': 10, 'PRO444': 10, 'TYR460': 10, 'LEU465': 10, 'ALA464': 10, 'ALA461': 10}, 'PRO444': {'LEU453': 10, 'ASP449': 3, 'ALA448': 10, 'LEU447': 10, 'ASP445': 10, 'LEU465': 10, 'TRP468': 10, 'ASP446': 10, 'HIE527': 8, 'CYS442': 10, 'ALA464': 10, 'GLU469': 8}, 'ASP445': {'LEU453': 1, 'ASP449': 10, 'PRO443': 10, 'ARG450': 10, 'ALA448': 10, 'TYR438': 1, 'SER440': 1, 'CYS439': 10, 'LEU447': 10, 'ASP446': 10, 'CYS442': 10, 'HIE527': 10}, 'ASP446': {'ARG242': 3, 'ASP449': 10, 'MET239': 10, 'PRO443': 10, 'ARG450': 10, 'ALA448': 10, 'CYS439': 10, 'LEU441': 10, 'LEU447': 10, 'THR437': 2, 'GLY451': 8, 'CYS442': 10, 'PRO444': 10, 'LEU452': 3, 'SER440': 10}, 'LEU447': {'ARG450': 10, 'GLY451': 10, 'PRO444': 10, 'ALA461': 10, 'MET239': 10, 'PRO443': 10, 'GLY454': 4, 'LEU465': 10, 'LEU452': 10, 'TYR460': 10, 'PRO238': 10, 'LEU453': 10, 'ASP449': 10, 'ASP445': 10, 'CYS442': 10, 'ALA458': 5, 'ALA464': 10, 'ALA448': 10, 'HID462': 3, 'GLN539': 3}, 'ALA448': {'LEU453': 10, 'ASP449': 10, 'PRO443': 10, 'ARG450': 10, 'GLY454': 7, 'ASP445': 10, 'ASP446': 10, 'PRO444': 10, 'GLY451': 10, 'LEU452': 10, 'LEU465': 3}, 'ASP449': {'ARG242': 10, 'ARG450': 10, 'ARG243': 7, 'CYS439': 6, 'LEU447': 10, 'ASP445': 10, 'ASP446': 10, 'GLY451': 10, 'PRO444': 3, 'LEU452': 6}, 'ARG450': {'ARG242': 10, 'LEU453': 1, 'MET239': 10, 'LEU241': 10, 'ASP277': 9, 'GLN435': 7, 'ALA448': 10, 'ARG243': 10, 'LEU447': 10, 'LEU240': 10, 'ASP445': 10, 'THR437': 1, 'ASP446': 10, 'GLY451': 10, 'LEU452': 10, 'SER440': 9, 'TYR359': 10, 'PHE274': 10}, 'GLY451': {'ARG242': 7, 'LEU453': 10, 'ASP449': 10, 'ALA448': 10, 'GLY454': 10, 'ARG243': 10, 'LEU447': 10, 'ASP446': 8, 'LEU452': 10, 'LEU455': 10, 'PHE274': 10}, 'LEU452': {'ARG242': 10, 'PHE232': 9, 'ARG450': 10, 'LEU241': 10, 'ALA461': 10, 'MET239': 10, 'GLY454': 10, 'TYR460': 10, 'PRO238': 10, 'PHE274': 10, 'LEU453': 10, 'ASP449': 6, 'ARG243': 9, 'LEU447': 10, 'LEU240': 7, 'LEU455': 10, 'LEU233': 10, 'ALA458': 10, 'ALA448': 10, 'ASP446': 3, 'HID462': 9, 'LEU311': 3}, 'LEU453': {'PRO456': 8, 'PRO443': 10, 'ARG450': 1, 'ALA448': 10, 'GLY454': 10, 'LEU447': 10, 'LEU465': 10, 'ASP445': 1, 'GLY451': 10, 'PRO444': 10, 'LEU455': 10, 'HID462': 10, 'ALA458': 10, 'ALA461': 10}, 'GLY454': {'PRO456': 10, 'ALA448': 7, 'ARG243': 4, 'LEU447': 4, 'GLY451': 10, 'LEU452': 10, 'HID462': 10, 'LEU455': 10, 'PHE274': 1}, 'LEU455': {'GLY309': 10, 'LEU453': 10, 'PRO456': 10, 'MET305': 10, 'LEU241': 10, 'ARG243': 7, 'LEU311': 10, 'GLY457': 10, 'SER272': 1, 'GLU306': 7, 'GLY451': 10, 'LEU452': 10, 'LEU233': 10, 'HID462': 4, 'LYS310': 10, 'ALA458': 10, 'PHE274': 10}, 'PRO456': {'GLY309': 10, 'LEU453': 8, 'MET305': 9, 'GLY454': 10, 'LEU459': 10, 'GLY457': 10, 'ASN308': 10, 'HID462': 8, 'LYS310': 8, 'ALA458': 10, 'TRP647': 8}, 'GLY457': {'GLY309': 10, 'LEU233': 10, 'MET305': 1, 'TYR460': 7, 'TYR230': 5, 'LEU459': 10, 'ASN234': 10, 'LEU311': 9, 'LEU455': 10, 'ASN308': 9, 'LYS310': 10, 'ALA458': 10, 'TRP647': 10}, 'ALA458': {'GLY309': 6, 'LEU453': 10, 'PRO456': 10, 'ASP463': 2, 'TYR460': 10, 'LEU241': 4, 'LEU459': 10, 'ASN234': 10, 'HID462': 10, 'LEU447': 5, 'LEU452': 10, 'LEU233': 10, 'PRO238': 10, 'LEU455': 10, 'ALA461': 10}, 'LEU459': {'PRO456': 10, 'ARG641': 4, 'LEU233': 10, 'ASP463': 10, 'PRO648': 10, 'ASN642': 10, 'TYR649': 10, 'GLY457': 10, 'ASN234': 10, 'LEU645': 10, 'TYR460': 10, 'HID462': 10, 'ALA464': 10, 'TRP647': 10, 'ALA461': 10}, 'TYR460': {'ASP463': 10, 'TYR651': 1, 'TYR649': 10, 'ALA461': 10, 'PHE534': 10, 'MET239': 10, 'PRO443': 10, 'LEU652': 10, 'ALA538': 3, 'ASN234': 10, 'LEU465': 10, 'LEU452': 10, 'PRO238': 10, 'GLY235': 2, 'LEU447': 10, 'ALA542': 10, 'LEU233': 10, 'ALA458': 10, 'ALA464': 10, 'GLY457': 7, 'HID462': 10, 'GLN539': 10}, 'ALA461': {'LEU453': 10, 'ASP463': 10, 'PRO443': 10, 'LEU459': 10, 'ASN234': 4, 'LEU447': 10, 'ARG466': 10, 'LEU465': 10, 'PRO238': 3, 'LEU452': 10, 'LEU233': 10, 'HID462': 10, 'ALA458': 10, 'ALA464': 10}, 'HID462': {'LEU453': 10, 'PRO456': 8, 'ARG641': 6, 'ASP463': 10, 'LEU634': 5, 'GLY454': 10, 'LEU459': 10, 'LEU447': 3, 'ARG466': 10, 'LEU467': 10, 'LEU452': 9, 'TYR460': 10, 'LEU465': 10, 'LEU455': 4, 'ALA458': 10, 'ALA464': 10}, 'ASP463': {'LYS653': 2, 'ARG641': 9, 'PHE534': 5, 'LEU634': 10, 'LEU652': 10, 'LEU459': 10, 'ILE638': 10, 'ARG466': 10, 'TRP468': 10, 'GLU650': 4, 'LEU467': 10, 'TYR460': 10, 'LEU465': 10, 'ALA458': 2, 'TYR649': 10, 'ALA461': 10, 'ALA464': 10}, 'ALA464': {'GLN539': 6, 'PHE534': 10, 'PRO443': 10, 'LEU652': 10, 'LEU459': 10, 'ALA538': 8, 'LEU447': 10, 'ARG466': 10, 'LEU465': 10, 'TRP468': 10, 'PRO444': 10, 'LEU467': 10, 'TYR460': 10, 'HID462': 10, 'TYR649': 9, 'ALA461': 10, 'GLU469': 10}, 'LEU465': {'LEU453': 10, 'ASP463': 10, 'PRO443': 10, 'PHE534': 6, 'ALA448': 3, 'LEU447': 10, 'ARG466': 10, 'TRP468': 10, 'PRO444': 10, 'LEU467': 10, 'ILE470': 9, 'TYR460': 10, 'HID462': 10, 'ALA461': 10, 'GLU469': 10}, 'ARG466': {'ARG641': 10, 'ILE471': 5, 'ASP463': 10, 'LEU634': 10, 'TRP468': 10, 'GLU637': 10, 'LEU467': 10, 'ILE470': 10, 'ILE638': 9, 'HID462': 10, 'ALA464': 10, 'ALA461': 10, 'GLU469': 10}, 'LEU467': {'ASP463': 10, 'PHE627': 9, 'HID462': 10, 'ILE657': 10, 'PHE534': 10, 'LEU652': 10, 'ALA538': 10, 'TRP468': 10, 'LEU465': 10, 'LEU607': 10, 'PRO654': 10, 'ILE470': 10, 'ILE638': 9, 'LEU634': 10, 'ALA464': 10, 'GLU469': 10, 'ALA472': 10, 'LEU631': 10, 'ILE471': 10}, 'TRP468': {'ASP463': 10, 'MET531': 10, 'TYR438': 1, 'LEU441': 10, 'PRO444': 10, 'HIE527': 10, 'ILE471': 10, 'THR530': 10, 'PHE534': 10, 'PRO443': 10, 'THR535': 10, 'ARG466': 10, 'PHE528': 2, 'CYS442': 10, 'ILE470': 10, 'ALA464': 10, 'GLU469': 10, 'ALA472': 10, 'LEU465': 10, 'ARG473': 10, 'GLN539': 1}, 'GLU469': {'ARG473': 10, 'ARG466': 10, 'TYR474': 10, 'ALA472': 10, 'PRO444': 8, 'LEU467': 10, 'ILE470': 10, 'LEU465': 10, 'ALA464': 10, 'ILE471': 10, 'THR530': 5}, 'ILE470': {'VAL475': 9, 'LYS633': 4, 'LEU634': 10, 'ASP630': 10, 'ARG466': 10, 'TRP468': 10, 'PHE627': 10, 'TYR474': 10, 'ALA472': 10, 'GLN626': 9, 'LEU467': 10, 'LEU631': 10, 'PRO654': 2, 'LEU465': 9, 'ARG473': 10, 'ILE471': 10}, 'ILE471': {'PHE627': 10, 'THR530': 10, 'ILE657': 5, 'PHE534': 10, 'ALA538': 10, 'ARG466': 5, 'TRP468': 10, 'LEU607': 10, 'VAL475': 10, 'GLU476': 10, 'TYR474': 10, 'LEU631': 10, 'GLU469': 10, 'VAL533': 10, 'LEU529': 3, 'ALA472': 10, 'LEU467': 10, 'THR537': 10, 'ARG473': 10}, 'ALA472': {'THR530': 10, 'VAL475': 10, 'GLY477': 10, 'CYS526': 10, 'TRP468': 10, 'GLU476': 10, 'TYR474': 10, 'HIE527': 4, 'LEU467': 10, 'ILE470': 10, 'ARG473': 10, 'GLU469': 10}, 'ARG473': {'VAL475': 10, 'GLY477': 10, 'ASP630': 10, 'VAL623': 10, 'ILE478': 10, 'TRP468': 10, 'GLU476': 10, 'TYR474': 10, 'GLN626': 10, 'PHE627': 2, 'ILE470': 10, 'ILE471': 10, 'GLU469': 10}, 'TYR474': {'GLY477': 10, 'PHE627': 10, 'TYR614': 10, 'VAL623': 10, 'PRO620': 9, 'VAL479': 10, 'GLN626': 5, 'LEU607': 4, 'VAL475': 10, 'PHE388': 8, 'LEU624': 10, 'HID392': 8, 'GLU476': 10, 'HID610': 10, 'ILE470': 10, 'GLU469': 10, 'ILE478': 10, 'VAL533': 10, 'ALA472': 10, 'ILE471': 10}, 'VAL475': {'HID480': 10, 'TYR614': 10, 'GLY477': 10, 'CYS526': 10, 'VAL623': 7, 'PHE388': 10, 'ILE478': 10, 'VAL479': 10, 'VAL533': 10, 'GLU476': 10, 'LEU529': 10, 'ALA472': 10, 'ILE470': 9, 'ILE471': 10, 'ARG473': 10, 'THR530': 10}, 'GLU476': {'HID480': 10, 'GLN522': 10, 'SER523': 10, 'GLY477': 10, 'CYS526': 10, 'VAL623': 1, 'PHE388': 5, 'ILE478': 10, 'VAL479': 10, 'LEU481': 9, 'LEU529': 7, 'TYR474': 10, 'ALA472': 10, 'ILE471': 10, 'ARG473': 10, 'THR530': 3}, 'GLY477': {'VAL475': 10, 'HID480': 10, 'VAL623': 10, 'LEU481': 10, 'ILE478': 10, 'VAL479': 10, 'TYR474': 10, 'ALA472': 10, 'ARG473': 10}, 'ILE478': {'VAL475': 10, 'HID480': 10, 'TYR483': 5, 'LYS619': 7, 'TYR614': 10, 'PHE615': 10, 'PHE482': 10, 'VAL623': 10, 'PHE388': 10, 'VAL479': 10, 'LEU481': 10, 'PRO620': 10, 'GLU476': 10, 'TYR474': 10, 'LEU624': 10, 'LYS621': 1, 'ARG473': 10}, 'VAL479': {'VAL475': 10, 'ILE385': 10, 'HID480': 10, 'TYR483': 10, 'GLN522': 10, 'PHE482': 10, 'GLY477': 10, 'CYS526': 10, 'PHE388': 10, 'LEU481': 10, 'VAL489': 5, 'GLU476': 10, 'TYR474': 10, 'LEU529': 10, 'GLN484': 9, 'LEU525': 10}, 'HID480': {'VAL475': 10, 'TYR483': 10, 'GLN522': 10, 'PHE482': 10, 'GLY477': 10, 'ARG485': 5, 'CYS526': 6, 'LEU481': 10, 'ILE478': 10, 'GLU476': 10, 'GLN484': 10}, 'LEU481': {'TYR483': 10, 'LYS619': 10, 'PHE615': 8, 'PHE482': 10, 'GLY477': 10, 'VAL623': 10, 'PRO620': 10, 'ILE478': 10, 'VAL479': 10, 'PHE388': 5, 'GLU476': 9, 'GLN484': 7}, 'PHE482': {'ILE385': 7, 'HID480': 10, 'TYR483': 10, 'ILE488': 10, 'PHE615': 10, 'PHE388': 10, 'ILE478': 10, 'VAL479': 10, 'PRO620': 10, 'ARG114': 4, 'PRO384': 10, 'THR112': 7, 'GLN484': 10, 'ALA113': 10, 'LEU115': 10}, 'TYR483': {'HID480': 10, 'GLN522': 10, 'LEU495': 10, 'ILE385': 10, 'VAL479': 10, 'PRO384': 10, 'PRO116': 5, 'ASP486': 4, 'LEU115': 10, 'ILE488': 10, 'ARG485': 10, 'HIP383': 10, 'PHE388': 4, 'LEU525': 10, 'ASP492': 10, 'LEU481': 10, 'ILE478': 5, 'VAL489': 10, 'GLN484': 10}, 'GLN484': {'HID480': 10, 'ILE488': 10, 'GLN522': 10, 'PHE482': 10, 'ARG485': 10, 'LEU481': 7, 'VAL479': 9, 'ASP486': 10}, 'ARG485': {'HID480': 5, 'ILE488': 10, 'TYR483': 10, 'GLN522': 10, 'LYS490': 5, 'VAL489': 10, 'ASP487': 10, 'ASP486': 10}, 'ASP486': {'TYR483': 4, 'ILE488': 10, 'LYS490': 10, 'GLN522': 10, 'GLY491': 9, 'SER523': 10, 'VAL489': 10, 'GLN520': 10, 'ASP487': 10, 'GLN484': 10, 'SER521': 10}, 'ASP487': {'ILE488': 10, 'LYS490': 10, 'GLN522': 1, 'ARG485': 10, 'GLY491': 10, 'VAL489': 10}, 'ILE488': {'TYR483': 10, 'LYS490': 10, 'ASP492': 10, 'GLN522': 4, 'PHE482': 10, 'ARG485': 10, 'GLY491': 10, 'VAL489': 10, 'GLN484': 10, 'ASP486': 10, 'LEU115': 10}, 'VAL489': {'TYR483': 10, 'LYS490': 10, 'ASP492': 10, 'GLN522': 10, 'ARG485': 10, 'GLY491': 10, 'LEU495': 10, 'VAL479': 5, 'PHE519': 10, 'GLN520': 7, 'ASP487': 10, 'ASP486': 10, 'SER521': 10, 'LEU525': 10}, 'LYS490': {'ILE488': 10, 'PRO493': 3, 'ASP492': 10, 'GLY491': 10, 'ARG485': 5, 'LEU495': 1, 'PHE519': 10, 'SER518': 1, 'GLN520': 10, 'ASP487': 10, 'ASP486': 10, 'SER521': 10}, 'GLY491': {'ILE488': 10, 'PRO493': 10, 'ASP492': 10, 'VAL489': 10, 'ASP487': 10, 'PRO116': 10, 'ASP486': 9, 'LEU115': 1}, 'ASP492': {'GLH494': 10, 'TYR483': 10, 'ILE488': 10, 'ALA497': 7, 'LYS490': 10, 'PRO493': 10, 'GLN496': 10, 'HIP383': 10, 'LEU495': 10, 'VAL489': 10, 'PRO384': 9, 'PRO116': 10, 'LEU115': 10}, 'PRO493': {'GLH494': 10, 'ALA497': 10, 'GLN496': 10, 'LYS490': 3, 'GLY491': 10, 'HIP383': 6, 'LEU495': 10, 'TRP498': 10, 'PRO116': 10}, 'GLH494': {'CYS499': 9, 'ALA497': 10, 'PRO380': 10, 'ASP492': 10, 'GLN496': 10, 'HIP383': 10, 'LEU495': 10, 'ARG128': 10, 'LEU379': 10, 'TRP498': 10, 'LEU132': 10, 'PRO116': 10, 'CYS378': 10}, 'LEU495': {'CYS499': 10, 'ILE385': 10, 'TYR483': 10, 'ALA497': 10, 'PRO493': 10, 'ASP492': 10, 'GLN496': 10, 'LYS490': 1, 'HIP383': 10, 'VAL489': 10, 'PHE519': 10, 'LEU379': 10, 'PRO384': 8, 'TRP498': 10, 'PRO116': 6, 'ARG500': 10, 'LEU115': 2, 'LEU525': 10}, 'GLN496': {'GLH494': 10, 'CYS499': 10, 'ALA497': 10, 'GLU501': 10, 'PRO493': 10, 'ASP492': 10, 'VAL517': 10, 'SER518': 10, 'PHE519': 10, 'TRP498': 10, 'ARG500': 10}, 'ALA497': {'GLH494': 10, 'CYS499': 10, 'GLU501': 10, 'PRO493': 10, 'ASP492': 7, 'LEU495': 10, 'ILE502': 8, 'LEU379': 3, 'TRP498': 10, 'LEU132': 2, 'CYS378': 10, 'ARG500': 10}, 'TRP498': {'CYS499': 10, 'GLU501': 10, 'PRO493': 10, 'GLN496': 10, 'THR503': 10, 'LEU495': 10, 'ALA374': 10, 'ARG500': 10, 'GLH494': 10, 'ILE385': 10, 'ILE502': 10, 'PHE528': 10, 'LEU379': 10, 'CYS532': 10, 'CYS378': 10, 'LEU389': 10, 'MET376': 2, 'THR375': 10, 'PHE519': 10, 'LEU525': 10, 'ILE371': 10, 'LEU529': 10, 'ARG377': 2}, 'CYS499': {'GLH494': 9, 'GLU501': 10, 'ALA497': 10, 'VAL517': 10, 'GLN496': 10, 'PHE515': 1, 'THR503': 10, 'LEU495': 10, 'ILE502': 10, 'ALA374': 3, 'PRO516': 10, 'SER518': 10, 'PHE519': 10, 'PHE528': 9, 'GLU504': 10, 'ARG500': 10}, 'ARG500': {'GLU501': 10, 'VAL517': 10, 'ALA497': 10, 'GLN496': 10, 'THR503': 10, 'LEU495': 10, 'ILE502': 10, 'TRP141': 7, 'SER518': 2, 'GLU504': 10, 'VAL505': 10, 'ARG377': 6, 'TRP498': 10, 'GLY506': 10, 'CYS378': 3}, 'GLU501': {'GLN136': 10, 'CYS499': 10, 'ALA497': 10, 'GLN496': 10, 'ARG377': 10, 'THR503': 10, 'ILE502': 10, 'ALA374': 10, 'TRP141': 10, 'GLU504': 10, 'VAL505': 10, 'VAL370': 10, 'TRP498': 10, 'GLY506': 10, 'LEU507': 10, 'CYS378': 10, 'VAL373': 10}, 'ILE502': {'CYS499': 10, 'ALA497': 8, 'CYS508': 10, 'MET531': 10, 'THR503': 10, 'ALA374': 10, 'VAL505': 10, 'ARG500': 10, 'PRO516': 6, 'PHE528': 10, 'CYS532': 10, 'TRP498': 10, 'LEU507': 10, 'GLU504': 10, 'CYS378': 1, 'PHE519': 2, 'VAL370': 10, 'GLY506': 10, 'GLN509': 6, 'VAL517': 2, 'PHE515': 10, 'VAL367': 10, 'ILE371': 10, 'ALA510': 5}, 'THR503': {'CYS499': 10, 'GLU501': 10, 'CYS508': 10, 'VAL517': 10, 'PHE515': 10, 'GLN511': 10, 'ALA510': 10, 'PRO516': 10, 'PHE528': 10, 'SER518': 1, 'VAL505': 10, 'VAL370': 1, 'TRP498': 10, 'GLY506': 10, 'LEU507': 10, 'GLU504': 10, 'ARG500': 10, 'GLN509': 10}, 'GLU504': {'CYS499': 10, 'GLU501': 10, 'CYS508': 10, 'VAL517': 10, 'ILE502': 10, 'GLN511': 10, 'ALA510': 10, 'VAL505': 10, 'GLY506': 10, 'LEU507': 10, 'ARG500': 10, 'GLN509': 10}, 'VAL505': {'GLU501': 10, 'VAL517': 1, 'CYS508': 10, 'GLY506': 10, 'THR503': 10, 'ILE502': 10, 'TRP141': 10, 'ARG500': 10, 'ARG377': 6, 'PRO149': 4, 'LEU507': 10, 'LEU150': 10, 'VAL373': 1, 'GLN509': 10}, 'GLY506': {'THR151': 9, 'GLU501': 10, 'CYS508': 10, 'LEU148': 1, 'VAL373': 4, 'THR503': 10, 'ILE502': 10, 'ALA374': 5, 'TRP141': 6, 'VAL370': 10, 'PRO149': 10, 'LEU507': 10, 'GLU504': 10, 'LEU150': 10, 'ARG500': 10, 'GLN509': 3}, 'LEU507': {'GLU501': 10, 'CYS508': 10, 'ARG513': 10, 'MET531': 1, 'THR503': 10, 'VAL505': 10, 'LEU362': 6, 'ILE502': 10, 'PHE528': 4, 'GLU504': 10, 'LEU436': 10, 'ALA433': 1, 'VAL370': 10, 'PRO149': 10, 'LEU150': 10, 'GLN509': 10, 'PHE515': 10, 'VAL367': 10, 'LEU366': 1, 'ALA510': 10, 'ILE371': 2}, 'CYS508': {'GLU146': 4, 'GLY506': 10, 'ARG513': 10, 'THR503': 10, 'GLY147': 10, 'ILE502': 10, 'ALA510': 10, 'VAL505': 10, 'LEU148': 10, 'PRO149': 10, 'GLU504': 10, 'LEU150': 10, 'GLN509': 10}, 'GLN509': {'ARG513': 10, 'THR503': 10, 'ASP512': 10, 'ILE502': 6, 'GLN511': 10, 'ALA510': 10, 'VAL505': 10, 'GLY506': 3, 'LEU507': 10, 'GLU504': 10}, 'ALA510': {'CYS508': 10, 'ARG513': 10, 'PHE515': 10, 'THR503': 10, 'ASP512': 10, 'TYR438': 4, 'ILE502': 5, 'GLN511': 10, 'GLY514': 10, 'THR437': 1, 'GLU504': 10, 'LEU507': 10, 'LEU436': 8}, 'GLN511': {'VAL517': 10, 'ARG513': 10, 'PHE515': 10, 'THR503': 10, 'ASP512': 10, 'GLY514': 10, 'PRO516': 10, 'GLU504': 10, 'GLN509': 10}, 'ASP512': {'ARG513': 10, 'PHE515': 6, 'GLY514': 10, 'ALA510': 10, 'THR437': 4, 'GLN509': 10}, 'ARG513': {'CYS508': 10, 'ALA432': 4, 'ALA433': 10, 'GLN435': 10, 'TYR438': 10, 'PHE515': 10, 'CYS439': 10, 'GLN511': 10, 'GLY514': 10, 'THR437': 10, 'ALA510': 10, 'ALA434': 10, 'PRO149': 9, 'LEU507': 10, 'LEU436': 10, 'GLN509': 10}, 'GLY514': {'PHE515': 10, 'TYR438': 10, 'ASP512': 10, 'GLN511': 10, 'CYS439': 10, 'THR437': 9, 'ALA510': 10, 'PRO516': 10}, 'PHE515': {'CYS499': 1, 'ARG513': 10, 'MET531': 10, 'TYR438': 10, 'THR503': 10, 'ASP512': 6, 'GLN511': 10, 'LEU441': 10, 'ILE502': 10, 'THR437': 10, 'PRO516': 10, 'PHE528': 10, 'LEU507': 10, 'LEU436': 10, 'VAL517': 10, 'VAL367': 10, 'ALA510': 10, 'ILE371': 3, 'SER440': 5}, 'PRO516': {'CYS499': 10, 'VAL517': 10, 'MET531': 3, 'TYR438': 10, 'THR503': 10, 'ILE502': 6, 'GLN511': 10, 'GLY514': 10, 'GLN524': 10, 'PHE528': 10, 'SER518': 10, 'PHE519': 10, 'GLN520': 4}, 'VAL517': {'CYS499': 10, 'GLN496': 10, 'PHE515': 10, 'THR503': 10, 'ILE502': 2, 'GLN511': 10, 'GLN524': 1, 'SER518': 10, 'PHE519': 10, 'VAL505': 1, 'GLU504': 10, 'ARG500': 10}, 'SER518': {'CYS499': 10, 'GLN496': 10, 'LYS490': 1, 'THR503': 1, 'GLN524': 10, 'PRO516': 10, 'PHE519': 10, 'GLN520': 10, 'ARG500': 2}, 'PHE519': {'CYS499': 10, 'VAL517': 10, 'LYS490': 10, 'GLN496': 10, 'GLN522': 7, 'TYR438': 6, 'LEU495': 10, 'ILE502': 2, 'GLN524': 10, 'VAL489': 10, 'PRO516': 10, 'PHE528': 10, 'LEU529': 3, 'GLN520': 10, 'TRP498': 10, 'SER521': 10, 'LEU525': 10}, 'GLN520': {'LYS490': 10, 'GLN522': 10, 'GLN524': 10, 'VAL489': 7, 'PRO516': 4, 'SER518': 10, 'ASP486': 10, 'SER521': 10}, 'SER521': {'GLN522': 10, 'LYS490': 10, 'SER523': 10, 'CYS526': 7, 'GLN524': 10, 'VAL489': 10, 'PHE519': 10, 'ASP486': 10, 'LEU525': 10}, 'GLN522': {'HID480': 10, 'TYR483': 10, 'ILE488': 4, 'ARG485': 10, 'SER523': 10, 'CYS526': 10, 'GLN524': 10, 'VAL479': 10, 'VAL489': 10, 'GLU476': 10, 'PHE519': 7, 'GLN520': 10, 'HIE527': 10, 'ASP487': 1, 'GLN484': 10, 'ASP486': 10, 'LEU525': 10}, 'SER523': {'CYS526': 10, 'GLN524': 10, 'PHE528': 10, 'GLU476': 10, 'HIE527': 10, 'ASP486': 10, 'SER521': 10, 'LEU525': 10}, 'GLN524': {'VAL517': 1, 'GLN522': 10, 'TYR438': 10, 'CYS526': 10, 'PRO516': 10, 'SER518': 10, 'PHE519': 10, 'GLN520': 10, 'PHE528': 10, 'LEU529': 10, 'HIE527': 10, 'SER521': 10, 'LEU525': 10}, 'LEU525': {'ILE385': 10, 'TYR483': 10, 'GLN522': 10, 'SER523': 10, 'CYS526': 10, 'LEU495': 0, 'VAL479': 10, 'VAL489': 10, 'PHE528': 10, 'PHE519': 10, 'LEU529': 10, 'TRP498': 10, 'HIE527': 10, 'SER521': 10, 'THR530': 10}, 'CYS526': {'VAL475': 10, 'HID480': 6, 'GLN522': 10, 'MET531': 10, 'SER523': 10, 'GLN524': 10, 'VAL479': 10, 'PHE528': 10, 'GLU476': 10, 'LEU529': 10, 'ALA472': 10, 'HIE527': 10, 'SER521': 7, 'THR530': 10}, 'HIE527': {'THR530': 10, 'GLN522': 10, 'MET531': 10, 'SER523': 10, 'TYR438': 10, 'CYS439': 7, 'GLN524': 10, 'ASP445': 10, 'TRP468': 10, 'PHE528': 10, 'LEU529': 10, 'PRO444': 8, 'CYS532': 10, 'ALA472': 4, 'CYS442': 10, 'LEU525': 10}, 'PHE528': {'CYS499': 9, 'MET531': 10, 'TYR438': 10, 'THR503': 10, 'CYS526': 10, 'THR530': 10, 'ILE502': 10, 'PRO516': 10, 'TRP468': 2, 'CYS532': 10, 'TRP498': 10, 'LEU507': 4, 'PHE519': 10, 'LEU525': 10, 'PHE515': 10, 'SER523': 10, 'GLN524': 10, 'VAL533': 10, 'ILE371': 10, 'LEU529': 10}, 'LEU529': {'MET531': 10, 'CYS526': 10, 'HIE527': 10, 'THR530': 10, 'ILE385': 10, 'PHE534': 10, 'TYR614': 4, 'VAL479': 10, 'CYS532': 10, 'TRP498': 10, 'LEU389': 10, 'VAL475': 10, 'PHE388': 10, 'HID392': 1, 'GLU476': 7, 'PHE519': 3, 'LEU525': 10, 'GLN524': 10, 'VAL533': 10, 'ILE471': 3}, 'THR530': {'VAL475': 10, 'MET531': 10, 'PHE534': 10, 'THR535': 10, 'CYS526': 10, 'VAL533': 10, 'TRP468': 10, 'GLU476': 3, 'PHE528': 10, 'ALA472': 10, 'CYS532': 10, 'CYS442': 6, 'GLU469': 5, 'ILE471': 10, 'HIE527': 10, 'LEU525': 10}, 'MET531': {'TYR438': 10, 'CYS526': 10, 'LEU441': 10, 'HIE527': 10, 'CYS536': 10, 'PHE534': 10, 'THR535': 10, 'ILE502': 10, 'TRP468': 10, 'PHE528': 10, 'PRO516': 3, 'CYS532': 10, 'LEU507': 1, 'PHE515': 10, 'VAL367': 6, 'VAL533': 10, 'ILE371': 10, 'LEU529': 10, 'CYS442': 10}, 'CYS532': {'CYS536': 10, 'PHE534': 10, 'THR535': 10, 'TYR438': 1, 'VAL367': 2, 'ILE502': 10, 'VAL533': 10, 'ILE371': 10, 'HID392': 5, 'PHE528': 10, 'LEU529': 10, 'TRP498': 10, 'THR537': 8, 'HIE527': 10, 'LEU389': 10, 'THR530': 10}, 'VAL533': {'MET531': 10, 'PHE627': 8, 'THR530': 10, 'CYS536': 10, 'PHE534': 10, 'TYR614': 10, 'THR535': 10, 'ALA538': 10, 'PHE528': 10, 'LEU389': 10, 'LEU607': 10, 'VAL475': 10, 'PHE388': 10, 'HID392': 10, 'TYR474': 10, 'ILE371': 5, 'LEU529': 10, 'ILE471': 10, 'THR537': 10, 'GLN539': 1}, 'PHE534': {'MET531': 10, 'ASP463': 5, 'LEU441': 10, 'ILE471': 10, 'THR530': 10, 'CYS536': 10, 'PRO443': 10, 'THR535': 10, 'LEU652': 10, 'ALA538': 10, 'TRP468': 10, 'CYS532': 10, 'TYR460': 10, 'LEU607': 4, 'CYS442': 6, 'ALA542': 2, 'ALA464': 10, 'LEU529': 10, 'LEU467': 10, 'LEU465': 6, 'THR537': 10, 'GLN539': 10}, 'THR535': {'ALA368': 10, 'CYS536': 10, 'ALA541': 2, 'MET531': 10, 'VAL661': 10, 'VAL367': 10, 'ALA538': 8, 'THR364': 10, 'ASN363': 10, 'ILE371': 10, 'LEU441': 10, 'TRP468': 10, 'VAL533': 10, 'CYS532': 10, 'THR537': 10, 'GLN539': 10, 'THR530': 10}, 'CYS536': {'MET531': 10, 'ASN659': 8, 'THR396': 3, 'ILE393': 10, 'ALA368': 10, 'PHE534': 10, 'ALA538': 10, 'THR364': 10, 'CYS532': 10, 'LEU389': 7, 'ALA541': 1, 'SER660': 10, 'GLH658': 6, 'HID392': 10, 'VAL661': 10, 'VAL367': 2, 'VAL533': 10, 'ILE371': 10, 'THR537': 10, 'GLN539': 9}, 'THR537': {'ILE657': 10, 'ALA541': 10, 'PHE534': 10, 'VAL661': 10, 'THR535': 10, 'LEU607': 10, 'SER660': 10, 'ALA538': 10, 'ASN659': 10, 'VAL533': 10, 'GLH658': 10, 'HID392': 10, 'CYS532': 8, 'ALA542': 10, 'ILE471': 10, 'GLN539': 10}, 'ALA538': {'CYS536': 10, 'ALA541': 10, 'ILE657': 10, 'PHE534': 10, 'THR535': 8, 'LEU607': 10, 'LEU652': 10, 'VAL533': 10, 'GLH658': 3, 'ILE543': 10, 'ALA542': 10, 'LEU467': 10, 'TYR460': 3, 'ILE471': 10, 'ALA464': 8, 'GLN539': 10}, 'GLN539': {'LEU441': 10, 'ASN363': 10, 'CYS536': 9, 'PHE534': 10, 'MET239': 10, 'PRO443': 10, 'THR535': 10, 'LEU652': 10, 'SER440': 10, 'THR364': 10, 'TRP468': 1, 'TYR460': 10, 'TYR359': 3, 'ALA541': 10, 'LEU447': 3, 'ALA542': 10, 'ILE543': 10, 'ALA464': 6, 'VAL533': 1, 'CYS442': 9, 'THR537': 10}, 'ALA541': {'ILE657': 10, 'CYS536': 1, 'CYS656': 9, 'LEU652': 10, 'THR535': 2, 'ALA538': 10, 'GLY546': 10, 'GLN545': 10, 'GLH658': 10, 'TYR651': 10, 'ALA542': 10, 'ILE543': 10, 'THR537': 10, 'GLN539': 10}, 'ALA542': {'PHE534': 2, 'MET239': 10, 'LEU652': 10, 'ASN237': 1, 'HIE355': 7, 'ALA236': 10, 'ALA538': 10, 'GLY546': 9, 'GLN545': 10, 'GLY235': 10, 'THR364': 1, 'TYR651': 10, 'ILE543': 10, 'TYR460': 10, 'PRO238': 10, 'THR537': 10, 'TYR649': 10, 'GLN539': 10}, 'ILE543': {'GLN547': 10, 'GLH356': 1, 'ALA236': 10, 'ASN363': 2, 'TYR651': 3, 'TYR649': 1, 'MET239': 10, 'ALA538': 10, 'GLY546': 10, 'TYR551': 1, 'THR364': 10, 'PRO238': 10, 'TYR359': 10, 'ALA541': 10, 'HIE355': 10, 'GLY235': 1, 'ASN237': 10, 'GLN545': 10, 'GLN539': 10}, 'GLN545': {'LEU548': 10, 'ALA541': 10, 'ARG600': 8, 'CYS656': 10, 'GLN547': 10, 'GLY546': 10, 'LEU597': 4, 'SER598': 10, 'GLH658': 2, 'TYR651': 10, 'ALA542': 10, 'ILE543': 10, 'ASP549': 10}, 'GLY546': {'LEU548': 10, 'ALA541': 10, 'TRP550': 10, 'LEU214': 3, 'GLN547': 10, 'HIE355': 10, 'ALA236': 10, 'TYR551': 10, 'GLY235': 7, 'SER598': 7, 'TYR651': 10, 'ALA542': 9, 'ILE543': 10, 'ASP549': 10}, 'GLN547': {'LEU548': 10, 'ILE593': 4, 'TRP550': 10, 'GLH356': 10, 'HIE355': 10, 'ALA236': 10, 'SER594': 10, 'TYR551': 10, 'LEU597': 10, 'SER598': 10, 'PHE352': 3, 'ILE543': 10, 'ALA552': 10, 'GLN545': 10, 'ASP549': 10}, 'LEU548': {'TRP553': 10, 'ALA215': 10, 'TRP550': 10, 'TRP595': 10, 'SER212': 10, 'PHE207': 1, 'ARG599': 9, 'GLN210': 10, 'GLY546': 10, 'TYR551': 10, 'MET591': 8, 'LYS211': 10, 'SER594': 10, 'SER598': 10, 'ALA552': 10, 'GLN545': 10, 'ASP549': 10}, 'ASP549': {'TRP553': 9, 'ALA215': 10, 'TRP550': 10, 'LEU214': 10, 'SER212': 10, 'GLN547': 10, 'ALA213': 1, 'ALA236': 6, 'GLY546': 10, 'GLN545': 10, 'TYR551': 10, 'SER598': 10, 'ASN556': 6, 'TYR651': 10, 'ALA552': 10, 'VAL218': 10}, 'TRP550': {'LEU214': 10, 'TYR230': 10, 'GLN547': 10, 'ALA236': 10, 'ASN556': 10, 'TYR651': 10, 'ALA552': 10, 'TYR649': 3, 'ALA557': 10, 'GLY546': 10, 'ASN234': 10, 'TYR551': 10, 'GLN231': 10, 'LEU548': 10, 'TRP553': 9, 'ALA215': 8, 'ARG348': 10, 'GLY235': 10, 'LEU227': 10, 'PHE228': 10, 'VAL218': 10}, 'TYR551': {'GLN590': 10, 'GLN547': 10, 'GLH356': 4, 'ALA236': 10, 'PHE352': 10, 'ASN556': 10, 'ALA552': 10, 'ALA557': 10, 'VAL554': 10, 'GLY546': 10, 'SER594': 10, 'MET591': 6, 'GLN231': 8, 'LEU548': 10, 'TRP553': 10, 'ASN349': 2, 'PRO558': 8, 'ARG348': 10, 'HIE355': 10, 'ILE543': 1, 'ASP351': 10, 'ASN237': 10, 'VAL218': 1, 'ASP549': 10}, 'ALA552': {'LEU548': 10, 'TRP553': 10, 'VAL554': 10, 'ALA215': 10, 'TRP550': 10, 'GLN590': 10, 'SER212': 2, 'GLN547': 10, 'TRP595': 3, 'PHE207': 10, 'GLN210': 10, 'MET591': 10, 'SER594': 10, 'LYS211': 10, 'ASN556': 9, 'VAL218': 1, 'ASP549': 10}, 'TRP553': {'GLN590': 6, 'PHE207': 10, 'GLN223': 2, 'ASN556': 10, 'ARG219': 10, 'ALA557': 10, 'VAL554': 10, 'MET591': 10, 'TYR551': 10, 'ILE206': 6, 'LEU548': 10, 'ALA215': 10, 'TRP550': 9, 'GLU201': 4, 'ASP204': 10, 'TRP222': 10, 'PRO555': 10, 'PHE203': 10, 'LYS211': 2, 'LEU200': 10, 'VAL218': 10, 'ASP549': 9}, 'VAL554': {'CYS559': 10, 'LEU581': 10, 'GLN590': 10, 'PHE207': 10, 'ASN556': 10, 'PRO582': 10, 'CYS588': 10, 'ALA552': 10, 'ALA557': 10, 'TRP197': 4, 'MET591': 10, 'TYR551': 10, 'GLN586': 7, 'PRO558': 10, 'ALA587': 10, 'PRO555': 10, 'PHE203': 10, 'ILE206': 9, 'THR560': 3}, 'PRO555': {'TRP553': 10, 'LEU581': 10, 'PRO558': 10, 'VAL577': 3, 'PHE203': 10, 'TRP222': 10, 'MET591': 6, 'SER345': 10, 'ASN556': 10, 'LEU341': 5, 'PRO582': 10, 'LEU200': 10, 'ILE206': 1, 'ALA587': 1, 'ALA557': 10}, 'ASN556': {'TRP553': 10, 'VAL554': 10, 'PRO558': 10, 'TRP550': 10, 'LEU581': 4, 'ARG348': 10, 'ALA557': 10, 'GLN590': 2, 'TYR551': 10, 'SER345': 10, 'PHE228': 2, 'LEU341': 6, 'ARG219': 3, 'ALA552': 9, 'TRP222': 10, 'VAL218': 10, 'ASP549': 6, 'LYS344': 2}, 'ALA557': {'PRO555': 10, 'ASN349': 10, 'TRP553': 10, 'VAL554': 10, 'ASP351': 9, 'TRP550': 10, 'ARG348': 10, 'PRO558': 10, 'CYS559': 10, 'GLN590': 10, 'TYR551': 10, 'SER345': 10, 'PHE352': 10}, 'PRO558': {'PRO555': 10, 'ASN349': 10, 'VAL554': 10, 'CYS559': 10, 'LEU581': 10, 'GLN590': 10, 'ARG348': 10, 'TRP346': 7, 'PHE203': 3, 'TYR551': 8, 'SER345': 10, 'GLN353': 10, 'PHE352': 10, 'ASN556': 10, 'PRO582': 10, 'SER350': 1, 'THR560': 10, 'MET561': 10}, 'CYS559': {'ASN349': 10, 'THR420': 10, 'VAL554': 10, 'GLN590': 10, 'ALA417': 10, 'GLN353': 10, 'VAL418': 10, 'PHE352': 10, 'PRO582': 10, 'GLN586': 10, 'SER419': 10, 'ALA587': 10, 'THR560': 10, 'ALA557': 10, 'MET561': 10}, 'THR560': {'THR420': 2, 'LEU581': 10, 'GLN590': 10, 'GLN294': 10, 'PHE352': 1, 'PRO582': 10, 'TYR295': 10, 'MET561': 10, 'VAL554': 3, 'ALA417': 10, 'VAL418': 10, 'GLN586': 10, 'SER419': 10, 'ASN349': 10, 'ILE289': 10, 'PRO558': 10, 'ARG562': 10, 'GLN353': 10, 'ASN287': 9, 'LEU296': 5, 'SER580': 9, 'ASP583': 10}, 'MET561': {'SER580': 10, 'ASN349': 10, 'CYS559': 10, 'PRO558': 10, 'ARG562': 10, 'LEU581': 10, 'TRP346': 10, 'VAL577': 7, 'LYS293': 9, 'GLN294': 10, 'SER345': 10, 'PRO565': 1, 'PRO582': 10, 'MET563': 10, 'TYR295': 10, 'SER419': 10, 'LEU296': 10, 'LEU342': 10}, 'ARG562': {'SER580': 10, 'GLY579': 10, 'LEU581': 10, 'ASP583': 10, 'TRP197': 10, 'TRP346': 6, 'GLU292': 10, 'LYS293': 10, 'GLN294': 10, 'ARG290': 1, 'VAL584': 10, 'GLN586': 10, 'PRO564': 9, 'MET563': 10, 'TYR295': 10, 'PRO582': 10, 'MET578': 10, 'THR560': 10}, 'MET563': {'SER580': 10, 'GLY579': 10, 'THR576': 10, 'TRP346': 10, 'ALA575': 5, 'GLU292': 1, 'LYS293': 10, 'GLN294': 10, 'PRO565': 10, 'PRO564': 10, 'TYR295': 10, 'MET561': 10}, 'PRO564': {'GLN319': 10, 'SER580': 9, 'ARG562': 9, 'TRP346': 10, 'LYS293': 2, 'PRO565': 10, 'PRO566': 10, 'TYR295': 10}, 'PRO565': {'GLN319': 10, 'SER580': 4, 'PHE331': 9, 'TRP346': 10, 'THR567': 10, 'PRO566': 10, 'ALA297': 10, 'ILE318': 10, 'TYR295': 10, 'MET563': 10, 'LEU296': 10, 'LEU342': 9, 'MET561': 1}, 'PRO566': {'SER580': 3, 'THR576': 10, 'TRP346': 10, 'VAL572': 10, 'LYS569': 10, 'THR329': 10, 'THR567': 10, 'ALA339': 10, 'THR568': 10, 'ILE318': 10, 'PRO564': 10, 'ALA343': 5, 'PHE331': 10, 'ASP571': 5, 'LEU342': 10}, 'THR567': {'LYS569': 10, 'THR329': 10, 'ASP571': 3, 'PRO565': 10, 'THR568': 10, 'PHE331': 3, 'PRO328': 7}, 'THR568': {'VAL572': 10, 'LYS569': 10, 'THR329': 10, 'PRO566': 10, 'GLU570': 10, 'PHE331': 4, 'ASP571': 10}, 'LYS569': {'LEU332': 10, 'LEU330': 10, 'ASP335': 10, 'VAL572': 10, 'ASP571': 10, 'THR329': 10, 'THR567': 10, 'ALA339': 10, 'PRO566': 10, 'TRP340': 3, 'ALA343': 1, 'GLU570': 10, 'PHE331': 10, 'MET257': 10, 'PRO336': 10}, 'GLU570': {'VAL572': 10, 'ALA339': 2, 'THR568': 10, 'ASP571': 10, 'PRO336': 10}, 'ASP571': {'THR573': 10, 'THR576': 10, 'VAL572': 10, 'LYS569': 10, 'THR567': 3, 'THR568': 10, 'PRO566': 5}, 'VAL572': {'THR573': 10, 'VAL577': 10, 'MET574': 10, 'THR576': 10, 'LYS569': 10, 'ALA339': 10, 'LEU338': 10, 'PRO566': 10, 'THR568': 10, 'GLU570': 10, 'PHE331': 10, 'LEU342': 10, 'PRO336': 9}, 'THR573': {'VAL577': 10, 'MET574': 10, 'THR576': 10, 'ALA575': 10, 'LEU338': 10, 'MET578': 10, 'ASP571': 10, 'LEU342': 3}, 'MET574': {'GLY579': 10, 'VAL577': 10, 'THR576': 10, 'VAL572': 10, 'ALA575': 10, 'GLU201': 1, 'ASN198': 10, 'LEU338': 10, 'CYS199': 10, 'MET578': 10, 'LEU200': 10}, 'ALA575': {'THR573': 10, 'GLY579': 10, 'VAL577': 10, 'SER580': 10, 'THR576': 10, 'ASN198': 10, 'CYS199': 4, 'MET563': 5, 'MET578': 10}, 'THR576': {'THR573': 10, 'GLY579': 10, 'VAL577': 10, 'SER580': 10, 'MET574': 10, 'LEU581': 10, 'TRP346': 3, 'VAL572': 10, 'PRO566': 10, 'MET563': 10, 'MET578': 10, 'ASP571': 10, 'LEU342': 10}, 'VAL577': {'THR573': 10, 'GLY579': 10, 'SER580': 10, 'LEU581': 10, 'MET574': 10, 'PRO555': 3, 'VAL572': 10, 'TRP346': 1, 'ALA575': 10, 'TRP222': 10, 'ALA339': 5, 'LEU338': 10, 'SER345': 9, 'LEU341': 10, 'MET578': 10, 'LEU200': 10, 'LEU342': 10, 'MET561': 7}, 'MET578': {'THR573': 10, 'GLY579': 10, 'ARG562': 10, 'SER580': 10, 'MET574': 10, 'THR576': 10, 'LEU581': 10, 'TRP197': 10, 'PHE203': 10, 'ALA575': 10, 'ASN198': 10, 'CYS199': 10, 'PRO582': 8, 'LEU200': 10, 'LEU342': 1, 'ASP202': 10}, 'GLY579': {'SER580': 10, 'ARG562': 10, 'VAL577': 10, 'MET574': 10, 'THR576': 10, 'LEU581': 10, 'ALA575': 10, 'MET563': 10}, 'SER580': {'ARG562': 10, 'VAL577': 10, 'LEU581': 10, 'THR576': 10, 'TRP346': 10, 'ALA575': 10, 'PRO565': 4, 'PRO566': 3, 'PRO564': 9, 'MET563': 10, 'PRO582': 10, 'MET578': 10, 'THR560': 9, 'LEU342': 10, 'MET561': 10}, 'LEU581': {'GLY579': 10, 'VAL577': 10, 'ASN556': 4, 'PRO582': 10, 'LEU341': 2, 'MET561': 10, 'VAL554': 10, 'THR576': 10, 'TRP197': 10, 'MET578': 10, 'LEU342': 10, 'ARG562': 10, 'PRO558': 10, 'SER345': 10, 'TRP222': 10, 'ASP583': 10, 'PRO555': 10, 'PHE203': 10, 'LEU200': 10, 'THR560': 10}, 'PRO582': {'CYS559': 10, 'GLN590': 10, 'GLN294': 8, 'VAL584': 10, 'MET561': 10, 'VAL554': 10, 'TRP197': 10, 'ALA417': 1, 'GLN586': 10, 'MET578': 8, 'ARG562': 10, 'ASN349': 1, 'PRO558': 10, 'ALA587': 10, 'SER580': 10, 'PRO555': 10, 'ASP583': 10, 'PHE203': 10, 'THR560': 10}, 'ASP583': {'ARG562': 10, 'LEU581': 10, 'ILE289': 3, 'PHE203': 3, 'TRP197': 10, 'GLU292': 10, 'ARG290': 10, 'GLN294': 7, 'VAL584': 10, 'GLN586': 10, 'CYS588': 9, 'ALA587': 10, 'THR560': 10, 'ARG585': 10}, 'VAL584': {'ARG562': 10, 'ARG189': 10, 'PHE203': 1, 'LYS188': 9, 'TRP197': 10, 'MET185': 1, 'LEU589': 10, 'ARG290': 5, 'THR192': 10, 'PRO582': 10, 'GLN586': 10, 'CYS588': 10, 'ILE206': 10, 'ALA587': 10, 'ARG585': 10}, 'ARG585': {'ASP583': 10, 'ILE289': 7, 'ARG189': 10, 'GLN590': 9, 'LYS416': 10, 'LEU589': 10, 'ARG290': 10, 'GLN586': 10, 'CYS588': 10, 'MET185': 10, 'ALA587': 10}, 'GLN586': {'CYS559': 10, 'GLN590': 10, 'GLN294': 10, 'CYS588': 10, 'PRO582': 10, 'VAL584': 10, 'VAL554': 7, 'ALA417': 10, 'GLU292': 2, 'MET591': 9, 'VAL418': 6, 'ILE289': 10, 'ARG562': 10, 'MET185': 8, 'ALA587': 10, 'ASP583': 10, 'LYS416': 10, 'LEU589': 10, 'ARG290': 10, 'THR560': 10}, 'ALA587': {'ASP583': 10, 'PRO555': 1, 'VAL554': 10, 'CYS559': 10, 'GLN590': 10, 'PHE203': 10, 'TRP197': 10, 'ALA592': 8, 'LEU589': 10, 'MET591': 10, 'CYS588': 10, 'PRO582': 10, 'ILE206': 10, 'VAL584': 10, 'ARG585': 10}, 'CYS588': {'ASP583': 9, 'VAL554': 10, 'ILE593': 10, 'MET185': 10, 'ARG189': 10, 'LYS188': 10, 'GLN590': 10, 'GLU184': 4, 'ALA592': 10, 'TRP197': 3, 'LEU589': 10, 'MET591': 10, 'THR192': 7, 'GLN586': 10, 'ILE206': 10, 'VAL584': 10, 'ARG585': 10}, 'LEU589': {'ALA182': 6, 'ILE593': 10, 'GLN590': 10, 'ALA417': 10, 'LYS416': 10, 'ALA592': 10, 'MET591': 10, 'SER594': 10, 'VAL418': 2, 'ILE413': 10, 'GLN586': 10, 'MET185': 10, 'ALA587': 10, 'VAL584': 10, 'ARG585': 10}, 'GLN590': {'CYS559': 10, 'ALA592': 10, 'PHE352': 10, 'PRO582': 10, 'CYS588': 10, 'ASN556': 2, 'ALA552': 10, 'ALA557': 10, 'VAL554': 10, 'ILE593': 10, 'ALA417': 8, 'MET591': 10, 'TYR551': 10, 'SER594': 10, 'GLN586': 10, 'TRP553': 6, 'PRO558': 10, 'TRP595': 10, 'ALA587': 10, 'THR560': 10, 'ARG585': 9}, 'MET591': {'PHE207': 10, 'ALA592': 10, 'CYS588': 10, 'ALA552': 10, 'VAL554': 10, 'ILE593': 10, 'TRP197': 4, 'SER594': 10, 'TYR551': 6, 'GLN586': 9, 'HIE596': 9, 'LEU548': 8, 'TRP553': 10, 'TRP595': 10, 'LYS188': 10, 'ASP204': 1, 'ALA587': 10, 'PRO555': 6, 'PHE203': 10, 'GLU184': 2, 'LEU589': 10, 'ILE206': 10}, 'ALA592': {'ALA182': 10, 'ILE593': 10, 'TRP595': 10, 'GLN590': 10, 'LYS188': 1, 'GLU184': 9, 'LEU589': 10, 'SER594': 10, 'LEU597': 8, 'GLY181': 10, 'CYS588': 10, 'HIE596': 10, 'LEU178': 4, 'MET185': 10, 'ALA587': 8}, 'ILE593': {'ALA182': 1, 'TRP595': 10, 'GLN590': 10, 'GLN547': 4, 'GLH356': 6, 'LEU589': 10, 'MET591': 10, 'LEU407': 10, 'SER594': 10, 'LEU597': 10, 'SER598': 9, 'ILE413': 10, 'CYS588': 10, 'HIE596': 10, 'LEU178': 10}, 'SER594': {'LEU548': 10, 'TRP595': 10, 'GLN590': 10, 'GLN547': 10, 'GLH356': 1, 'ALA592': 10, 'ARG599': 10, 'LEU589': 10, 'MET591': 10, 'TYR551': 10, 'LEU597': 10, 'SER598': 10, 'PHE352': 4, 'HIE596': 10, 'ALA552': 10}, 'TRP595': {'THR177': 10, 'LEU548': 10, 'ARG600': 6, 'ILE593': 10, 'GLN590': 10, 'SER212': 1, 'ALA592': 10, 'ARG599': 10, 'GLN210': 10, 'MET591': 10, 'TRP176': 5, 'LEU597': 10, 'LYS211': 6, 'GLY181': 8, 'SER598': 10, 'HIE596': 10, 'ALA552': 3, 'ALA180': 3}, 'HIE596': {'THR177': 10, 'ASP173': 9, 'ILE593': 10, 'PHE174': 10, 'GLU175': 8, 'ILE399': 1, 'ALA592': 10, 'ARG599': 10, 'MET591': 9, 'SER594': 10, 'LEU597': 10, 'SER598': 10, 'GLY181': 5, 'GLN601': 7, 'LEU178': 10}, 'LEU597': {'ILE593': 10, 'PHE174': 9, 'TRP595': 10, 'GLN547': 10, 'GLH356': 10, 'ILE399': 2, 'ALA592': 8, 'ARG599': 10, 'HIE355': 1, 'SER594': 10, 'LEU407': 1, 'SER598': 10, 'GLN601': 5, 'GLN545': 4}, 'SER598': {'LEU548': 10, 'ARG600': 10, 'ILE593': 9, 'TRP595': 10, 'GLN547': 10, 'ARG599': 10, 'GLY546': 7, 'SER594': 10, 'HIE596': 10, 'GLN545': 10, 'ASP549': 10}, 'ARG599': {'THR177': 10, 'LEU548': 9, 'ASP173': 10, 'ARG600': 10, 'PHE174': 10, 'TRP595': 10, 'SER594': 10, 'LYS170': 10, 'TRP176': 5, 'LEU597': 10, 'PRO602': 10, 'GLN601': 10, 'HIE596': 10}, 'ARG600': {'SER655': 6, 'TRP595': 6, 'SER598': 10, 'PRO602': 10, 'PRO606': 6, 'GLN601': 10, 'MET604': 10, 'CYS656': 6, 'GLN545': 8}, 'GLN601': {'ASP173': 10, 'PHE174': 10, 'ARG394': 10, 'THR662': 10, 'ARG599': 10, 'LYS170': 10, 'LEU597': 5, 'GLH658': 10, 'PRO602': 10, 'MET604': 10, 'HIE596': 7, 'VAL605': 9, 'ASP603': 10}, 'PRO602': {'ASP173': 10, 'ARG600': 10, 'ARG599': 10, 'LYS170': 10, 'TRP176': 2, 'MET604': 10, 'ASP603': 10}, 'ASP603': {'ARG394': 10, 'TYR395': 7, 'THR662': 9, 'LYS170': 10, 'GLH658': 10, 'TRP13': 10, 'GLN601': 10, 'MET604': 10, 'HIE167': 10, 'VAL605': 10}, 'MET604': {'SER655': 10, 'ILE657': 10, 'ARG600': 10, 'ARG394': 10, 'LEU607': 10, 'LYS611': 1, 'THR662': 7, 'ASN659': 10, 'GLH658': 10, 'PRO606': 10, 'PRO602': 10, 'GLN601': 10, 'PRO654': 3, 'CYS656': 10, 'VAL605': 10}, 'VAL605': {'ILE657': 10, 'ARG394': 10, 'LYS611': 10, 'SER660': 10, 'THR662': 3, 'ASN659': 10, 'GLH658': 10, 'PRO606': 10, 'TRP13': 2, 'GLU612': 1, 'GLN601': 9, 'HID609': 10, 'HID610': 10, 'LEU607': 10, 'ASP603': 10}, 'PRO606': {'SER655': 10, 'ILE657': 10, 'ARG600': 6, 'CYS656': 9, 'LYS611': 10, 'ASN659': 10, 'GLY608': 10, 'ARG628': 9, 'GLH658': 10, 'HID609': 10, 'MET604': 10, 'HID610': 10, 'PRO654': 10, 'LEU607': 10}, 'LEU607': {'ASN659': 10, 'PHE627': 10, 'VAL605': 10, 'ILE657': 10, 'TYR614': 2, 'PHE534': 4, 'ALA538': 10, 'MET604': 10, 'SER655': 5, 'SER660': 2, 'GLH658': 10, 'HID392': 10, 'LEU631': 10, 'TYR474': 4, 'HID609': 10, 'HID610': 10, 'PRO654': 10, 'PRO391': 1, 'GLY608': 10, 'VAL533': 10, 'LEU467': 10, 'ILE471': 10, 'THR537': 10}, 'GLY608': {'SER655': 4, 'ILE657': 7, 'ASN659': 1, 'ARG628': 10, 'LEU624': 10, 'PRO606': 10, 'PHE627': 10, 'HID609': 10, 'HID610': 10, 'LEU631': 10, 'PRO654': 10}, 'HID609': {'LYS611': 10, 'ASN659': 8, 'ARG628': 10, 'LEU624': 4, 'PRO606': 10, 'LEU631': 1, 'PHE627': 2, 'HID610': 10, 'VAL605': 10, 'LYS613': 4, 'LEU607': 10}, 'HID610': {'TYR614': 10, 'LEU607': 10, 'LYS611': 10, 'ASN659': 10, 'GLY608': 10, 'LEU624': 10, 'PRO606': 10, 'PRO391': 10, 'TYR474': 10, 'HID392': 10, 'GLU612': 10, 'PHE627': 10, 'LYS613': 10, 'VAL605': 10}, 'LYS611': {'ASN659': 10, 'PRO606': 10, 'PRO391': 10, 'TRP13': 8, 'HID609': 10, 'GLU612': 10, 'MET604': 1, 'LYS613': 10, 'VAL605': 10}, 'GLU612': {'TYR614': 10, 'ASN659': 10, 'ASP79': 3, 'PRO391': 10, 'ILE390': 10, 'TRP13': 10, 'LYS387': 10, 'HID610': 10, 'TYR97': 10, 'LYS613': 10, 'VAL605': 1}, 'LYS613': {'TYR614': 10, 'LYS611': 10, 'PHE615': 10, 'SER616': 6, 'LEU624': 10, 'PRO391': 10, 'LYS387': 10, 'HID609': 4, 'HID610': 10, 'TYR97': 10, 'LYS621': 9}, 'TYR614': {'PHE615': 10, 'GLU612': 10, 'PHE627': 1, 'TYR97': 6, 'PRO620': 10, 'ILE390': 8, 'LEU389': 10, 'LEU607': 2, 'VAL475': 10, 'PHE388': 10, 'LEU624': 10, 'PRO391': 10, 'HID392': 10, 'LYS387': 10, 'TYR474': 10, 'HID610': 10, 'SER616': 10, 'ILE478': 10, 'VAL533': 10, 'LEU529': 4}, 'PHE615': {'LEU624': 1, 'ILE385': 10, 'GLY111': 8, 'PHE482': 10, 'HIP383': 4, 'SER616': 10, 'PHE388': 10, 'ILE478': 10, 'LEU481': 8, 'PRO620': 10, 'PHE386': 1, 'LYS387': 10, 'GLY617': 10, 'PRO384': 10, 'LYS621': 10, 'THR112': 8, 'LYS613': 10, 'ALA113': 10}, 'SER616': {'TYR614': 10, 'PRO620': 10, 'GLY617': 10, 'LYS387': 6, 'LYS621': 10, 'LYS613': 6, 'PRO618': 10, 'ARG80': 4}, 'GLY617': {'LYS619': 10, 'ALA622': 10, 'PHE615': 10, 'PRO620': 10, 'LYS621': 10, 'PRO618': 10}, 'PRO618': {'LYS619': 10, 'ALA622': 10, 'SER616': 10, 'PRO620': 10, 'VAL623': 10, 'LYS621': 10}, 'LYS619': {'ALA622': 10, 'VAL623': 10, 'PRO620': 10, 'LEU481': 10, 'LEU624': 10, 'ILE478': 7, 'GLY617': 10, 'LYS621': 10}, 'PRO620': {'TYR614': 10, 'ALA622': 10, 'PHE615': 10, 'PHE482': 10, 'ASN625': 10, 'SER616': 10, 'LEU481': 10, 'ILE478': 10, 'VAL623': 10, 'LEU624': 10, 'TYR474': 9, 'GLY617': 10, 'LYS621': 10, 'PRO618': 10}, 'LYS621': {'LYS619': 10, 'ALA622': 10, 'PHE615': 10, 'ASN625': 10, 'SER616': 10, 'VAL623': 10, 'LEU624': 10, 'ILE478': 1, 'GLY617': 10, 'GLN626': 10, 'LYS613': 9, 'PRO618': 10}, 'ALA622': {'LYS619': 10, 'ASN625': 10, 'VAL623': 10, 'PRO620': 10, 'LEU624': 10, 'PHE627': 10, 'GLY617': 10, 'GLN626': 10, 'PRO618': 10}, 'VAL623': {'VAL475': 7, 'LYS619': 10, 'GLY477': 10, 'ASN625': 10, 'LEU481': 10, 'ILE478': 10, 'PRO620': 10, 'LEU624': 10, 'PHE627': 10, 'TYR474': 10, 'GLN626': 10, 'LYS621': 10, 'ARG628': 10, 'GLU476': 1, 'PRO618': 10, 'ARG473': 10}, 'LEU624': {'LYS619': 10, 'THR629': 10, 'TYR614': 10, 'ALA622': 10, 'PHE615': 1, 'ASN625': 10, 'GLY608': 10, 'ILE478': 10, 'PRO620': 10, 'PHE627': 10, 'ARG628': 10, 'TYR474': 10, 'HID609': 4, 'HID610': 10, 'LYS621': 10, 'GLN626': 10, 'LYS613': 10}, 'ASN625': {'THR629': 10, 'ALA622': 10, 'ASP630': 10, 'VAL623': 10, 'PRO620': 10, 'ARG628': 10, 'PHE627': 10, 'GLN626': 10, 'LYS621': 10}, 'GLN626': {'THR629': 10, 'ALA622': 10, 'ASP630': 10, 'VAL623': 10, 'ARG628': 10, 'LEU624': 10, 'PHE627': 10, 'TYR474': 5, 'LYS621': 10, 'LEU631': 10, 'ILE470': 9, 'ARG473': 10}, 'PHE627': {'GLU632': 10, 'ALA622': 10, 'ASN625': 10, 'ASP630': 10, 'ARG628': 10, 'THR629': 10, 'TYR614': 1, 'VAL623': 10, 'LEU607': 10, 'LEU624': 10, 'HID392': 5, 'TYR474': 10, 'HID609': 2, 'HID610': 10, 'ILE470': 10, 'GLY608': 10, 'VAL533': 8, 'LEU631': 10, 'ILE471': 10, 'LEU467': 9, 'ARG473': 2}, 'ARG628': {'SER655': 3, 'THR629': 10, 'LYS633': 10, 'GLU632': 10, 'ASN625': 10, 'VAL623': 10, 'GLY608': 10, 'LEU624': 10, 'ASP630': 10, 'PRO606': 9, 'GLN626': 10, 'HID609': 10, 'LEU631': 10}, 'THR629': {'LYS633': 10, 'GLU632': 10, 'ASN625': 10, 'ASP630': 10, 'LEU624': 10, 'PHE627': 10, 'GLN626': 10, 'LEU631': 10, 'LEU634': 10}, 'ASP630': {'LYS633': 10, 'GLU632': 10, 'LEU634': 10, 'ASN625': 10, 'GLU635': 10, 'ARG628': 10, 'PHE627': 10, 'GLN626': 10, 'PRO654': 2, 'LEU631': 10, 'ILE470': 10, 'ARG473': 10}, 'LEU631': {'ILE657': 10, 'SER655': 1, 'THR629': 10, 'LYS633': 10, 'GLU632': 10, 'GLU635': 10, 'GLY608': 10, 'ARG628': 10, 'PHE627': 10, 'GLN626': 10, 'PRO654': 10, 'HID609': 1, 'LEU467': 10, 'ILE470': 10, 'ILE471': 10, 'LEU634': 10, 'LEU607': 10, 'LYS636': 10}, 'GLU632': {'SER655': 7, 'THR629': 10, 'LYS633': 10, 'ASP630': 10, 'GLU635': 10, 'ARG628': 10, 'PHE627': 10, 'PRO654': 10, 'GLU637': 10, 'LEU634': 10, 'LYS636': 10}, 'LYS633': {'THR629': 10, 'ASP630': 10, 'GLU635': 10, 'ARG628': 10, 'GLU637': 10, 'LEU631': 10, 'ILE470': 4, 'ILE638': 10, 'LEU634': 10, 'LYS636': 10}, 'LEU634': {'LYS653': 9, 'THR639': 10, 'THR629': 10, 'ILE657': 3, 'ASP463': 10, 'GLU632': 10, 'LEU652': 10, 'ASP630': 10, 'GLU635': 10, 'ARG466': 10, 'LEU631': 10, 'PRO654': 10, 'GLU637': 10, 'LEU467': 10, 'ILE470': 10, 'ILE638': 10, 'HID462': 5, 'LYS636': 10}, 'GLU635': {'LYS653': 10, 'THR639': 10, 'SER655': 10, 'CYS656': 9, 'LYS633': 10, 'GLU632': 10, 'ASP630': 10, 'ALA640': 9, 'PRO654': 10, 'GLU637': 10, 'LEU631': 10, 'ILE638': 10, 'LYS636': 10}, 'LYS636': {'LYS653': 1, 'THR639': 10, 'ARG641': 9, 'LYS633': 10, 'GLU632': 10, 'ALA640': 10, 'GLU637': 10, 'LEU631': 10, 'ILE638': 10, 'LEU634': 10}, 'GLU637': {'THR639': 10, 'ARG641': 10, 'LYS633': 10, 'GLU632': 10, 'ASN642': 10, 'GLU635': 10, 'ALA640': 10, 'ARG466': 10, 'ILE638': 10, 'LEU634': 10}, 'ILE638': {'GLU643': 10, 'THR639': 10, 'ARG641': 10, 'LYS653': 10, 'LYS633': 10, 'ASP463': 10, 'ASN642': 10, 'LEU652': 10, 'GLU635': 10, 'ALA640': 10, 'GLU650': 10, 'ARG466': 9, 'PRO654': 10, 'LEU467': 9, 'LEU634': 10, 'TYR649': 10, 'LYS636': 10}, 'THR639': {'GLU643': 10, 'LYS653': 10, 'ARG641': 10, 'ASN642': 10, 'ALA640': 10, 'GLU635': 10, 'GLU650': 4, 'GLU637': 10, 'LEU634': 10, 'LYS636': 10}, 'ALA640': {'GLU643': 10, 'GLN644': 10, 'ARG641': 10, 'GLU635': 9, 'GLU637': 10, 'ILE638': 10, 'ASN642': 10, 'LYS636': 10}, 'ARG641': {'GLU643': 10, 'THR639': 10, 'GLN644': 10, 'ASP463': 9, 'LEU459': 4, 'LEU645': 10, 'ARG466': 10, 'GLU637': 10, 'ILE638': 10, 'HID462': 6, 'ASN642': 10, 'LYS636': 9}, 'ASN642': {'GLU643': 10, 'THR639': 10, 'GLN644': 10, 'LYS653': 10, 'PRO648': 10, 'TRP647': 1, 'LEU459': 10, 'ALA640': 10, 'LEU645': 10, 'GLU650': 10, 'GLU637': 10, 'ILE638': 10, 'TYR649': 10}, 'GLU643': {'THR639': 10, 'ARG641': 10, 'GLN644': 10, 'PRO648': 9, 'ALA640': 10, 'LEU645': 10, 'ILE638': 10}, 'GLN644': {'ARG641': 10, 'ALA640': 10, 'LEU645': 10, 'ASP646': 10, 'ASN308': 3, 'ASN642': 10, 'TRP647': 1}, 'LEU645': {'GLU643': 10, 'ARG641': 10, 'PRO648': 10, 'LEU459': 10, 'TYR649': 10, 'ASP646': 10, 'ASN308': 9, 'ASN642': 10, 'TRP647': 10}, 'ASP646': {'GLN644': 10, 'PRO648': 10, 'TYR230': 10, 'LYS217': 4, 'LYS310': 3, 'GLU226': 4, 'TRP647': 10}, 'TRP647': {'PRO456': 8, 'GLN644': 1, 'PRO648': 10, 'TYR230': 10, 'ASN642': 1, 'LEU459': 10, 'ASN234': 10, 'GLY457': 10, 'LEU645': 10, 'LYS217': 1, 'ASN308': 10, 'LYS310': 10, 'GLU226': 5, 'TYR649': 10}, 'PRO648': {'GLU643': 9, 'LEU214': 5, 'TYR230': 8, 'LEU459': 10, 'ASN234': 10, 'LEU645': 10, 'GLU650': 10, 'ASP646': 10, 'ASN642': 10, 'TYR649': 10}, 'TYR649': {'LYS653': 10, 'ASP463': 10, 'TRP550': 3, 'ASN642': 10, 'LEU652': 10, 'LEU459': 10, 'ASN234': 10, 'GLY235': 10, 'ILE638': 10, 'LEU645': 10, 'GLU650': 10, 'TYR651': 10, 'ALA542': 10, 'ILE543': 1, 'TYR460': 10, 'PRO238': 10, 'ALA464': 9, 'TRP647': 10}, 'GLU650': {'LYS653': 10, 'THR639': 4, 'PRO648': 10, 'ASP463': 4, 'LEU652': 10, 'TYR651': 10, 'CYS656': 10, 'ILE638': 10, 'ASN642': 10}, 'TYR651': {'LYS653': 10, 'ALA541': 10, 'ILE657': 10, 'CYS656': 10, 'TRP550': 10, 'LEU214': 10, 'LEU652': 10, 'ALA236': 10, 'GLY546': 10, 'ASN234': 9, 'GLY235': 10, 'ALA542': 10, 'ILE543': 3, 'TYR460': 1, 'PRO238': 3, 'GLN545': 10, 'TYR649': 10, 'ASP549': 10}, 'LEU652': {'LYS653': 10, 'ALA541': 10, 'ILE657': 10, 'CYS656': 10, 'PHE534': 10, 'ASP463': 10, 'LEU634': 10, 'ALA538': 10, 'ILE638': 10, 'GLU650': 10, 'PRO654': 10, 'ALA542': 10, 'LEU467': 10, 'TYR460': 10, 'ALA464': 10, 'TYR649': 10, 'GLN539': 10}, 'LYS653': {'SER655': 10, 'THR639': 10, 'ILE657': 10, 'ASP463': 2, 'ASN642': 10, 'GLU635': 10, 'GLU650': 10, 'TYR651': 10, 'PRO654': 10, 'CYS656': 10, 'ILE638': 10, 'LEU634': 9, 'TYR649': 10, 'LYS636': 1}, 'PRO654': {'SER655': 10, 'ILE657': 10, 'CYS656': 10, 'GLU632': 10, 'LEU652': 10, 'GLU635': 10, 'ASP630': 2, 'GLY608': 10, 'PRO606': 10, 'MET604': 3, 'LEU631': 10, 'ILE470': 2, 'ILE638': 10, 'LEU467': 10, 'LEU634': 10, 'LEU607': 10}, 'SER655': {'LYS653': 10, 'ILE657': 10, 'ARG600': 6, 'GLU632': 7, 'GLU635': 10, 'GLY608': 4, 'ARG628': 3, 'PRO606': 10, 'LEU631': 1, 'MET604': 10, 'CYS656': 10, 'LEU607': 5}, 'CYS656': {'LYS653': 10, 'ALA541': 9, 'ARG600': 6, 'ILE657': 10, 'LEU652': 10, 'GLU635': 9, 'GLU650': 10, 'GLH658': 10, 'PRO606': 9, 'TYR651': 10, 'PRO654': 10, 'MET604': 10, 'GLN545': 10}, 'ILE657': {'ASN659': 10, 'TYR651': 10, 'VAL605': 10, 'LEU652': 10, 'ALA538': 10, 'MET604': 10, 'LEU607': 10, 'SER655': 10, 'ALA541': 10, 'SER660': 3, 'GLH658': 10, 'PRO654': 10, 'LEU634': 3, 'LYS653': 10, 'GLY608': 7, 'THR537': 0, 'PRO606': 10, 'LEU631': 10, 'ILE471': 5, 'LEU467': 10}, 'GLH658': {'CYS536': 6, 'ALA541': 10, 'VAL661': 10, 'ARG394': 10, 'LEU607': 10, 'SER660': 10, 'THR662': 10, 'ALA538': 3, 'ASN659': 10, 'GLN545': 2, 'PRO606': 10, 'GLN601': 10, 'MET604': 10, 'CYS656': 10, 'THR537': 10, 'VAL605': 10, 'ASP603': 10}, 'ASN659': {'CYS536': 8, 'ILE657': 10, 'VAL661': 10, 'ARG394': 10, 'LEU607': 10, 'LYS611': 10, 'SER660': 10, 'THR662': 6, 'GLY608': 1, 'PRO391': 10, 'HID392': 10, 'PRO606': 10, 'HID609': 8, 'MET604': 10, 'HID610': 10, 'GLU612': 10, 'THR537': 10, 'VAL605': 10}, 'SER660': {'CYS536': 10, 'ILE657': 3, 'VAL661': 10, 'ARG394': 10, 'TYR395': 10, 'LEU607': 2, 'THR662': 10, 'GLH658': 10, 'PRO391': 10, 'HID392': 10, 'THR396': 5, 'ILE393': 10, 'THR537': 10, 'VAL605': 10}, 'VAL661': {'ALA368': 10, 'CYS536': 10, 'ASN400': 9, 'ARG394': 10, 'TYR395': 10, 'THR535': 10, 'ILE399': 10, 'THR662': 10, 'ASN659': 10, 'THR364': 10, 'THR396': 10, 'HID392': 10, 'PRO391': 1, 'GLH658': 10, 'ILE393': 10, 'THR537': 10}, 'THR662': {'PHE174': 8, 'ARG394': 10, 'PHE166': 10, 'TYR395': 10, 'SER660': 10, 'ILE399': 10, 'LYS170': 10, 'ASN659': 6, 'THR396': 9, 'GLH658': 10, 'GLN601': 10, 'MET604': 7, 'VAL605': 3, 'ASP603': 9}}


In [3]:
from ..EMDA.plotters import plot_contacts_frequencies_differences

ImportError: attempted relative import with no known parent package